In [1]:
%%bash

# Instal Java
apt-get update --fix-missing
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,306 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,651 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,229 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/u

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [29]:
import os
from datetime import datetime
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, max

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enable", True)

In [4]:
#Criando os dataframes ou tabelas
clientes = [
    (1, "Joao", "M", "MG", "Betim"),
    (2, "Luiza", "F", "SP", "São Caetano"),
    (3, "Gabriela", "F", "MG", "Belo Horizonte"),
    (4, "Rafael", "M", "RJ", "Niteroi"),
    (5, "Jean", "M", "RJ", "Cabo Frio"),
    (6, "Daniela", "F", "PR", "Curitiba"),
]

# id_cliente: (id_conta, dt_criacao)
contas = {
    2: (191821, 1608987600),
    1: (191822, 1608552000),
    3: (191823, 1609761600),
    6: (191824, 1609769460),
    5: (191825, 1610449920),
    4: (191826, 1610712000)
}

produtos = [
    { "conta_id": 191821, "fl_cartao_credito": 1, "fl_investidor": 0, "fl_credito": 1},
    { "conta_id": 191822, "fl_cartao_credito": 1, "fl_investidor": 1, "fl_credito": 0},
    { "conta_id": 191823, "fl_cartao_credito": 0, "fl_investidor": 0, "fl_credito": 1},
    { "conta_id": 191824, "fl_cartao_credito": 0, "fl_investidor": 1, "fl_credito": 0},
    { "conta_id": 191825, "fl_cartao_credito": 1, "fl_investidor": 1, "fl_credito": 1},
    { "conta_id": 191826, "fl_cartao_credito": 1, "fl_investidor": 1, "fl_credito": 0}
]


transacoes = [
    {"conta_id": 191821, "valor": 10.0, "dt_transacao": "2021-01-03 16:32:01"},
    {"conta_id": 191821, "valor": 100.0, "dt_transacao": "2021-01-10 17:00:00"},
    {"conta_id": 191821, "valor": 300.0, "dt_transacao": "2021-03-03 09:11:12"},
    {"conta_id": 191821, "valor": 132.0, "dt_transacao": "2021-03-03 10:15:10"},
    {"conta_id": 191822, "valor": 5.0, "dt_transacao": "2021-01-03 23:12:11"},
    {"conta_id": 191822, "valor": 2.0, "dt_transacao": "2021-01-03 11:18:19"},
    {"conta_id": 191823, "valor": 49.99, "dt_transacao": "2021-01-03 15:18:19"},
    {"conta_id": 191823, "valor": 40.0, "dt_transacao": "2021-01-03 18:19:21"},
    {"conta_id": 191823, "valor": 9.30, "dt_transacao": "2021-01-03 22:11:10"},
    {"conta_id": 191826, "valor": 1000.0, "dt_transacao": "2021-01-16 08:12:11"},
    {"conta_id": 191825, "valor": 23.43, "dt_transacao": "2021-01-03 09:11:56"},
    {"conta_id": 191825, "valor": 10.09, "dt_transacao": "2021-01-03 18:19:18"}
]

In [25]:
# Dataframes
# Renomeando uma coluna
# Alterando um tipo do dado
# Transformações

df_clientes = spark.createDataFrame(clientes, schema=['id', 'nome', 'sexo', 'estado', 'cidade'])
df_contas = spark.createDataFrame([Row(k, *v) for k, v in contas.items()], schema=['cliente_id', 'conta_id', 'dt_criacao']) \
  .withColumn('dt_criacao', col('dt_criacao').cast('timestamp')) \
  .withColumnRenamed('dt_criacao', 'dt_criacao_novo')
df_produtos = spark.createDataFrame(produtos)
df_transacoes = spark.createDataFrame(transacoes).withColumn('dt_transacao', col('dt_transacao').cast('timestamp'))

In [ ]:
# Comandos para exibir os dados

df_clientes.show()
df_contas.show()
df_produtos.show()
df_transacoes.show()

Exercicios - Praticas Spark

In [26]:
# filtrando dados
# Exibir o valor
clientes_cartao = df_produtos.filter(col("fl_cartao_credito") == 1)

In [27]:
# Realizando Joins
# Exibir Valor
resultado = df_transacoes.join(clientes_cartao, "conta_id") \
                         .filter(col("valor") > 100) \
                         .select("conta_id", "valor")

In [37]:
# Encontrando a maior transação & Exibir o valor
# Aplicando alias
maior_transacao = df_transacoes.agg(max("valor").alias("max_valor")).collect()[0]["max_valor"]

In [43]:
maior_transacao.sho

1000.0

In [ ]:
# Realize select de colunas especificas dos seus dataframes - .select()
sel_transacoes = df_transacoes.select("valor")
sel_transacoes.show()

Exercicios - Praticas SQL

In [51]:
# Criação de viewTemporaria
df_clientes.createOrReplaceTempView('CLIENTES')
sel_clientes = spark.sql(f'''
                          SELECT count(nome), estado
                            FROM CLIENTES
                            group by estado
                        ''')

sel_clientes.show()

+--------+------+
|    nome|estado|
+--------+------+
|    Joao|    MG|
|   Luiza|    SP|
|Gabriela|    MG|
|  Rafael|    RJ|
|    Jean|    RJ|
| Daniela|    PR|
+--------+------+



In [ ]:
# filtrando dados .sql() - Where
# Exibir o valor


In [ ]:
# Realizando Joins
# Exibir Valor

In [ ]:
# Encontrando a maior transação & Exibir o valor
# Aplicando alias

Quantidade de Clientes por Estado

In [56]:
contagem_cliente_estado = df_clientes.groupBy("estado").count()
print(contagem_cliente_estado.show())

+------+-----+
|estado|count|
+------+-----+
|    SP|    1|
|    MG|    2|
|    RJ|    2|
|    PR|    1|
+------+-----+

None


Cliente e Conta

In [59]:
juncao = df_contas.join(
    df_clientes,
    df_contas["cliente_id"] == df_clientes["id"],
    how="inner"
).select(
    df_clientes["id"].alias("cliente_id"),
    df_contas["conta_id"]
)

Soma das transações por cliente e conta

In [60]:
df_contas_clientes = df_contas.join(df_clientes, df_contas.cliente_id == df_clientes.id, how='inner') \
                              .select(df_contas.conta_id, df_clientes.nome)

df_transacao = df_transacoes.join(df_contas_clientes, 'conta_id', how='inner')

df = df_transacao.groupBy('nome', 'conta_id') \
                                     .agg(sum('valor').alias('soma'))

In [ ]:

# Base de dados sintética (1000 linhas)

clientes = [(1, 'Luiza_1', 'M', 'BA', 'Salvador'), (2, 'Joao_2', 'M', 'SC', 'Joinville'), (3, 'Daniela_3', 'F', 'MG', 'Uberlândia'), (4, 'Jean_4', 'F', 'RS', 'Porto Alegre'), (5, 'Rafael_5', 'M', 'PE', 'Olinda'), (6, 'Daniela_6', 'M', 'PE', 'Olinda'), (7, 'Patricia_7', 'F', 'RS', 'Porto Alegre'), (8, 'Carlos_8', 'M', 'SC', 'Joinville'), (9, 'Daniela_9', 'M', 'BA', 'Feira de Santana'), (10, 'Patricia_10', 'F', 'PE', 'Recife'), (11, 'Luiza_11', 'F', 'SP', 'Campinas'), (12, 'Daniela_12', 'F', 'PE', 'Recife'), (13, 'Rafael_13', 'M', 'SP', 'Campinas'), (14, 'Rafael_14', 'F', 'PE', 'Recife'), (15, 'Luiza_15', 'M', 'SC', 'Joinville'), (16, 'Carlos_16', 'F', 'PE', 'Recife'), (17, 'Jean_17', 'F', 'PE', 'Recife'), (18, 'Luiza_18', 'M', 'SC', 'Florianópolis'), (19, 'Jean_19', 'F', 'RS', 'Porto Alegre'), (20, 'Joao_20', 'F', 'SP', 'São Paulo')]  # exibindo amostra de 20 para não poluir
contas = {k: v for k, v in list({1: (190001, 1587686400), 2: (190002, 1590105600), 3: (190003, 1598140800), 4: (190004, 1594166400), 5: (190005, 1589760000), 6: (190006, 1606262400), 7: (190007, 1583884800), 8: (190008, 1604188800), 9: (190009, 1584748800), 10: (190010, 1582761600), 11: (190011, 1583366400), 12: (190012, 1588723200), 13: (190013, 1605744000), 14: (190014, 1601683200), 15: (190015, 1582588800), 16: (190016, 1595548800), 17: (190017, 1586217600), 18: (190018, 1602979200), 19: (190019, 1578182400), 20: (190020, 1589500800), 21: (190021, 1596499200), 22: (190022, 1579392000), 23: (190023, 1605225600), 24: (190024, 1582588800), 25: (190025, 1590105600), 26: (190026, 1589068800), 27: (190027, 1595635200), 28: (190028, 1597363200), 29: (190029, 1586563200), 30: (190030, 1588550400), 31: (190031, 1602028800), 32: (190032, 1583884800), 33: (190033, 1595030400), 34: (190034, 1589846400), 35: (190035, 1598227200), 36: (190036, 1606521600), 37: (190037, 1594080000), 38: (190038, 1591142400), 39: (190039, 1600905600), 40: (190040, 1592179200), 41: (190041, 1602374400), 42: (190042, 1589760000), 43: (190043, 1599177600), 44: (190044, 1602288000), 45: (190045, 1585958400), 46: (190046, 1602720000), 47: (190047, 1597363200), 48: (190048, 1588032000), 49: (190049, 1601856000), 50: (190050, 1608768000), 51: (190051, 1598572800), 52: (190052, 1588982400), 53: (190053, 1608681600), 54: (190054, 1598659200), 55: (190055, 1584403200), 56: (190056, 1591833600), 57: (190057, 1597968000), 58: (190058, 1605484800), 59: (190059, 1591056000), 60: (190060, 1585353600), 61: (190061, 1608336000), 62: (190062, 1606953600), 63: (190063, 1597276800), 64: (190064, 1601683200), 65: (190065, 1595289600), 66: (190066, 1598313600), 67: (190067, 1594166400), 68: (190068, 1597449600), 69: (190069, 1586217600), 70: (190070, 1586995200), 71: (190071, 1607040000), 72: (190072, 1593820800), 73: (190073, 1583366400), 74: (190074, 1595462400), 75: (190075, 1595980800), 76: (190076, 1603152000), 77: (190077, 1596326400), 78: (190078, 1599264000), 79: (190079, 1587513600), 80: (190080, 1585526400), 81: (190081, 1585353600), 82: (190082, 1593302400), 83: (190083, 1607904000), 84: (190084, 1606348800), 85: (190085, 1609113600), 86: (190086, 1602460800), 87: (190087, 1607472000), 88: (190088, 1583884800), 89: (190089, 1604188800), 90: (190090, 1597622400), 91: (190091, 1606176000), 92: (190092, 1594857600), 93: (190093, 1590451200), 94: (190094, 1596326400), 95: (190095, 1593734400), 96: (190096, 1593388800), 97: (190097, 1579392000), 98: (190098, 1597276800), 99: (190099, 1592265600), 100: (190100, 1584144000), 101: (190101, 1584576000), 102: (190102, 1589328000), 103: (190103, 1609286400), 104: (190104, 1598486400), 105: (190105, 1583452800), 106: (190106, 1591228800), 107: (190107, 1603324800), 108: (190108, 1604620800), 109: (190109, 1579996800), 110: (190110, 1591401600), 111: (190111, 1589500800), 112: (190112, 1594857600), 113: (190113, 1599955200), 114: (190114, 1595203200), 115: (190115, 1600992000), 116: (190116, 1596326400), 117: (190117, 1598140800), 118: (190118, 1607731200), 119: (190119, 1601596800), 120: (190120, 1580083200), 121: (190121, 1603152000), 122: (190122, 1606176000), 123: (190123, 1604793600), 124: (190124, 1578096000), 125: (190125, 1608163200), 126: (190126, 1607904000), 127: (190127, 1592265600), 128: (190128, 1579824000), 129: (190129, 1607040000), 130: (190130, 1601337600), 131: (190131, 1579305600), 132: (190132, 1590624000), 133: (190133, 1607817600), 134: (190134, 1587772800), 135: (190135, 1589068800), 136: (190136, 1592524800), 137: (190137, 1602201600), 138: (190138, 1578528000), 139: (190139, 1606176000), 140: (190140, 1583884800), 141: (190141, 1578096000), 142: (190142, 1581292800), 143: (190143, 1597017600), 144: (190144, 1578528000), 145: (190145, 1583971200), 146: (190146, 1605830400), 147: (190147, 1609372800), 148: (190148, 1597449600), 149: (190149, 1602288000), 150: (190150, 1594598400), 151: (190151, 1598054400), 152: (190152, 1596844800), 153: (190153, 1607990400), 154: (190154, 1604275200), 155: (190155, 1582502400), 156: (190156, 1607817600), 157: (190157, 1580774400), 158: (190158, 1584835200), 159: (190159, 1604016000), 160: (190160, 1582848000), 161: (190161, 1585872000), 162: (190162, 1605225600), 163: (190163, 1579219200), 164: (190164, 1601856000), 165: (190165, 1605052800), 166: (190166, 1590624000), 167: (190167, 1594425600), 168: (190168, 1595376000), 169: (190169, 1597276800), 170: (190170, 1607212800), 171: (190171, 1578182400), 172: (190172, 1594684800), 173: (190173, 1591747200), 174: (190174, 1591142400), 175: (190175, 1594944000), 176: (190176, 1581465600), 177: (190177, 1580342400), 178: (190178, 1609113600), 179: (190179, 1609286400), 180: (190180, 1594771200), 181: (190181, 1580860800), 182: (190182, 1601942400), 183: (190183, 1580428800), 184: (190184, 1581984000), 185: (190185, 1582502400), 186: (190186, 1587340800), 187: (190187, 1602374400), 188: (190188, 1586563200), 189: (190189, 1579132800), 190: (190190, 1580515200), 191: (190191, 1593043200), 192: (190192, 1582243200), 193: (190193, 1601769600), 194: (190194, 1592438400), 195: (190195, 1594339200), 196: (190196, 1594512000), 197: (190197, 1599782400), 198: (190198, 1587081600), 199: (190199, 1603324800), 200: (190200, 1581033600), 201: (190201, 1597017600), 202: (190202, 1584576000), 203: (190203, 1599436800), 204: (190204, 1589673600), 205: (190205, 1600905600), 206: (190206, 1579651200), 207: (190207, 1602201600), 208: (190208, 1584403200), 209: (190209, 1592438400), 210: (190210, 1597363200), 211: (190211, 1586304000), 212: (190212, 1600128000), 213: (190213, 1589241600), 214: (190214, 1585440000), 215: (190215, 1607126400), 216: (190216, 1586131200), 217: (190217, 1608163200), 218: (190218, 1587600000), 219: (190219, 1599955200), 220: (190220, 1605830400), 221: (190221, 1594771200), 222: (190222, 1594771200), 223: (190223, 1592956800), 224: (190224, 1609286400), 225: (190225, 1607040000), 226: (190226, 1590883200), 227: (190227, 1581811200), 228: (190228, 1596672000), 229: (190229, 1605312000), 230: (190230, 1580428800), 231: (190231, 1606780800), 232: (190232, 1606262400), 233: (190233, 1588204800), 234: (190234, 1592870400), 235: (190235, 1608854400), 236: (190236, 1605830400), 237: (190237, 1604707200), 238: (190238, 1587081600), 239: (190239, 1596499200), 240: (190240, 1582329600), 241: (190241, 1586390400), 242: (190242, 1596758400), 243: (190243, 1605484800), 244: (190244, 1584835200), 245: (190245, 1593129600), 246: (190246, 1578873600), 247: (190247, 1601596800), 248: (190248, 1598140800), 249: (190249, 1594944000), 250: (190250, 1608163200), 251: (190251, 1602374400), 252: (190252, 1593043200), 253: (190253, 1590624000), 254: (190254, 1586390400), 255: (190255, 1587600000), 256: (190256, 1604620800), 257: (190257, 1595116800), 258: (190258, 1602806400), 259: (190259, 1579651200), 260: (190260, 1593993600), 261: (190261, 1587945600), 262: (190262, 1599091200), 263: (190263, 1607040000), 264: (190264, 1587945600), 265: (190265, 1598832000), 266: (190266, 1593302400), 267: (190267, 1606780800), 268: (190268, 1602201600), 269: (190269, 1584921600), 270: (190270, 1594771200), 271: (190271, 1600387200), 272: (190272, 1608940800), 273: (190273, 1582934400), 274: (190274, 1596067200), 275: (190275, 1593475200), 276: (190276, 1584662400), 277: (190277, 1601078400), 278: (190278, 1589414400), 279: (190279, 1602374400), 280: (190280, 1591056000), 281: (190281, 1592611200), 282: (190282, 1594684800), 283: (190283, 1587859200), 284: (190284, 1588204800), 285: (190285, 1582416000), 286: (190286, 1583798400), 287: (190287, 1598572800), 288: (190288, 1606348800), 289: (190289, 1580860800), 290: (190290, 1578614400), 291: (190291, 1578441600), 292: (190292, 1588896000), 293: (190293, 1595030400), 294: (190294, 1586563200), 295: (190295, 1595289600), 296: (190296, 1588118400), 297: (190297, 1586217600), 298: (190298, 1605225600), 299: (190299, 1590278400), 300: (190300, 1586736000), 301: (190301, 1587600000), 302: (190302, 1581206400), 303: (190303, 1603324800), 304: (190304, 1604361600), 305: (190305, 1595376000), 306: (190306, 1584662400), 307: (190307, 1603324800), 308: (190308, 1591920000), 309: (190309, 1595203200), 310: (190310, 1599955200), 311: (190311, 1596067200), 312: (190312, 1579737600), 313: (190313, 1608508800), 314: (190314, 1585353600), 315: (190315, 1599264000), 316: (190316, 1605484800), 317: (190317, 1604534400), 318: (190318, 1606780800), 319: (190319, 1583625600), 320: (190320, 1585008000), 321: (190321, 1596672000), 322: (190322, 1600732800), 323: (190323, 1599955200), 324: (190324, 1609027200), 325: (190325, 1597536000), 326: (190326, 1599609600), 327: (190327, 1593216000), 328: (190328, 1579392000), 329: (190329, 1608595200), 330: (190330, 1585785600), 331: (190331, 1579824000), 332: (190332, 1600214400), 333: (190333, 1583798400), 334: (190334, 1592784000), 335: (190335, 1588291200), 336: (190336, 1599955200), 337: (190337, 1578009600), 338: (190338, 1590451200), 339: (190339, 1601769600), 340: (190340, 1583539200), 341: (190341, 1591574400), 342: (190342, 1585872000), 343: (190343, 1591142400), 344: (190344, 1602892800), 345: (190345, 1597449600), 346: (190346, 1581811200), 347: (190347, 1591056000), 348: (190348, 1594166400), 349: (190349, 1595721600), 350: (190350, 1578268800), 351: (190351, 1593475200), 352: (190352, 1601078400), 353: (190353, 1603152000), 354: (190354, 1590710400), 355: (190355, 1589155200), 356: (190356, 1594771200), 357: (190357, 1591142400), 358: (190358, 1604793600), 359: (190359, 1602028800), 360: (190360, 1588896000), 361: (190361, 1590796800), 362: (190362, 1580428800), 363: (190363, 1581033600), 364: (190364, 1584489600), 365: (190365, 1606435200), 366: (190366, 1607040000), 367: (190367, 1579392000), 368: (190368, 1606694400), 369: (190369, 1608854400), 370: (190370, 1582848000), 371: (190371, 1579824000), 372: (190372, 1603497600), 373: (190373, 1589760000), 374: (190374, 1578009600), 375: (190375, 1607472000), 376: (190376, 1600560000), 377: (190377, 1581033600), 378: (190378, 1602979200), 379: (190379, 1604534400), 380: (190380, 1578441600), 381: (190381, 1599004800), 382: (190382, 1578528000), 383: (190383, 1608336000), 384: (190384, 1593993600), 385: (190385, 1587772800), 386: (190386, 1598054400), 387: (190387, 1600819200), 388: (190388, 1598054400), 389: (190389, 1585180800), 390: (190390, 1597276800), 391: (190391, 1591142400), 392: (190392, 1599004800), 393: (190393, 1578787200), 394: (190394, 1602633600), 395: (190395, 1603929600), 396: (190396, 1603670400), 397: (190397, 1585699200), 398: (190398, 1589241600), 399: (190399, 1600560000), 400: (190400, 1595980800), 401: (190401, 1581206400), 402: (190402, 1584057600), 403: (190403, 1586476800), 404: (190404, 1590105600), 405: (190405, 1608681600), 406: (190406, 1582675200), 407: (190407, 1590624000), 408: (190408, 1608508800), 409: (190409, 1593561600), 410: (190410, 1605312000), 411: (190411, 1594425600), 412: (190412, 1591574400), 413: (190413, 1579046400), 414: (190414, 1607040000), 415: (190415, 1589587200), 416: (190416, 1591056000), 417: (190417, 1580774400), 418: (190418, 1607472000), 419: (190419, 1608076800), 420: (190420, 1582848000), 421: (190421, 1583798400), 422: (190422, 1582761600), 423: (190423, 1587427200), 424: (190424, 1601942400), 425: (190425, 1591488000), 426: (190426, 1598659200), 427: (190427, 1604448000), 428: (190428, 1586649600), 429: (190429, 1598918400), 430: (190430, 1608854400), 431: (190431, 1590537600), 432: (190432, 1588809600), 433: (190433, 1604188800), 434: (190434, 1602547200), 435: (190435, 1582934400), 436: (190436, 1599523200), 437: (190437, 1579564800), 438: (190438, 1603065600), 439: (190439, 1592092800), 440: (190440, 1606262400), 441: (190441, 1588377600), 442: (190442, 1591488000), 443: (190443, 1605657600), 444: (190444, 1608076800), 445: (190445, 1603670400), 446: (190446, 1587168000), 447: (190447, 1606780800), 448: (190448, 1587081600), 449: (190449, 1597708800), 450: (190450, 1588032000), 451: (190451, 1588032000), 452: (190452, 1586995200), 453: (190453, 1578614400), 454: (190454, 1587686400), 455: (190455, 1582502400), 456: (190456, 1606435200), 457: (190457, 1590451200), 458: (190458, 1595894400), 459: (190459, 1595548800), 460: (190460, 1601856000), 461: (190461, 1600387200), 462: (190462, 1600128000), 463: (190463, 1603497600), 464: (190464, 1582156800), 465: (190465, 1594166400), 466: (190466, 1598745600), 467: (190467, 1600905600), 468: (190468, 1599436800), 469: (190469, 1594512000), 470: (190470, 1609113600), 471: (190471, 1581552000), 472: (190472, 1585872000), 473: (190473, 1596067200), 474: (190474, 1582848000), 475: (190475, 1602892800), 476: (190476, 1589846400), 477: (190477, 1608681600), 478: (190478, 1578528000), 479: (190479, 1606521600), 480: (190480, 1578182400), 481: (190481, 1605484800), 482: (190482, 1602201600), 483: (190483, 1606176000), 484: (190484, 1586390400), 485: (190485, 1605484800), 486: (190486, 1605484800), 487: (190487, 1580256000), 488: (190488, 1605139200), 489: (190489, 1609372800), 490: (190490, 1591488000), 491: (190491, 1596499200), 492: (190492, 1596326400), 493: (190493, 1585958400), 494: (190494, 1605744000), 495: (190495, 1578787200), 496: (190496, 1597968000), 497: (190497, 1601337600), 498: (190498, 1598572800), 499: (190499, 1597795200), 500: (190500, 1603584000), 501: (190501, 1599955200), 502: (190502, 1607644800), 503: (190503, 1598659200), 504: (190504, 1591660800), 505: (190505, 1592784000), 506: (190506, 1595116800), 507: (190507, 1580860800), 508: (190508, 1595116800), 509: (190509, 1586736000), 510: (190510, 1598227200), 511: (190511, 1588377600), 512: (190512, 1582416000), 513: (190513, 1586390400), 514: (190514, 1590105600), 515: (190515, 1605916800), 516: (190516, 1587513600), 517: (190517, 1596758400), 518: (190518, 1587168000), 519: (190519, 1604361600), 520: (190520, 1600041600), 521: (190521, 1583712000), 522: (190522, 1597363200), 523: (190523, 1585440000), 524: (190524, 1593734400), 525: (190525, 1594598400), 526: (190526, 1595721600), 527: (190527, 1606608000), 528: (190528, 1589068800), 529: (190529, 1586131200), 530: (190530, 1589673600), 531: (190531, 1606780800), 532: (190532, 1586649600), 533: (190533, 1590019200), 534: (190534, 1592697600), 535: (190535, 1583971200), 536: (190536, 1592006400), 537: (190537, 1609286400), 538: (190538, 1597017600), 539: (190539, 1592956800), 540: (190540, 1594080000), 541: (190541, 1598140800), 542: (190542, 1589932800), 543: (190543, 1595116800), 544: (190544, 1609200000), 545: (190545, 1580256000), 546: (190546, 1583020800), 547: (190547, 1578268800), 548: (190548, 1608508800), 549: (190549, 1603670400), 550: (190550, 1586908800), 551: (190551, 1606867200), 552: (190552, 1599523200), 553: (190553, 1587340800), 554: (190554, 1608508800), 555: (190555, 1597017600), 556: (190556, 1606089600), 557: (190557, 1601164800), 558: (190558, 1594512000), 559: (190559, 1600819200), 560: (190560, 1590451200), 561: (190561, 1604620800), 562: (190562, 1582761600), 563: (190563, 1580342400), 564: (190564, 1600560000), 565: (190565, 1593475200), 566: (190566, 1586563200), 567: (190567, 1608076800), 568: (190568, 1608768000), 569: (190569, 1586304000), 570: (190570, 1590192000), 571: (190571, 1588982400), 572: (190572, 1587081600), 573: (190573, 1601942400), 574: (190574, 1605657600), 575: (190575, 1585094400), 576: (190576, 1590883200), 577: (190577, 1588809600), 578: (190578, 1605484800), 579: (190579, 1592179200), 580: (190580, 1585612800), 581: (190581, 1582243200), 582: (190582, 1592784000), 583: (190583, 1590883200), 584: (190584, 1606780800), 585: (190585, 1605052800), 586: (190586, 1607817600), 587: (190587, 1580774400), 588: (190588, 1604448000), 589: (190589, 1593820800), 590: (190590, 1587081600), 591: (190591, 1580169600), 592: (190592, 1597449600), 593: (190593, 1607904000), 594: (190594, 1587513600), 595: (190595, 1579046400), 596: (190596, 1591488000), 597: (190597, 1594684800), 598: (190598, 1584921600), 599: (190599, 1578441600), 600: (190600, 1597363200), 601: (190601, 1600473600), 602: (190602, 1602979200), 603: (190603, 1580688000), 604: (190604, 1587254400), 605: (190605, 1578787200), 606: (190606, 1585958400), 607: (190607, 1602633600), 608: (190608, 1583366400), 609: (190609, 1586217600), 610: (190610, 1595980800), 611: (190611, 1582416000), 612: (190612, 1590624000), 613: (190613, 1600214400), 614: (190614, 1603065600), 615: (190615, 1591833600), 616: (190616, 1592092800), 617: (190617, 1594684800), 618: (190618, 1605571200), 619: (190619, 1581033600), 620: (190620, 1604966400), 621: (190621, 1582848000), 622: (190622, 1599696000), 623: (190623, 1597536000), 624: (190624, 1582848000), 625: (190625, 1608249600), 626: (190626, 1591833600), 627: (190627, 1597795200), 628: (190628, 1579392000), 629: (190629, 1595462400), 630: (190630, 1585353600), 631: (190631, 1583366400), 632: (190632, 1593388800), 633: (190633, 1586131200), 634: (190634, 1587513600), 635: (190635, 1598832000), 636: (190636, 1600041600), 637: (190637, 1579996800), 638: (190638, 1582243200), 639: (190639, 1585958400), 640: (190640, 1578700800), 641: (190641, 1582675200), 642: (190642, 1601942400), 643: (190643, 1595980800), 644: (190644, 1589500800), 645: (190645, 1598400000), 646: (190646, 1596844800), 647: (190647, 1587513600), 648: (190648, 1597104000), 649: (190649, 1582070400), 650: (190650, 1605225600), 651: (190651, 1606521600), 652: (190652, 1599696000), 653: (190653, 1582416000), 654: (190654, 1605916800), 655: (190655, 1607299200), 656: (190656, 1602720000), 657: (190657, 1598486400), 658: (190658, 1584489600), 659: (190659, 1603238400), 660: (190660, 1594771200), 661: (190661, 1599523200), 662: (190662, 1579305600), 663: (190663, 1596672000), 664: (190664, 1590624000), 665: (190665, 1605139200), 666: (190666, 1599609600), 667: (190667, 1589760000), 668: (190668, 1599091200), 669: (190669, 1592956800), 670: (190670, 1581206400), 671: (190671, 1606867200), 672: (190672, 1595980800), 673: (190673, 1588204800), 674: (190674, 1580428800), 675: (190675, 1585958400), 676: (190676, 1579046400), 677: (190677, 1585008000), 678: (190678, 1579478400), 679: (190679, 1578182400), 680: (190680, 1599696000), 681: (190681, 1602288000), 682: (190682, 1586822400), 683: (190683, 1595808000), 684: (190684, 1606867200), 685: (190685, 1580083200), 686: (190686, 1588464000), 687: (190687, 1596326400), 688: (190688, 1578441600), 689: (190689, 1583452800), 690: (190690, 1593475200), 691: (190691, 1592006400), 692: (190692, 1594771200), 693: (190693, 1593129600), 694: (190694, 1602201600), 695: (190695, 1589414400), 696: (190696, 1600214400), 697: (190697, 1594425600), 698: (190698, 1609200000), 699: (190699, 1605657600), 700: (190700, 1593129600), 701: (190701, 1596585600), 702: (190702, 1604793600), 703: (190703, 1579996800), 704: (190704, 1578355200), 705: (190705, 1600128000), 706: (190706, 1596758400), 707: (190707, 1597536000), 708: (190708, 1605052800), 709: (190709, 1606608000), 710: (190710, 1596585600), 711: (190711, 1604966400), 712: (190712, 1600992000), 713: (190713, 1580774400), 714: (190714, 1589068800), 715: (190715, 1603756800), 716: (190716, 1592265600), 717: (190717, 1608076800), 718: (190718, 1594339200), 719: (190719, 1579737600), 720: (190720, 1583107200), 721: (190721, 1607212800), 722: (190722, 1579737600), 723: (190723, 1598313600), 724: (190724, 1581379200), 725: (190725, 1586131200), 726: (190726, 1594857600), 727: (190727, 1598486400), 728: (190728, 1592611200), 729: (190729, 1588550400), 730: (190730, 1604188800), 731: (190731, 1591142400), 732: (190732, 1584835200), 733: (190733, 1601424000), 734: (190734, 1597017600), 735: (190735, 1595376000), 736: (190736, 1606694400), 737: (190737, 1596585600), 738: (190738, 1579910400), 739: (190739, 1592697600), 740: (190740, 1601337600), 741: (190741, 1593043200), 742: (190742, 1592697600), 743: (190743, 1603065600), 744: (190744, 1599955200), 745: (190745, 1587772800), 746: (190746, 1598832000), 747: (190747, 1607040000), 748: (190748, 1607644800), 749: (190749, 1587427200), 750: (190750, 1605657600), 751: (190751, 1601337600), 752: (190752, 1605139200), 753: (190753, 1594080000), 754: (190754, 1588464000), 755: (190755, 1599091200), 756: (190756, 1584230400), 757: (190757, 1589328000), 758: (190758, 1602720000), 759: (190759, 1590710400), 760: (190760, 1600300800), 761: (190761, 1594598400), 762: (190762, 1602979200), 763: (190763, 1602892800), 764: (190764, 1586217600), 765: (190765, 1601510400), 766: (190766, 1602979200), 767: (190767, 1606348800), 768: (190768, 1585008000), 769: (190769, 1601251200), 770: (190770, 1598659200), 771: (190771, 1606348800), 772: (190772, 1592006400), 773: (190773, 1604620800), 774: (190774, 1598659200), 775: (190775, 1580774400), 776: (190776, 1583107200), 777: (190777, 1581120000), 778: (190778, 1603238400), 779: (190779, 1587772800), 780: (190780, 1586908800), 781: (190781, 1591747200), 782: (190782, 1594598400), 783: (190783, 1596153600), 784: (190784, 1606608000), 785: (190785, 1580515200), 786: (190786, 1579046400), 787: (190787, 1598832000), 788: (190788, 1598918400), 789: (190789, 1587772800), 790: (190790, 1603411200), 791: (190791, 1599091200), 792: (190792, 1607385600), 793: (190793, 1597795200), 794: (190794, 1608508800), 795: (190795, 1604793600), 796: (190796, 1592006400), 797: (190797, 1595980800), 798: (190798, 1580947200), 799: (190799, 1607040000), 800: (190800, 1600905600), 801: (190801, 1588896000), 802: (190802, 1604620800), 803: (190803, 1605571200), 804: (190804, 1608681600), 805: (190805, 1595721600), 806: (190806, 1605139200), 807: (190807, 1595376000), 808: (190808, 1608508800), 809: (190809, 1603497600), 810: (190810, 1582070400), 811: (190811, 1598918400), 812: (190812, 1608940800), 813: (190813, 1580169600), 814: (190814, 1608595200), 815: (190815, 1586649600), 816: (190816, 1601164800), 817: (190817, 1597276800), 818: (190818, 1607990400), 819: (190819, 1584057600), 820: (190820, 1592006400), 821: (190821, 1599782400), 822: (190822, 1579046400), 823: (190823, 1587254400), 824: (190824, 1586908800), 825: (190825, 1604275200), 826: (190826, 1608595200), 827: (190827, 1592265600), 828: (190828, 1600732800), 829: (190829, 1597104000), 830: (190830, 1586649600), 831: (190831, 1580860800), 832: (190832, 1592006400), 833: (190833, 1587686400), 834: (190834, 1601510400), 835: (190835, 1609200000), 836: (190836, 1583712000), 837: (190837, 1595203200), 838: (190838, 1607904000), 839: (190839, 1581552000), 840: (190840, 1592784000), 841: (190841, 1605139200), 842: (190842, 1585353600), 843: (190843, 1586476800), 844: (190844, 1591920000), 845: (190845, 1603152000), 846: (190846, 1595808000), 847: (190847, 1582502400), 848: (190848, 1607212800), 849: (190849, 1581292800), 850: (190850, 1586563200), 851: (190851, 1595289600), 852: (190852, 1580515200), 853: (190853, 1603843200), 854: (190854, 1593388800), 855: (190855, 1605744000), 856: (190856, 1578528000), 857: (190857, 1584835200), 858: (190858, 1600732800), 859: (190859, 1594166400), 860: (190860, 1583452800), 861: (190861, 1599523200), 862: (190862, 1599264000), 863: (190863, 1578873600), 864: (190864, 1609027200), 865: (190865, 1604880000), 866: (190866, 1596067200), 867: (190867, 1607990400), 868: (190868, 1609372800), 869: (190869, 1587168000), 870: (190870, 1590451200), 871: (190871, 1580169600), 872: (190872, 1605052800), 873: (190873, 1606694400), 874: (190874, 1585526400), 875: (190875, 1608422400), 876: (190876, 1589587200), 877: (190877, 1590451200), 878: (190878, 1593129600), 879: (190879, 1603152000), 880: (190880, 1578009600), 881: (190881, 1608249600), 882: (190882, 1591315200), 883: (190883, 1580947200), 884: (190884, 1594944000), 885: (190885, 1608249600), 886: (190886, 1591488000), 887: (190887, 1585785600), 888: (190888, 1599264000), 889: (190889, 1608768000), 890: (190890, 1596326400), 891: (190891, 1586044800), 892: (190892, 1607212800), 893: (190893, 1582156800), 894: (190894, 1605484800), 895: (190895, 1604966400), 896: (190896, 1601510400), 897: (190897, 1589760000), 898: (190898, 1607385600), 899: (190899, 1590710400), 900: (190900, 1589068800), 901: (190901, 1596672000), 902: (190902, 1579564800), 903: (190903, 1578787200), 904: (190904, 1585440000), 905: (190905, 1588032000), 906: (190906, 1597363200), 907: (190907, 1587427200), 908: (190908, 1603065600), 909: (190909, 1587945600), 910: (190910, 1581292800), 911: (190911, 1580083200), 912: (190912, 1584576000), 913: (190913, 1583625600), 914: (190914, 1602115200), 915: (190915, 1591056000), 916: (190916, 1597190400), 917: (190917, 1606694400), 918: (190918, 1584230400), 919: (190919, 1600560000), 920: (190920, 1578355200), 921: (190921, 1581292800), 922: (190922, 1597017600), 923: (190923, 1596326400), 924: (190924, 1596153600), 925: (190925, 1592697600), 926: (190926, 1581897600), 927: (190927, 1579478400), 928: (190928, 1580688000), 929: (190929, 1607299200), 930: (190930, 1597017600), 931: (190931, 1588982400), 932: (190932, 1596153600), 933: (190933, 1578182400), 934: (190934, 1579564800), 935: (190935, 1598313600), 936: (190936, 1596844800), 937: (190937, 1585612800), 938: (190938, 1589241600), 939: (190939, 1577923200), 940: (190940, 1581465600), 941: (190941, 1592956800), 942: (190942, 1602460800), 943: (190943, 1598313600), 944: (190944, 1580256000), 945: (190945, 1609286400), 946: (190946, 1602115200), 947: (190947, 1580083200), 948: (190948, 1587427200), 949: (190949, 1595721600), 950: (190950, 1603929600), 951: (190951, 1577923200), 952: (190952, 1586908800), 953: (190953, 1598054400), 954: (190954, 1598745600), 955: (190955, 1578528000), 956: (190956, 1589846400), 957: (190957, 1605571200), 958: (190958, 1586390400), 959: (190959, 1579824000), 960: (190960, 1595980800), 961: (190961, 1595894400), 962: (190962, 1608249600), 963: (190963, 1578960000), 964: (190964, 1598140800), 965: (190965, 1579305600), 966: (190966, 1587945600), 967: (190967, 1590883200), 968: (190968, 1588464000), 969: (190969, 1596499200), 970: (190970, 1585094400), 971: (190971, 1607558400), 972: (190972, 1599955200), 973: (190973, 1595721600), 974: (190974, 1595808000), 975: (190975, 1605484800), 976: (190976, 1600905600), 977: (190977, 1581465600), 978: (190978, 1601856000), 979: (190979, 1600473600), 980: (190980, 1590624000), 981: (190981, 1587600000), 982: (190982, 1586563200), 983: (190983, 1581984000), 984: (190984, 1582761600), 985: (190985, 1580342400), 986: (190986, 1588032000), 987: (190987, 1581033600), 988: (190988, 1605657600), 989: (190989, 1595030400), 990: (190990, 1599264000), 991: (190991, 1587168000), 992: (190992, 1590624000), 993: (190993, 1590883200), 994: (190994, 1595635200), 995: (190995, 1606867200), 996: (190996, 1601856000), 997: (190997, 1587340800), 998: (190998, 1588896000), 999: (190999, 1580256000), 1000: (191000, 1594425600)}.items())[:20]}  # amostra de 20
produtos = [{'conta_id': 190001, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190002, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190003, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190004, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190005, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190006, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190007, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190008, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190009, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190010, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190011, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190012, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190013, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190014, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190015, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190016, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190017, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190018, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190019, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190020, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}]
transacoes = [{'conta_id': 190001, 'valor': 844.42, 'dt_transacao': '2021-01-04 02:00:00'}, {'conta_id': 190001, 'valor': 438.06, 'dt_transacao': '2021-03-06 19:00:00'}, {'conta_id': 190001, 'valor': 54.05, 'dt_transacao': '2021-01-26 22:00:00'}, {'conta_id': 190001, 'valor': 1300.12, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190001, 'valor': 441.66, 'dt_transacao': '2021-03-17 08:00:00'}, {'conta_id': 190002, 'valor': 186.4, 'dt_transacao': '2021-01-13 11:00:00'}, {'conta_id': 190003, 'valor': 587.06, 'dt_transacao': '2021-03-22 19:00:00'}, {'conta_id': 190003, 'valor': 1771.02, 'dt_transacao': '2021-02-16 18:00:00'}, {'conta_id': 190003, 'valor': 385.38, 'dt_transacao': '2021-01-09 01:00:00'}, {'conta_id': 190003, 'valor': 1322.87, 'dt_transacao': '2021-02-07 02:00:00'}, {'conta_id': 190003, 'valor': 1710.78, 'dt_transacao': '2021-01-13 12:00:00'}, {'conta_id': 190004, 'valor': 1218.65, 'dt_transacao': '2021-01-22 17:00:00'}, {'conta_id': 190005, 'valor': 1643.78, 'dt_transacao': '2021-02-10 12:00:00'}, {'conta_id': 190005, 'valor': 536.21, 'dt_transacao': '2021-01-28 18:00:00'}, {'conta_id': 190006, 'valor': 494.01, 'dt_transacao': '2021-03-13 17:00:00'}, {'conta_id': 190006, 'valor': 526.22, 'dt_transacao': '2021-03-16 13:00:00'}, {'conta_id': 190007, 'valor': 306.53, 'dt_transacao': '2021-01-21 21:00:00'}, {'conta_id': 190008, 'valor': 1363.74, 'dt_transacao': '2021-03-10 08:00:00'}, {'conta_id': 190009, 'valor': 1524.16, 'dt_transacao': '2021-03-06 03:00:00'}, {'conta_id': 190009, 'valor': 1741.17, 'dt_transacao': '2021-02-08 20:00:00'}]

print("Total clientes:", len([(1, 'Luiza_1', 'M', 'BA', 'Salvador'), (2, 'Joao_2', 'M', 'SC', 'Joinville'), (3, 'Daniela_3', 'F', 'MG', 'Uberlândia'), (4, 'Jean_4', 'F', 'RS', 'Porto Alegre'), (5, 'Rafael_5', 'M', 'PE', 'Olinda'), (6, 'Daniela_6', 'M', 'PE', 'Olinda'), (7, 'Patricia_7', 'F', 'RS', 'Porto Alegre'), (8, 'Carlos_8', 'M', 'SC', 'Joinville'), (9, 'Daniela_9', 'M', 'BA', 'Feira de Santana'), (10, 'Patricia_10', 'F', 'PE', 'Recife'), (11, 'Luiza_11', 'F', 'SP', 'Campinas'), (12, 'Daniela_12', 'F', 'PE', 'Recife'), (13, 'Rafael_13', 'M', 'SP', 'Campinas'), (14, 'Rafael_14', 'F', 'PE', 'Recife'), (15, 'Luiza_15', 'M', 'SC', 'Joinville'), (16, 'Carlos_16', 'F', 'PE', 'Recife'), (17, 'Jean_17', 'F', 'PE', 'Recife'), (18, 'Luiza_18', 'M', 'SC', 'Florianópolis'), (19, 'Jean_19', 'F', 'RS', 'Porto Alegre'), (20, 'Joao_20', 'F', 'SP', 'São Paulo'), (21, 'Jean_21', 'M', 'SP', 'Campinas'), (22, 'Luiza_22', 'M', 'RJ', 'Cabo Frio'), (23, 'Rafael_23', 'M', 'RS', 'Caxias do Sul'), (24, 'Rafael_24', 'F', 'RJ', 'Cabo Frio'), (25, 'Joao_25', 'M', 'SC', 'Joinville'), (26, 'Patricia_26', 'M', 'SC', 'Florianópolis'), (27, 'Jean_27', 'F', 'SC', 'Joinville'), (28, 'Rafael_28', 'F', 'RS', 'Caxias do Sul'), (29, 'Luiza_29', 'M', 'BA', 'Salvador'), (30, 'Rafael_30', 'F', 'PE', 'Olinda'), (31, 'Marcos_31', 'F', 'PE', 'Olinda'), (32, 'Jean_32', 'F', 'RS', 'Porto Alegre'), (33, 'Carlos_33', 'M', 'PR', 'Londrina'), (34, 'Carlos_34', 'M', 'PE', 'Recife'), (35, 'Joao_35', 'M', 'SC', 'Florianópolis'), (36, 'Marcos_36', 'M', 'RS', 'Porto Alegre'), (37, 'Luiza_37', 'M', 'PE', 'Olinda'), (38, 'Patricia_38', 'M', 'SP', 'Campinas'), (39, 'Gabriela_39', 'F', 'RJ', 'Cabo Frio'), (40, 'Carlos_40', 'F', 'MG', 'Betim'), (41, 'Luiza_41', 'M', 'SC', 'Joinville'), (42, 'Carlos_42', 'F', 'RS', 'Caxias do Sul'), (43, 'Marcos_43', 'M', 'PR', 'Curitiba'), (44, 'Gabriela_44', 'F', 'MG', 'Belo Horizonte'), (45, 'Fernanda_45', 'F', 'PE', 'Olinda'), (46, 'Luiza_46', 'M', 'RJ', 'Rio de Janeiro'), (47, 'Patricia_47', 'M', 'SC', 'Joinville'), (48, 'Fernanda_48', 'F', 'BA', 'Salvador'), (49, 'Patricia_49', 'M', 'SC', 'Florianópolis'), (50, 'Fernanda_50', 'M', 'PE', 'Olinda'), (51, 'Joao_51', 'F', 'MG', 'Belo Horizonte'), (52, 'Joao_52', 'F', 'SP', 'São Caetano'), (53, 'Jean_53', 'M', 'BA', 'Feira de Santana'), (54, 'Jean_54', 'F', 'SC', 'Joinville'), (55, 'Joao_55', 'F', 'BA', 'Feira de Santana'), (56, 'Carlos_56', 'M', 'SC', 'Florianópolis'), (57, 'Luiza_57', 'F', 'SP', 'Campinas'), (58, 'Luiza_58', 'F', 'SC', 'Joinville'), (59, 'Luiza_59', 'F', 'RJ', 'Cabo Frio'), (60, 'Gabriela_60', 'M', 'SC', 'Florianópolis'), (61, 'Fernanda_61', 'F', 'BA', 'Feira de Santana'), (62, 'Patricia_62', 'F', 'BA', 'Salvador'), (63, 'Gabriela_63', 'M', 'MG', 'Betim'), (64, 'Carlos_64', 'M', 'PR', 'Curitiba'), (65, 'Rafael_65', 'F', 'RS', 'Porto Alegre'), (66, 'Patricia_66', 'M', 'SP', 'Campinas'), (67, 'Rafael_67', 'M', 'SC', 'Joinville'), (68, 'Jean_68', 'M', 'RS', 'Porto Alegre'), (69, 'Daniela_69', 'M', 'SC', 'Joinville'), (70, 'Gabriela_70', 'M', 'SP', 'Campinas'), (71, 'Marcos_71', 'F', 'RJ', 'Rio de Janeiro'), (72, 'Luiza_72', 'M', 'PE', 'Olinda'), (73, 'Luiza_73', 'F', 'RJ', 'Rio de Janeiro'), (74, 'Rafael_74', 'F', 'RS', 'Caxias do Sul'), (75, 'Marcos_75', 'M', 'RS', 'Porto Alegre'), (76, 'Daniela_76', 'F', 'SP', 'São Caetano'), (77, 'Joao_77', 'M', 'PR', 'Londrina'), (78, 'Luiza_78', 'F', 'PE', 'Olinda'), (79, 'Patricia_79', 'F', 'BA', 'Salvador'), (80, 'Luiza_80', 'M', 'PE', 'Olinda'), (81, 'Luiza_81', 'F', 'MG', 'Betim'), (82, 'Luiza_82', 'M', 'PR', 'Londrina'), (83, 'Gabriela_83', 'F', 'PE', 'Olinda'), (84, 'Joao_84', 'F', 'BA', 'Salvador'), (85, 'Luiza_85', 'M', 'MG', 'Belo Horizonte'), (86, 'Daniela_86', 'M', 'RS', 'Porto Alegre'), (87, 'Marcos_87', 'F', 'RS', 'Porto Alegre'), (88, 'Daniela_88', 'F', 'PR', 'Curitiba'), (89, 'Marcos_89', 'M', 'RS', 'Porto Alegre'), (90, 'Daniela_90', 'M', 'PE', 'Olinda'), (91, 'Rafael_91', 'M', 'BA', 'Salvador'), (92, 'Joao_92', 'F', 'RS', 'Porto Alegre'), (93, 'Daniela_93', 'M', 'RJ', 'Rio de Janeiro'), (94, 'Jean_94', 'M', 'MG', 'Belo Horizonte'), (95, 'Carlos_95', 'M', 'SP', 'Campinas'), (96, 'Patricia_96', 'F', 'RJ', 'Rio de Janeiro'), (97, 'Marcos_97', 'F', 'PE', 'Olinda'), (98, 'Luiza_98', 'F', 'SC', 'Joinville'), (99, 'Jean_99', 'F', 'SP', 'São Paulo'), (100, 'Carlos_100', 'F', 'RJ', 'Rio de Janeiro'), (101, 'Daniela_101', 'F', 'MG', 'Uberlândia'), (102, 'Marcos_102', 'F', 'SP', 'São Caetano'), (103, 'Marcos_103', 'F', 'BA', 'Salvador'), (104, 'Fernanda_104', 'M', 'RS', 'Caxias do Sul'), (105, 'Luiza_105', 'F', 'PE', 'Olinda'), (106, 'Marcos_106', 'F', 'RJ', 'Cabo Frio'), (107, 'Joao_107', 'M', 'MG', 'Uberlândia'), (108, 'Fernanda_108', 'F', 'PR', 'Londrina'), (109, 'Fernanda_109', 'M', 'RJ', 'Rio de Janeiro'), (110, 'Gabriela_110', 'F', 'PE', 'Olinda'), (111, 'Joao_111', 'M', 'SC', 'Florianópolis'), (112, 'Rafael_112', 'F', 'BA', 'Salvador'), (113, 'Daniela_113', 'M', 'PR', 'Londrina'), (114, 'Gabriela_114', 'F', 'RS', 'Porto Alegre'), (115, 'Rafael_115', 'M', 'PE', 'Recife'), (116, 'Rafael_116', 'M', 'PE', 'Olinda'), (117, 'Marcos_117', 'F', 'RS', 'Porto Alegre'), (118, 'Jean_118', 'F', 'SP', 'Campinas'), (119, 'Jean_119', 'F', 'PR', 'Curitiba'), (120, 'Rafael_120', 'M', 'PR', 'Curitiba'), (121, 'Jean_121', 'M', 'SP', 'Campinas'), (122, 'Luiza_122', 'F', 'BA', 'Feira de Santana'), (123, 'Daniela_123', 'F', 'PR', 'Curitiba'), (124, 'Gabriela_124', 'F', 'RS', 'Porto Alegre'), (125, 'Fernanda_125', 'F', 'RJ', 'Niteroi'), (126, 'Marcos_126', 'F', 'PR', 'Curitiba'), (127, 'Marcos_127', 'M', 'RJ', 'Rio de Janeiro'), (128, 'Jean_128', 'M', 'PR', 'Curitiba'), (129, 'Carlos_129', 'F', 'SP', 'Campinas'), (130, 'Joao_130', 'F', 'PE', 'Olinda'), (131, 'Rafael_131', 'M', 'SP', 'Campinas'), (132, 'Rafael_132', 'M', 'RJ', 'Cabo Frio'), (133, 'Jean_133', 'M', 'RJ', 'Cabo Frio'), (134, 'Carlos_134', 'M', 'PE', 'Recife'), (135, 'Marcos_135', 'M', 'SC', 'Florianópolis'), (136, 'Patricia_136', 'M', 'PE', 'Recife'), (137, 'Patricia_137', 'M', 'SC', 'Florianópolis'), (138, 'Carlos_138', 'M', 'PR', 'Curitiba'), (139, 'Marcos_139', 'F', 'BA', 'Feira de Santana'), (140, 'Luiza_140', 'M', 'MG', 'Betim'), (141, 'Joao_141', 'F', 'PE', 'Recife'), (142, 'Jean_142', 'F', 'BA', 'Feira de Santana'), (143, 'Joao_143', 'M', 'SP', 'São Caetano'), (144, 'Jean_144', 'M', 'RJ', 'Niteroi'), (145, 'Patricia_145', 'F', 'RS', 'Caxias do Sul'), (146, 'Patricia_146', 'F', 'RS', 'Porto Alegre'), (147, 'Carlos_147', 'M', 'BA', 'Salvador'), (148, 'Luiza_148', 'M', 'MG', 'Betim'), (149, 'Luiza_149', 'M', 'SP', 'São Caetano'), (150, 'Marcos_150', 'M', 'RJ', 'Cabo Frio'), (151, 'Jean_151', 'M', 'PE', 'Recife'), (152, 'Carlos_152', 'F', 'PE', 'Olinda'), (153, 'Daniela_153', 'M', 'PR', 'Curitiba'), (154, 'Carlos_154', 'F', 'SP', 'São Caetano'), (155, 'Joao_155', 'F', 'RS', 'Caxias do Sul'), (156, 'Fernanda_156', 'F', 'SC', 'Joinville'), (157, 'Luiza_157', 'F', 'PE', 'Olinda'), (158, 'Gabriela_158', 'M', 'RJ', 'Cabo Frio'), (159, 'Carlos_159', 'M', 'PR', 'Londrina'), (160, 'Daniela_160', 'F', 'PR', 'Londrina'), (161, 'Luiza_161', 'M', 'BA', 'Salvador'), (162, 'Gabriela_162', 'M', 'RS', 'Caxias do Sul'), (163, 'Luiza_163', 'M', 'PR', 'Londrina'), (164, 'Marcos_164', 'M', 'SP', 'São Caetano'), (165, 'Rafael_165', 'F', 'SC', 'Florianópolis'), (166, 'Patricia_166', 'M', 'RS', 'Caxias do Sul'), (167, 'Fernanda_167', 'M', 'BA', 'Feira de Santana'), (168, 'Fernanda_168', 'F', 'SP', 'Campinas'), (169, 'Fernanda_169', 'M', 'SP', 'São Paulo'), (170, 'Daniela_170', 'F', 'SP', 'São Caetano'), (171, 'Gabriela_171', 'M', 'PE', 'Recife'), (172, 'Daniela_172', 'F', 'BA', 'Salvador'), (173, 'Luiza_173', 'F', 'RS', 'Caxias do Sul'), (174, 'Patricia_174', 'F', 'MG', 'Uberlândia'), (175, 'Joao_175', 'F', 'RS', 'Porto Alegre'), (176, 'Marcos_176', 'F', 'SC', 'Joinville'), (177, 'Luiza_177', 'F', 'SC', 'Florianópolis'), (178, 'Jean_178', 'M', 'SP', 'São Paulo'), (179, 'Carlos_179', 'F', 'SP', 'São Caetano'), (180, 'Rafael_180', 'M', 'PE', 'Olinda'), (181, 'Gabriela_181', 'M', 'MG', 'Betim'), (182, 'Fernanda_182', 'F', 'PE', 'Recife'), (183, 'Rafael_183', 'M', 'SC', 'Joinville'), (184, 'Daniela_184', 'F', 'SP', 'Campinas'), (185, 'Rafael_185', 'M', 'SP', 'São Paulo'), (186, 'Luiza_186', 'M', 'BA', 'Salvador'), (187, 'Gabriela_187', 'M', 'SC', 'Joinville'), (188, 'Joao_188', 'M', 'PR', 'Curitiba'), (189, 'Gabriela_189', 'F', 'SC', 'Joinville'), (190, 'Jean_190', 'F', 'SC', 'Joinville'), (191, 'Carlos_191', 'F', 'PE', 'Olinda'), (192, 'Fernanda_192', 'F', 'PR', 'Curitiba'), (193, 'Daniela_193', 'M', 'PR', 'Londrina'), (194, 'Rafael_194', 'M', 'PE', 'Recife'), (195, 'Gabriela_195', 'M', 'RS', 'Porto Alegre'), (196, 'Marcos_196', 'F', 'PR', 'Curitiba'), (197, 'Gabriela_197', 'F', 'PR', 'Curitiba'), (198, 'Daniela_198', 'F', 'MG', 'Betim'), (199, 'Daniela_199', 'F', 'SP', 'São Caetano'), (200, 'Joao_200', 'F', 'BA', 'Salvador'), (201, 'Joao_201', 'F', 'RJ', 'Cabo Frio'), (202, 'Luiza_202', 'M', 'SC', 'Joinville'), (203, 'Luiza_203', 'F', 'SC', 'Joinville'), (204, 'Fernanda_204', 'F', 'BA', 'Salvador'), (205, 'Patricia_205', 'M', 'MG', 'Betim'), (206, 'Gabriela_206', 'M', 'PR', 'Curitiba'), (207, 'Luiza_207', 'M', 'SC', 'Joinville'), (208, 'Marcos_208', 'F', 'SP', 'São Caetano'), (209, 'Patricia_209', 'F', 'MG', 'Betim'), (210, 'Luiza_210', 'M', 'RJ', 'Niteroi'), (211, 'Carlos_211', 'M', 'MG', 'Belo Horizonte'), (212, 'Rafael_212', 'F', 'RS', 'Caxias do Sul'), (213, 'Marcos_213', 'F', 'RS', 'Porto Alegre'), (214, 'Joao_214', 'F', 'RJ', 'Cabo Frio'), (215, 'Carlos_215', 'M', 'MG', 'Uberlândia'), (216, 'Daniela_216', 'F', 'BA', 'Feira de Santana'), (217, 'Patricia_217', 'M', 'RS', 'Porto Alegre'), (218, 'Jean_218', 'M', 'RS', 'Porto Alegre'), (219, 'Patricia_219', 'M', 'PE', 'Recife'), (220, 'Fernanda_220', 'F', 'RJ', 'Niteroi'), (221, 'Carlos_221', 'F', 'SC', 'Joinville'), (222, 'Daniela_222', 'F', 'RJ', 'Rio de Janeiro'), (223, 'Marcos_223', 'F', 'SC', 'Florianópolis'), (224, 'Joao_224', 'M', 'SP', 'São Caetano'), (225, 'Rafael_225', 'M', 'BA', 'Feira de Santana'), (226, 'Marcos_226', 'M', 'RS', 'Caxias do Sul'), (227, 'Luiza_227', 'M', 'PE', 'Recife'), (228, 'Joao_228', 'F', 'RS', 'Caxias do Sul'), (229, 'Gabriela_229', 'F', 'RS', 'Caxias do Sul'), (230, 'Gabriela_230', 'M', 'RJ', 'Cabo Frio'), (231, 'Fernanda_231', 'M', 'SP', 'São Caetano'), (232, 'Carlos_232', 'M', 'MG', 'Belo Horizonte'), (233, 'Marcos_233', 'M', 'SC', 'Joinville'), (234, 'Patricia_234', 'F', 'PR', 'Londrina'), (235, 'Daniela_235', 'F', 'BA', 'Feira de Santana'), (236, 'Daniela_236', 'M', 'SC', 'Florianópolis'), (237, 'Carlos_237', 'F', 'SC', 'Joinville'), (238, 'Marcos_238', 'F', 'PR', 'Curitiba'), (239, 'Marcos_239', 'F', 'SC', 'Joinville'), (240, 'Marcos_240', 'M', 'RS', 'Caxias do Sul'), (241, 'Fernanda_241', 'M', 'RS', 'Porto Alegre'), (242, 'Marcos_242', 'M', 'SC', 'Florianópolis'), (243, 'Joao_243', 'M', 'RS', 'Porto Alegre'), (244, 'Rafael_244', 'M', 'SP', 'Campinas'), (245, 'Fernanda_245', 'M', 'SP', 'Campinas'), (246, 'Luiza_246', 'F', 'BA', 'Feira de Santana'), (247, 'Daniela_247', 'M', 'SP', 'São Paulo'), (248, 'Carlos_248', 'M', 'RJ', 'Rio de Janeiro'), (249, 'Luiza_249', 'M', 'BA', 'Feira de Santana'), (250, 'Luiza_250', 'F', 'SP', 'São Caetano'), (251, 'Jean_251', 'F', 'MG', 'Uberlândia'), (252, 'Marcos_252', 'M', 'BA', 'Salvador'), (253, 'Patricia_253', 'M', 'PE', 'Recife'), (254, 'Rafael_254', 'M', 'BA', 'Salvador'), (255, 'Gabriela_255', 'M', 'RS', 'Porto Alegre'), (256, 'Fernanda_256', 'M', 'RJ', 'Cabo Frio'), (257, 'Marcos_257', 'F', 'RJ', 'Niteroi'), (258, 'Fernanda_258', 'M', 'RJ', 'Cabo Frio'), (259, 'Luiza_259', 'F', 'RS', 'Porto Alegre'), (260, 'Marcos_260', 'M', 'SC', 'Joinville'), (261, 'Joao_261', 'M', 'PE', 'Recife'), (262, 'Rafael_262', 'F', 'RS', 'Porto Alegre'), (263, 'Marcos_263', 'F', 'MG', 'Uberlândia'), (264, 'Gabriela_264', 'F', 'PE', 'Recife'), (265, 'Carlos_265', 'M', 'SP', 'São Paulo'), (266, 'Carlos_266', 'M', 'PE', 'Olinda'), (267, 'Joao_267', 'M', 'MG', 'Betim'), (268, 'Jean_268', 'F', 'PR', 'Londrina'), (269, 'Marcos_269', 'F', 'RS', 'Porto Alegre'), (270, 'Daniela_270', 'M', 'MG', 'Uberlândia'), (271, 'Jean_271', 'M', 'MG', 'Uberlândia'), (272, 'Patricia_272', 'M', 'MG', 'Uberlândia'), (273, 'Luiza_273', 'M', 'BA', 'Feira de Santana'), (274, 'Carlos_274', 'M', 'PR', 'Londrina'), (275, 'Marcos_275', 'M', 'PE', 'Olinda'), (276, 'Rafael_276', 'M', 'PR', 'Londrina'), (277, 'Carlos_277', 'M', 'RJ', 'Rio de Janeiro'), (278, 'Daniela_278', 'F', 'RJ', 'Cabo Frio'), (279, 'Rafael_279', 'M', 'BA', 'Salvador'), (280, 'Fernanda_280', 'M', 'RS', 'Caxias do Sul'), (281, 'Jean_281', 'F', 'SP', 'São Caetano'), (282, 'Daniela_282', 'F', 'PE', 'Recife'), (283, 'Luiza_283', 'M', 'RJ', 'Niteroi'), (284, 'Gabriela_284', 'M', 'MG', 'Belo Horizonte'), (285, 'Joao_285', 'M', 'SC', 'Florianópolis'), (286, 'Fernanda_286', 'F', 'BA', 'Feira de Santana'), (287, 'Marcos_287', 'M', 'RS', 'Caxias do Sul'), (288, 'Carlos_288', 'F', 'PR', 'Curitiba'), (289, 'Marcos_289', 'F', 'SC', 'Florianópolis'), (290, 'Jean_290', 'F', 'PR', 'Londrina'), (291, 'Joao_291', 'M', 'SP', 'São Paulo'), (292, 'Marcos_292', 'F', 'RJ', 'Rio de Janeiro'), (293, 'Fernanda_293', 'M', 'PR', 'Londrina'), (294, 'Luiza_294', 'F', 'PE', 'Olinda'), (295, 'Joao_295', 'M', 'BA', 'Salvador'), (296, 'Gabriela_296', 'M', 'RJ', 'Niteroi'), (297, 'Fernanda_297', 'F', 'BA', 'Feira de Santana'), (298, 'Marcos_298', 'F', 'RJ', 'Rio de Janeiro'), (299, 'Rafael_299', 'F', 'RJ', 'Cabo Frio'), (300, 'Joao_300', 'M', 'RJ', 'Rio de Janeiro'), (301, 'Fernanda_301', 'F', 'MG', 'Betim'), (302, 'Jean_302', 'M', 'SP', 'São Caetano'), (303, 'Gabriela_303', 'F', 'SP', 'Campinas'), (304, 'Carlos_304', 'F', 'RJ', 'Cabo Frio'), (305, 'Jean_305', 'F', 'PE', 'Olinda'), (306, 'Luiza_306', 'M', 'PE', 'Olinda'), (307, 'Patricia_307', 'M', 'BA', 'Salvador'), (308, 'Carlos_308', 'M', 'BA', 'Salvador'), (309, 'Luiza_309', 'M', 'SC', 'Joinville'), (310, 'Carlos_310', 'F', 'SP', 'São Caetano'), (311, 'Daniela_311', 'M', 'SP', 'Campinas'), (312, 'Daniela_312', 'F', 'RJ', 'Niteroi'), (313, 'Gabriela_313', 'F', 'MG', 'Betim'), (314, 'Joao_314', 'F', 'MG', 'Betim'), (315, 'Rafael_315', 'F', 'SP', 'São Caetano'), (316, 'Jean_316', 'M', 'BA', 'Salvador'), (317, 'Patricia_317', 'M', 'SC', 'Joinville'), (318, 'Jean_318', 'M', 'SC', 'Florianópolis'), (319, 'Jean_319', 'M', 'PR', 'Curitiba'), (320, 'Carlos_320', 'F', 'PE', 'Recife'), (321, 'Fernanda_321', 'F', 'PE', 'Recife'), (322, 'Fernanda_322', 'F', 'MG', 'Belo Horizonte'), (323, 'Gabriela_323', 'F', 'PE', 'Recife'), (324, 'Gabriela_324', 'M', 'SP', 'São Paulo'), (325, 'Joao_325', 'F', 'BA', 'Feira de Santana'), (326, 'Fernanda_326', 'F', 'MG', 'Uberlândia'), (327, 'Gabriela_327', 'M', 'MG', 'Uberlândia'), (328, 'Marcos_328', 'M', 'PE', 'Olinda'), (329, 'Jean_329', 'F', 'SC', 'Florianópolis'), (330, 'Gabriela_330', 'F', 'PE', 'Olinda'), (331, 'Luiza_331', 'M', 'SC', 'Joinville'), (332, 'Jean_332', 'M', 'BA', 'Feira de Santana'), (333, 'Fernanda_333', 'F', 'MG', 'Uberlândia'), (334, 'Luiza_334', 'M', 'BA', 'Salvador'), (335, 'Jean_335', 'F', 'SP', 'São Caetano'), (336, 'Luiza_336', 'F', 'MG', 'Betim'), (337, 'Joao_337', 'F', 'SC', 'Florianópolis'), (338, 'Rafael_338', 'F', 'PR', 'Curitiba'), (339, 'Luiza_339', 'M', 'SP', 'São Paulo'), (340, 'Patricia_340', 'F', 'PE', 'Recife'), (341, 'Carlos_341', 'M', 'PE', 'Olinda'), (342, 'Patricia_342', 'F', 'RS', 'Caxias do Sul'), (343, 'Carlos_343', 'M', 'RS', 'Porto Alegre'), (344, 'Daniela_344', 'M', 'SC', 'Joinville'), (345, 'Luiza_345', 'F', 'RS', 'Porto Alegre'), (346, 'Carlos_346', 'F', 'PE', 'Recife'), (347, 'Fernanda_347', 'M', 'PR', 'Londrina'), (348, 'Carlos_348', 'M', 'BA', 'Salvador'), (349, 'Carlos_349', 'F', 'RS', 'Caxias do Sul'), (350, 'Gabriela_350', 'F', 'RJ', 'Rio de Janeiro'), (351, 'Fernanda_351', 'F', 'PE', 'Olinda'), (352, 'Patricia_352', 'F', 'SP', 'São Paulo'), (353, 'Joao_353', 'F', 'PE', 'Olinda'), (354, 'Fernanda_354', 'M', 'MG', 'Uberlândia'), (355, 'Jean_355', 'M', 'BA', 'Feira de Santana'), (356, 'Patricia_356', 'M', 'PE', 'Olinda'), (357, 'Gabriela_357', 'F', 'SC', 'Joinville'), (358, 'Joao_358', 'M', 'RS', 'Caxias do Sul'), (359, 'Rafael_359', 'F', 'RJ', 'Niteroi'), (360, 'Fernanda_360', 'F', 'PR', 'Londrina'), (361, 'Gabriela_361', 'M', 'RJ', 'Niteroi'), (362, 'Joao_362', 'F', 'SC', 'Joinville'), (363, 'Fernanda_363', 'F', 'SP', 'São Paulo'), (364, 'Joao_364', 'M', 'SC', 'Joinville'), (365, 'Jean_365', 'F', 'MG', 'Uberlândia'), (366, 'Rafael_366', 'M', 'MG', 'Betim'), (367, 'Luiza_367', 'F', 'PR', 'Curitiba'), (368, 'Joao_368', 'F', 'SP', 'São Paulo'), (369, 'Joao_369', 'F', 'MG', 'Belo Horizonte'), (370, 'Patricia_370', 'M', 'SP', 'São Paulo'), (371, 'Marcos_371', 'M', 'RS', 'Porto Alegre'), (372, 'Daniela_372', 'M', 'PE', 'Recife'), (373, 'Gabriela_373', 'M', 'RJ', 'Cabo Frio'), (374, 'Luiza_374', 'F', 'RS', 'Caxias do Sul'), (375, 'Fernanda_375', 'F', 'RJ', 'Rio de Janeiro'), (376, 'Gabriela_376', 'M', 'MG', 'Betim'), (377, 'Carlos_377', 'M', 'SC', 'Florianópolis'), (378, 'Daniela_378', 'M', 'MG', 'Uberlândia'), (379, 'Gabriela_379', 'F', 'PR', 'Londrina'), (380, 'Rafael_380', 'F', 'PE', 'Olinda'), (381, 'Fernanda_381', 'M', 'SC', 'Joinville'), (382, 'Daniela_382', 'F', 'PR', 'Curitiba'), (383, 'Fernanda_383', 'M', 'SP', 'São Caetano'), (384, 'Daniela_384', 'M', 'SP', 'São Caetano'), (385, 'Marcos_385', 'M', 'PE', 'Recife'), (386, 'Patricia_386', 'F', 'SP', 'São Paulo'), (387, 'Gabriela_387', 'F', 'RJ', 'Rio de Janeiro'), (388, 'Daniela_388', 'F', 'PE', 'Olinda'), (389, 'Gabriela_389', 'F', 'BA', 'Feira de Santana'), (390, 'Luiza_390', 'M', 'RS', 'Porto Alegre'), (391, 'Daniela_391', 'F', 'RJ', 'Cabo Frio'), (392, 'Gabriela_392', 'F', 'RS', 'Porto Alegre'), (393, 'Patricia_393', 'M', 'BA', 'Salvador'), (394, 'Daniela_394', 'F', 'BA', 'Feira de Santana'), (395, 'Jean_395', 'F', 'RS', 'Porto Alegre'), (396, 'Marcos_396', 'F', 'MG', 'Betim'), (397, 'Fernanda_397', 'M', 'BA', 'Feira de Santana'), (398, 'Patricia_398', 'F', 'PE', 'Olinda'), (399, 'Joao_399', 'F', 'BA', 'Salvador'), (400, 'Gabriela_400', 'F', 'RS', 'Caxias do Sul'), (401, 'Marcos_401', 'M', 'SC', 'Joinville'), (402, 'Luiza_402', 'M', 'RJ', 'Cabo Frio'), (403, 'Joao_403', 'F', 'SC', 'Florianópolis'), (404, 'Daniela_404', 'M', 'SP', 'São Paulo'), (405, 'Luiza_405', 'M', 'PE', 'Olinda'), (406, 'Luiza_406', 'F', 'SP', 'São Caetano'), (407, 'Jean_407', 'F', 'PR', 'Londrina'), (408, 'Luiza_408', 'M', 'SC', 'Joinville'), (409, 'Fernanda_409', 'F', 'BA', 'Feira de Santana'), (410, 'Carlos_410', 'F', 'RJ', 'Niteroi'), (411, 'Luiza_411', 'F', 'PE', 'Olinda'), (412, 'Gabriela_412', 'F', 'SC', 'Florianópolis'), (413, 'Daniela_413', 'F', 'RS', 'Porto Alegre'), (414, 'Daniela_414', 'F', 'RS', 'Porto Alegre'), (415, 'Joao_415', 'F', 'SC', 'Joinville'), (416, 'Fernanda_416', 'F', 'RJ', 'Cabo Frio'), (417, 'Luiza_417', 'M', 'RS', 'Caxias do Sul'), (418, 'Carlos_418', 'F', 'BA', 'Feira de Santana'), (419, 'Gabriela_419', 'M', 'RJ', 'Cabo Frio'), (420, 'Jean_420', 'M', 'SC', 'Florianópolis'), (421, 'Joao_421', 'M', 'RJ', 'Niteroi'), (422, 'Luiza_422', 'F', 'RJ', 'Rio de Janeiro'), (423, 'Daniela_423', 'F', 'PR', 'Londrina'), (424, 'Fernanda_424', 'F', 'SP', 'São Caetano'), (425, 'Joao_425', 'M', 'BA', 'Feira de Santana'), (426, 'Joao_426', 'F', 'SP', 'São Caetano'), (427, 'Marcos_427', 'M', 'RS', 'Caxias do Sul'), (428, 'Fernanda_428', 'M', 'MG', 'Betim'), (429, 'Gabriela_429', 'F', 'RS', 'Caxias do Sul'), (430, 'Luiza_430', 'F', 'PR', 'Londrina'), (431, 'Rafael_431', 'M', 'PR', 'Londrina'), (432, 'Jean_432', 'F', 'PR', 'Londrina'), (433, 'Carlos_433', 'M', 'SC', 'Joinville'), (434, 'Jean_434', 'F', 'RS', 'Porto Alegre'), (435, 'Marcos_435', 'F', 'RJ', 'Niteroi'), (436, 'Daniela_436', 'F', 'RJ', 'Niteroi'), (437, 'Fernanda_437', 'F', 'SP', 'São Caetano'), (438, 'Patricia_438', 'M', 'MG', 'Uberlândia'), (439, 'Luiza_439', 'F', 'RS', 'Porto Alegre'), (440, 'Patricia_440', 'M', 'PR', 'Curitiba'), (441, 'Luiza_441', 'F', 'BA', 'Salvador'), (442, 'Luiza_442', 'F', 'PE', 'Olinda'), (443, 'Carlos_443', 'F', 'RJ', 'Niteroi'), (444, 'Fernanda_444', 'F', 'RJ', 'Rio de Janeiro'), (445, 'Fernanda_445', 'F', 'RS', 'Caxias do Sul'), (446, 'Patricia_446', 'F', 'RS', 'Porto Alegre'), (447, 'Carlos_447', 'M', 'MG', 'Belo Horizonte'), (448, 'Luiza_448', 'M', 'PR', 'Curitiba'), (449, 'Fernanda_449', 'F', 'PE', 'Recife'), (450, 'Fernanda_450', 'F', 'PE', 'Olinda'), (451, 'Rafael_451', 'F', 'RS', 'Caxias do Sul'), (452, 'Daniela_452', 'F', 'RJ', 'Cabo Frio'), (453, 'Jean_453', 'M', 'PR', 'Curitiba'), (454, 'Luiza_454', 'M', 'SP', 'Campinas'), (455, 'Joao_455', 'F', 'PE', 'Recife'), (456, 'Fernanda_456', 'F', 'SP', 'São Paulo'), (457, 'Joao_457', 'M', 'PE', 'Olinda'), (458, 'Rafael_458', 'M', 'SC', 'Joinville'), (459, 'Gabriela_459', 'M', 'BA', 'Feira de Santana'), (460, 'Gabriela_460', 'F', 'PE', 'Recife'), (461, 'Gabriela_461', 'F', 'PR', 'Curitiba'), (462, 'Jean_462', 'M', 'BA', 'Feira de Santana'), (463, 'Joao_463', 'F', 'RS', 'Porto Alegre'), (464, 'Marcos_464', 'M', 'RS', 'Caxias do Sul'), (465, 'Fernanda_465', 'M', 'PE', 'Recife'), (466, 'Marcos_466', 'M', 'PR', 'Londrina'), (467, 'Luiza_467', 'F', 'BA', 'Salvador'), (468, 'Jean_468', 'F', 'PR', 'Curitiba'), (469, 'Patricia_469', 'F', 'PE', 'Olinda'), (470, 'Fernanda_470', 'F', 'SP', 'São Paulo'), (471, 'Jean_471', 'M', 'PR', 'Londrina'), (472, 'Marcos_472', 'M', 'BA', 'Salvador'), (473, 'Carlos_473', 'M', 'RS', 'Porto Alegre'), (474, 'Patricia_474', 'M', 'PE', 'Olinda'), (475, 'Luiza_475', 'M', 'RJ', 'Niteroi'), (476, 'Gabriela_476', 'M', 'MG', 'Betim'), (477, 'Luiza_477', 'F', 'SP', 'São Caetano'), (478, 'Daniela_478', 'M', 'PE', 'Olinda'), (479, 'Carlos_479', 'F', 'SC', 'Florianópolis'), (480, 'Daniela_480', 'F', 'RJ', 'Cabo Frio'), (481, 'Daniela_481', 'M', 'SP', 'Campinas'), (482, 'Luiza_482', 'F', 'PE', 'Recife'), (483, 'Marcos_483', 'F', 'MG', 'Betim'), (484, 'Fernanda_484', 'F', 'MG', 'Betim'), (485, 'Daniela_485', 'F', 'SP', 'Campinas'), (486, 'Rafael_486', 'M', 'PE', 'Recife'), (487, 'Luiza_487', 'M', 'MG', 'Betim'), (488, 'Jean_488', 'F', 'SC', 'Florianópolis'), (489, 'Luiza_489', 'M', 'MG', 'Uberlândia'), (490, 'Jean_490', 'M', 'RJ', 'Niteroi'), (491, 'Gabriela_491', 'M', 'PR', 'Curitiba'), (492, 'Fernanda_492', 'F', 'RS', 'Caxias do Sul'), (493, 'Daniela_493', 'F', 'PR', 'Londrina'), (494, 'Gabriela_494', 'F', 'PE', 'Recife'), (495, 'Marcos_495', 'M', 'RJ', 'Rio de Janeiro'), (496, 'Luiza_496', 'F', 'PR', 'Londrina'), (497, 'Gabriela_497', 'M', 'PR', 'Curitiba'), (498, 'Marcos_498', 'F', 'PR', 'Curitiba'), (499, 'Joao_499', 'M', 'PR', 'Curitiba'), (500, 'Jean_500', 'F', 'RS', 'Porto Alegre'), (501, 'Luiza_501', 'F', 'SP', 'São Caetano'), (502, 'Marcos_502', 'M', 'RS', 'Porto Alegre'), (503, 'Daniela_503', 'F', 'SC', 'Florianópolis'), (504, 'Fernanda_504', 'F', 'RS', 'Porto Alegre'), (505, 'Jean_505', 'M', 'PE', 'Olinda'), (506, 'Patricia_506', 'F', 'SC', 'Florianópolis'), (507, 'Fernanda_507', 'M', 'RJ', 'Rio de Janeiro'), (508, 'Joao_508', 'M', 'RS', 'Caxias do Sul'), (509, 'Patricia_509', 'M', 'PE', 'Olinda'), (510, 'Marcos_510', 'F', 'PE', 'Olinda'), (511, 'Gabriela_511', 'M', 'SC', 'Florianópolis'), (512, 'Carlos_512', 'M', 'RS', 'Porto Alegre'), (513, 'Fernanda_513', 'F', 'RS', 'Caxias do Sul'), (514, 'Rafael_514', 'F', 'RJ', 'Niteroi'), (515, 'Marcos_515', 'M', 'PE', 'Recife'), (516, 'Joao_516', 'F', 'PR', 'Curitiba'), (517, 'Daniela_517', 'M', 'PR', 'Londrina'), (518, 'Carlos_518', 'F', 'MG', 'Betim'), (519, 'Luiza_519', 'M', 'BA', 'Salvador'), (520, 'Patricia_520', 'F', 'SC', 'Florianópolis'), (521, 'Joao_521', 'M', 'PE', 'Olinda'), (522, 'Rafael_522', 'F', 'RS', 'Porto Alegre'), (523, 'Gabriela_523', 'M', 'RJ', 'Rio de Janeiro'), (524, 'Rafael_524', 'F', 'PE', 'Recife'), (525, 'Marcos_525', 'M', 'RJ', 'Niteroi'), (526, 'Joao_526', 'M', 'RS', 'Caxias do Sul'), (527, 'Daniela_527', 'F', 'RS', 'Porto Alegre'), (528, 'Gabriela_528', 'F', 'PE', 'Recife'), (529, 'Carlos_529', 'F', 'SP', 'Campinas'), (530, 'Luiza_530', 'M', 'RS', 'Porto Alegre'), (531, 'Luiza_531', 'M', 'SC', 'Joinville'), (532, 'Luiza_532', 'F', 'PR', 'Curitiba'), (533, 'Jean_533', 'M', 'SC', 'Florianópolis'), (534, 'Patricia_534', 'F', 'PE', 'Olinda'), (535, 'Luiza_535', 'M', 'PE', 'Olinda'), (536, 'Carlos_536', 'M', 'MG', 'Uberlândia'), (537, 'Luiza_537', 'F', 'BA', 'Salvador'), (538, 'Rafael_538', 'M', 'SP', 'São Paulo'), (539, 'Daniela_539', 'M', 'SP', 'Campinas'), (540, 'Patricia_540', 'M', 'SP', 'Campinas'), (541, 'Luiza_541', 'F', 'MG', 'Uberlândia'), (542, 'Joao_542', 'F', 'BA', 'Salvador'), (543, 'Rafael_543', 'M', 'RJ', 'Niteroi'), (544, 'Luiza_544', 'M', 'RS', 'Porto Alegre'), (545, 'Gabriela_545', 'F', 'SC', 'Florianópolis'), (546, 'Gabriela_546', 'F', 'PR', 'Curitiba'), (547, 'Carlos_547', 'F', 'PE', 'Olinda'), (548, 'Rafael_548', 'M', 'PE', 'Olinda'), (549, 'Carlos_549', 'F', 'SP', 'Campinas'), (550, 'Luiza_550', 'M', 'RJ', 'Cabo Frio'), (551, 'Jean_551', 'F', 'RJ', 'Niteroi'), (552, 'Marcos_552', 'F', 'RJ', 'Cabo Frio'), (553, 'Joao_553', 'M', 'RJ', 'Niteroi'), (554, 'Fernanda_554', 'M', 'BA', 'Feira de Santana'), (555, 'Gabriela_555', 'F', 'RJ', 'Niteroi'), (556, 'Marcos_556', 'F', 'SP', 'São Caetano'), (557, 'Gabriela_557', 'F', 'PE', 'Recife'), (558, 'Marcos_558', 'M', 'RJ', 'Rio de Janeiro'), (559, 'Fernanda_559', 'F', 'BA', 'Feira de Santana'), (560, 'Patricia_560', 'M', 'SP', 'Campinas'), (561, 'Carlos_561', 'F', 'PR', 'Curitiba'), (562, 'Fernanda_562', 'F', 'RJ', 'Niteroi'), (563, 'Marcos_563', 'M', 'PE', 'Olinda'), (564, 'Fernanda_564', 'M', 'SP', 'Campinas'), (565, 'Fernanda_565', 'F', 'RJ', 'Rio de Janeiro'), (566, 'Daniela_566', 'M', 'RJ', 'Cabo Frio'), (567, 'Gabriela_567', 'M', 'RS', 'Porto Alegre'), (568, 'Rafael_568', 'F', 'SP', 'São Caetano'), (569, 'Gabriela_569', 'F', 'MG', 'Betim'), (570, 'Jean_570', 'M', 'BA', 'Feira de Santana'), (571, 'Jean_571', 'M', 'SC', 'Florianópolis'), (572, 'Patricia_572', 'F', 'PR', 'Londrina'), (573, 'Rafael_573', 'F', 'SP', 'São Paulo'), (574, 'Jean_574', 'F', 'RS', 'Caxias do Sul'), (575, 'Rafael_575', 'F', 'PE', 'Recife'), (576, 'Marcos_576', 'F', 'BA', 'Salvador'), (577, 'Gabriela_577', 'F', 'MG', 'Uberlândia'), (578, 'Patricia_578', 'F', 'RS', 'Porto Alegre'), (579, 'Jean_579', 'M', 'PE', 'Recife'), (580, 'Fernanda_580', 'M', 'PE', 'Recife'), (581, 'Carlos_581', 'M', 'PE', 'Olinda'), (582, 'Joao_582', 'M', 'SP', 'Campinas'), (583, 'Joao_583', 'M', 'SP', 'Campinas'), (584, 'Carlos_584', 'F', 'RJ', 'Niteroi'), (585, 'Luiza_585', 'F', 'RS', 'Porto Alegre'), (586, 'Gabriela_586', 'F', 'RS', 'Porto Alegre'), (587, 'Gabriela_587', 'F', 'PE', 'Olinda'), (588, 'Fernanda_588', 'F', 'PE', 'Olinda'), (589, 'Patricia_589', 'M', 'SP', 'São Paulo'), (590, 'Fernanda_590', 'M', 'SC', 'Florianópolis'), (591, 'Carlos_591', 'M', 'SC', 'Joinville'), (592, 'Jean_592', 'F', 'SC', 'Joinville'), (593, 'Gabriela_593', 'M', 'SC', 'Joinville'), (594, 'Gabriela_594', 'M', 'SC', 'Florianópolis'), (595, 'Luiza_595', 'M', 'RS', 'Caxias do Sul'), (596, 'Joao_596', 'M', 'RS', 'Porto Alegre'), (597, 'Marcos_597', 'M', 'MG', 'Uberlândia'), (598, 'Rafael_598', 'M', 'RJ', 'Cabo Frio'), (599, 'Luiza_599', 'M', 'PE', 'Recife'), (600, 'Fernanda_600', 'F', 'PE', 'Olinda'), (601, 'Carlos_601', 'M', 'PE', 'Olinda'), (602, 'Luiza_602', 'F', 'SC', 'Florianópolis'), (603, 'Carlos_603', 'M', 'SC', 'Joinville'), (604, 'Gabriela_604', 'F', 'RJ', 'Cabo Frio'), (605, 'Fernanda_605', 'M', 'MG', 'Belo Horizonte'), (606, 'Fernanda_606', 'F', 'PR', 'Londrina'), (607, 'Carlos_607', 'F', 'SC', 'Florianópolis'), (608, 'Carlos_608', 'F', 'MG', 'Betim'), (609, 'Carlos_609', 'M', 'RS', 'Porto Alegre'), (610, 'Luiza_610', 'F', 'PE', 'Olinda'), (611, 'Carlos_611', 'F', 'SP', 'São Paulo'), (612, 'Luiza_612', 'F', 'MG', 'Uberlândia'), (613, 'Carlos_613', 'M', 'RS', 'Porto Alegre'), (614, 'Marcos_614', 'F', 'RS', 'Porto Alegre'), (615, 'Gabriela_615', 'F', 'RJ', 'Niteroi'), (616, 'Luiza_616', 'F', 'BA', 'Salvador'), (617, 'Gabriela_617', 'M', 'SP', 'Campinas'), (618, 'Joao_618', 'F', 'SP', 'São Paulo'), (619, 'Luiza_619', 'F', 'MG', 'Betim'), (620, 'Luiza_620', 'F', 'RS', 'Caxias do Sul'), (621, 'Joao_621', 'M', 'PR', 'Londrina'), (622, 'Luiza_622', 'F', 'BA', 'Feira de Santana'), (623, 'Gabriela_623', 'M', 'PE', 'Olinda'), (624, 'Patricia_624', 'F', 'SC', 'Florianópolis'), (625, 'Daniela_625', 'M', 'MG', 'Betim'), (626, 'Patricia_626', 'M', 'SC', 'Joinville'), (627, 'Jean_627', 'F', 'SP', 'São Paulo'), (628, 'Fernanda_628', 'F', 'RJ', 'Niteroi'), (629, 'Jean_629', 'F', 'RS', 'Caxias do Sul'), (630, 'Luiza_630', 'F', 'SC', 'Florianópolis'), (631, 'Gabriela_631', 'M', 'PE', 'Recife'), (632, 'Luiza_632', 'F', 'BA', 'Salvador'), (633, 'Joao_633', 'M', 'PE', 'Olinda'), (634, 'Fernanda_634', 'M', 'SP', 'São Paulo'), (635, 'Fernanda_635', 'M', 'BA', 'Salvador'), (636, 'Carlos_636', 'M', 'RJ', 'Niteroi'), (637, 'Fernanda_637', 'F', 'SP', 'Campinas'), (638, 'Jean_638', 'M', 'SP', 'Campinas'), (639, 'Patricia_639', 'F', 'MG', 'Belo Horizonte'), (640, 'Joao_640', 'F', 'SC', 'Joinville'), (641, 'Joao_641', 'F', 'PR', 'Londrina'), (642, 'Daniela_642', 'F', 'PE', 'Olinda'), (643, 'Luiza_643', 'F', 'MG', 'Belo Horizonte'), (644, 'Luiza_644', 'F', 'PE', 'Recife'), (645, 'Luiza_645', 'M', 'PE', 'Recife'), (646, 'Joao_646', 'F', 'BA', 'Salvador'), (647, 'Rafael_647', 'F', 'SP', 'São Caetano'), (648, 'Jean_648', 'F', 'RJ', 'Cabo Frio'), (649, 'Marcos_649', 'M', 'SC', 'Joinville'), (650, 'Jean_650', 'M', 'RS', 'Porto Alegre'), (651, 'Joao_651', 'M', 'MG', 'Betim'), (652, 'Joao_652', 'M', 'RJ', 'Cabo Frio'), (653, 'Luiza_653', 'M', 'PR', 'Curitiba'), (654, 'Patricia_654', 'M', 'SC', 'Florianópolis'), (655, 'Jean_655', 'M', 'PE', 'Olinda'), (656, 'Joao_656', 'F', 'PE', 'Recife'), (657, 'Fernanda_657', 'M', 'RS', 'Porto Alegre'), (658, 'Carlos_658', 'M', 'RJ', 'Cabo Frio'), (659, 'Fernanda_659', 'F', 'PE', 'Recife'), (660, 'Rafael_660', 'M', 'SP', 'São Paulo'), (661, 'Marcos_661', 'M', 'PR', 'Londrina'), (662, 'Patricia_662', 'M', 'SP', 'São Paulo'), (663, 'Daniela_663', 'M', 'SP', 'São Caetano'), (664, 'Patricia_664', 'F', 'MG', 'Belo Horizonte'), (665, 'Luiza_665', 'M', 'BA', 'Feira de Santana'), (666, 'Rafael_666', 'F', 'MG', 'Belo Horizonte'), (667, 'Patricia_667', 'M', 'PE', 'Recife'), (668, 'Patricia_668', 'F', 'MG', 'Uberlândia'), (669, 'Patricia_669', 'F', 'RS', 'Porto Alegre'), (670, 'Joao_670', 'M', 'PR', 'Curitiba'), (671, 'Gabriela_671', 'F', 'PE', 'Recife'), (672, 'Marcos_672', 'M', 'MG', 'Betim'), (673, 'Luiza_673', 'F', 'RJ', 'Rio de Janeiro'), (674, 'Jean_674', 'F', 'BA', 'Salvador'), (675, 'Joao_675', 'M', 'PR', 'Londrina'), (676, 'Patricia_676', 'F', 'BA', 'Salvador'), (677, 'Jean_677', 'F', 'SC', 'Joinville'), (678, 'Marcos_678', 'F', 'RS', 'Caxias do Sul'), (679, 'Fernanda_679', 'F', 'MG', 'Betim'), (680, 'Gabriela_680', 'F', 'BA', 'Feira de Santana'), (681, 'Gabriela_681', 'F', 'RS', 'Porto Alegre'), (682, 'Fernanda_682', 'M', 'SP', 'São Paulo'), (683, 'Joao_683', 'F', 'RS', 'Caxias do Sul'), (684, 'Daniela_684', 'M', 'SC', 'Florianópolis'), (685, 'Marcos_685', 'F', 'SP', 'Campinas'), (686, 'Fernanda_686', 'F', 'SP', 'Campinas'), (687, 'Daniela_687', 'F', 'PR', 'Londrina'), (688, 'Fernanda_688', 'M', 'RJ', 'Niteroi'), (689, 'Gabriela_689', 'F', 'SC', 'Joinville'), (690, 'Patricia_690', 'M', 'MG', 'Belo Horizonte'), (691, 'Patricia_691', 'F', 'PE', 'Olinda'), (692, 'Joao_692', 'F', 'BA', 'Feira de Santana'), (693, 'Marcos_693', 'F', 'RS', 'Porto Alegre'), (694, 'Jean_694', 'M', 'SP', 'São Paulo'), (695, 'Luiza_695', 'F', 'SC', 'Joinville'), (696, 'Rafael_696', 'M', 'RS', 'Porto Alegre'), (697, 'Joao_697', 'M', 'RJ', 'Cabo Frio'), (698, 'Carlos_698', 'M', 'PE', 'Recife'), (699, 'Rafael_699', 'F', 'PE', 'Olinda'), (700, 'Gabriela_700', 'F', 'RS', 'Porto Alegre'), (701, 'Daniela_701', 'F', 'SC', 'Joinville'), (702, 'Gabriela_702', 'M', 'SP', 'Campinas'), (703, 'Daniela_703', 'F', 'SP', 'Campinas'), (704, 'Daniela_704', 'F', 'PR', 'Curitiba'), (705, 'Jean_705', 'M', 'SP', 'São Paulo'), (706, 'Jean_706', 'F', 'SC', 'Joinville'), (707, 'Patricia_707', 'M', 'SC', 'Florianópolis'), (708, 'Rafael_708', 'M', 'RJ', 'Niteroi'), (709, 'Rafael_709', 'F', 'PE', 'Olinda'), (710, 'Rafael_710', 'F', 'SP', 'São Paulo'), (711, 'Carlos_711', 'F', 'BA', 'Feira de Santana'), (712, 'Rafael_712', 'F', 'RJ', 'Rio de Janeiro'), (713, 'Luiza_713', 'M', 'PE', 'Recife'), (714, 'Carlos_714', 'F', 'RS', 'Caxias do Sul'), (715, 'Jean_715', 'M', 'RJ', 'Cabo Frio'), (716, 'Marcos_716', 'F', 'SC', 'Joinville'), (717, 'Marcos_717', 'F', 'PE', 'Olinda'), (718, 'Carlos_718', 'M', 'PR', 'Londrina'), (719, 'Joao_719', 'F', 'PE', 'Olinda'), (720, 'Gabriela_720', 'M', 'SP', 'São Caetano'), (721, 'Daniela_721', 'M', 'RS', 'Porto Alegre'), (722, 'Jean_722', 'M', 'RJ', 'Niteroi'), (723, 'Daniela_723', 'F', 'SP', 'Campinas'), (724, 'Joao_724', 'M', 'RS', 'Caxias do Sul'), (725, 'Patricia_725', 'F', 'SC', 'Florianópolis'), (726, 'Fernanda_726', 'M', 'MG', 'Uberlândia'), (727, 'Marcos_727', 'M', 'BA', 'Salvador'), (728, 'Joao_728', 'M', 'SC', 'Joinville'), (729, 'Patricia_729', 'F', 'SC', 'Joinville'), (730, 'Joao_730', 'M', 'RJ', 'Niteroi'), (731, 'Joao_731', 'M', 'PR', 'Curitiba'), (732, 'Marcos_732', 'F', 'MG', 'Betim'), (733, 'Daniela_733', 'F', 'SC', 'Joinville'), (734, 'Rafael_734', 'M', 'PR', 'Londrina'), (735, 'Jean_735', 'F', 'RJ', 'Rio de Janeiro'), (736, 'Carlos_736', 'F', 'MG', 'Betim'), (737, 'Daniela_737', 'F', 'SP', 'São Paulo'), (738, 'Jean_738', 'M', 'PE', 'Olinda'), (739, 'Joao_739', 'F', 'BA', 'Feira de Santana'), (740, 'Joao_740', 'M', 'RS', 'Porto Alegre'), (741, 'Gabriela_741', 'F', 'SC', 'Joinville'), (742, 'Gabriela_742', 'M', 'SC', 'Florianópolis'), (743, 'Luiza_743', 'F', 'SP', 'São Paulo'), (744, 'Carlos_744', 'F', 'RJ', 'Niteroi'), (745, 'Patricia_745', 'F', 'SP', 'São Paulo'), (746, 'Patricia_746', 'F', 'RS', 'Caxias do Sul'), (747, 'Carlos_747', 'F', 'SC', 'Florianópolis'), (748, 'Jean_748', 'F', 'RS', 'Caxias do Sul'), (749, 'Patricia_749', 'M', 'SP', 'São Caetano'), (750, 'Patricia_750', 'F', 'RS', 'Caxias do Sul'), (751, 'Gabriela_751', 'F', 'BA', 'Salvador'), (752, 'Carlos_752', 'F', 'PR', 'Curitiba'), (753, 'Marcos_753', 'M', 'RJ', 'Niteroi'), (754, 'Fernanda_754', 'F', 'PR', 'Londrina'), (755, 'Jean_755', 'M', 'MG', 'Betim'), (756, 'Fernanda_756', 'F', 'SP', 'São Paulo'), (757, 'Luiza_757', 'F', 'PE', 'Recife'), (758, 'Jean_758', 'M', 'PR', 'Londrina'), (759, 'Gabriela_759', 'M', 'RJ', 'Cabo Frio'), (760, 'Gabriela_760', 'M', 'BA', 'Feira de Santana'), (761, 'Gabriela_761', 'F', 'RJ', 'Niteroi'), (762, 'Rafael_762', 'F', 'PE', 'Olinda'), (763, 'Marcos_763', 'M', 'MG', 'Betim'), (764, 'Joao_764', 'F', 'BA', 'Salvador'), (765, 'Patricia_765', 'F', 'PE', 'Recife'), (766, 'Patricia_766', 'M', 'PE', 'Olinda'), (767, 'Gabriela_767', 'F', 'PE', 'Olinda'), (768, 'Joao_768', 'F', 'PE', 'Olinda'), (769, 'Rafael_769', 'F', 'BA', 'Feira de Santana'), (770, 'Carlos_770', 'F', 'PR', 'Londrina'), (771, 'Carlos_771', 'M', 'SC', 'Joinville'), (772, 'Jean_772', 'F', 'RS', 'Porto Alegre'), (773, 'Carlos_773', 'F', 'PR', 'Londrina'), (774, 'Daniela_774', 'F', 'BA', 'Salvador'), (775, 'Marcos_775', 'F', 'MG', 'Belo Horizonte'), (776, 'Jean_776', 'F', 'MG', 'Belo Horizonte'), (777, 'Fernanda_777', 'F', 'RS', 'Porto Alegre'), (778, 'Daniela_778', 'M', 'RJ', 'Cabo Frio'), (779, 'Patricia_779', 'F', 'MG', 'Belo Horizonte'), (780, 'Daniela_780', 'M', 'PR', 'Curitiba'), (781, 'Marcos_781', 'F', 'SP', 'São Caetano'), (782, 'Carlos_782', 'M', 'PE', 'Recife'), (783, 'Luiza_783', 'F', 'RS', 'Caxias do Sul'), (784, 'Carlos_784', 'F', 'PR', 'Curitiba'), (785, 'Gabriela_785', 'M', 'PE', 'Recife'), (786, 'Carlos_786', 'F', 'BA', 'Feira de Santana'), (787, 'Luiza_787', 'F', 'SP', 'São Paulo'), (788, 'Gabriela_788', 'F', 'RS', 'Porto Alegre'), (789, 'Carlos_789', 'F', 'SP', 'São Paulo'), (790, 'Daniela_790', 'F', 'RS', 'Porto Alegre'), (791, 'Carlos_791', 'F', 'MG', 'Uberlândia'), (792, 'Daniela_792', 'F', 'PE', 'Recife'), (793, 'Jean_793', 'M', 'RS', 'Caxias do Sul'), (794, 'Gabriela_794', 'M', 'RS', 'Porto Alegre'), (795, 'Patricia_795', 'M', 'RJ', 'Cabo Frio'), (796, 'Carlos_796', 'F', 'PE', 'Olinda'), (797, 'Jean_797', 'M', 'SP', 'São Paulo'), (798, 'Carlos_798', 'M', 'PR', 'Londrina'), (799, 'Marcos_799', 'M', 'PE', 'Recife'), (800, 'Gabriela_800', 'F', 'BA', 'Salvador'), (801, 'Joao_801', 'M', 'BA', 'Feira de Santana'), (802, 'Jean_802', 'M', 'PE', 'Olinda'), (803, 'Fernanda_803', 'F', 'MG', 'Betim'), (804, 'Fernanda_804', 'M', 'RS', 'Caxias do Sul'), (805, 'Jean_805', 'M', 'SP', 'Campinas'), (806, 'Jean_806', 'F', 'RJ', 'Niteroi'), (807, 'Jean_807', 'M', 'RS', 'Caxias do Sul'), (808, 'Jean_808', 'M', 'PE', 'Recife'), (809, 'Rafael_809', 'F', 'RJ', 'Cabo Frio'), (810, 'Gabriela_810', 'F', 'PR', 'Curitiba'), (811, 'Joao_811', 'F', 'MG', 'Betim'), (812, 'Jean_812', 'M', 'PE', 'Olinda'), (813, 'Patricia_813', 'F', 'MG', 'Uberlândia'), (814, 'Carlos_814', 'F', 'SC', 'Joinville'), (815, 'Luiza_815', 'M', 'BA', 'Feira de Santana'), (816, 'Daniela_816', 'M', 'RJ', 'Rio de Janeiro'), (817, 'Luiza_817', 'M', 'RS', 'Porto Alegre'), (818, 'Jean_818', 'F', 'SP', 'São Caetano'), (819, 'Luiza_819', 'F', 'PR', 'Londrina'), (820, 'Daniela_820', 'F', 'PR', 'Londrina'), (821, 'Marcos_821', 'F', 'SC', 'Joinville'), (822, 'Gabriela_822', 'F', 'MG', 'Uberlândia'), (823, 'Marcos_823', 'M', 'RS', 'Porto Alegre'), (824, 'Patricia_824', 'M', 'MG', 'Betim'), (825, 'Carlos_825', 'M', 'RJ', 'Rio de Janeiro'), (826, 'Jean_826', 'M', 'SC', 'Florianópolis'), (827, 'Fernanda_827', 'M', 'RJ', 'Cabo Frio'), (828, 'Marcos_828', 'F', 'PR', 'Curitiba'), (829, 'Gabriela_829', 'M', 'SP', 'São Paulo'), (830, 'Jean_830', 'M', 'RS', 'Caxias do Sul'), (831, 'Daniela_831', 'F', 'MG', 'Betim'), (832, 'Marcos_832', 'M', 'RJ', 'Cabo Frio'), (833, 'Joao_833', 'M', 'SC', 'Florianópolis'), (834, 'Fernanda_834', 'F', 'RJ', 'Niteroi'), (835, 'Gabriela_835', 'M', 'PE', 'Olinda'), (836, 'Daniela_836', 'F', 'RS', 'Caxias do Sul'), (837, 'Carlos_837', 'F', 'BA', 'Salvador'), (838, 'Rafael_838', 'F', 'RJ', 'Niteroi'), (839, 'Daniela_839', 'M', 'SP', 'São Paulo'), (840, 'Patricia_840', 'F', 'PE', 'Olinda'), (841, 'Gabriela_841', 'F', 'SP', 'Campinas'), (842, 'Carlos_842', 'F', 'SC', 'Joinville'), (843, 'Carlos_843', 'M', 'SP', 'Campinas'), (844, 'Rafael_844', 'F', 'MG', 'Betim'), (845, 'Joao_845', 'M', 'MG', 'Uberlândia'), (846, 'Carlos_846', 'F', 'RS', 'Caxias do Sul'), (847, 'Marcos_847', 'M', 'MG', 'Belo Horizonte'), (848, 'Luiza_848', 'M', 'PE', 'Olinda'), (849, 'Carlos_849', 'M', 'PR', 'Curitiba'), (850, 'Gabriela_850', 'M', 'SP', 'São Paulo'), (851, 'Gabriela_851', 'F', 'PR', 'Curitiba'), (852, 'Jean_852', 'F', 'RJ', 'Rio de Janeiro'), (853, 'Fernanda_853', 'M', 'SP', 'São Caetano'), (854, 'Patricia_854', 'M', 'MG', 'Uberlândia'), (855, 'Joao_855', 'M', 'RS', 'Caxias do Sul'), (856, 'Gabriela_856', 'M', 'PR', 'Curitiba'), (857, 'Patricia_857', 'F', 'SC', 'Joinville'), (858, 'Fernanda_858', 'F', 'RJ', 'Rio de Janeiro'), (859, 'Jean_859', 'F', 'RS', 'Porto Alegre'), (860, 'Marcos_860', 'F', 'RS', 'Porto Alegre'), (861, 'Gabriela_861', 'F', 'SP', 'Campinas'), (862, 'Fernanda_862', 'M', 'BA', 'Feira de Santana'), (863, 'Jean_863', 'M', 'RJ', 'Niteroi'), (864, 'Jean_864', 'F', 'BA', 'Salvador'), (865, 'Daniela_865', 'F', 'RJ', 'Niteroi'), (866, 'Daniela_866', 'F', 'RS', 'Porto Alegre'), (867, 'Rafael_867', 'M', 'MG', 'Uberlândia'), (868, 'Jean_868', 'F', 'MG', 'Uberlândia'), (869, 'Luiza_869', 'F', 'RJ', 'Niteroi'), (870, 'Jean_870', 'F', 'SP', 'Campinas'), (871, 'Rafael_871', 'F', 'RS', 'Caxias do Sul'), (872, 'Marcos_872', 'F', 'PR', 'Curitiba'), (873, 'Daniela_873', 'M', 'BA', 'Feira de Santana'), (874, 'Rafael_874', 'M', 'MG', 'Betim'), (875, 'Patricia_875', 'F', 'SP', 'Campinas'), (876, 'Jean_876', 'F', 'PE', 'Recife'), (877, 'Marcos_877', 'F', 'PR', 'Curitiba'), (878, 'Gabriela_878', 'F', 'RS', 'Caxias do Sul'), (879, 'Marcos_879', 'M', 'RS', 'Porto Alegre'), (880, 'Marcos_880', 'M', 'SP', 'Campinas'), (881, 'Carlos_881', 'M', 'RS', 'Caxias do Sul'), (882, 'Fernanda_882', 'M', 'SP', 'Campinas'), (883, 'Luiza_883', 'M', 'RJ', 'Cabo Frio'), (884, 'Carlos_884', 'M', 'BA', 'Feira de Santana'), (885, 'Fernanda_885', 'F', 'SP', 'São Caetano'), (886, 'Patricia_886', 'F', 'MG', 'Betim'), (887, 'Joao_887', 'F', 'SP', 'Campinas'), (888, 'Gabriela_888', 'M', 'PR', 'Londrina'), (889, 'Patricia_889', 'F', 'PR', 'Londrina'), (890, 'Carlos_890', 'M', 'RS', 'Porto Alegre'), (891, 'Joao_891', 'M', 'SP', 'Campinas'), (892, 'Joao_892', 'M', 'SP', 'Campinas'), (893, 'Fernanda_893', 'F', 'PE', 'Recife'), (894, 'Marcos_894', 'M', 'RJ', 'Rio de Janeiro'), (895, 'Fernanda_895', 'M', 'RJ', 'Niteroi'), (896, 'Jean_896', 'F', 'RS', 'Caxias do Sul'), (897, 'Marcos_897', 'M', 'BA', 'Feira de Santana'), (898, 'Carlos_898', 'F', 'BA', 'Feira de Santana'), (899, 'Fernanda_899', 'M', 'SP', 'Campinas'), (900, 'Jean_900', 'F', 'RS', 'Porto Alegre'), (901, 'Carlos_901', 'M', 'BA', 'Salvador'), (902, 'Gabriela_902', 'M', 'SC', 'Florianópolis'), (903, 'Marcos_903', 'F', 'PR', 'Curitiba'), (904, 'Rafael_904', 'M', 'PE', 'Recife'), (905, 'Patricia_905', 'F', 'PE', 'Recife'), (906, 'Daniela_906', 'M', 'PE', 'Olinda'), (907, 'Carlos_907', 'M', 'SC', 'Florianópolis'), (908, 'Luiza_908', 'F', 'MG', 'Betim'), (909, 'Luiza_909', 'F', 'BA', 'Feira de Santana'), (910, 'Gabriela_910', 'M', 'MG', 'Betim'), (911, 'Luiza_911', 'M', 'SP', 'Campinas'), (912, 'Luiza_912', 'F', 'RS', 'Porto Alegre'), (913, 'Gabriela_913', 'M', 'PE', 'Recife'), (914, 'Jean_914', 'F', 'PE', 'Olinda'), (915, 'Luiza_915', 'F', 'SC', 'Joinville'), (916, 'Rafael_916', 'M', 'MG', 'Belo Horizonte'), (917, 'Joao_917', 'F', 'PE', 'Olinda'), (918, 'Marcos_918', 'M', 'MG', 'Belo Horizonte'), (919, 'Luiza_919', 'F', 'PE', 'Olinda'), (920, 'Rafael_920', 'M', 'MG', 'Betim'), (921, 'Fernanda_921', 'M', 'PE', 'Recife'), (922, 'Jean_922', 'M', 'PR', 'Curitiba'), (923, 'Marcos_923', 'F', 'SC', 'Joinville'), (924, 'Carlos_924', 'F', 'SC', 'Florianópolis'), (925, 'Rafael_925', 'F', 'PR', 'Curitiba'), (926, 'Joao_926', 'M', 'SC', 'Joinville'), (927, 'Gabriela_927', 'F', 'RS', 'Porto Alegre'), (928, 'Patricia_928', 'M', 'BA', 'Salvador'), (929, 'Carlos_929', 'F', 'PR', 'Londrina'), (930, 'Daniela_930', 'F', 'PR', 'Londrina'), (931, 'Daniela_931', 'M', 'MG', 'Belo Horizonte'), (932, 'Gabriela_932', 'M', 'RJ', 'Niteroi'), (933, 'Rafael_933', 'M', 'SC', 'Florianópolis'), (934, 'Marcos_934', 'F', 'PR', 'Londrina'), (935, 'Patricia_935', 'M', 'PE', 'Olinda'), (936, 'Fernanda_936', 'M', 'SC', 'Florianópolis'), (937, 'Luiza_937', 'M', 'SC', 'Joinville'), (938, 'Joao_938', 'M', 'RJ', 'Cabo Frio'), (939, 'Luiza_939', 'M', 'RJ', 'Rio de Janeiro'), (940, 'Marcos_940', 'M', 'PE', 'Olinda'), (941, 'Luiza_941', 'M', 'RJ', 'Rio de Janeiro'), (942, 'Fernanda_942', 'M', 'PR', 'Londrina'), (943, 'Daniela_943', 'F', 'MG', 'Betim'), (944, 'Carlos_944', 'F', 'MG', 'Belo Horizonte'), (945, 'Joao_945', 'F', 'RS', 'Porto Alegre'), (946, 'Rafael_946', 'M', 'BA', 'Feira de Santana'), (947, 'Carlos_947', 'M', 'RS', 'Caxias do Sul'), (948, 'Patricia_948', 'M', 'MG', 'Betim'), (949, 'Rafael_949', 'F', 'RS', 'Porto Alegre'), (950, 'Fernanda_950', 'M', 'SP', 'São Caetano'), (951, 'Rafael_951', 'F', 'SP', 'Campinas'), (952, 'Jean_952', 'F', 'PR', 'Curitiba'), (953, 'Fernanda_953', 'M', 'MG', 'Belo Horizonte'), (954, 'Jean_954', 'M', 'SC', 'Florianópolis'), (955, 'Fernanda_955', 'M', 'PE', 'Recife'), (956, 'Jean_956', 'F', 'BA', 'Salvador'), (957, 'Jean_957', 'F', 'PR', 'Curitiba'), (958, 'Carlos_958', 'F', 'MG', 'Betim'), (959, 'Fernanda_959', 'M', 'PR', 'Londrina'), (960, 'Luiza_960', 'M', 'RJ', 'Rio de Janeiro'), (961, 'Joao_961', 'F', 'MG', 'Uberlândia'), (962, 'Marcos_962', 'M', 'MG', 'Betim'), (963, 'Daniela_963', 'F', 'PE', 'Recife'), (964, 'Rafael_964', 'M', 'RJ', 'Niteroi'), (965, 'Carlos_965', 'M', 'RJ', 'Niteroi'), (966, 'Gabriela_966', 'F', 'SP', 'São Caetano'), (967, 'Rafael_967', 'M', 'PR', 'Londrina'), (968, 'Marcos_968', 'M', 'PR', 'Curitiba'), (969, 'Luiza_969', 'M', 'PE', 'Recife'), (970, 'Joao_970', 'M', 'SP', 'São Caetano'), (971, 'Gabriela_971', 'F', 'RS', 'Caxias do Sul'), (972, 'Luiza_972', 'M', 'RJ', 'Niteroi'), (973, 'Luiza_973', 'F', 'BA', 'Salvador'), (974, 'Joao_974', 'M', 'PE', 'Recife'), (975, 'Jean_975', 'M', 'MG', 'Betim'), (976, 'Gabriela_976', 'F', 'BA', 'Salvador'), (977, 'Rafael_977', 'M', 'BA', 'Feira de Santana'), (978, 'Carlos_978', 'M', 'RS', 'Caxias do Sul'), (979, 'Jean_979', 'F', 'BA', 'Feira de Santana'), (980, 'Patricia_980', 'M', 'RS', 'Porto Alegre'), (981, 'Patricia_981', 'F', 'SP', 'São Caetano'), (982, 'Fernanda_982', 'M', 'MG', 'Uberlândia'), (983, 'Marcos_983', 'F', 'RS', 'Caxias do Sul'), (984, 'Joao_984', 'M', 'RS', 'Caxias do Sul'), (985, 'Carlos_985', 'F', 'RS', 'Porto Alegre'), (986, 'Jean_986', 'M', 'BA', 'Salvador'), (987, 'Gabriela_987', 'M', 'PR', 'Curitiba'), (988, 'Marcos_988', 'F', 'MG', 'Betim'), (989, 'Rafael_989', 'M', 'PR', 'Londrina'), (990, 'Carlos_990', 'M', 'SC', 'Florianópolis'), (991, 'Marcos_991', 'M', 'SC', 'Florianópolis'), (992, 'Joao_992', 'M', 'RS', 'Porto Alegre'), (993, 'Jean_993', 'F', 'SP', 'Campinas'), (994, 'Rafael_994', 'F', 'RJ', 'Niteroi'), (995, 'Daniela_995', 'F', 'PR', 'Curitiba'), (996, 'Marcos_996', 'M', 'SC', 'Joinville'), (997, 'Carlos_997', 'F', 'SC', 'Florianópolis'), (998, 'Rafael_998', 'F', 'RS', 'Porto Alegre'), (999, 'Patricia_999', 'F', 'SC', 'Florianópolis'), (1000, 'Daniela_1000', 'M', 'MG', 'Belo Horizonte')]))
print("Total contas:", len({1: (190001, 1587686400), 2: (190002, 1590105600), 3: (190003, 1598140800), 4: (190004, 1594166400), 5: (190005, 1589760000), 6: (190006, 1606262400), 7: (190007, 1583884800), 8: (190008, 1604188800), 9: (190009, 1584748800), 10: (190010, 1582761600), 11: (190011, 1583366400), 12: (190012, 1588723200), 13: (190013, 1605744000), 14: (190014, 1601683200), 15: (190015, 1582588800), 16: (190016, 1595548800), 17: (190017, 1586217600), 18: (190018, 1602979200), 19: (190019, 1578182400), 20: (190020, 1589500800), 21: (190021, 1596499200), 22: (190022, 1579392000), 23: (190023, 1605225600), 24: (190024, 1582588800), 25: (190025, 1590105600), 26: (190026, 1589068800), 27: (190027, 1595635200), 28: (190028, 1597363200), 29: (190029, 1586563200), 30: (190030, 1588550400), 31: (190031, 1602028800), 32: (190032, 1583884800), 33: (190033, 1595030400), 34: (190034, 1589846400), 35: (190035, 1598227200), 36: (190036, 1606521600), 37: (190037, 1594080000), 38: (190038, 1591142400), 39: (190039, 1600905600), 40: (190040, 1592179200), 41: (190041, 1602374400), 42: (190042, 1589760000), 43: (190043, 1599177600), 44: (190044, 1602288000), 45: (190045, 1585958400), 46: (190046, 1602720000), 47: (190047, 1597363200), 48: (190048, 1588032000), 49: (190049, 1601856000), 50: (190050, 1608768000), 51: (190051, 1598572800), 52: (190052, 1588982400), 53: (190053, 1608681600), 54: (190054, 1598659200), 55: (190055, 1584403200), 56: (190056, 1591833600), 57: (190057, 1597968000), 58: (190058, 1605484800), 59: (190059, 1591056000), 60: (190060, 1585353600), 61: (190061, 1608336000), 62: (190062, 1606953600), 63: (190063, 1597276800), 64: (190064, 1601683200), 65: (190065, 1595289600), 66: (190066, 1598313600), 67: (190067, 1594166400), 68: (190068, 1597449600), 69: (190069, 1586217600), 70: (190070, 1586995200), 71: (190071, 1607040000), 72: (190072, 1593820800), 73: (190073, 1583366400), 74: (190074, 1595462400), 75: (190075, 1595980800), 76: (190076, 1603152000), 77: (190077, 1596326400), 78: (190078, 1599264000), 79: (190079, 1587513600), 80: (190080, 1585526400), 81: (190081, 1585353600), 82: (190082, 1593302400), 83: (190083, 1607904000), 84: (190084, 1606348800), 85: (190085, 1609113600), 86: (190086, 1602460800), 87: (190087, 1607472000), 88: (190088, 1583884800), 89: (190089, 1604188800), 90: (190090, 1597622400), 91: (190091, 1606176000), 92: (190092, 1594857600), 93: (190093, 1590451200), 94: (190094, 1596326400), 95: (190095, 1593734400), 96: (190096, 1593388800), 97: (190097, 1579392000), 98: (190098, 1597276800), 99: (190099, 1592265600), 100: (190100, 1584144000), 101: (190101, 1584576000), 102: (190102, 1589328000), 103: (190103, 1609286400), 104: (190104, 1598486400), 105: (190105, 1583452800), 106: (190106, 1591228800), 107: (190107, 1603324800), 108: (190108, 1604620800), 109: (190109, 1579996800), 110: (190110, 1591401600), 111: (190111, 1589500800), 112: (190112, 1594857600), 113: (190113, 1599955200), 114: (190114, 1595203200), 115: (190115, 1600992000), 116: (190116, 1596326400), 117: (190117, 1598140800), 118: (190118, 1607731200), 119: (190119, 1601596800), 120: (190120, 1580083200), 121: (190121, 1603152000), 122: (190122, 1606176000), 123: (190123, 1604793600), 124: (190124, 1578096000), 125: (190125, 1608163200), 126: (190126, 1607904000), 127: (190127, 1592265600), 128: (190128, 1579824000), 129: (190129, 1607040000), 130: (190130, 1601337600), 131: (190131, 1579305600), 132: (190132, 1590624000), 133: (190133, 1607817600), 134: (190134, 1587772800), 135: (190135, 1589068800), 136: (190136, 1592524800), 137: (190137, 1602201600), 138: (190138, 1578528000), 139: (190139, 1606176000), 140: (190140, 1583884800), 141: (190141, 1578096000), 142: (190142, 1581292800), 143: (190143, 1597017600), 144: (190144, 1578528000), 145: (190145, 1583971200), 146: (190146, 1605830400), 147: (190147, 1609372800), 148: (190148, 1597449600), 149: (190149, 1602288000), 150: (190150, 1594598400), 151: (190151, 1598054400), 152: (190152, 1596844800), 153: (190153, 1607990400), 154: (190154, 1604275200), 155: (190155, 1582502400), 156: (190156, 1607817600), 157: (190157, 1580774400), 158: (190158, 1584835200), 159: (190159, 1604016000), 160: (190160, 1582848000), 161: (190161, 1585872000), 162: (190162, 1605225600), 163: (190163, 1579219200), 164: (190164, 1601856000), 165: (190165, 1605052800), 166: (190166, 1590624000), 167: (190167, 1594425600), 168: (190168, 1595376000), 169: (190169, 1597276800), 170: (190170, 1607212800), 171: (190171, 1578182400), 172: (190172, 1594684800), 173: (190173, 1591747200), 174: (190174, 1591142400), 175: (190175, 1594944000), 176: (190176, 1581465600), 177: (190177, 1580342400), 178: (190178, 1609113600), 179: (190179, 1609286400), 180: (190180, 1594771200), 181: (190181, 1580860800), 182: (190182, 1601942400), 183: (190183, 1580428800), 184: (190184, 1581984000), 185: (190185, 1582502400), 186: (190186, 1587340800), 187: (190187, 1602374400), 188: (190188, 1586563200), 189: (190189, 1579132800), 190: (190190, 1580515200), 191: (190191, 1593043200), 192: (190192, 1582243200), 193: (190193, 1601769600), 194: (190194, 1592438400), 195: (190195, 1594339200), 196: (190196, 1594512000), 197: (190197, 1599782400), 198: (190198, 1587081600), 199: (190199, 1603324800), 200: (190200, 1581033600), 201: (190201, 1597017600), 202: (190202, 1584576000), 203: (190203, 1599436800), 204: (190204, 1589673600), 205: (190205, 1600905600), 206: (190206, 1579651200), 207: (190207, 1602201600), 208: (190208, 1584403200), 209: (190209, 1592438400), 210: (190210, 1597363200), 211: (190211, 1586304000), 212: (190212, 1600128000), 213: (190213, 1589241600), 214: (190214, 1585440000), 215: (190215, 1607126400), 216: (190216, 1586131200), 217: (190217, 1608163200), 218: (190218, 1587600000), 219: (190219, 1599955200), 220: (190220, 1605830400), 221: (190221, 1594771200), 222: (190222, 1594771200), 223: (190223, 1592956800), 224: (190224, 1609286400), 225: (190225, 1607040000), 226: (190226, 1590883200), 227: (190227, 1581811200), 228: (190228, 1596672000), 229: (190229, 1605312000), 230: (190230, 1580428800), 231: (190231, 1606780800), 232: (190232, 1606262400), 233: (190233, 1588204800), 234: (190234, 1592870400), 235: (190235, 1608854400), 236: (190236, 1605830400), 237: (190237, 1604707200), 238: (190238, 1587081600), 239: (190239, 1596499200), 240: (190240, 1582329600), 241: (190241, 1586390400), 242: (190242, 1596758400), 243: (190243, 1605484800), 244: (190244, 1584835200), 245: (190245, 1593129600), 246: (190246, 1578873600), 247: (190247, 1601596800), 248: (190248, 1598140800), 249: (190249, 1594944000), 250: (190250, 1608163200), 251: (190251, 1602374400), 252: (190252, 1593043200), 253: (190253, 1590624000), 254: (190254, 1586390400), 255: (190255, 1587600000), 256: (190256, 1604620800), 257: (190257, 1595116800), 258: (190258, 1602806400), 259: (190259, 1579651200), 260: (190260, 1593993600), 261: (190261, 1587945600), 262: (190262, 1599091200), 263: (190263, 1607040000), 264: (190264, 1587945600), 265: (190265, 1598832000), 266: (190266, 1593302400), 267: (190267, 1606780800), 268: (190268, 1602201600), 269: (190269, 1584921600), 270: (190270, 1594771200), 271: (190271, 1600387200), 272: (190272, 1608940800), 273: (190273, 1582934400), 274: (190274, 1596067200), 275: (190275, 1593475200), 276: (190276, 1584662400), 277: (190277, 1601078400), 278: (190278, 1589414400), 279: (190279, 1602374400), 280: (190280, 1591056000), 281: (190281, 1592611200), 282: (190282, 1594684800), 283: (190283, 1587859200), 284: (190284, 1588204800), 285: (190285, 1582416000), 286: (190286, 1583798400), 287: (190287, 1598572800), 288: (190288, 1606348800), 289: (190289, 1580860800), 290: (190290, 1578614400), 291: (190291, 1578441600), 292: (190292, 1588896000), 293: (190293, 1595030400), 294: (190294, 1586563200), 295: (190295, 1595289600), 296: (190296, 1588118400), 297: (190297, 1586217600), 298: (190298, 1605225600), 299: (190299, 1590278400), 300: (190300, 1586736000), 301: (190301, 1587600000), 302: (190302, 1581206400), 303: (190303, 1603324800), 304: (190304, 1604361600), 305: (190305, 1595376000), 306: (190306, 1584662400), 307: (190307, 1603324800), 308: (190308, 1591920000), 309: (190309, 1595203200), 310: (190310, 1599955200), 311: (190311, 1596067200), 312: (190312, 1579737600), 313: (190313, 1608508800), 314: (190314, 1585353600), 315: (190315, 1599264000), 316: (190316, 1605484800), 317: (190317, 1604534400), 318: (190318, 1606780800), 319: (190319, 1583625600), 320: (190320, 1585008000), 321: (190321, 1596672000), 322: (190322, 1600732800), 323: (190323, 1599955200), 324: (190324, 1609027200), 325: (190325, 1597536000), 326: (190326, 1599609600), 327: (190327, 1593216000), 328: (190328, 1579392000), 329: (190329, 1608595200), 330: (190330, 1585785600), 331: (190331, 1579824000), 332: (190332, 1600214400), 333: (190333, 1583798400), 334: (190334, 1592784000), 335: (190335, 1588291200), 336: (190336, 1599955200), 337: (190337, 1578009600), 338: (190338, 1590451200), 339: (190339, 1601769600), 340: (190340, 1583539200), 341: (190341, 1591574400), 342: (190342, 1585872000), 343: (190343, 1591142400), 344: (190344, 1602892800), 345: (190345, 1597449600), 346: (190346, 1581811200), 347: (190347, 1591056000), 348: (190348, 1594166400), 349: (190349, 1595721600), 350: (190350, 1578268800), 351: (190351, 1593475200), 352: (190352, 1601078400), 353: (190353, 1603152000), 354: (190354, 1590710400), 355: (190355, 1589155200), 356: (190356, 1594771200), 357: (190357, 1591142400), 358: (190358, 1604793600), 359: (190359, 1602028800), 360: (190360, 1588896000), 361: (190361, 1590796800), 362: (190362, 1580428800), 363: (190363, 1581033600), 364: (190364, 1584489600), 365: (190365, 1606435200), 366: (190366, 1607040000), 367: (190367, 1579392000), 368: (190368, 1606694400), 369: (190369, 1608854400), 370: (190370, 1582848000), 371: (190371, 1579824000), 372: (190372, 1603497600), 373: (190373, 1589760000), 374: (190374, 1578009600), 375: (190375, 1607472000), 376: (190376, 1600560000), 377: (190377, 1581033600), 378: (190378, 1602979200), 379: (190379, 1604534400), 380: (190380, 1578441600), 381: (190381, 1599004800), 382: (190382, 1578528000), 383: (190383, 1608336000), 384: (190384, 1593993600), 385: (190385, 1587772800), 386: (190386, 1598054400), 387: (190387, 1600819200), 388: (190388, 1598054400), 389: (190389, 1585180800), 390: (190390, 1597276800), 391: (190391, 1591142400), 392: (190392, 1599004800), 393: (190393, 1578787200), 394: (190394, 1602633600), 395: (190395, 1603929600), 396: (190396, 1603670400), 397: (190397, 1585699200), 398: (190398, 1589241600), 399: (190399, 1600560000), 400: (190400, 1595980800), 401: (190401, 1581206400), 402: (190402, 1584057600), 403: (190403, 1586476800), 404: (190404, 1590105600), 405: (190405, 1608681600), 406: (190406, 1582675200), 407: (190407, 1590624000), 408: (190408, 1608508800), 409: (190409, 1593561600), 410: (190410, 1605312000), 411: (190411, 1594425600), 412: (190412, 1591574400), 413: (190413, 1579046400), 414: (190414, 1607040000), 415: (190415, 1589587200), 416: (190416, 1591056000), 417: (190417, 1580774400), 418: (190418, 1607472000), 419: (190419, 1608076800), 420: (190420, 1582848000), 421: (190421, 1583798400), 422: (190422, 1582761600), 423: (190423, 1587427200), 424: (190424, 1601942400), 425: (190425, 1591488000), 426: (190426, 1598659200), 427: (190427, 1604448000), 428: (190428, 1586649600), 429: (190429, 1598918400), 430: (190430, 1608854400), 431: (190431, 1590537600), 432: (190432, 1588809600), 433: (190433, 1604188800), 434: (190434, 1602547200), 435: (190435, 1582934400), 436: (190436, 1599523200), 437: (190437, 1579564800), 438: (190438, 1603065600), 439: (190439, 1592092800), 440: (190440, 1606262400), 441: (190441, 1588377600), 442: (190442, 1591488000), 443: (190443, 1605657600), 444: (190444, 1608076800), 445: (190445, 1603670400), 446: (190446, 1587168000), 447: (190447, 1606780800), 448: (190448, 1587081600), 449: (190449, 1597708800), 450: (190450, 1588032000), 451: (190451, 1588032000), 452: (190452, 1586995200), 453: (190453, 1578614400), 454: (190454, 1587686400), 455: (190455, 1582502400), 456: (190456, 1606435200), 457: (190457, 1590451200), 458: (190458, 1595894400), 459: (190459, 1595548800), 460: (190460, 1601856000), 461: (190461, 1600387200), 462: (190462, 1600128000), 463: (190463, 1603497600), 464: (190464, 1582156800), 465: (190465, 1594166400), 466: (190466, 1598745600), 467: (190467, 1600905600), 468: (190468, 1599436800), 469: (190469, 1594512000), 470: (190470, 1609113600), 471: (190471, 1581552000), 472: (190472, 1585872000), 473: (190473, 1596067200), 474: (190474, 1582848000), 475: (190475, 1602892800), 476: (190476, 1589846400), 477: (190477, 1608681600), 478: (190478, 1578528000), 479: (190479, 1606521600), 480: (190480, 1578182400), 481: (190481, 1605484800), 482: (190482, 1602201600), 483: (190483, 1606176000), 484: (190484, 1586390400), 485: (190485, 1605484800), 486: (190486, 1605484800), 487: (190487, 1580256000), 488: (190488, 1605139200), 489: (190489, 1609372800), 490: (190490, 1591488000), 491: (190491, 1596499200), 492: (190492, 1596326400), 493: (190493, 1585958400), 494: (190494, 1605744000), 495: (190495, 1578787200), 496: (190496, 1597968000), 497: (190497, 1601337600), 498: (190498, 1598572800), 499: (190499, 1597795200), 500: (190500, 1603584000), 501: (190501, 1599955200), 502: (190502, 1607644800), 503: (190503, 1598659200), 504: (190504, 1591660800), 505: (190505, 1592784000), 506: (190506, 1595116800), 507: (190507, 1580860800), 508: (190508, 1595116800), 509: (190509, 1586736000), 510: (190510, 1598227200), 511: (190511, 1588377600), 512: (190512, 1582416000), 513: (190513, 1586390400), 514: (190514, 1590105600), 515: (190515, 1605916800), 516: (190516, 1587513600), 517: (190517, 1596758400), 518: (190518, 1587168000), 519: (190519, 1604361600), 520: (190520, 1600041600), 521: (190521, 1583712000), 522: (190522, 1597363200), 523: (190523, 1585440000), 524: (190524, 1593734400), 525: (190525, 1594598400), 526: (190526, 1595721600), 527: (190527, 1606608000), 528: (190528, 1589068800), 529: (190529, 1586131200), 530: (190530, 1589673600), 531: (190531, 1606780800), 532: (190532, 1586649600), 533: (190533, 1590019200), 534: (190534, 1592697600), 535: (190535, 1583971200), 536: (190536, 1592006400), 537: (190537, 1609286400), 538: (190538, 1597017600), 539: (190539, 1592956800), 540: (190540, 1594080000), 541: (190541, 1598140800), 542: (190542, 1589932800), 543: (190543, 1595116800), 544: (190544, 1609200000), 545: (190545, 1580256000), 546: (190546, 1583020800), 547: (190547, 1578268800), 548: (190548, 1608508800), 549: (190549, 1603670400), 550: (190550, 1586908800), 551: (190551, 1606867200), 552: (190552, 1599523200), 553: (190553, 1587340800), 554: (190554, 1608508800), 555: (190555, 1597017600), 556: (190556, 1606089600), 557: (190557, 1601164800), 558: (190558, 1594512000), 559: (190559, 1600819200), 560: (190560, 1590451200), 561: (190561, 1604620800), 562: (190562, 1582761600), 563: (190563, 1580342400), 564: (190564, 1600560000), 565: (190565, 1593475200), 566: (190566, 1586563200), 567: (190567, 1608076800), 568: (190568, 1608768000), 569: (190569, 1586304000), 570: (190570, 1590192000), 571: (190571, 1588982400), 572: (190572, 1587081600), 573: (190573, 1601942400), 574: (190574, 1605657600), 575: (190575, 1585094400), 576: (190576, 1590883200), 577: (190577, 1588809600), 578: (190578, 1605484800), 579: (190579, 1592179200), 580: (190580, 1585612800), 581: (190581, 1582243200), 582: (190582, 1592784000), 583: (190583, 1590883200), 584: (190584, 1606780800), 585: (190585, 1605052800), 586: (190586, 1607817600), 587: (190587, 1580774400), 588: (190588, 1604448000), 589: (190589, 1593820800), 590: (190590, 1587081600), 591: (190591, 1580169600), 592: (190592, 1597449600), 593: (190593, 1607904000), 594: (190594, 1587513600), 595: (190595, 1579046400), 596: (190596, 1591488000), 597: (190597, 1594684800), 598: (190598, 1584921600), 599: (190599, 1578441600), 600: (190600, 1597363200), 601: (190601, 1600473600), 602: (190602, 1602979200), 603: (190603, 1580688000), 604: (190604, 1587254400), 605: (190605, 1578787200), 606: (190606, 1585958400), 607: (190607, 1602633600), 608: (190608, 1583366400), 609: (190609, 1586217600), 610: (190610, 1595980800), 611: (190611, 1582416000), 612: (190612, 1590624000), 613: (190613, 1600214400), 614: (190614, 1603065600), 615: (190615, 1591833600), 616: (190616, 1592092800), 617: (190617, 1594684800), 618: (190618, 1605571200), 619: (190619, 1581033600), 620: (190620, 1604966400), 621: (190621, 1582848000), 622: (190622, 1599696000), 623: (190623, 1597536000), 624: (190624, 1582848000), 625: (190625, 1608249600), 626: (190626, 1591833600), 627: (190627, 1597795200), 628: (190628, 1579392000), 629: (190629, 1595462400), 630: (190630, 1585353600), 631: (190631, 1583366400), 632: (190632, 1593388800), 633: (190633, 1586131200), 634: (190634, 1587513600), 635: (190635, 1598832000), 636: (190636, 1600041600), 637: (190637, 1579996800), 638: (190638, 1582243200), 639: (190639, 1585958400), 640: (190640, 1578700800), 641: (190641, 1582675200), 642: (190642, 1601942400), 643: (190643, 1595980800), 644: (190644, 1589500800), 645: (190645, 1598400000), 646: (190646, 1596844800), 647: (190647, 1587513600), 648: (190648, 1597104000), 649: (190649, 1582070400), 650: (190650, 1605225600), 651: (190651, 1606521600), 652: (190652, 1599696000), 653: (190653, 1582416000), 654: (190654, 1605916800), 655: (190655, 1607299200), 656: (190656, 1602720000), 657: (190657, 1598486400), 658: (190658, 1584489600), 659: (190659, 1603238400), 660: (190660, 1594771200), 661: (190661, 1599523200), 662: (190662, 1579305600), 663: (190663, 1596672000), 664: (190664, 1590624000), 665: (190665, 1605139200), 666: (190666, 1599609600), 667: (190667, 1589760000), 668: (190668, 1599091200), 669: (190669, 1592956800), 670: (190670, 1581206400), 671: (190671, 1606867200), 672: (190672, 1595980800), 673: (190673, 1588204800), 674: (190674, 1580428800), 675: (190675, 1585958400), 676: (190676, 1579046400), 677: (190677, 1585008000), 678: (190678, 1579478400), 679: (190679, 1578182400), 680: (190680, 1599696000), 681: (190681, 1602288000), 682: (190682, 1586822400), 683: (190683, 1595808000), 684: (190684, 1606867200), 685: (190685, 1580083200), 686: (190686, 1588464000), 687: (190687, 1596326400), 688: (190688, 1578441600), 689: (190689, 1583452800), 690: (190690, 1593475200), 691: (190691, 1592006400), 692: (190692, 1594771200), 693: (190693, 1593129600), 694: (190694, 1602201600), 695: (190695, 1589414400), 696: (190696, 1600214400), 697: (190697, 1594425600), 698: (190698, 1609200000), 699: (190699, 1605657600), 700: (190700, 1593129600), 701: (190701, 1596585600), 702: (190702, 1604793600), 703: (190703, 1579996800), 704: (190704, 1578355200), 705: (190705, 1600128000), 706: (190706, 1596758400), 707: (190707, 1597536000), 708: (190708, 1605052800), 709: (190709, 1606608000), 710: (190710, 1596585600), 711: (190711, 1604966400), 712: (190712, 1600992000), 713: (190713, 1580774400), 714: (190714, 1589068800), 715: (190715, 1603756800), 716: (190716, 1592265600), 717: (190717, 1608076800), 718: (190718, 1594339200), 719: (190719, 1579737600), 720: (190720, 1583107200), 721: (190721, 1607212800), 722: (190722, 1579737600), 723: (190723, 1598313600), 724: (190724, 1581379200), 725: (190725, 1586131200), 726: (190726, 1594857600), 727: (190727, 1598486400), 728: (190728, 1592611200), 729: (190729, 1588550400), 730: (190730, 1604188800), 731: (190731, 1591142400), 732: (190732, 1584835200), 733: (190733, 1601424000), 734: (190734, 1597017600), 735: (190735, 1595376000), 736: (190736, 1606694400), 737: (190737, 1596585600), 738: (190738, 1579910400), 739: (190739, 1592697600), 740: (190740, 1601337600), 741: (190741, 1593043200), 742: (190742, 1592697600), 743: (190743, 1603065600), 744: (190744, 1599955200), 745: (190745, 1587772800), 746: (190746, 1598832000), 747: (190747, 1607040000), 748: (190748, 1607644800), 749: (190749, 1587427200), 750: (190750, 1605657600), 751: (190751, 1601337600), 752: (190752, 1605139200), 753: (190753, 1594080000), 754: (190754, 1588464000), 755: (190755, 1599091200), 756: (190756, 1584230400), 757: (190757, 1589328000), 758: (190758, 1602720000), 759: (190759, 1590710400), 760: (190760, 1600300800), 761: (190761, 1594598400), 762: (190762, 1602979200), 763: (190763, 1602892800), 764: (190764, 1586217600), 765: (190765, 1601510400), 766: (190766, 1602979200), 767: (190767, 1606348800), 768: (190768, 1585008000), 769: (190769, 1601251200), 770: (190770, 1598659200), 771: (190771, 1606348800), 772: (190772, 1592006400), 773: (190773, 1604620800), 774: (190774, 1598659200), 775: (190775, 1580774400), 776: (190776, 1583107200), 777: (190777, 1581120000), 778: (190778, 1603238400), 779: (190779, 1587772800), 780: (190780, 1586908800), 781: (190781, 1591747200), 782: (190782, 1594598400), 783: (190783, 1596153600), 784: (190784, 1606608000), 785: (190785, 1580515200), 786: (190786, 1579046400), 787: (190787, 1598832000), 788: (190788, 1598918400), 789: (190789, 1587772800), 790: (190790, 1603411200), 791: (190791, 1599091200), 792: (190792, 1607385600), 793: (190793, 1597795200), 794: (190794, 1608508800), 795: (190795, 1604793600), 796: (190796, 1592006400), 797: (190797, 1595980800), 798: (190798, 1580947200), 799: (190799, 1607040000), 800: (190800, 1600905600), 801: (190801, 1588896000), 802: (190802, 1604620800), 803: (190803, 1605571200), 804: (190804, 1608681600), 805: (190805, 1595721600), 806: (190806, 1605139200), 807: (190807, 1595376000), 808: (190808, 1608508800), 809: (190809, 1603497600), 810: (190810, 1582070400), 811: (190811, 1598918400), 812: (190812, 1608940800), 813: (190813, 1580169600), 814: (190814, 1608595200), 815: (190815, 1586649600), 816: (190816, 1601164800), 817: (190817, 1597276800), 818: (190818, 1607990400), 819: (190819, 1584057600), 820: (190820, 1592006400), 821: (190821, 1599782400), 822: (190822, 1579046400), 823: (190823, 1587254400), 824: (190824, 1586908800), 825: (190825, 1604275200), 826: (190826, 1608595200), 827: (190827, 1592265600), 828: (190828, 1600732800), 829: (190829, 1597104000), 830: (190830, 1586649600), 831: (190831, 1580860800), 832: (190832, 1592006400), 833: (190833, 1587686400), 834: (190834, 1601510400), 835: (190835, 1609200000), 836: (190836, 1583712000), 837: (190837, 1595203200), 838: (190838, 1607904000), 839: (190839, 1581552000), 840: (190840, 1592784000), 841: (190841, 1605139200), 842: (190842, 1585353600), 843: (190843, 1586476800), 844: (190844, 1591920000), 845: (190845, 1603152000), 846: (190846, 1595808000), 847: (190847, 1582502400), 848: (190848, 1607212800), 849: (190849, 1581292800), 850: (190850, 1586563200), 851: (190851, 1595289600), 852: (190852, 1580515200), 853: (190853, 1603843200), 854: (190854, 1593388800), 855: (190855, 1605744000), 856: (190856, 1578528000), 857: (190857, 1584835200), 858: (190858, 1600732800), 859: (190859, 1594166400), 860: (190860, 1583452800), 861: (190861, 1599523200), 862: (190862, 1599264000), 863: (190863, 1578873600), 864: (190864, 1609027200), 865: (190865, 1604880000), 866: (190866, 1596067200), 867: (190867, 1607990400), 868: (190868, 1609372800), 869: (190869, 1587168000), 870: (190870, 1590451200), 871: (190871, 1580169600), 872: (190872, 1605052800), 873: (190873, 1606694400), 874: (190874, 1585526400), 875: (190875, 1608422400), 876: (190876, 1589587200), 877: (190877, 1590451200), 878: (190878, 1593129600), 879: (190879, 1603152000), 880: (190880, 1578009600), 881: (190881, 1608249600), 882: (190882, 1591315200), 883: (190883, 1580947200), 884: (190884, 1594944000), 885: (190885, 1608249600), 886: (190886, 1591488000), 887: (190887, 1585785600), 888: (190888, 1599264000), 889: (190889, 1608768000), 890: (190890, 1596326400), 891: (190891, 1586044800), 892: (190892, 1607212800), 893: (190893, 1582156800), 894: (190894, 1605484800), 895: (190895, 1604966400), 896: (190896, 1601510400), 897: (190897, 1589760000), 898: (190898, 1607385600), 899: (190899, 1590710400), 900: (190900, 1589068800), 901: (190901, 1596672000), 902: (190902, 1579564800), 903: (190903, 1578787200), 904: (190904, 1585440000), 905: (190905, 1588032000), 906: (190906, 1597363200), 907: (190907, 1587427200), 908: (190908, 1603065600), 909: (190909, 1587945600), 910: (190910, 1581292800), 911: (190911, 1580083200), 912: (190912, 1584576000), 913: (190913, 1583625600), 914: (190914, 1602115200), 915: (190915, 1591056000), 916: (190916, 1597190400), 917: (190917, 1606694400), 918: (190918, 1584230400), 919: (190919, 1600560000), 920: (190920, 1578355200), 921: (190921, 1581292800), 922: (190922, 1597017600), 923: (190923, 1596326400), 924: (190924, 1596153600), 925: (190925, 1592697600), 926: (190926, 1581897600), 927: (190927, 1579478400), 928: (190928, 1580688000), 929: (190929, 1607299200), 930: (190930, 1597017600), 931: (190931, 1588982400), 932: (190932, 1596153600), 933: (190933, 1578182400), 934: (190934, 1579564800), 935: (190935, 1598313600), 936: (190936, 1596844800), 937: (190937, 1585612800), 938: (190938, 1589241600), 939: (190939, 1577923200), 940: (190940, 1581465600), 941: (190941, 1592956800), 942: (190942, 1602460800), 943: (190943, 1598313600), 944: (190944, 1580256000), 945: (190945, 1609286400), 946: (190946, 1602115200), 947: (190947, 1580083200), 948: (190948, 1587427200), 949: (190949, 1595721600), 950: (190950, 1603929600), 951: (190951, 1577923200), 952: (190952, 1586908800), 953: (190953, 1598054400), 954: (190954, 1598745600), 955: (190955, 1578528000), 956: (190956, 1589846400), 957: (190957, 1605571200), 958: (190958, 1586390400), 959: (190959, 1579824000), 960: (190960, 1595980800), 961: (190961, 1595894400), 962: (190962, 1608249600), 963: (190963, 1578960000), 964: (190964, 1598140800), 965: (190965, 1579305600), 966: (190966, 1587945600), 967: (190967, 1590883200), 968: (190968, 1588464000), 969: (190969, 1596499200), 970: (190970, 1585094400), 971: (190971, 1607558400), 972: (190972, 1599955200), 973: (190973, 1595721600), 974: (190974, 1595808000), 975: (190975, 1605484800), 976: (190976, 1600905600), 977: (190977, 1581465600), 978: (190978, 1601856000), 979: (190979, 1600473600), 980: (190980, 1590624000), 981: (190981, 1587600000), 982: (190982, 1586563200), 983: (190983, 1581984000), 984: (190984, 1582761600), 985: (190985, 1580342400), 986: (190986, 1588032000), 987: (190987, 1581033600), 988: (190988, 1605657600), 989: (190989, 1595030400), 990: (190990, 1599264000), 991: (190991, 1587168000), 992: (190992, 1590624000), 993: (190993, 1590883200), 994: (190994, 1595635200), 995: (190995, 1606867200), 996: (190996, 1601856000), 997: (190997, 1587340800), 998: (190998, 1588896000), 999: (190999, 1580256000), 1000: (191000, 1594425600)}))
print("Total produtos:", len([{'conta_id': 190001, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190002, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190003, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190004, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190005, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190006, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190007, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190008, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190009, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190010, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190011, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190012, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190013, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190014, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190015, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190016, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190017, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190018, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190019, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190020, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190021, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190022, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190023, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190024, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190025, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190026, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190027, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190028, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190029, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190030, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190031, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190032, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190033, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190034, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190035, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190036, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190037, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190038, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190039, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190040, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190041, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190042, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190043, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190044, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190045, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190046, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190047, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190048, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190049, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190050, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190051, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190052, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190053, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190054, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190055, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190056, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190057, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190058, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190059, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190060, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190061, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190062, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190063, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190064, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190065, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190066, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190067, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190068, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190069, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190070, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190071, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190072, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190073, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190074, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190075, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190076, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190077, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190078, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190079, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190080, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190081, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190082, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190083, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190084, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190085, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190086, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190087, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190088, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190089, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190090, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190091, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190092, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190093, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190094, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190095, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190096, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190097, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190098, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190099, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190100, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190101, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190102, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190103, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190104, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190105, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190106, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190107, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190108, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190109, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190110, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190111, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190112, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190113, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190114, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190115, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190116, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190117, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190118, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190119, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190120, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190121, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190122, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190123, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190124, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190125, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190126, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190127, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190128, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190129, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190130, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190131, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190132, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190133, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190134, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190135, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190136, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190137, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190138, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190139, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190140, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190141, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190142, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190143, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190144, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190145, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190146, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190147, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190148, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190149, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190150, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190151, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190152, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190153, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190154, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190155, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190156, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190157, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190158, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190159, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190160, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190161, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190162, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190163, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190164, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190165, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190166, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190167, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190168, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190169, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190170, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190171, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190172, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190173, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190174, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190175, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190176, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190177, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190178, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190179, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190180, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190181, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190182, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190183, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190184, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190185, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190186, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190187, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190188, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190189, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190190, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190191, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190192, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190193, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190194, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190195, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190196, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190197, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190198, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190199, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190200, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190201, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190202, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190203, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190204, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190205, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190206, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190207, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190208, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190209, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190210, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190211, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190212, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190213, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190214, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190215, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190216, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190217, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190218, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190219, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190220, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190221, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190222, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190223, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190224, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190225, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190226, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190227, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190228, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190229, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190230, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190231, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190232, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190233, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190234, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190235, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190236, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190237, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190238, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190239, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190240, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190241, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190242, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190243, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190244, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190245, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190246, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190247, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190248, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190249, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190250, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190251, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190252, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190253, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190254, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190255, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190256, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190257, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190258, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190259, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190260, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190261, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190262, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190263, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190264, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190265, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190266, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190267, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190268, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190269, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190270, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190271, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190272, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190273, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190274, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190275, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190276, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190277, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190278, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190279, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190280, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190281, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190282, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190283, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190284, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190285, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190286, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190287, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190288, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190289, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190290, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190291, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190292, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190293, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190294, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190295, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190296, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190297, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190298, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190299, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190300, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190301, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190302, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190303, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190304, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190305, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190306, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190307, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190308, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190309, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190310, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190311, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190312, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190313, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190314, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190315, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190316, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190317, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190318, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190319, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190320, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190321, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190322, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190323, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190324, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190325, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190326, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190327, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190328, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190329, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190330, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190331, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190332, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190333, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190334, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190335, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190336, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190337, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190338, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190339, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190340, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190341, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190342, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190343, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190344, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190345, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190346, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190347, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190348, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190349, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190350, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190351, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190352, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190353, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190354, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190355, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190356, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190357, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190358, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190359, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190360, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190361, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190362, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190363, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190364, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190365, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190366, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190367, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190368, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190369, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190370, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190371, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190372, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190373, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190374, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190375, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190376, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190377, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190378, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190379, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190380, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190381, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190382, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190383, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190384, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190385, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190386, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190387, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190388, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190389, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190390, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190391, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190392, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190393, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190394, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190395, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190396, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190397, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190398, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190399, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190400, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190401, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190402, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190403, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190404, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190405, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190406, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190407, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190408, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190409, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190410, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190411, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190412, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190413, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190414, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190415, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190416, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190417, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190418, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190419, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190420, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190421, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190422, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190423, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190424, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190425, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190426, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190427, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190428, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190429, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190430, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190431, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190432, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190433, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190434, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190435, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190436, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190437, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190438, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190439, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190440, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190441, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190442, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190443, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190444, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190445, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190446, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190447, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190448, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190449, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190450, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190451, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190452, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190453, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190454, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190455, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190456, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190457, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190458, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190459, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190460, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190461, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190462, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190463, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190464, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190465, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190466, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190467, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190468, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190469, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190470, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190471, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190472, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190473, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190474, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190475, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190476, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190477, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190478, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190479, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190480, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190481, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190482, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190483, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190484, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190485, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190486, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190487, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190488, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190489, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190490, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190491, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190492, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190493, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190494, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190495, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190496, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190497, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190498, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190499, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190500, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190501, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190502, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190503, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190504, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190505, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190506, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190507, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190508, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190509, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190510, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190511, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190512, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190513, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190514, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190515, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190516, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190517, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190518, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190519, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190520, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190521, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190522, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190523, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190524, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190525, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190526, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190527, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190528, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190529, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190530, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190531, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190532, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190533, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190534, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190535, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190536, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190537, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190538, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190539, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190540, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190541, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190542, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190543, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190544, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190545, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190546, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190547, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190548, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190549, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190550, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190551, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190552, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190553, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190554, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190555, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190556, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190557, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190558, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190559, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190560, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190561, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190562, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190563, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190564, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190565, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190566, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190567, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190568, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190569, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190570, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190571, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190572, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190573, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190574, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190575, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190576, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190577, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190578, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190579, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190580, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190581, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190582, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190583, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190584, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190585, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190586, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190587, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190588, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190589, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190590, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190591, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190592, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190593, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190594, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190595, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190596, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190597, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190598, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190599, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190600, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190601, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190602, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190603, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190604, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190605, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190606, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190607, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190608, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190609, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190610, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190611, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190612, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190613, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190614, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190615, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190616, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190617, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190618, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190619, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190620, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190621, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190622, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190623, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190624, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190625, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190626, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190627, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190628, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190629, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190630, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190631, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190632, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190633, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190634, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190635, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190636, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190637, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190638, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190639, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190640, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190641, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190642, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190643, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190644, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190645, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190646, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190647, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190648, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190649, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190650, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190651, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190652, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190653, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190654, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190655, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190656, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190657, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190658, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190659, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190660, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190661, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190662, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190663, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190664, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190665, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190666, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190667, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190668, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190669, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190670, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190671, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190672, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190673, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190674, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190675, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190676, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190677, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190678, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190679, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190680, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190681, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190682, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190683, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190684, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190685, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190686, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190687, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190688, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190689, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190690, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190691, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190692, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190693, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190694, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190695, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190696, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190697, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190698, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190699, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190700, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190701, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190702, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190703, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190704, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190705, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190706, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190707, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190708, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190709, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190710, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190711, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190712, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190713, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190714, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190715, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190716, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190717, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190718, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190719, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190720, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190721, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190722, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190723, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190724, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190725, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190726, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190727, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190728, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190729, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190730, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190731, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190732, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190733, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190734, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190735, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190736, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190737, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190738, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190739, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190740, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190741, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190742, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190743, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190744, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190745, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190746, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190747, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190748, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190749, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190750, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190751, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190752, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190753, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190754, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190755, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190756, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190757, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190758, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190759, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190760, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190761, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190762, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190763, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190764, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190765, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190766, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190767, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190768, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190769, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190770, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190771, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190772, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190773, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190774, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190775, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190776, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190777, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190778, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190779, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190780, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190781, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190782, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190783, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190784, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190785, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190786, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190787, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190788, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190789, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190790, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190791, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190792, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190793, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190794, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190795, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190796, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190797, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190798, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190799, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190800, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190801, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190802, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190803, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190804, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190805, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190806, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190807, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190808, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190809, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190810, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190811, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190812, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190813, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190814, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190815, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190816, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190817, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190818, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190819, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190820, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190821, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190822, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190823, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190824, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190825, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190826, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190827, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190828, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190829, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190830, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190831, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190832, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190833, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190834, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190835, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190836, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190837, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190838, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190839, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190840, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190841, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190842, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190843, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190844, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190845, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190846, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190847, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190848, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190849, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190850, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190851, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190852, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190853, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190854, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190855, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190856, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190857, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190858, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190859, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190860, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190861, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190862, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190863, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190864, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190865, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190866, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190867, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190868, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190869, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190870, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190871, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190872, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190873, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190874, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190875, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190876, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190877, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190878, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190879, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190880, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190881, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190882, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190883, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190884, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190885, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190886, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190887, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190888, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190889, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190890, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190891, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190892, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190893, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190894, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190895, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190896, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190897, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190898, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190899, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190900, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190901, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190902, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190903, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190904, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190905, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190906, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190907, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190908, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190909, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190910, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190911, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190912, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190913, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190914, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190915, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190916, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190917, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190918, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190919, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190920, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190921, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190922, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190923, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190924, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190925, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190926, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190927, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190928, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190929, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190930, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190931, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190932, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190933, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190934, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190935, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190936, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190937, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190938, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190939, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190940, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190941, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190942, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190943, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190944, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190945, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190946, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190947, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190948, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190949, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190950, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190951, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190952, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190953, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190954, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190955, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190956, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190957, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190958, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190959, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190960, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190961, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190962, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190963, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190964, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190965, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190966, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190967, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190968, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190969, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190970, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190971, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190972, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190973, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190974, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190975, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190976, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190977, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 1}, {'conta_id': 190978, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190979, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190980, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190981, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190982, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190983, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190984, 'fl_cartao_credito': 0, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190985, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190986, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190987, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190988, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190989, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190990, 'fl_cartao_credito': 1, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190991, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190992, 'fl_cartao_credito': 1, 'fl_investidor': 0, 'fl_credito': 0}, {'conta_id': 190993, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190994, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190995, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 190996, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190997, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190998, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 1}, {'conta_id': 190999, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}, {'conta_id': 191000, 'fl_cartao_credito': 0, 'fl_investidor': 1, 'fl_credito': 0}]))
print("Total transações:", len([{'conta_id': 190001, 'valor': 844.42, 'dt_transacao': '2021-01-04 02:00:00'}, {'conta_id': 190001, 'valor': 438.06, 'dt_transacao': '2021-03-06 19:00:00'}, {'conta_id': 190001, 'valor': 54.05, 'dt_transacao': '2021-01-26 22:00:00'}, {'conta_id': 190001, 'valor': 1300.12, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190001, 'valor': 441.66, 'dt_transacao': '2021-03-17 08:00:00'}, {'conta_id': 190002, 'valor': 186.4, 'dt_transacao': '2021-01-13 11:00:00'}, {'conta_id': 190003, 'valor': 587.06, 'dt_transacao': '2021-03-22 19:00:00'}, {'conta_id': 190003, 'valor': 1771.02, 'dt_transacao': '2021-02-16 18:00:00'}, {'conta_id': 190003, 'valor': 385.38, 'dt_transacao': '2021-01-09 01:00:00'}, {'conta_id': 190003, 'valor': 1322.87, 'dt_transacao': '2021-02-07 02:00:00'}, {'conta_id': 190003, 'valor': 1710.78, 'dt_transacao': '2021-01-13 12:00:00'}, {'conta_id': 190004, 'valor': 1218.65, 'dt_transacao': '2021-01-22 17:00:00'}, {'conta_id': 190005, 'valor': 1643.78, 'dt_transacao': '2021-02-10 12:00:00'}, {'conta_id': 190005, 'valor': 536.21, 'dt_transacao': '2021-01-28 18:00:00'}, {'conta_id': 190006, 'valor': 494.01, 'dt_transacao': '2021-03-13 17:00:00'}, {'conta_id': 190006, 'valor': 526.22, 'dt_transacao': '2021-03-16 13:00:00'}, {'conta_id': 190007, 'valor': 306.53, 'dt_transacao': '2021-01-21 21:00:00'}, {'conta_id': 190008, 'valor': 1363.74, 'dt_transacao': '2021-03-10 08:00:00'}, {'conta_id': 190009, 'valor': 1524.16, 'dt_transacao': '2021-03-06 03:00:00'}, {'conta_id': 190009, 'valor': 1741.17, 'dt_transacao': '2021-02-08 20:00:00'}, {'conta_id': 190009, 'valor': 1015.82, 'dt_transacao': '2021-01-26 04:00:00'}, {'conta_id': 190009, 'valor': 748.45, 'dt_transacao': '2021-01-21 17:00:00'}, {'conta_id': 190009, 'valor': 1907.05, 'dt_transacao': '2021-03-09 00:00:00'}, {'conta_id': 190010, 'valor': 482.5, 'dt_transacao': '2021-03-14 02:00:00'}, {'conta_id': 190011, 'valor': 1055.82, 'dt_transacao': '2021-03-19 13:00:00'}, {'conta_id': 190011, 'valor': 1928.76, 'dt_transacao': '2021-03-11 23:00:00'}, {'conta_id': 190011, 'valor': 1380.05, 'dt_transacao': '2021-02-09 12:00:00'}, {'conta_id': 190012, 'valor': 1177.03, 'dt_transacao': '2021-01-30 18:00:00'}, {'conta_id': 190013, 'valor': 1719.41, 'dt_transacao': '2021-01-10 16:00:00'}, {'conta_id': 190014, 'valor': 1615.19, 'dt_transacao': '2021-01-25 03:00:00'}, {'conta_id': 190014, 'valor': 194.76, 'dt_transacao': '2021-02-25 11:00:00'}, {'conta_id': 190014, 'valor': 847.73, 'dt_transacao': '2021-03-01 23:00:00'}, {'conta_id': 190014, 'valor': 109.29, 'dt_transacao': '2021-03-25 20:00:00'}, {'conta_id': 190015, 'valor': 897.78, 'dt_transacao': '2021-02-24 05:00:00'}, {'conta_id': 190015, 'valor': 557.81, 'dt_transacao': '2021-02-01 02:00:00'}, {'conta_id': 190015, 'valor': 886.82, 'dt_transacao': '2021-03-12 03:00:00'}, {'conta_id': 190015, 'valor': 102.13, 'dt_transacao': '2021-03-11 00:00:00'}, {'conta_id': 190015, 'valor': 1938.02, 'dt_transacao': '2021-01-31 05:00:00'}, {'conta_id': 190016, 'valor': 1970.63, 'dt_transacao': '2021-02-03 14:00:00'}, {'conta_id': 190017, 'valor': 1471.74, 'dt_transacao': '2021-01-08 23:00:00'}, {'conta_id': 190017, 'valor': 627.9, 'dt_transacao': '2021-01-07 18:00:00'}, {'conta_id': 190017, 'valor': 954.1, 'dt_transacao': '2021-03-09 05:00:00'}, {'conta_id': 190017, 'valor': 114.7, 'dt_transacao': '2021-03-07 02:00:00'}, {'conta_id': 190017, 'valor': 1702.83, 'dt_transacao': '2021-01-09 19:00:00'}, {'conta_id': 190018, 'valor': 1315.09, 'dt_transacao': '2021-03-14 16:00:00'}, {'conta_id': 190018, 'valor': 633.42, 'dt_transacao': '2021-02-03 06:00:00'}, {'conta_id': 190018, 'valor': 1339.78, 'dt_transacao': '2021-02-10 07:00:00'}, {'conta_id': 190018, 'valor': 531.96, 'dt_transacao': '2021-01-17 21:00:00'}, {'conta_id': 190019, 'valor': 427.1, 'dt_transacao': '2021-02-03 04:00:00'}, {'conta_id': 190019, 'valor': 1866.59, 'dt_transacao': '2021-01-09 07:00:00'}, {'conta_id': 190019, 'valor': 739.68, 'dt_transacao': '2021-01-21 14:00:00'}, {'conta_id': 190019, 'valor': 1667.66, 'dt_transacao': '2021-04-01 09:00:00'}, {'conta_id': 190019, 'valor': 1223.74, 'dt_transacao': '2021-03-25 16:00:00'}, {'conta_id': 190020, 'valor': 564.23, 'dt_transacao': '2021-01-27 22:00:00'}, {'conta_id': 190020, 'valor': 686.42, 'dt_transacao': '2021-03-29 20:00:00'}, {'conta_id': 190020, 'valor': 1705.97, 'dt_transacao': '2021-03-06 15:00:00'}, {'conta_id': 190021, 'valor': 1542.47, 'dt_transacao': '2021-03-23 08:00:00'}, {'conta_id': 190021, 'valor': 324.0, 'dt_transacao': '2021-02-26 17:00:00'}, {'conta_id': 190021, 'valor': 1411.63, 'dt_transacao': '2021-03-13 00:00:00'}, {'conta_id': 190022, 'valor': 84.62, 'dt_transacao': '2021-02-16 01:00:00'}, {'conta_id': 190022, 'valor': 1797.95, 'dt_transacao': '2021-01-27 21:00:00'}, {'conta_id': 190023, 'valor': 1762.95, 'dt_transacao': '2021-01-04 05:00:00'}, {'conta_id': 190023, 'valor': 1473.39, 'dt_transacao': '2021-02-12 13:00:00'}, {'conta_id': 190024, 'valor': 399.95, 'dt_transacao': '2021-02-28 11:00:00'}, {'conta_id': 190024, 'valor': 611.08, 'dt_transacao': '2021-01-30 07:00:00'}, {'conta_id': 190025, 'valor': 799.96, 'dt_transacao': '2021-03-10 10:00:00'}, {'conta_id': 190025, 'valor': 1878.54, 'dt_transacao': '2021-01-15 08:00:00'}, {'conta_id': 190025, 'valor': 357.96, 'dt_transacao': '2021-02-03 01:00:00'}, {'conta_id': 190025, 'valor': 217.7, 'dt_transacao': '2021-02-25 11:00:00'}, {'conta_id': 190025, 'valor': 1457.36, 'dt_transacao': '2021-02-10 13:00:00'}, {'conta_id': 190026, 'valor': 1851.22, 'dt_transacao': '2021-03-10 21:00:00'}, {'conta_id': 190026, 'valor': 1439.14, 'dt_transacao': '2021-03-27 06:00:00'}, {'conta_id': 190026, 'valor': 729.08, 'dt_transacao': '2021-01-09 21:00:00'}, {'conta_id': 190026, 'valor': 1841.62, 'dt_transacao': '2021-03-21 10:00:00'}, {'conta_id': 190026, 'valor': 1327.11, 'dt_transacao': '2021-01-16 23:00:00'}, {'conta_id': 190027, 'valor': 1330.19, 'dt_transacao': '2021-02-18 21:00:00'}, {'conta_id': 190027, 'valor': 1496.52, 'dt_transacao': '2021-01-23 19:00:00'}, {'conta_id': 190027, 'valor': 1138.64, 'dt_transacao': '2021-02-21 17:00:00'}, {'conta_id': 190027, 'valor': 1667.62, 'dt_transacao': '2021-02-08 09:00:00'}, {'conta_id': 190028, 'valor': 1318.04, 'dt_transacao': '2021-02-06 16:00:00'}, {'conta_id': 190028, 'valor': 1328.04, 'dt_transacao': '2021-03-21 10:00:00'}, {'conta_id': 190029, 'valor': 1967.89, 'dt_transacao': '2021-03-20 02:00:00'}, {'conta_id': 190029, 'valor': 911.38, 'dt_transacao': '2021-03-22 18:00:00'}, {'conta_id': 190030, 'valor': 438.46, 'dt_transacao': '2021-03-31 16:00:00'}, {'conta_id': 190030, 'valor': 929.64, 'dt_transacao': '2021-03-13 07:00:00'}, {'conta_id': 190030, 'valor': 1835.02, 'dt_transacao': '2021-01-16 14:00:00'}, {'conta_id': 190030, 'valor': 267.54, 'dt_transacao': '2021-03-01 21:00:00'}, {'conta_id': 190031, 'valor': 519.12, 'dt_transacao': '2021-02-01 20:00:00'}, {'conta_id': 190031, 'valor': 555.35, 'dt_transacao': '2021-03-08 15:00:00'}, {'conta_id': 190031, 'valor': 1253.87, 'dt_transacao': '2021-02-05 14:00:00'}, {'conta_id': 190031, 'valor': 155.89, 'dt_transacao': '2021-02-06 07:00:00'}, {'conta_id': 190032, 'valor': 1413.13, 'dt_transacao': '2021-01-09 13:00:00'}, {'conta_id': 190032, 'valor': 815.79, 'dt_transacao': '2021-03-11 14:00:00'}, {'conta_id': 190033, 'valor': 1885.54, 'dt_transacao': '2021-02-08 12:00:00'}, {'conta_id': 190034, 'valor': 672.92, 'dt_transacao': '2021-03-28 12:00:00'}, {'conta_id': 190034, 'valor': 1448.64, 'dt_transacao': '2021-03-01 04:00:00'}, {'conta_id': 190034, 'valor': 1961.46, 'dt_transacao': '2021-03-10 00:00:00'}, {'conta_id': 190034, 'valor': 1813.95, 'dt_transacao': '2021-03-17 18:00:00'}, {'conta_id': 190035, 'valor': 655.38, 'dt_transacao': '2021-02-28 10:00:00'}, {'conta_id': 190035, 'valor': 675.66, 'dt_transacao': '2021-02-18 08:00:00'}, {'conta_id': 190035, 'valor': 1504.27, 'dt_transacao': '2021-02-23 08:00:00'}, {'conta_id': 190035, 'valor': 1669.99, 'dt_transacao': '2021-03-02 00:00:00'}, {'conta_id': 190036, 'valor': 399.52, 'dt_transacao': '2021-01-03 19:00:00'}, {'conta_id': 190036, 'valor': 305.61, 'dt_transacao': '2021-01-17 15:00:00'}, {'conta_id': 190037, 'valor': 217.09, 'dt_transacao': '2021-01-04 09:00:00'}, {'conta_id': 190037, 'valor': 1152.02, 'dt_transacao': '2021-02-18 12:00:00'}, {'conta_id': 190037, 'valor': 1883.0, 'dt_transacao': '2021-01-26 02:00:00'}, {'conta_id': 190038, 'valor': 857.33, 'dt_transacao': '2021-02-17 02:00:00'}, {'conta_id': 190038, 'valor': 1012.42, 'dt_transacao': '2021-02-13 00:00:00'}, {'conta_id': 190038, 'valor': 1699.3, 'dt_transacao': '2021-03-04 03:00:00'}, {'conta_id': 190038, 'valor': 867.61, 'dt_transacao': '2021-02-16 20:00:00'}, {'conta_id': 190038, 'valor': 1783.26, 'dt_transacao': '2021-02-28 22:00:00'}, {'conta_id': 190039, 'valor': 1651.64, 'dt_transacao': '2021-03-17 08:00:00'}, {'conta_id': 190039, 'valor': 645.29, 'dt_transacao': '2021-02-01 02:00:00'}, {'conta_id': 190039, 'valor': 558.58, 'dt_transacao': '2021-02-27 07:00:00'}, {'conta_id': 190039, 'valor': 1501.2, 'dt_transacao': '2021-03-14 19:00:00'}, {'conta_id': 190040, 'valor': 1595.87, 'dt_transacao': '2021-02-13 08:00:00'}, {'conta_id': 190040, 'valor': 1760.76, 'dt_transacao': '2021-03-31 08:00:00'}, {'conta_id': 190040, 'valor': 1112.03, 'dt_transacao': '2021-03-08 06:00:00'}, {'conta_id': 190041, 'valor': 1586.07, 'dt_transacao': '2021-01-12 09:00:00'}, {'conta_id': 190041, 'valor': 443.97, 'dt_transacao': '2021-03-30 07:00:00'}, {'conta_id': 190041, 'valor': 613.11, 'dt_transacao': '2021-03-16 11:00:00'}, {'conta_id': 190041, 'valor': 947.05, 'dt_transacao': '2021-03-09 11:00:00'}, {'conta_id': 190042, 'valor': 1442.85, 'dt_transacao': '2021-02-10 03:00:00'}, {'conta_id': 190043, 'valor': 593.2, 'dt_transacao': '2021-02-16 05:00:00'}, {'conta_id': 190043, 'valor': 605.23, 'dt_transacao': '2021-04-01 17:00:00'}, {'conta_id': 190044, 'valor': 1745.57, 'dt_transacao': '2021-03-15 09:00:00'}, {'conta_id': 190045, 'valor': 1644.41, 'dt_transacao': '2021-02-21 15:00:00'}, {'conta_id': 190046, 'valor': 1116.59, 'dt_transacao': '2021-02-23 19:00:00'}, {'conta_id': 190047, 'valor': 1242.43, 'dt_transacao': '2021-03-20 23:00:00'}, {'conta_id': 190047, 'valor': 199.37, 'dt_transacao': '2021-01-27 20:00:00'}, {'conta_id': 190047, 'valor': 422.84, 'dt_transacao': '2021-03-26 02:00:00'}, {'conta_id': 190047, 'valor': 314.96, 'dt_transacao': '2021-01-23 17:00:00'}, {'conta_id': 190047, 'valor': 151.05, 'dt_transacao': '2021-01-01 13:00:00'}, {'conta_id': 190048, 'valor': 1375.08, 'dt_transacao': '2021-02-03 20:00:00'}, {'conta_id': 190049, 'valor': 143.75, 'dt_transacao': '2021-01-22 09:00:00'}, {'conta_id': 190049, 'valor': 1190.49, 'dt_transacao': '2021-03-14 09:00:00'}, {'conta_id': 190050, 'valor': 161.8, 'dt_transacao': '2021-01-06 13:00:00'}, {'conta_id': 190050, 'valor': 1469.33, 'dt_transacao': '2021-03-19 08:00:00'}, {'conta_id': 190050, 'valor': 52.7, 'dt_transacao': '2021-01-30 21:00:00'}, {'conta_id': 190050, 'valor': 1672.08, 'dt_transacao': '2021-03-15 18:00:00'}, {'conta_id': 190051, 'valor': 872.33, 'dt_transacao': '2021-03-04 02:00:00'}, {'conta_id': 190051, 'valor': 940.55, 'dt_transacao': '2021-02-22 10:00:00'}, {'conta_id': 190051, 'valor': 642.8, 'dt_transacao': '2021-01-14 05:00:00'}, {'conta_id': 190051, 'valor': 660.28, 'dt_transacao': '2021-03-30 15:00:00'}, {'conta_id': 190051, 'valor': 577.15, 'dt_transacao': '2021-02-21 17:00:00'}, {'conta_id': 190052, 'valor': 1650.03, 'dt_transacao': '2021-01-01 21:00:00'}, {'conta_id': 190052, 'valor': 1738.87, 'dt_transacao': '2021-03-01 13:00:00'}, {'conta_id': 190052, 'valor': 109.35, 'dt_transacao': '2021-03-08 11:00:00'}, {'conta_id': 190052, 'valor': 1245.65, 'dt_transacao': '2021-03-05 20:00:00'}, {'conta_id': 190052, 'valor': 884.64, 'dt_transacao': '2021-01-07 06:00:00'}, {'conta_id': 190053, 'valor': 1598.83, 'dt_transacao': '2021-04-01 05:00:00'}, {'conta_id': 190053, 'valor': 622.19, 'dt_transacao': '2021-01-02 17:00:00'}, {'conta_id': 190053, 'valor': 816.57, 'dt_transacao': '2021-01-29 03:00:00'}, {'conta_id': 190053, 'valor': 923.43, 'dt_transacao': '2021-01-16 20:00:00'}, {'conta_id': 190053, 'valor': 1664.45, 'dt_transacao': '2021-03-05 22:00:00'}, {'conta_id': 190054, 'valor': 382.03, 'dt_transacao': '2021-03-18 16:00:00'}, {'conta_id': 190054, 'valor': 1492.84, 'dt_transacao': '2021-01-18 02:00:00'}, {'conta_id': 190054, 'valor': 553.22, 'dt_transacao': '2021-02-23 10:00:00'}, {'conta_id': 190054, 'valor': 1868.56, 'dt_transacao': '2021-03-06 08:00:00'}, {'conta_id': 190055, 'valor': 1104.28, 'dt_transacao': '2021-03-11 12:00:00'}, {'conta_id': 190055, 'valor': 911.23, 'dt_transacao': '2021-02-11 06:00:00'}, {'conta_id': 190055, 'valor': 1961.57, 'dt_transacao': '2021-01-31 18:00:00'}, {'conta_id': 190056, 'valor': 991.16, 'dt_transacao': '2021-01-05 04:00:00'}, {'conta_id': 190056, 'valor': 1005.08, 'dt_transacao': '2021-03-17 10:00:00'}, {'conta_id': 190057, 'valor': 150.58, 'dt_transacao': '2021-02-03 10:00:00'}, {'conta_id': 190057, 'valor': 1246.9, 'dt_transacao': '2021-02-20 20:00:00'}, {'conta_id': 190058, 'valor': 1262.69, 'dt_transacao': '2021-02-06 07:00:00'}, {'conta_id': 190058, 'valor': 1493.79, 'dt_transacao': '2021-02-25 03:00:00'}, {'conta_id': 190059, 'valor': 587.44, 'dt_transacao': '2021-02-17 13:00:00'}, {'conta_id': 190060, 'valor': 1366.59, 'dt_transacao': '2021-03-05 18:00:00'}, {'conta_id': 190060, 'valor': 287.09, 'dt_transacao': '2021-03-01 20:00:00'}, {'conta_id': 190060, 'valor': 508.7, 'dt_transacao': '2021-02-02 21:00:00'}, {'conta_id': 190060, 'valor': 19.79, 'dt_transacao': '2021-03-01 09:00:00'}, {'conta_id': 190060, 'valor': 1355.54, 'dt_transacao': '2021-01-21 02:00:00'}, {'conta_id': 190061, 'valor': 1998.75, 'dt_transacao': '2021-03-03 03:00:00'}, {'conta_id': 190061, 'valor': 475.17, 'dt_transacao': '2021-03-15 11:00:00'}, {'conta_id': 190062, 'valor': 1730.27, 'dt_transacao': '2021-01-07 19:00:00'}, {'conta_id': 190062, 'valor': 1490.6, 'dt_transacao': '2021-02-06 07:00:00'}, {'conta_id': 190062, 'valor': 1214.52, 'dt_transacao': '2021-02-15 07:00:00'}, {'conta_id': 190062, 'valor': 1273.6, 'dt_transacao': '2021-03-21 08:00:00'}, {'conta_id': 190062, 'valor': 1355.85, 'dt_transacao': '2021-03-26 21:00:00'}, {'conta_id': 190063, 'valor': 1819.35, 'dt_transacao': '2021-02-11 23:00:00'}, {'conta_id': 190064, 'valor': 1828.18, 'dt_transacao': '2021-03-03 09:00:00'}, {'conta_id': 190064, 'valor': 1493.37, 'dt_transacao': '2021-02-13 03:00:00'}, {'conta_id': 190064, 'valor': 937.19, 'dt_transacao': '2021-01-10 04:00:00'}, {'conta_id': 190065, 'valor': 737.8, 'dt_transacao': '2021-03-28 08:00:00'}, {'conta_id': 190065, 'valor': 1169.54, 'dt_transacao': '2021-03-23 11:00:00'}, {'conta_id': 190065, 'valor': 217.58, 'dt_transacao': '2021-01-30 15:00:00'}, {'conta_id': 190065, 'valor': 51.02, 'dt_transacao': '2021-03-13 10:00:00'}, {'conta_id': 190066, 'valor': 91.65, 'dt_transacao': '2021-01-05 09:00:00'}, {'conta_id': 190066, 'valor': 1991.86, 'dt_transacao': '2021-01-15 03:00:00'}, {'conta_id': 190067, 'valor': 1667.55, 'dt_transacao': '2021-03-20 13:00:00'}, {'conta_id': 190068, 'valor': 1961.87, 'dt_transacao': '2021-02-17 17:00:00'}, {'conta_id': 190069, 'valor': 1283.77, 'dt_transacao': '2021-03-18 00:00:00'}, {'conta_id': 190069, 'valor': 102.11, 'dt_transacao': '2021-02-12 07:00:00'}, {'conta_id': 190070, 'valor': 1577.5, 'dt_transacao': '2021-03-18 00:00:00'}, {'conta_id': 190070, 'valor': 555.27, 'dt_transacao': '2021-01-19 04:00:00'}, {'conta_id': 190071, 'valor': 1580.18, 'dt_transacao': '2021-01-31 18:00:00'}, {'conta_id': 190071, 'valor': 648.22, 'dt_transacao': '2021-01-23 08:00:00'}, {'conta_id': 190071, 'valor': 105.75, 'dt_transacao': '2021-02-23 16:00:00'}, {'conta_id': 190072, 'valor': 87.69, 'dt_transacao': '2021-03-26 16:00:00'}, {'conta_id': 190072, 'valor': 603.96, 'dt_transacao': '2021-03-24 00:00:00'}, {'conta_id': 190072, 'valor': 122.59, 'dt_transacao': '2021-03-03 12:00:00'}, {'conta_id': 190072, 'valor': 853.12, 'dt_transacao': '2021-01-14 15:00:00'}, {'conta_id': 190072, 'valor': 1425.09, 'dt_transacao': '2021-02-26 02:00:00'}, {'conta_id': 190073, 'valor': 1061.58, 'dt_transacao': '2021-02-28 16:00:00'}, {'conta_id': 190073, 'valor': 1211.25, 'dt_transacao': '2021-01-13 22:00:00'}, {'conta_id': 190073, 'valor': 229.71, 'dt_transacao': '2021-03-25 20:00:00'}, {'conta_id': 190073, 'valor': 1755.53, 'dt_transacao': '2021-03-12 23:00:00'}, {'conta_id': 190073, 'valor': 1731.12, 'dt_transacao': '2021-02-25 14:00:00'}, {'conta_id': 190074, 'valor': 625.73, 'dt_transacao': '2021-02-02 11:00:00'}, {'conta_id': 190074, 'valor': 1907.23, 'dt_transacao': '2021-03-29 15:00:00'}, {'conta_id': 190074, 'valor': 135.2, 'dt_transacao': '2021-01-11 02:00:00'}, {'conta_id': 190074, 'valor': 864.32, 'dt_transacao': '2021-02-17 04:00:00'}, {'conta_id': 190075, 'valor': 1201.22, 'dt_transacao': '2021-02-15 03:00:00'}, {'conta_id': 190075, 'valor': 1156.26, 'dt_transacao': '2021-01-28 04:00:00'}, {'conta_id': 190075, 'valor': 1313.82, 'dt_transacao': '2021-01-29 03:00:00'}, {'conta_id': 190076, 'valor': 1858.24, 'dt_transacao': '2021-03-30 08:00:00'}, {'conta_id': 190076, 'valor': 58.88, 'dt_transacao': '2021-02-04 22:00:00'}, {'conta_id': 190076, 'valor': 1525.06, 'dt_transacao': '2021-02-13 11:00:00'}, {'conta_id': 190076, 'valor': 13.21, 'dt_transacao': '2021-01-19 18:00:00'}, {'conta_id': 190076, 'valor': 1315.03, 'dt_transacao': '2021-01-09 04:00:00'}, {'conta_id': 190077, 'valor': 623.96, 'dt_transacao': '2021-02-11 18:00:00'}, {'conta_id': 190077, 'valor': 1192.85, 'dt_transacao': '2021-01-07 04:00:00'}, {'conta_id': 190077, 'valor': 315.66, 'dt_transacao': '2021-03-21 01:00:00'}, {'conta_id': 190078, 'valor': 1510.13, 'dt_transacao': '2021-01-15 11:00:00'}, {'conta_id': 190078, 'valor': 860.4, 'dt_transacao': '2021-02-06 21:00:00'}, {'conta_id': 190079, 'valor': 603.54, 'dt_transacao': '2021-01-28 04:00:00'}, {'conta_id': 190079, 'valor': 1528.56, 'dt_transacao': '2021-02-07 10:00:00'}, {'conta_id': 190080, 'valor': 1117.89, 'dt_transacao': '2021-03-27 11:00:00'}, {'conta_id': 190080, 'valor': 145.08, 'dt_transacao': '2021-01-06 13:00:00'}, {'conta_id': 190080, 'valor': 38.46, 'dt_transacao': '2021-01-10 10:00:00'}, {'conta_id': 190080, 'valor': 1151.94, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190080, 'valor': 1419.29, 'dt_transacao': '2021-02-23 09:00:00'}, {'conta_id': 190081, 'valor': 1689.12, 'dt_transacao': '2021-02-01 13:00:00'}, {'conta_id': 190081, 'valor': 1178.15, 'dt_transacao': '2021-03-09 02:00:00'}, {'conta_id': 190081, 'valor': 792.14, 'dt_transacao': '2021-02-09 23:00:00'}, {'conta_id': 190081, 'valor': 679.69, 'dt_transacao': '2021-02-12 05:00:00'}, {'conta_id': 190082, 'valor': 512.68, 'dt_transacao': '2021-01-23 19:00:00'}, {'conta_id': 190082, 'valor': 306.59, 'dt_transacao': '2021-03-25 02:00:00'}, {'conta_id': 190083, 'valor': 137.5, 'dt_transacao': '2021-02-26 20:00:00'}, {'conta_id': 190083, 'valor': 606.34, 'dt_transacao': '2021-02-05 18:00:00'}, {'conta_id': 190083, 'valor': 113.29, 'dt_transacao': '2021-03-06 02:00:00'}, {'conta_id': 190083, 'valor': 621.51, 'dt_transacao': '2021-02-27 01:00:00'}, {'conta_id': 190084, 'valor': 1109.08, 'dt_transacao': '2021-01-06 14:00:00'}, {'conta_id': 190084, 'valor': 1818.19, 'dt_transacao': '2021-03-15 20:00:00'}, {'conta_id': 190084, 'valor': 377.29, 'dt_transacao': '2021-03-19 15:00:00'}, {'conta_id': 190084, 'valor': 1003.23, 'dt_transacao': '2021-01-08 14:00:00'}, {'conta_id': 190084, 'valor': 207.9, 'dt_transacao': '2021-02-13 22:00:00'}, {'conta_id': 190085, 'valor': 746.33, 'dt_transacao': '2021-02-06 12:00:00'}, {'conta_id': 190085, 'valor': 818.13, 'dt_transacao': '2021-02-13 21:00:00'}, {'conta_id': 190085, 'valor': 1196.17, 'dt_transacao': '2021-03-22 20:00:00'}, {'conta_id': 190086, 'valor': 1746.69, 'dt_transacao': '2021-02-12 02:00:00'}, {'conta_id': 190086, 'valor': 1165.9, 'dt_transacao': '2021-01-19 11:00:00'}, {'conta_id': 190086, 'valor': 621.15, 'dt_transacao': '2021-03-25 22:00:00'}, {'conta_id': 190086, 'valor': 1324.61, 'dt_transacao': '2021-01-17 19:00:00'}, {'conta_id': 190086, 'valor': 1417.96, 'dt_transacao': '2021-01-11 09:00:00'}, {'conta_id': 190087, 'valor': 725.83, 'dt_transacao': '2021-01-24 06:00:00'}, {'conta_id': 190088, 'valor': 1015.87, 'dt_transacao': '2021-03-11 23:00:00'}, {'conta_id': 190089, 'valor': 1662.86, 'dt_transacao': '2021-03-21 05:00:00'}, {'conta_id': 190089, 'valor': 1442.64, 'dt_transacao': '2021-01-06 13:00:00'}, {'conta_id': 190090, 'valor': 1808.04, 'dt_transacao': '2021-01-25 11:00:00'}, {'conta_id': 190090, 'valor': 1356.94, 'dt_transacao': '2021-03-15 14:00:00'}, {'conta_id': 190090, 'valor': 923.9, 'dt_transacao': '2021-02-06 12:00:00'}, {'conta_id': 190090, 'valor': 1006.31, 'dt_transacao': '2021-02-23 05:00:00'}, {'conta_id': 190091, 'valor': 1702.48, 'dt_transacao': '2021-02-06 02:00:00'}, {'conta_id': 190091, 'valor': 1526.47, 'dt_transacao': '2021-02-04 14:00:00'}, {'conta_id': 190091, 'valor': 1810.71, 'dt_transacao': '2021-01-19 13:00:00'}, {'conta_id': 190091, 'valor': 184.34, 'dt_transacao': '2021-01-29 14:00:00'}, {'conta_id': 190091, 'valor': 1769.37, 'dt_transacao': '2021-01-04 13:00:00'}, {'conta_id': 190092, 'valor': 638.05, 'dt_transacao': '2021-01-13 22:00:00'}, {'conta_id': 190093, 'valor': 1231.08, 'dt_transacao': '2021-01-11 00:00:00'}, {'conta_id': 190093, 'valor': 512.59, 'dt_transacao': '2021-01-20 17:00:00'}, {'conta_id': 190093, 'valor': 1880.51, 'dt_transacao': '2021-03-19 16:00:00'}, {'conta_id': 190093, 'valor': 847.09, 'dt_transacao': '2021-02-06 07:00:00'}, {'conta_id': 190094, 'valor': 1193.88, 'dt_transacao': '2021-01-03 20:00:00'}, {'conta_id': 190094, 'valor': 1031.24, 'dt_transacao': '2021-01-31 22:00:00'}, {'conta_id': 190094, 'valor': 288.01, 'dt_transacao': '2021-02-24 00:00:00'}, {'conta_id': 190094, 'valor': 1230.0, 'dt_transacao': '2021-01-31 18:00:00'}, {'conta_id': 190095, 'valor': 88.03, 'dt_transacao': '2021-02-19 11:00:00'}, {'conta_id': 190095, 'valor': 508.05, 'dt_transacao': '2021-01-03 11:00:00'}, {'conta_id': 190095, 'valor': 1577.25, 'dt_transacao': '2021-02-14 07:00:00'}, {'conta_id': 190095, 'valor': 1466.15, 'dt_transacao': '2021-03-22 03:00:00'}, {'conta_id': 190096, 'valor': 1263.84, 'dt_transacao': '2021-01-18 02:00:00'}, {'conta_id': 190096, 'valor': 1432.37, 'dt_transacao': '2021-02-28 01:00:00'}, {'conta_id': 190096, 'valor': 587.59, 'dt_transacao': '2021-01-06 06:00:00'}, {'conta_id': 190096, 'valor': 1771.69, 'dt_transacao': '2021-02-10 09:00:00'}, {'conta_id': 190097, 'valor': 1731.73, 'dt_transacao': '2021-02-12 08:00:00'}, {'conta_id': 190098, 'valor': 1384.44, 'dt_transacao': '2021-02-17 16:00:00'}, {'conta_id': 190098, 'valor': 534.17, 'dt_transacao': '2021-01-11 23:00:00'}, {'conta_id': 190098, 'valor': 849.57, 'dt_transacao': '2021-02-25 19:00:00'}, {'conta_id': 190098, 'valor': 1990.2, 'dt_transacao': '2021-01-24 17:00:00'}, {'conta_id': 190099, 'valor': 1435.61, 'dt_transacao': '2021-01-22 22:00:00'}, {'conta_id': 190099, 'valor': 888.37, 'dt_transacao': '2021-02-27 01:00:00'}, {'conta_id': 190099, 'valor': 1453.9, 'dt_transacao': '2021-02-15 19:00:00'}, {'conta_id': 190099, 'valor': 1996.3, 'dt_transacao': '2021-02-05 20:00:00'}, {'conta_id': 190099, 'valor': 1960.07, 'dt_transacao': '2021-01-08 02:00:00'}, {'conta_id': 190100, 'valor': 472.75, 'dt_transacao': '2021-03-24 11:00:00'}, {'conta_id': 190101, 'valor': 1526.41, 'dt_transacao': '2021-03-15 04:00:00'}, {'conta_id': 190101, 'valor': 1059.39, 'dt_transacao': '2021-02-20 10:00:00'}, {'conta_id': 190101, 'valor': 1299.47, 'dt_transacao': '2021-02-01 03:00:00'}, {'conta_id': 190101, 'valor': 52.8, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190102, 'valor': 401.16, 'dt_transacao': '2021-01-18 02:00:00'}, {'conta_id': 190102, 'valor': 904.63, 'dt_transacao': '2021-02-26 02:00:00'}, {'conta_id': 190102, 'valor': 1621.22, 'dt_transacao': '2021-02-10 21:00:00'}, {'conta_id': 190102, 'valor': 695.5, 'dt_transacao': '2021-01-09 17:00:00'}, {'conta_id': 190103, 'valor': 1970.28, 'dt_transacao': '2021-01-18 07:00:00'}, {'conta_id': 190104, 'valor': 796.92, 'dt_transacao': '2021-03-03 16:00:00'}, {'conta_id': 190104, 'valor': 822.18, 'dt_transacao': '2021-02-22 18:00:00'}, {'conta_id': 190104, 'valor': 148.43, 'dt_transacao': '2021-02-10 20:00:00'}, {'conta_id': 190105, 'valor': 1821.51, 'dt_transacao': '2021-01-08 03:00:00'}, {'conta_id': 190105, 'valor': 1036.69, 'dt_transacao': '2021-02-08 05:00:00'}, {'conta_id': 190105, 'valor': 324.77, 'dt_transacao': '2021-04-01 07:00:00'}, {'conta_id': 190105, 'valor': 692.7, 'dt_transacao': '2021-03-08 09:00:00'}, {'conta_id': 190105, 'valor': 1692.02, 'dt_transacao': '2021-02-02 06:00:00'}, {'conta_id': 190106, 'valor': 1317.81, 'dt_transacao': '2021-03-19 10:00:00'}, {'conta_id': 190106, 'valor': 1685.22, 'dt_transacao': '2021-03-14 01:00:00'}, {'conta_id': 190107, 'valor': 1278.34, 'dt_transacao': '2021-03-05 20:00:00'}, {'conta_id': 190107, 'valor': 1091.61, 'dt_transacao': '2021-03-24 09:00:00'}, {'conta_id': 190107, 'valor': 966.41, 'dt_transacao': '2021-03-29 12:00:00'}, {'conta_id': 190107, 'valor': 595.57, 'dt_transacao': '2021-01-10 22:00:00'}, {'conta_id': 190107, 'valor': 120.77, 'dt_transacao': '2021-02-26 13:00:00'}, {'conta_id': 190108, 'valor': 1950.77, 'dt_transacao': '2021-01-17 11:00:00'}, {'conta_id': 190108, 'valor': 1030.45, 'dt_transacao': '2021-01-14 10:00:00'}, {'conta_id': 190108, 'valor': 484.23, 'dt_transacao': '2021-01-16 08:00:00'}, {'conta_id': 190109, 'valor': 1925.41, 'dt_transacao': '2021-01-21 10:00:00'}, {'conta_id': 190109, 'valor': 1867.57, 'dt_transacao': '2021-02-10 06:00:00'}, {'conta_id': 190110, 'valor': 1011.38, 'dt_transacao': '2021-01-31 06:00:00'}, {'conta_id': 190110, 'valor': 1167.1, 'dt_transacao': '2021-01-07 01:00:00'}, {'conta_id': 190110, 'valor': 563.43, 'dt_transacao': '2021-03-18 20:00:00'}, {'conta_id': 190110, 'valor': 1345.37, 'dt_transacao': '2021-02-06 17:00:00'}, {'conta_id': 190111, 'valor': 801.76, 'dt_transacao': '2021-03-14 17:00:00'}, {'conta_id': 190112, 'valor': 1950.19, 'dt_transacao': '2021-02-20 11:00:00'}, {'conta_id': 190112, 'valor': 1590.0, 'dt_transacao': '2021-02-13 17:00:00'}, {'conta_id': 190113, 'valor': 1427.47, 'dt_transacao': '2021-03-02 16:00:00'}, {'conta_id': 190113, 'valor': 914.52, 'dt_transacao': '2021-03-19 16:00:00'}, {'conta_id': 190114, 'valor': 600.28, 'dt_transacao': '2021-01-03 11:00:00'}, {'conta_id': 190114, 'valor': 663.15, 'dt_transacao': '2021-01-15 13:00:00'}, {'conta_id': 190114, 'valor': 1167.82, 'dt_transacao': '2021-03-30 20:00:00'}, {'conta_id': 190114, 'valor': 54.27, 'dt_transacao': '2021-03-02 08:00:00'}, {'conta_id': 190114, 'valor': 1934.17, 'dt_transacao': '2021-03-16 18:00:00'}, {'conta_id': 190115, 'valor': 1145.57, 'dt_transacao': '2021-03-31 03:00:00'}, {'conta_id': 190116, 'valor': 1004.14, 'dt_transacao': '2021-03-20 15:00:00'}, {'conta_id': 190116, 'valor': 1745.76, 'dt_transacao': '2021-01-08 22:00:00'}, {'conta_id': 190116, 'valor': 277.03, 'dt_transacao': '2021-01-27 17:00:00'}, {'conta_id': 190116, 'valor': 650.77, 'dt_transacao': '2021-03-26 15:00:00'}, {'conta_id': 190117, 'valor': 968.88, 'dt_transacao': '2021-02-01 08:00:00'}, {'conta_id': 190117, 'valor': 1116.62, 'dt_transacao': '2021-01-29 09:00:00'}, {'conta_id': 190117, 'valor': 1542.58, 'dt_transacao': '2021-04-01 06:00:00'}, {'conta_id': 190117, 'valor': 1380.79, 'dt_transacao': '2021-03-04 10:00:00'}, {'conta_id': 190117, 'valor': 960.13, 'dt_transacao': '2021-03-13 23:00:00'}, {'conta_id': 190118, 'valor': 581.61, 'dt_transacao': '2021-02-06 22:00:00'}, {'conta_id': 190118, 'valor': 158.98, 'dt_transacao': '2021-02-26 20:00:00'}, {'conta_id': 190119, 'valor': 1231.74, 'dt_transacao': '2021-03-17 07:00:00'}, {'conta_id': 190120, 'valor': 201.99, 'dt_transacao': '2021-01-18 00:00:00'}, {'conta_id': 190120, 'valor': 1929.07, 'dt_transacao': '2021-01-21 13:00:00'}, {'conta_id': 190120, 'valor': 1306.13, 'dt_transacao': '2021-03-02 15:00:00'}, {'conta_id': 190120, 'valor': 1301.43, 'dt_transacao': '2021-02-06 10:00:00'}, {'conta_id': 190121, 'valor': 1764.61, 'dt_transacao': '2021-01-23 01:00:00'}, {'conta_id': 190121, 'valor': 1994.35, 'dt_transacao': '2021-02-20 15:00:00'}, {'conta_id': 190121, 'valor': 373.44, 'dt_transacao': '2021-02-07 01:00:00'}, {'conta_id': 190121, 'valor': 19.53, 'dt_transacao': '2021-03-14 19:00:00'}, {'conta_id': 190122, 'valor': 386.43, 'dt_transacao': '2021-01-15 10:00:00'}, {'conta_id': 190122, 'valor': 325.83, 'dt_transacao': '2021-02-28 20:00:00'}, {'conta_id': 190122, 'valor': 515.27, 'dt_transacao': '2021-01-24 00:00:00'}, {'conta_id': 190123, 'valor': 802.27, 'dt_transacao': '2021-01-13 05:00:00'}, {'conta_id': 190124, 'valor': 1507.76, 'dt_transacao': '2021-02-12 09:00:00'}, {'conta_id': 190124, 'valor': 1165.34, 'dt_transacao': '2021-03-15 00:00:00'}, {'conta_id': 190124, 'valor': 523.84, 'dt_transacao': '2021-02-16 22:00:00'}, {'conta_id': 190124, 'valor': 473.37, 'dt_transacao': '2021-03-27 03:00:00'}, {'conta_id': 190125, 'valor': 1229.64, 'dt_transacao': '2021-01-29 04:00:00'}, {'conta_id': 190125, 'valor': 956.89, 'dt_transacao': '2021-02-28 23:00:00'}, {'conta_id': 190125, 'valor': 1977.69, 'dt_transacao': '2021-02-17 13:00:00'}, {'conta_id': 190126, 'valor': 1834.57, 'dt_transacao': '2021-01-15 01:00:00'}, {'conta_id': 190127, 'valor': 978.17, 'dt_transacao': '2021-03-07 14:00:00'}, {'conta_id': 190127, 'valor': 1617.58, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190127, 'valor': 1026.98, 'dt_transacao': '2021-02-28 18:00:00'}, {'conta_id': 190127, 'valor': 116.42, 'dt_transacao': '2021-03-01 21:00:00'}, {'conta_id': 190127, 'valor': 1614.02, 'dt_transacao': '2021-01-03 12:00:00'}, {'conta_id': 190128, 'valor': 1988.25, 'dt_transacao': '2021-01-09 15:00:00'}, {'conta_id': 190128, 'valor': 64.44, 'dt_transacao': '2021-02-22 05:00:00'}, {'conta_id': 190128, 'valor': 1538.18, 'dt_transacao': '2021-03-24 23:00:00'}, {'conta_id': 190128, 'valor': 1292.12, 'dt_transacao': '2021-02-23 11:00:00'}, {'conta_id': 190128, 'valor': 1985.72, 'dt_transacao': '2021-02-18 14:00:00'}, {'conta_id': 190129, 'valor': 1985.7, 'dt_transacao': '2021-02-20 16:00:00'}, {'conta_id': 190129, 'valor': 255.57, 'dt_transacao': '2021-01-29 00:00:00'}, {'conta_id': 190130, 'valor': 692.97, 'dt_transacao': '2021-02-05 06:00:00'}, {'conta_id': 190130, 'valor': 1874.88, 'dt_transacao': '2021-01-15 01:00:00'}, {'conta_id': 190130, 'valor': 1619.15, 'dt_transacao': '2021-02-23 19:00:00'}, {'conta_id': 190130, 'valor': 1532.34, 'dt_transacao': '2021-01-03 07:00:00'}, {'conta_id': 190131, 'valor': 805.13, 'dt_transacao': '2021-01-30 17:00:00'}, {'conta_id': 190132, 'valor': 604.23, 'dt_transacao': '2021-01-19 21:00:00'}, {'conta_id': 190132, 'valor': 1042.98, 'dt_transacao': '2021-02-22 09:00:00'}, {'conta_id': 190133, 'valor': 1883.23, 'dt_transacao': '2021-03-27 12:00:00'}, {'conta_id': 190133, 'valor': 1571.82, 'dt_transacao': '2021-02-10 22:00:00'}, {'conta_id': 190133, 'valor': 833.23, 'dt_transacao': '2021-01-20 09:00:00'}, {'conta_id': 190134, 'valor': 1125.21, 'dt_transacao': '2021-02-23 17:00:00'}, {'conta_id': 190135, 'valor': 1694.42, 'dt_transacao': '2021-01-12 17:00:00'}, {'conta_id': 190135, 'valor': 1449.44, 'dt_transacao': '2021-01-25 01:00:00'}, {'conta_id': 190135, 'valor': 537.65, 'dt_transacao': '2021-03-28 19:00:00'}, {'conta_id': 190135, 'valor': 1206.24, 'dt_transacao': '2021-01-10 06:00:00'}, {'conta_id': 190136, 'valor': 1924.19, 'dt_transacao': '2021-01-16 23:00:00'}, {'conta_id': 190136, 'valor': 1999.14, 'dt_transacao': '2021-03-03 07:00:00'}, {'conta_id': 190137, 'valor': 1067.77, 'dt_transacao': '2021-02-15 09:00:00'}, {'conta_id': 190137, 'valor': 523.86, 'dt_transacao': '2021-03-07 15:00:00'}, {'conta_id': 190137, 'valor': 932.54, 'dt_transacao': '2021-03-02 10:00:00'}, {'conta_id': 190137, 'valor': 1828.75, 'dt_transacao': '2021-02-17 10:00:00'}, {'conta_id': 190138, 'valor': 393.19, 'dt_transacao': '2021-01-30 12:00:00'}, {'conta_id': 190138, 'valor': 1140.01, 'dt_transacao': '2021-02-01 15:00:00'}, {'conta_id': 190139, 'valor': 339.36, 'dt_transacao': '2021-03-11 15:00:00'}, {'conta_id': 190139, 'valor': 368.91, 'dt_transacao': '2021-03-11 20:00:00'}, {'conta_id': 190139, 'valor': 1914.52, 'dt_transacao': '2021-03-09 01:00:00'}, {'conta_id': 190140, 'valor': 438.22, 'dt_transacao': '2021-02-28 11:00:00'}, {'conta_id': 190141, 'valor': 1444.66, 'dt_transacao': '2021-03-10 09:00:00'}, {'conta_id': 190141, 'valor': 35.27, 'dt_transacao': '2021-03-31 21:00:00'}, {'conta_id': 190141, 'valor': 861.72, 'dt_transacao': '2021-01-23 03:00:00'}, {'conta_id': 190141, 'valor': 1836.18, 'dt_transacao': '2021-03-09 04:00:00'}, {'conta_id': 190141, 'valor': 482.7, 'dt_transacao': '2021-03-21 16:00:00'}, {'conta_id': 190142, 'valor': 487.17, 'dt_transacao': '2021-03-31 07:00:00'}, {'conta_id': 190142, 'valor': 600.65, 'dt_transacao': '2021-01-11 20:00:00'}, {'conta_id': 190142, 'valor': 1983.36, 'dt_transacao': '2021-03-25 01:00:00'}, {'conta_id': 190142, 'valor': 187.91, 'dt_transacao': '2021-02-18 12:00:00'}, {'conta_id': 190142, 'valor': 1105.85, 'dt_transacao': '2021-01-08 20:00:00'}, {'conta_id': 190143, 'valor': 423.62, 'dt_transacao': '2021-03-30 18:00:00'}, {'conta_id': 190143, 'valor': 951.53, 'dt_transacao': '2021-01-06 02:00:00'}, {'conta_id': 190144, 'valor': 599.25, 'dt_transacao': '2021-03-14 07:00:00'}, {'conta_id': 190145, 'valor': 874.74, 'dt_transacao': '2021-01-10 10:00:00'}, {'conta_id': 190146, 'valor': 712.27, 'dt_transacao': '2021-02-18 13:00:00'}, {'conta_id': 190146, 'valor': 372.29, 'dt_transacao': '2021-03-16 21:00:00'}, {'conta_id': 190147, 'valor': 760.62, 'dt_transacao': '2021-03-23 04:00:00'}, {'conta_id': 190147, 'valor': 1479.57, 'dt_transacao': '2021-02-10 11:00:00'}, {'conta_id': 190148, 'valor': 175.51, 'dt_transacao': '2021-02-25 14:00:00'}, {'conta_id': 190148, 'valor': 1114.41, 'dt_transacao': '2021-03-07 13:00:00'}, {'conta_id': 190149, 'valor': 584.4, 'dt_transacao': '2021-02-19 02:00:00'}, {'conta_id': 190150, 'valor': 537.55, 'dt_transacao': '2021-01-19 02:00:00'}, {'conta_id': 190150, 'valor': 335.25, 'dt_transacao': '2021-02-05 13:00:00'}, {'conta_id': 190150, 'valor': 600.2, 'dt_transacao': '2021-01-10 11:00:00'}, {'conta_id': 190151, 'valor': 1897.7, 'dt_transacao': '2021-02-12 19:00:00'}, {'conta_id': 190152, 'valor': 1894.61, 'dt_transacao': '2021-01-26 15:00:00'}, {'conta_id': 190152, 'valor': 629.05, 'dt_transacao': '2021-02-20 10:00:00'}, {'conta_id': 190152, 'valor': 583.6, 'dt_transacao': '2021-03-30 20:00:00'}, {'conta_id': 190152, 'valor': 982.52, 'dt_transacao': '2021-03-15 07:00:00'}, {'conta_id': 190152, 'valor': 652.45, 'dt_transacao': '2021-02-05 12:00:00'}, {'conta_id': 190153, 'valor': 1595.57, 'dt_transacao': '2021-03-30 02:00:00'}, {'conta_id': 190153, 'valor': 1638.11, 'dt_transacao': '2021-02-22 21:00:00'}, {'conta_id': 190153, 'valor': 999.11, 'dt_transacao': '2021-03-23 09:00:00'}, {'conta_id': 190153, 'valor': 484.8, 'dt_transacao': '2021-03-26 04:00:00'}, {'conta_id': 190154, 'valor': 74.49, 'dt_transacao': '2021-02-16 22:00:00'}, {'conta_id': 190154, 'valor': 1791.11, 'dt_transacao': '2021-03-10 19:00:00'}, {'conta_id': 190154, 'valor': 1276.72, 'dt_transacao': '2021-01-11 03:00:00'}, {'conta_id': 190154, 'valor': 1538.14, 'dt_transacao': '2021-03-26 21:00:00'}, {'conta_id': 190154, 'valor': 710.66, 'dt_transacao': '2021-03-24 19:00:00'}, {'conta_id': 190155, 'valor': 1021.84, 'dt_transacao': '2021-03-08 17:00:00'}, {'conta_id': 190155, 'valor': 1161.46, 'dt_transacao': '2021-03-15 07:00:00'}, {'conta_id': 190156, 'valor': 209.99, 'dt_transacao': '2021-02-23 02:00:00'}, {'conta_id': 190156, 'valor': 233.02, 'dt_transacao': '2021-01-19 11:00:00'}, {'conta_id': 190156, 'valor': 624.47, 'dt_transacao': '2021-02-28 06:00:00'}, {'conta_id': 190156, 'valor': 1042.71, 'dt_transacao': '2021-02-14 15:00:00'}, {'conta_id': 190157, 'valor': 1698.51, 'dt_transacao': '2021-03-24 03:00:00'}, {'conta_id': 190158, 'valor': 1595.71, 'dt_transacao': '2021-03-22 05:00:00'}, {'conta_id': 190158, 'valor': 373.59, 'dt_transacao': '2021-03-26 13:00:00'}, {'conta_id': 190159, 'valor': 412.36, 'dt_transacao': '2021-04-01 02:00:00'}, {'conta_id': 190159, 'valor': 1155.12, 'dt_transacao': '2021-03-10 05:00:00'}, {'conta_id': 190160, 'valor': 777.89, 'dt_transacao': '2021-02-20 18:00:00'}, {'conta_id': 190160, 'valor': 229.35, 'dt_transacao': '2021-02-15 14:00:00'}, {'conta_id': 190160, 'valor': 1235.11, 'dt_transacao': '2021-03-28 22:00:00'}, {'conta_id': 190160, 'valor': 1601.43, 'dt_transacao': '2021-03-20 18:00:00'}, {'conta_id': 190160, 'valor': 170.42, 'dt_transacao': '2021-01-18 10:00:00'}, {'conta_id': 190161, 'valor': 1193.26, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190161, 'valor': 846.65, 'dt_transacao': '2021-03-04 20:00:00'}, {'conta_id': 190161, 'valor': 1075.22, 'dt_transacao': '2021-03-24 05:00:00'}, {'conta_id': 190161, 'valor': 1110.27, 'dt_transacao': '2021-03-04 09:00:00'}, {'conta_id': 190162, 'valor': 273.93, 'dt_transacao': '2021-02-19 17:00:00'}, {'conta_id': 190162, 'valor': 1011.37, 'dt_transacao': '2021-03-05 13:00:00'}, {'conta_id': 190162, 'valor': 1369.71, 'dt_transacao': '2021-02-23 22:00:00'}, {'conta_id': 190162, 'valor': 1679.94, 'dt_transacao': '2021-03-04 05:00:00'}, {'conta_id': 190163, 'valor': 1561.7, 'dt_transacao': '2021-03-14 23:00:00'}, {'conta_id': 190163, 'valor': 1545.32, 'dt_transacao': '2021-03-05 17:00:00'}, {'conta_id': 190164, 'valor': 437.87, 'dt_transacao': '2021-02-24 03:00:00'}, {'conta_id': 190164, 'valor': 1472.24, 'dt_transacao': '2021-03-25 07:00:00'}, {'conta_id': 190164, 'valor': 596.12, 'dt_transacao': '2021-01-05 14:00:00'}, {'conta_id': 190164, 'valor': 526.37, 'dt_transacao': '2021-01-12 14:00:00'}, {'conta_id': 190164, 'valor': 1730.72, 'dt_transacao': '2021-01-16 07:00:00'}, {'conta_id': 190165, 'valor': 1140.58, 'dt_transacao': '2021-03-10 08:00:00'}, {'conta_id': 190166, 'valor': 1837.62, 'dt_transacao': '2021-02-05 20:00:00'}, {'conta_id': 190166, 'valor': 254.65, 'dt_transacao': '2021-01-17 07:00:00'}, {'conta_id': 190166, 'valor': 293.34, 'dt_transacao': '2021-02-11 07:00:00'}, {'conta_id': 190166, 'valor': 1523.19, 'dt_transacao': '2021-03-28 12:00:00'}, {'conta_id': 190167, 'valor': 1031.71, 'dt_transacao': '2021-02-05 22:00:00'}, {'conta_id': 190167, 'valor': 1872.74, 'dt_transacao': '2021-02-17 14:00:00'}, {'conta_id': 190167, 'valor': 1900.56, 'dt_transacao': '2021-02-11 18:00:00'}, {'conta_id': 190167, 'valor': 5.94, 'dt_transacao': '2021-01-12 23:00:00'}, {'conta_id': 190168, 'valor': 537.81, 'dt_transacao': '2021-02-06 10:00:00'}, {'conta_id': 190168, 'valor': 1095.07, 'dt_transacao': '2021-01-17 18:00:00'}, {'conta_id': 190168, 'valor': 1704.99, 'dt_transacao': '2021-02-13 21:00:00'}, {'conta_id': 190168, 'valor': 1524.75, 'dt_transacao': '2021-01-06 03:00:00'}, {'conta_id': 190169, 'valor': 1991.39, 'dt_transacao': '2021-01-17 20:00:00'}, {'conta_id': 190169, 'valor': 604.43, 'dt_transacao': '2021-02-05 02:00:00'}, {'conta_id': 190170, 'valor': 1452.25, 'dt_transacao': '2021-01-30 13:00:00'}, {'conta_id': 190170, 'valor': 1311.31, 'dt_transacao': '2021-04-01 03:00:00'}, {'conta_id': 190170, 'valor': 3.17, 'dt_transacao': '2021-03-03 02:00:00'}, {'conta_id': 190171, 'valor': 955.92, 'dt_transacao': '2021-01-30 12:00:00'}, {'conta_id': 190171, 'valor': 179.57, 'dt_transacao': '2021-01-14 03:00:00'}, {'conta_id': 190172, 'valor': 1127.02, 'dt_transacao': '2021-03-20 20:00:00'}, {'conta_id': 190172, 'valor': 329.07, 'dt_transacao': '2021-02-14 23:00:00'}, {'conta_id': 190172, 'valor': 426.9, 'dt_transacao': '2021-01-20 13:00:00'}, {'conta_id': 190172, 'valor': 1232.54, 'dt_transacao': '2021-02-26 06:00:00'}, {'conta_id': 190172, 'valor': 174.29, 'dt_transacao': '2021-01-13 04:00:00'}, {'conta_id': 190173, 'valor': 497.09, 'dt_transacao': '2021-03-18 19:00:00'}, {'conta_id': 190174, 'valor': 1292.69, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190174, 'valor': 772.39, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190174, 'valor': 1276.53, 'dt_transacao': '2021-01-10 16:00:00'}, {'conta_id': 190175, 'valor': 1908.95, 'dt_transacao': '2021-01-10 00:00:00'}, {'conta_id': 190175, 'valor': 1175.14, 'dt_transacao': '2021-01-02 08:00:00'}, {'conta_id': 190175, 'valor': 432.27, 'dt_transacao': '2021-01-08 12:00:00'}, {'conta_id': 190175, 'valor': 1013.71, 'dt_transacao': '2021-03-22 22:00:00'}, {'conta_id': 190176, 'valor': 158.96, 'dt_transacao': '2021-01-27 04:00:00'}, {'conta_id': 190176, 'valor': 1073.24, 'dt_transacao': '2021-02-20 18:00:00'}, {'conta_id': 190176, 'valor': 1268.95, 'dt_transacao': '2021-03-25 21:00:00'}, {'conta_id': 190177, 'valor': 907.29, 'dt_transacao': '2021-03-19 01:00:00'}, {'conta_id': 190178, 'valor': 1906.08, 'dt_transacao': '2021-03-08 16:00:00'}, {'conta_id': 190178, 'valor': 1566.19, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190178, 'valor': 744.37, 'dt_transacao': '2021-02-04 23:00:00'}, {'conta_id': 190178, 'valor': 1772.94, 'dt_transacao': '2021-01-04 10:00:00'}, {'conta_id': 190179, 'valor': 720.89, 'dt_transacao': '2021-03-05 18:00:00'}, {'conta_id': 190179, 'valor': 334.6, 'dt_transacao': '2021-01-21 08:00:00'}, {'conta_id': 190179, 'valor': 1546.88, 'dt_transacao': '2021-03-15 21:00:00'}, {'conta_id': 190179, 'valor': 1445.83, 'dt_transacao': '2021-01-30 23:00:00'}, {'conta_id': 190179, 'valor': 1018.14, 'dt_transacao': '2021-01-06 00:00:00'}, {'conta_id': 190180, 'valor': 1606.43, 'dt_transacao': '2021-03-25 13:00:00'}, {'conta_id': 190180, 'valor': 449.81, 'dt_transacao': '2021-02-01 13:00:00'}, {'conta_id': 190180, 'valor': 1831.79, 'dt_transacao': '2021-02-10 08:00:00'}, {'conta_id': 190180, 'valor': 1652.17, 'dt_transacao': '2021-03-15 11:00:00'}, {'conta_id': 190180, 'valor': 239.68, 'dt_transacao': '2021-03-28 01:00:00'}, {'conta_id': 190181, 'valor': 173.68, 'dt_transacao': '2021-02-28 00:00:00'}, {'conta_id': 190181, 'valor': 738.47, 'dt_transacao': '2021-02-07 18:00:00'}, {'conta_id': 190181, 'valor': 1569.23, 'dt_transacao': '2021-03-21 07:00:00'}, {'conta_id': 190182, 'valor': 48.8, 'dt_transacao': '2021-03-11 11:00:00'}, {'conta_id': 190182, 'valor': 1272.25, 'dt_transacao': '2021-03-30 00:00:00'}, {'conta_id': 190182, 'valor': 1902.03, 'dt_transacao': '2021-01-01 21:00:00'}, {'conta_id': 190182, 'valor': 753.17, 'dt_transacao': '2021-02-09 03:00:00'}, {'conta_id': 190183, 'valor': 1842.63, 'dt_transacao': '2021-03-08 13:00:00'}, {'conta_id': 190184, 'valor': 1136.07, 'dt_transacao': '2021-02-25 21:00:00'}, {'conta_id': 190184, 'valor': 1928.47, 'dt_transacao': '2021-01-31 09:00:00'}, {'conta_id': 190184, 'valor': 522.72, 'dt_transacao': '2021-02-10 12:00:00'}, {'conta_id': 190184, 'valor': 1401.75, 'dt_transacao': '2021-02-11 10:00:00'}, {'conta_id': 190184, 'valor': 906.06, 'dt_transacao': '2021-01-30 02:00:00'}, {'conta_id': 190185, 'valor': 813.62, 'dt_transacao': '2021-03-10 11:00:00'}, {'conta_id': 190185, 'valor': 1929.3, 'dt_transacao': '2021-01-27 14:00:00'}, {'conta_id': 190185, 'valor': 617.31, 'dt_transacao': '2021-02-03 03:00:00'}, {'conta_id': 190186, 'valor': 1936.83, 'dt_transacao': '2021-02-20 16:00:00'}, {'conta_id': 190186, 'valor': 98.85, 'dt_transacao': '2021-02-08 08:00:00'}, {'conta_id': 190187, 'valor': 1607.99, 'dt_transacao': '2021-01-18 03:00:00'}, {'conta_id': 190188, 'valor': 1172.65, 'dt_transacao': '2021-01-10 00:00:00'}, {'conta_id': 190188, 'valor': 1994.33, 'dt_transacao': '2021-01-26 20:00:00'}, {'conta_id': 190188, 'valor': 887.42, 'dt_transacao': '2021-01-17 02:00:00'}, {'conta_id': 190188, 'valor': 1810.25, 'dt_transacao': '2021-02-12 03:00:00'}, {'conta_id': 190188, 'valor': 86.35, 'dt_transacao': '2021-03-01 01:00:00'}, {'conta_id': 190189, 'valor': 430.22, 'dt_transacao': '2021-01-24 08:00:00'}, {'conta_id': 190189, 'valor': 560.43, 'dt_transacao': '2021-01-20 01:00:00'}, {'conta_id': 190189, 'valor': 1228.6, 'dt_transacao': '2021-03-20 07:00:00'}, {'conta_id': 190190, 'valor': 839.05, 'dt_transacao': '2021-03-24 16:00:00'}, {'conta_id': 190190, 'valor': 499.05, 'dt_transacao': '2021-03-11 00:00:00'}, {'conta_id': 190191, 'valor': 53.1, 'dt_transacao': '2021-03-18 07:00:00'}, {'conta_id': 190191, 'valor': 59.58, 'dt_transacao': '2021-02-05 01:00:00'}, {'conta_id': 190192, 'valor': 1632.75, 'dt_transacao': '2021-01-08 07:00:00'}, {'conta_id': 190192, 'valor': 1151.81, 'dt_transacao': '2021-02-19 11:00:00'}, {'conta_id': 190192, 'valor': 1870.48, 'dt_transacao': '2021-01-23 05:00:00'}, {'conta_id': 190192, 'valor': 473.87, 'dt_transacao': '2021-02-10 23:00:00'}, {'conta_id': 190192, 'valor': 1395.56, 'dt_transacao': '2021-03-17 00:00:00'}, {'conta_id': 190193, 'valor': 181.42, 'dt_transacao': '2021-01-30 19:00:00'}, {'conta_id': 190194, 'valor': 1837.64, 'dt_transacao': '2021-02-11 01:00:00'}, {'conta_id': 190194, 'valor': 44.8, 'dt_transacao': '2021-03-17 22:00:00'}, {'conta_id': 190195, 'valor': 645.88, 'dt_transacao': '2021-01-21 23:00:00'}, {'conta_id': 190195, 'valor': 1897.44, 'dt_transacao': '2021-03-23 15:00:00'}, {'conta_id': 190195, 'valor': 1298.71, 'dt_transacao': '2021-01-23 22:00:00'}, {'conta_id': 190196, 'valor': 1457.93, 'dt_transacao': '2021-03-31 18:00:00'}, {'conta_id': 190196, 'valor': 783.81, 'dt_transacao': '2021-03-14 01:00:00'}, {'conta_id': 190197, 'valor': 694.27, 'dt_transacao': '2021-02-10 05:00:00'}, {'conta_id': 190197, 'valor': 1272.74, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190198, 'valor': 1574.25, 'dt_transacao': '2021-01-01 01:00:00'}, {'conta_id': 190198, 'valor': 772.49, 'dt_transacao': '2021-02-23 05:00:00'}, {'conta_id': 190198, 'valor': 1769.22, 'dt_transacao': '2021-03-05 12:00:00'}, {'conta_id': 190199, 'valor': 1598.59, 'dt_transacao': '2021-01-05 09:00:00'}, {'conta_id': 190200, 'valor': 1464.24, 'dt_transacao': '2021-03-12 02:00:00'}, {'conta_id': 190200, 'valor': 676.56, 'dt_transacao': '2021-03-20 15:00:00'}, {'conta_id': 190201, 'valor': 411.9, 'dt_transacao': '2021-01-25 13:00:00'}, {'conta_id': 190201, 'valor': 1394.7, 'dt_transacao': '2021-03-11 08:00:00'}, {'conta_id': 190201, 'valor': 1321.54, 'dt_transacao': '2021-02-08 07:00:00'}, {'conta_id': 190202, 'valor': 176.17, 'dt_transacao': '2021-03-26 20:00:00'}, {'conta_id': 190202, 'valor': 1200.83, 'dt_transacao': '2021-01-26 03:00:00'}, {'conta_id': 190202, 'valor': 52.45, 'dt_transacao': '2021-01-22 20:00:00'}, {'conta_id': 190202, 'valor': 584.33, 'dt_transacao': '2021-01-11 15:00:00'}, {'conta_id': 190203, 'valor': 742.67, 'dt_transacao': '2021-02-18 11:00:00'}, {'conta_id': 190203, 'valor': 373.98, 'dt_transacao': '2021-01-06 08:00:00'}, {'conta_id': 190204, 'valor': 1336.61, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190205, 'valor': 598.41, 'dt_transacao': '2021-02-20 22:00:00'}, {'conta_id': 190205, 'valor': 70.42, 'dt_transacao': '2021-03-15 16:00:00'}, {'conta_id': 190205, 'valor': 1601.1, 'dt_transacao': '2021-03-28 19:00:00'}, {'conta_id': 190205, 'valor': 295.75, 'dt_transacao': '2021-02-27 05:00:00'}, {'conta_id': 190206, 'valor': 1994.54, 'dt_transacao': '2021-03-22 04:00:00'}, {'conta_id': 190206, 'valor': 1668.38, 'dt_transacao': '2021-02-03 10:00:00'}, {'conta_id': 190207, 'valor': 1573.31, 'dt_transacao': '2021-03-21 18:00:00'}, {'conta_id': 190208, 'valor': 1559.31, 'dt_transacao': '2021-02-06 17:00:00'}, {'conta_id': 190208, 'valor': 1760.39, 'dt_transacao': '2021-02-22 08:00:00'}, {'conta_id': 190208, 'valor': 766.7, 'dt_transacao': '2021-03-04 18:00:00'}, {'conta_id': 190208, 'valor': 1218.32, 'dt_transacao': '2021-01-05 05:00:00'}, {'conta_id': 190208, 'valor': 551.55, 'dt_transacao': '2021-01-19 19:00:00'}, {'conta_id': 190209, 'valor': 1794.17, 'dt_transacao': '2021-01-03 10:00:00'}, {'conta_id': 190209, 'valor': 1634.95, 'dt_transacao': '2021-02-08 09:00:00'}, {'conta_id': 190209, 'valor': 561.41, 'dt_transacao': '2021-03-04 22:00:00'}, {'conta_id': 190210, 'valor': 1291.46, 'dt_transacao': '2021-01-25 14:00:00'}, {'conta_id': 190210, 'valor': 423.57, 'dt_transacao': '2021-02-22 23:00:00'}, {'conta_id': 190210, 'valor': 969.44, 'dt_transacao': '2021-03-08 11:00:00'}, {'conta_id': 190210, 'valor': 1642.21, 'dt_transacao': '2021-03-07 05:00:00'}, {'conta_id': 190210, 'valor': 132.17, 'dt_transacao': '2021-04-01 16:00:00'}, {'conta_id': 190211, 'valor': 913.44, 'dt_transacao': '2021-02-16 22:00:00'}, {'conta_id': 190212, 'valor': 745.56, 'dt_transacao': '2021-03-18 14:00:00'}, {'conta_id': 190212, 'valor': 355.99, 'dt_transacao': '2021-03-02 18:00:00'}, {'conta_id': 190213, 'valor': 100.25, 'dt_transacao': '2021-03-15 06:00:00'}, {'conta_id': 190213, 'valor': 1714.37, 'dt_transacao': '2021-03-24 07:00:00'}, {'conta_id': 190213, 'valor': 1355.71, 'dt_transacao': '2021-02-04 20:00:00'}, {'conta_id': 190213, 'valor': 819.6, 'dt_transacao': '2021-01-03 21:00:00'}, {'conta_id': 190214, 'valor': 1610.37, 'dt_transacao': '2021-02-09 22:00:00'}, {'conta_id': 190214, 'valor': 1934.08, 'dt_transacao': '2021-03-24 08:00:00'}, {'conta_id': 190214, 'valor': 848.19, 'dt_transacao': '2021-02-18 11:00:00'}, {'conta_id': 190214, 'valor': 918.54, 'dt_transacao': '2021-01-28 14:00:00'}, {'conta_id': 190214, 'valor': 604.99, 'dt_transacao': '2021-03-08 19:00:00'}, {'conta_id': 190215, 'valor': 964.0, 'dt_transacao': '2021-02-13 08:00:00'}, {'conta_id': 190216, 'valor': 808.79, 'dt_transacao': '2021-02-10 21:00:00'}, {'conta_id': 190216, 'valor': 280.72, 'dt_transacao': '2021-01-04 20:00:00'}, {'conta_id': 190216, 'valor': 995.25, 'dt_transacao': '2021-02-07 08:00:00'}, {'conta_id': 190216, 'valor': 1813.92, 'dt_transacao': '2021-02-21 12:00:00'}, {'conta_id': 190216, 'valor': 1473.73, 'dt_transacao': '2021-03-16 22:00:00'}, {'conta_id': 190217, 'valor': 1681.44, 'dt_transacao': '2021-03-13 05:00:00'}, {'conta_id': 190217, 'valor': 588.27, 'dt_transacao': '2021-03-02 04:00:00'}, {'conta_id': 190218, 'valor': 1576.39, 'dt_transacao': '2021-02-12 11:00:00'}, {'conta_id': 190218, 'valor': 521.86, 'dt_transacao': '2021-02-07 14:00:00'}, {'conta_id': 190218, 'valor': 1702.58, 'dt_transacao': '2021-03-02 01:00:00'}, {'conta_id': 190218, 'valor': 1732.92, 'dt_transacao': '2021-03-16 18:00:00'}, {'conta_id': 190219, 'valor': 109.07, 'dt_transacao': '2021-01-20 04:00:00'}, {'conta_id': 190219, 'valor': 1984.7, 'dt_transacao': '2021-02-13 13:00:00'}, {'conta_id': 190219, 'valor': 1974.4, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190219, 'valor': 633.45, 'dt_transacao': '2021-01-24 02:00:00'}, {'conta_id': 190220, 'valor': 419.56, 'dt_transacao': '2021-01-22 18:00:00'}, {'conta_id': 190220, 'valor': 1529.2, 'dt_transacao': '2021-03-04 02:00:00'}, {'conta_id': 190220, 'valor': 259.68, 'dt_transacao': '2021-02-25 20:00:00'}, {'conta_id': 190221, 'valor': 1900.19, 'dt_transacao': '2021-02-11 16:00:00'}, {'conta_id': 190221, 'valor': 381.96, 'dt_transacao': '2021-03-28 00:00:00'}, {'conta_id': 190221, 'valor': 1253.75, 'dt_transacao': '2021-02-01 07:00:00'}, {'conta_id': 190222, 'valor': 597.11, 'dt_transacao': '2021-02-07 14:00:00'}, {'conta_id': 190223, 'valor': 1896.22, 'dt_transacao': '2021-02-21 00:00:00'}, {'conta_id': 190223, 'valor': 457.61, 'dt_transacao': '2021-01-17 06:00:00'}, {'conta_id': 190224, 'valor': 1246.65, 'dt_transacao': '2021-03-18 01:00:00'}, {'conta_id': 190224, 'valor': 1324.28, 'dt_transacao': '2021-03-25 00:00:00'}, {'conta_id': 190224, 'valor': 97.42, 'dt_transacao': '2021-01-15 13:00:00'}, {'conta_id': 190224, 'valor': 889.18, 'dt_transacao': '2021-01-16 17:00:00'}, {'conta_id': 190224, 'valor': 506.22, 'dt_transacao': '2021-03-31 04:00:00'}, {'conta_id': 190225, 'valor': 1661.47, 'dt_transacao': '2021-01-14 16:00:00'}, {'conta_id': 190225, 'valor': 1437.81, 'dt_transacao': '2021-01-16 09:00:00'}, {'conta_id': 190225, 'valor': 228.91, 'dt_transacao': '2021-01-14 15:00:00'}, {'conta_id': 190225, 'valor': 398.09, 'dt_transacao': '2021-01-26 08:00:00'}, {'conta_id': 190226, 'valor': 1376.72, 'dt_transacao': '2021-03-03 10:00:00'}, {'conta_id': 190226, 'valor': 469.78, 'dt_transacao': '2021-01-02 21:00:00'}, {'conta_id': 190227, 'valor': 914.05, 'dt_transacao': '2021-02-07 08:00:00'}, {'conta_id': 190227, 'valor': 929.09, 'dt_transacao': '2021-01-13 05:00:00'}, {'conta_id': 190228, 'valor': 1704.15, 'dt_transacao': '2021-03-26 20:00:00'}, {'conta_id': 190229, 'valor': 366.56, 'dt_transacao': '2021-02-21 12:00:00'}, {'conta_id': 190229, 'valor': 1924.66, 'dt_transacao': '2021-02-04 12:00:00'}, {'conta_id': 190229, 'valor': 1380.11, 'dt_transacao': '2021-03-03 13:00:00'}, {'conta_id': 190230, 'valor': 965.87, 'dt_transacao': '2021-03-14 21:00:00'}, {'conta_id': 190230, 'valor': 876.89, 'dt_transacao': '2021-03-20 12:00:00'}, {'conta_id': 190230, 'valor': 1558.55, 'dt_transacao': '2021-03-20 00:00:00'}, {'conta_id': 190230, 'valor': 1725.29, 'dt_transacao': '2021-01-17 15:00:00'}, {'conta_id': 190230, 'valor': 1840.88, 'dt_transacao': '2021-01-15 13:00:00'}, {'conta_id': 190231, 'valor': 1038.68, 'dt_transacao': '2021-03-30 22:00:00'}, {'conta_id': 190231, 'valor': 1236.86, 'dt_transacao': '2021-02-25 21:00:00'}, {'conta_id': 190231, 'valor': 1453.58, 'dt_transacao': '2021-03-03 11:00:00'}, {'conta_id': 190232, 'valor': 150.66, 'dt_transacao': '2021-03-04 10:00:00'}, {'conta_id': 190232, 'valor': 482.54, 'dt_transacao': '2021-02-05 01:00:00'}, {'conta_id': 190233, 'valor': 896.75, 'dt_transacao': '2021-03-05 14:00:00'}, {'conta_id': 190233, 'valor': 632.81, 'dt_transacao': '2021-03-05 23:00:00'}, {'conta_id': 190233, 'valor': 1307.09, 'dt_transacao': '2021-01-14 12:00:00'}, {'conta_id': 190233, 'valor': 825.26, 'dt_transacao': '2021-03-13 17:00:00'}, {'conta_id': 190233, 'valor': 1.34, 'dt_transacao': '2021-01-12 19:00:00'}, {'conta_id': 190234, 'valor': 664.61, 'dt_transacao': '2021-03-21 19:00:00'}, {'conta_id': 190234, 'valor': 946.79, 'dt_transacao': '2021-03-26 11:00:00'}, {'conta_id': 190234, 'valor': 1432.11, 'dt_transacao': '2021-01-15 03:00:00'}, {'conta_id': 190234, 'valor': 1518.88, 'dt_transacao': '2021-01-29 00:00:00'}, {'conta_id': 190235, 'valor': 281.76, 'dt_transacao': '2021-02-22 03:00:00'}, {'conta_id': 190235, 'valor': 509.32, 'dt_transacao': '2021-01-26 06:00:00'}, {'conta_id': 190235, 'valor': 242.37, 'dt_transacao': '2021-02-20 06:00:00'}, {'conta_id': 190235, 'valor': 914.63, 'dt_transacao': '2021-01-25 22:00:00'}, {'conta_id': 190236, 'valor': 1912.42, 'dt_transacao': '2021-01-18 15:00:00'}, {'conta_id': 190236, 'valor': 9.67, 'dt_transacao': '2021-01-07 17:00:00'}, {'conta_id': 190236, 'valor': 1870.68, 'dt_transacao': '2021-03-16 15:00:00'}, {'conta_id': 190236, 'valor': 1848.82, 'dt_transacao': '2021-01-23 22:00:00'}, {'conta_id': 190236, 'valor': 1157.53, 'dt_transacao': '2021-01-17 03:00:00'}, {'conta_id': 190237, 'valor': 1941.01, 'dt_transacao': '2021-02-27 04:00:00'}, {'conta_id': 190237, 'valor': 277.22, 'dt_transacao': '2021-03-19 22:00:00'}, {'conta_id': 190237, 'valor': 1772.99, 'dt_transacao': '2021-03-07 09:00:00'}, {'conta_id': 190238, 'valor': 1008.28, 'dt_transacao': '2021-03-30 20:00:00'}, {'conta_id': 190238, 'valor': 645.44, 'dt_transacao': '2021-01-15 11:00:00'}, {'conta_id': 190239, 'valor': 291.51, 'dt_transacao': '2021-03-27 01:00:00'}, {'conta_id': 190239, 'valor': 400.6, 'dt_transacao': '2021-01-21 09:00:00'}, {'conta_id': 190239, 'valor': 1367.1, 'dt_transacao': '2021-01-19 01:00:00'}, {'conta_id': 190239, 'valor': 1381.5, 'dt_transacao': '2021-02-07 06:00:00'}, {'conta_id': 190240, 'valor': 1568.83, 'dt_transacao': '2021-03-31 09:00:00'}, {'conta_id': 190240, 'valor': 1386.18, 'dt_transacao': '2021-01-07 12:00:00'}, {'conta_id': 190240, 'valor': 775.72, 'dt_transacao': '2021-03-29 10:00:00'}, {'conta_id': 190240, 'valor': 1931.7, 'dt_transacao': '2021-01-07 00:00:00'}, {'conta_id': 190240, 'valor': 1487.6, 'dt_transacao': '2021-01-26 19:00:00'}, {'conta_id': 190241, 'valor': 374.77, 'dt_transacao': '2021-01-23 23:00:00'}, {'conta_id': 190242, 'valor': 1168.39, 'dt_transacao': '2021-02-02 00:00:00'}, {'conta_id': 190242, 'valor': 661.8, 'dt_transacao': '2021-04-01 04:00:00'}, {'conta_id': 190243, 'valor': 1568.28, 'dt_transacao': '2021-02-10 16:00:00'}, {'conta_id': 190244, 'valor': 443.17, 'dt_transacao': '2021-01-28 21:00:00'}, {'conta_id': 190244, 'valor': 664.44, 'dt_transacao': '2021-02-16 09:00:00'}, {'conta_id': 190244, 'valor': 437.67, 'dt_transacao': '2021-03-21 16:00:00'}, {'conta_id': 190245, 'valor': 1365.42, 'dt_transacao': '2021-02-11 12:00:00'}, {'conta_id': 190246, 'valor': 593.57, 'dt_transacao': '2021-03-25 06:00:00'}, {'conta_id': 190247, 'valor': 1730.04, 'dt_transacao': '2021-02-24 03:00:00'}, {'conta_id': 190247, 'valor': 411.95, 'dt_transacao': '2021-03-31 23:00:00'}, {'conta_id': 190248, 'valor': 109.48, 'dt_transacao': '2021-02-02 17:00:00'}, {'conta_id': 190249, 'valor': 34.77, 'dt_transacao': '2021-03-28 03:00:00'}, {'conta_id': 190249, 'valor': 1070.13, 'dt_transacao': '2021-03-24 23:00:00'}, {'conta_id': 190249, 'valor': 1237.08, 'dt_transacao': '2021-03-16 01:00:00'}, {'conta_id': 190250, 'valor': 763.45, 'dt_transacao': '2021-03-26 00:00:00'}, {'conta_id': 190250, 'valor': 948.5, 'dt_transacao': '2021-03-10 07:00:00'}, {'conta_id': 190250, 'valor': 273.63, 'dt_transacao': '2021-02-20 17:00:00'}, {'conta_id': 190250, 'valor': 590.66, 'dt_transacao': '2021-03-23 04:00:00'}, {'conta_id': 190251, 'valor': 1145.86, 'dt_transacao': '2021-02-20 17:00:00'}, {'conta_id': 190252, 'valor': 368.8, 'dt_transacao': '2021-03-20 08:00:00'}, {'conta_id': 190252, 'valor': 389.46, 'dt_transacao': '2021-01-24 21:00:00'}, {'conta_id': 190253, 'valor': 1451.45, 'dt_transacao': '2021-03-04 10:00:00'}, {'conta_id': 190253, 'valor': 714.92, 'dt_transacao': '2021-03-27 20:00:00'}, {'conta_id': 190254, 'valor': 1233.07, 'dt_transacao': '2021-02-09 22:00:00'}, {'conta_id': 190254, 'valor': 1168.76, 'dt_transacao': '2021-03-14 18:00:00'}, {'conta_id': 190254, 'valor': 729.62, 'dt_transacao': '2021-02-10 13:00:00'}, {'conta_id': 190255, 'valor': 1359.48, 'dt_transacao': '2021-02-12 15:00:00'}, {'conta_id': 190255, 'valor': 926.23, 'dt_transacao': '2021-02-15 09:00:00'}, {'conta_id': 190256, 'valor': 393.67, 'dt_transacao': '2021-03-22 02:00:00'}, {'conta_id': 190256, 'valor': 1455.12, 'dt_transacao': '2021-01-03 16:00:00'}, {'conta_id': 190256, 'valor': 1263.62, 'dt_transacao': '2021-03-16 18:00:00'}, {'conta_id': 190256, 'valor': 810.86, 'dt_transacao': '2021-03-17 06:00:00'}, {'conta_id': 190256, 'valor': 1812.7, 'dt_transacao': '2021-03-12 08:00:00'}, {'conta_id': 190257, 'valor': 1002.75, 'dt_transacao': '2021-01-12 12:00:00'}, {'conta_id': 190257, 'valor': 728.37, 'dt_transacao': '2021-01-02 21:00:00'}, {'conta_id': 190257, 'valor': 478.52, 'dt_transacao': '2021-03-20 21:00:00'}, {'conta_id': 190257, 'valor': 332.52, 'dt_transacao': '2021-03-07 04:00:00'}, {'conta_id': 190258, 'valor': 438.42, 'dt_transacao': '2021-03-04 07:00:00'}, {'conta_id': 190259, 'valor': 1835.89, 'dt_transacao': '2021-01-06 01:00:00'}, {'conta_id': 190259, 'valor': 1961.2, 'dt_transacao': '2021-02-11 03:00:00'}, {'conta_id': 190259, 'valor': 1407.77, 'dt_transacao': '2021-03-08 20:00:00'}, {'conta_id': 190259, 'valor': 1659.46, 'dt_transacao': '2021-02-03 18:00:00'}, {'conta_id': 190260, 'valor': 1033.23, 'dt_transacao': '2021-02-15 11:00:00'}, {'conta_id': 190260, 'valor': 974.24, 'dt_transacao': '2021-03-03 19:00:00'}, {'conta_id': 190261, 'valor': 374.62, 'dt_transacao': '2021-02-16 19:00:00'}, {'conta_id': 190261, 'valor': 1434.89, 'dt_transacao': '2021-01-16 21:00:00'}, {'conta_id': 190262, 'valor': 323.33, 'dt_transacao': '2021-02-22 00:00:00'}, {'conta_id': 190262, 'valor': 488.1, 'dt_transacao': '2021-01-17 18:00:00'}, {'conta_id': 190262, 'valor': 1185.1, 'dt_transacao': '2021-01-18 20:00:00'}, {'conta_id': 190262, 'valor': 1025.77, 'dt_transacao': '2021-03-15 04:00:00'}, {'conta_id': 190262, 'valor': 112.77, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190263, 'valor': 1213.45, 'dt_transacao': '2021-03-02 15:00:00'}, {'conta_id': 190263, 'valor': 341.96, 'dt_transacao': '2021-02-22 19:00:00'}, {'conta_id': 190263, 'valor': 286.28, 'dt_transacao': '2021-03-16 05:00:00'}, {'conta_id': 190263, 'valor': 1246.8, 'dt_transacao': '2021-03-07 08:00:00'}, {'conta_id': 190264, 'valor': 34.11, 'dt_transacao': '2021-03-08 16:00:00'}, {'conta_id': 190265, 'valor': 1319.13, 'dt_transacao': '2021-03-19 17:00:00'}, {'conta_id': 190265, 'valor': 1655.22, 'dt_transacao': '2021-03-28 20:00:00'}, {'conta_id': 190265, 'valor': 863.56, 'dt_transacao': '2021-02-23 11:00:00'}, {'conta_id': 190265, 'valor': 1749.77, 'dt_transacao': '2021-03-31 16:00:00'}, {'conta_id': 190266, 'valor': 954.81, 'dt_transacao': '2021-01-30 02:00:00'}, {'conta_id': 190266, 'valor': 567.25, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190266, 'valor': 516.35, 'dt_transacao': '2021-01-01 22:00:00'}, {'conta_id': 190267, 'valor': 214.06, 'dt_transacao': '2021-02-24 20:00:00'}, {'conta_id': 190267, 'valor': 1171.15, 'dt_transacao': '2021-02-03 17:00:00'}, {'conta_id': 190267, 'valor': 847.7, 'dt_transacao': '2021-02-16 19:00:00'}, {'conta_id': 190268, 'valor': 1110.27, 'dt_transacao': '2021-02-02 09:00:00'}, {'conta_id': 190268, 'valor': 1155.72, 'dt_transacao': '2021-04-01 12:00:00'}, {'conta_id': 190268, 'valor': 1471.83, 'dt_transacao': '2021-03-26 03:00:00'}, {'conta_id': 190268, 'valor': 1495.88, 'dt_transacao': '2021-02-27 22:00:00'}, {'conta_id': 190268, 'valor': 1898.54, 'dt_transacao': '2021-01-13 06:00:00'}, {'conta_id': 190269, 'valor': 943.87, 'dt_transacao': '2021-03-18 13:00:00'}, {'conta_id': 190269, 'valor': 177.19, 'dt_transacao': '2021-02-03 07:00:00'}, {'conta_id': 190270, 'valor': 1280.94, 'dt_transacao': '2021-02-26 00:00:00'}, {'conta_id': 190270, 'valor': 404.21, 'dt_transacao': '2021-03-26 16:00:00'}, {'conta_id': 190271, 'valor': 479.27, 'dt_transacao': '2021-01-01 05:00:00'}, {'conta_id': 190271, 'valor': 174.6, 'dt_transacao': '2021-02-05 12:00:00'}, {'conta_id': 190271, 'valor': 706.9, 'dt_transacao': '2021-02-20 12:00:00'}, {'conta_id': 190272, 'valor': 1830.1, 'dt_transacao': '2021-02-14 11:00:00'}, {'conta_id': 190272, 'valor': 1926.61, 'dt_transacao': '2021-02-05 07:00:00'}, {'conta_id': 190272, 'valor': 1213.12, 'dt_transacao': '2021-01-25 14:00:00'}, {'conta_id': 190272, 'valor': 1859.5, 'dt_transacao': '2021-03-15 00:00:00'}, {'conta_id': 190272, 'valor': 522.95, 'dt_transacao': '2021-02-22 16:00:00'}, {'conta_id': 190273, 'valor': 1126.77, 'dt_transacao': '2021-02-01 10:00:00'}, {'conta_id': 190273, 'valor': 1922.52, 'dt_transacao': '2021-02-28 21:00:00'}, {'conta_id': 190273, 'valor': 921.98, 'dt_transacao': '2021-03-17 00:00:00'}, {'conta_id': 190273, 'valor': 964.46, 'dt_transacao': '2021-02-03 02:00:00'}, {'conta_id': 190274, 'valor': 575.29, 'dt_transacao': '2021-02-21 21:00:00'}, {'conta_id': 190274, 'valor': 1497.83, 'dt_transacao': '2021-02-13 02:00:00'}, {'conta_id': 190275, 'valor': 998.5, 'dt_transacao': '2021-01-26 07:00:00'}, {'conta_id': 190275, 'valor': 1471.12, 'dt_transacao': '2021-02-03 06:00:00'}, {'conta_id': 190276, 'valor': 1228.04, 'dt_transacao': '2021-03-11 01:00:00'}, {'conta_id': 190276, 'valor': 1696.17, 'dt_transacao': '2021-02-17 19:00:00'}, {'conta_id': 190276, 'valor': 631.52, 'dt_transacao': '2021-03-15 08:00:00'}, {'conta_id': 190276, 'valor': 982.12, 'dt_transacao': '2021-03-07 08:00:00'}, {'conta_id': 190277, 'valor': 695.23, 'dt_transacao': '2021-03-29 05:00:00'}, {'conta_id': 190277, 'valor': 914.93, 'dt_transacao': '2021-01-16 22:00:00'}, {'conta_id': 190277, 'valor': 1036.89, 'dt_transacao': '2021-02-10 22:00:00'}, {'conta_id': 190277, 'valor': 17.83, 'dt_transacao': '2021-03-28 06:00:00'}, {'conta_id': 190277, 'valor': 759.48, 'dt_transacao': '2021-01-13 10:00:00'}, {'conta_id': 190278, 'valor': 686.85, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190278, 'valor': 848.46, 'dt_transacao': '2021-02-23 22:00:00'}, {'conta_id': 190278, 'valor': 1504.64, 'dt_transacao': '2021-02-16 12:00:00'}, {'conta_id': 190278, 'valor': 559.98, 'dt_transacao': '2021-01-24 18:00:00'}, {'conta_id': 190278, 'valor': 263.15, 'dt_transacao': '2021-03-15 21:00:00'}, {'conta_id': 190279, 'valor': 1600.27, 'dt_transacao': '2021-01-20 15:00:00'}, {'conta_id': 190279, 'valor': 1611.53, 'dt_transacao': '2021-01-23 13:00:00'}, {'conta_id': 190280, 'valor': 1381.4, 'dt_transacao': '2021-01-08 06:00:00'}, {'conta_id': 190281, 'valor': 1115.43, 'dt_transacao': '2021-02-02 11:00:00'}, {'conta_id': 190281, 'valor': 40.65, 'dt_transacao': '2021-01-16 11:00:00'}, {'conta_id': 190281, 'valor': 1734.87, 'dt_transacao': '2021-03-19 06:00:00'}, {'conta_id': 190281, 'valor': 1154.13, 'dt_transacao': '2021-03-25 20:00:00'}, {'conta_id': 190281, 'valor': 1077.34, 'dt_transacao': '2021-02-06 11:00:00'}, {'conta_id': 190282, 'valor': 1281.52, 'dt_transacao': '2021-02-13 13:00:00'}, {'conta_id': 190282, 'valor': 1262.92, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190283, 'valor': 1211.16, 'dt_transacao': '2021-02-20 08:00:00'}, {'conta_id': 190283, 'valor': 1775.5, 'dt_transacao': '2021-02-16 23:00:00'}, {'conta_id': 190283, 'valor': 1174.25, 'dt_transacao': '2021-02-27 13:00:00'}, {'conta_id': 190284, 'valor': 794.69, 'dt_transacao': '2021-01-13 02:00:00'}, {'conta_id': 190284, 'valor': 1873.65, 'dt_transacao': '2021-03-02 17:00:00'}, {'conta_id': 190285, 'valor': 619.43, 'dt_transacao': '2021-02-01 23:00:00'}, {'conta_id': 190285, 'valor': 569.07, 'dt_transacao': '2021-03-22 04:00:00'}, {'conta_id': 190285, 'valor': 263.42, 'dt_transacao': '2021-01-22 00:00:00'}, {'conta_id': 190285, 'valor': 71.27, 'dt_transacao': '2021-02-15 10:00:00'}, {'conta_id': 190286, 'valor': 1279.93, 'dt_transacao': '2021-03-10 09:00:00'}, {'conta_id': 190286, 'valor': 1817.94, 'dt_transacao': '2021-03-03 17:00:00'}, {'conta_id': 190286, 'valor': 455.03, 'dt_transacao': '2021-01-12 14:00:00'}, {'conta_id': 190286, 'valor': 616.43, 'dt_transacao': '2021-02-22 08:00:00'}, {'conta_id': 190286, 'valor': 312.28, 'dt_transacao': '2021-01-02 00:00:00'}, {'conta_id': 190287, 'valor': 19.26, 'dt_transacao': '2021-02-25 09:00:00'}, {'conta_id': 190287, 'valor': 1194.41, 'dt_transacao': '2021-01-01 17:00:00'}, {'conta_id': 190287, 'valor': 747.91, 'dt_transacao': '2021-02-22 17:00:00'}, {'conta_id': 190288, 'valor': 159.34, 'dt_transacao': '2021-03-22 07:00:00'}, {'conta_id': 190288, 'valor': 1033.7, 'dt_transacao': '2021-03-20 22:00:00'}, {'conta_id': 190288, 'valor': 1369.6, 'dt_transacao': '2021-02-06 20:00:00'}, {'conta_id': 190289, 'valor': 1274.15, 'dt_transacao': '2021-03-21 14:00:00'}, {'conta_id': 190290, 'valor': 1944.69, 'dt_transacao': '2021-02-06 08:00:00'}, {'conta_id': 190290, 'valor': 250.95, 'dt_transacao': '2021-02-26 23:00:00'}, {'conta_id': 190291, 'valor': 623.84, 'dt_transacao': '2021-03-05 11:00:00'}, {'conta_id': 190291, 'valor': 927.91, 'dt_transacao': '2021-01-05 10:00:00'}, {'conta_id': 190291, 'valor': 161.37, 'dt_transacao': '2021-01-19 01:00:00'}, {'conta_id': 190291, 'valor': 533.74, 'dt_transacao': '2021-02-20 02:00:00'}, {'conta_id': 190292, 'valor': 770.21, 'dt_transacao': '2021-02-26 07:00:00'}, {'conta_id': 190292, 'valor': 1475.5, 'dt_transacao': '2021-01-05 06:00:00'}, {'conta_id': 190293, 'valor': 1489.7, 'dt_transacao': '2021-03-12 16:00:00'}, {'conta_id': 190294, 'valor': 472.7, 'dt_transacao': '2021-02-25 10:00:00'}, {'conta_id': 190295, 'valor': 1567.17, 'dt_transacao': '2021-03-02 22:00:00'}, {'conta_id': 190295, 'valor': 1192.83, 'dt_transacao': '2021-02-17 02:00:00'}, {'conta_id': 190295, 'valor': 498.2, 'dt_transacao': '2021-03-05 02:00:00'}, {'conta_id': 190295, 'valor': 1372.2, 'dt_transacao': '2021-02-13 20:00:00'}, {'conta_id': 190295, 'valor': 121.04, 'dt_transacao': '2021-02-15 20:00:00'}, {'conta_id': 190296, 'valor': 43.68, 'dt_transacao': '2021-02-18 11:00:00'}, {'conta_id': 190296, 'valor': 1449.07, 'dt_transacao': '2021-02-02 01:00:00'}, {'conta_id': 190297, 'valor': 551.37, 'dt_transacao': '2021-03-18 17:00:00'}, {'conta_id': 190297, 'valor': 321.53, 'dt_transacao': '2021-03-09 04:00:00'}, {'conta_id': 190297, 'valor': 899.32, 'dt_transacao': '2021-02-03 17:00:00'}, {'conta_id': 190297, 'valor': 734.86, 'dt_transacao': '2021-01-01 20:00:00'}, {'conta_id': 190298, 'valor': 1316.92, 'dt_transacao': '2021-03-10 12:00:00'}, {'conta_id': 190299, 'valor': 193.42, 'dt_transacao': '2021-03-26 02:00:00'}, {'conta_id': 190300, 'valor': 64.6, 'dt_transacao': '2021-03-24 12:00:00'}, {'conta_id': 190301, 'valor': 1856.88, 'dt_transacao': '2021-01-11 14:00:00'}, {'conta_id': 190301, 'valor': 1602.24, 'dt_transacao': '2021-01-08 11:00:00'}, {'conta_id': 190301, 'valor': 1116.48, 'dt_transacao': '2021-02-23 11:00:00'}, {'conta_id': 190301, 'valor': 1588.7, 'dt_transacao': '2021-01-08 19:00:00'}, {'conta_id': 190302, 'valor': 1063.79, 'dt_transacao': '2021-01-25 06:00:00'}, {'conta_id': 190302, 'valor': 1052.86, 'dt_transacao': '2021-02-01 18:00:00'}, {'conta_id': 190303, 'valor': 334.45, 'dt_transacao': '2021-01-24 17:00:00'}, {'conta_id': 190304, 'valor': 1982.93, 'dt_transacao': '2021-02-11 11:00:00'}, {'conta_id': 190304, 'valor': 825.27, 'dt_transacao': '2021-03-23 13:00:00'}, {'conta_id': 190304, 'valor': 1652.48, 'dt_transacao': '2021-02-11 19:00:00'}, {'conta_id': 190304, 'valor': 482.2, 'dt_transacao': '2021-03-11 06:00:00'}, {'conta_id': 190304, 'valor': 750.75, 'dt_transacao': '2021-03-12 23:00:00'}, {'conta_id': 190305, 'valor': 33.59, 'dt_transacao': '2021-01-18 09:00:00'}, {'conta_id': 190305, 'valor': 14.13, 'dt_transacao': '2021-01-29 06:00:00'}, {'conta_id': 190305, 'valor': 1711.59, 'dt_transacao': '2021-01-03 22:00:00'}, {'conta_id': 190305, 'valor': 1644.85, 'dt_transacao': '2021-03-12 16:00:00'}, {'conta_id': 190305, 'valor': 1113.09, 'dt_transacao': '2021-03-15 04:00:00'}, {'conta_id': 190306, 'valor': 500.46, 'dt_transacao': '2021-02-02 09:00:00'}, {'conta_id': 190306, 'valor': 184.72, 'dt_transacao': '2021-02-21 18:00:00'}, {'conta_id': 190306, 'valor': 1835.85, 'dt_transacao': '2021-01-10 23:00:00'}, {'conta_id': 190306, 'valor': 650.06, 'dt_transacao': '2021-03-03 05:00:00'}, {'conta_id': 190306, 'valor': 977.06, 'dt_transacao': '2021-03-30 22:00:00'}, {'conta_id': 190307, 'valor': 696.51, 'dt_transacao': '2021-02-25 13:00:00'}, {'conta_id': 190307, 'valor': 1860.56, 'dt_transacao': '2021-01-16 07:00:00'}, {'conta_id': 190307, 'valor': 1495.61, 'dt_transacao': '2021-03-19 00:00:00'}, {'conta_id': 190308, 'valor': 101.99, 'dt_transacao': '2021-02-20 14:00:00'}, {'conta_id': 190308, 'valor': 1505.93, 'dt_transacao': '2021-03-15 16:00:00'}, {'conta_id': 190309, 'valor': 952.23, 'dt_transacao': '2021-01-07 17:00:00'}, {'conta_id': 190310, 'valor': 1119.03, 'dt_transacao': '2021-03-04 12:00:00'}, {'conta_id': 190310, 'valor': 1304.84, 'dt_transacao': '2021-01-26 17:00:00'}, {'conta_id': 190310, 'valor': 1232.38, 'dt_transacao': '2021-03-09 07:00:00'}, {'conta_id': 190310, 'valor': 485.39, 'dt_transacao': '2021-02-04 13:00:00'}, {'conta_id': 190311, 'valor': 642.16, 'dt_transacao': '2021-03-18 20:00:00'}, {'conta_id': 190311, 'valor': 49.39, 'dt_transacao': '2021-02-09 05:00:00'}, {'conta_id': 190312, 'valor': 1515.71, 'dt_transacao': '2021-01-11 03:00:00'}, {'conta_id': 190312, 'valor': 1479.42, 'dt_transacao': '2021-02-01 19:00:00'}, {'conta_id': 190313, 'valor': 46.47, 'dt_transacao': '2021-01-27 23:00:00'}, {'conta_id': 190313, 'valor': 1076.47, 'dt_transacao': '2021-01-27 17:00:00'}, {'conta_id': 190313, 'valor': 1047.01, 'dt_transacao': '2021-02-03 17:00:00'}, {'conta_id': 190313, 'valor': 966.57, 'dt_transacao': '2021-01-19 22:00:00'}, {'conta_id': 190314, 'valor': 543.29, 'dt_transacao': '2021-03-01 20:00:00'}, {'conta_id': 190314, 'valor': 1890.41, 'dt_transacao': '2021-01-09 12:00:00'}, {'conta_id': 190314, 'valor': 1321.09, 'dt_transacao': '2021-03-26 21:00:00'}, {'conta_id': 190314, 'valor': 721.08, 'dt_transacao': '2021-01-14 00:00:00'}, {'conta_id': 190314, 'valor': 585.7, 'dt_transacao': '2021-03-25 12:00:00'}, {'conta_id': 190315, 'valor': 1933.62, 'dt_transacao': '2021-03-13 22:00:00'}, {'conta_id': 190315, 'valor': 43.96, 'dt_transacao': '2021-02-01 16:00:00'}, {'conta_id': 190315, 'valor': 610.55, 'dt_transacao': '2021-03-04 12:00:00'}, {'conta_id': 190315, 'valor': 1489.04, 'dt_transacao': '2021-01-03 11:00:00'}, {'conta_id': 190315, 'valor': 1368.94, 'dt_transacao': '2021-02-15 20:00:00'}, {'conta_id': 190316, 'valor': 1898.28, 'dt_transacao': '2021-01-29 04:00:00'}, {'conta_id': 190316, 'valor': 684.23, 'dt_transacao': '2021-02-20 21:00:00'}, {'conta_id': 190316, 'valor': 275.38, 'dt_transacao': '2021-02-18 16:00:00'}, {'conta_id': 190316, 'valor': 1921.25, 'dt_transacao': '2021-03-20 20:00:00'}, {'conta_id': 190316, 'valor': 1373.36, 'dt_transacao': '2021-02-23 07:00:00'}, {'conta_id': 190317, 'valor': 1305.29, 'dt_transacao': '2021-03-16 17:00:00'}, {'conta_id': 190317, 'valor': 1368.41, 'dt_transacao': '2021-01-26 04:00:00'}, {'conta_id': 190318, 'valor': 1066.39, 'dt_transacao': '2021-01-16 05:00:00'}, {'conta_id': 190318, 'valor': 635.88, 'dt_transacao': '2021-01-19 23:00:00'}, {'conta_id': 190318, 'valor': 59.94, 'dt_transacao': '2021-02-10 13:00:00'}, {'conta_id': 190318, 'valor': 1922.43, 'dt_transacao': '2021-02-08 14:00:00'}, {'conta_id': 190318, 'valor': 1491.47, 'dt_transacao': '2021-03-23 23:00:00'}, {'conta_id': 190319, 'valor': 1134.52, 'dt_transacao': '2021-01-30 19:00:00'}, {'conta_id': 190319, 'valor': 465.13, 'dt_transacao': '2021-03-29 00:00:00'}, {'conta_id': 190319, 'valor': 719.16, 'dt_transacao': '2021-03-30 15:00:00'}, {'conta_id': 190319, 'valor': 269.04, 'dt_transacao': '2021-03-19 15:00:00'}, {'conta_id': 190319, 'valor': 1424.33, 'dt_transacao': '2021-02-17 05:00:00'}, {'conta_id': 190320, 'valor': 1529.31, 'dt_transacao': '2021-01-02 08:00:00'}, {'conta_id': 190320, 'valor': 1900.45, 'dt_transacao': '2021-01-29 23:00:00'}, {'conta_id': 190320, 'valor': 1343.01, 'dt_transacao': '2021-01-26 06:00:00'}, {'conta_id': 190320, 'valor': 133.21, 'dt_transacao': '2021-02-26 19:00:00'}, {'conta_id': 190321, 'valor': 788.91, 'dt_transacao': '2021-03-15 19:00:00'}, {'conta_id': 190321, 'valor': 1072.78, 'dt_transacao': '2021-03-24 22:00:00'}, {'conta_id': 190321, 'valor': 1574.05, 'dt_transacao': '2021-03-08 03:00:00'}, {'conta_id': 190322, 'valor': 1530.55, 'dt_transacao': '2021-01-07 06:00:00'}, {'conta_id': 190322, 'valor': 583.07, 'dt_transacao': '2021-02-20 09:00:00'}, {'conta_id': 190322, 'valor': 1547.58, 'dt_transacao': '2021-02-06 14:00:00'}, {'conta_id': 190322, 'valor': 333.81, 'dt_transacao': '2021-02-07 19:00:00'}, {'conta_id': 190322, 'valor': 1838.15, 'dt_transacao': '2021-03-30 16:00:00'}, {'conta_id': 190323, 'valor': 827.71, 'dt_transacao': '2021-02-27 00:00:00'}, {'conta_id': 190323, 'valor': 358.58, 'dt_transacao': '2021-03-17 22:00:00'}, {'conta_id': 190323, 'valor': 1791.69, 'dt_transacao': '2021-02-28 02:00:00'}, {'conta_id': 190323, 'valor': 500.91, 'dt_transacao': '2021-03-30 20:00:00'}, {'conta_id': 190324, 'valor': 1414.85, 'dt_transacao': '2021-03-07 19:00:00'}, {'conta_id': 190324, 'valor': 1899.56, 'dt_transacao': '2021-02-18 17:00:00'}, {'conta_id': 190324, 'valor': 761.83, 'dt_transacao': '2021-01-22 10:00:00'}, {'conta_id': 190324, 'valor': 1131.42, 'dt_transacao': '2021-03-26 04:00:00'}, {'conta_id': 190324, 'valor': 325.09, 'dt_transacao': '2021-01-19 05:00:00'}, {'conta_id': 190325, 'valor': 1920.91, 'dt_transacao': '2021-01-01 19:00:00'}, {'conta_id': 190325, 'valor': 1531.03, 'dt_transacao': '2021-01-02 14:00:00'}, {'conta_id': 190326, 'valor': 820.89, 'dt_transacao': '2021-01-11 19:00:00'}, {'conta_id': 190326, 'valor': 1200.6, 'dt_transacao': '2021-02-01 17:00:00'}, {'conta_id': 190326, 'valor': 556.95, 'dt_transacao': '2021-03-22 17:00:00'}, {'conta_id': 190326, 'valor': 1640.42, 'dt_transacao': '2021-01-27 01:00:00'}, {'conta_id': 190327, 'valor': 1676.19, 'dt_transacao': '2021-02-08 10:00:00'}, {'conta_id': 190327, 'valor': 1840.85, 'dt_transacao': '2021-01-07 03:00:00'}, {'conta_id': 190327, 'valor': 834.5, 'dt_transacao': '2021-01-23 05:00:00'}, {'conta_id': 190327, 'valor': 1344.51, 'dt_transacao': '2021-03-20 21:00:00'}, {'conta_id': 190327, 'valor': 1383.34, 'dt_transacao': '2021-03-18 18:00:00'}, {'conta_id': 190328, 'valor': 1795.16, 'dt_transacao': '2021-01-09 13:00:00'}, {'conta_id': 190328, 'valor': 1686.78, 'dt_transacao': '2021-01-09 09:00:00'}, {'conta_id': 190328, 'valor': 942.16, 'dt_transacao': '2021-01-16 22:00:00'}, {'conta_id': 190329, 'valor': 1459.73, 'dt_transacao': '2021-03-22 16:00:00'}, {'conta_id': 190329, 'valor': 1974.39, 'dt_transacao': '2021-03-03 23:00:00'}, {'conta_id': 190329, 'valor': 1377.62, 'dt_transacao': '2021-04-01 09:00:00'}, {'conta_id': 190329, 'valor': 854.68, 'dt_transacao': '2021-02-10 08:00:00'}, {'conta_id': 190329, 'valor': 1051.26, 'dt_transacao': '2021-03-21 11:00:00'}, {'conta_id': 190330, 'valor': 237.3, 'dt_transacao': '2021-03-01 20:00:00'}, {'conta_id': 190331, 'valor': 1281.6, 'dt_transacao': '2021-03-30 09:00:00'}, {'conta_id': 190332, 'valor': 683.38, 'dt_transacao': '2021-03-14 19:00:00'}, {'conta_id': 190332, 'valor': 1920.27, 'dt_transacao': '2021-01-23 06:00:00'}, {'conta_id': 190332, 'valor': 661.7, 'dt_transacao': '2021-02-13 21:00:00'}, {'conta_id': 190333, 'valor': 1791.96, 'dt_transacao': '2021-03-01 13:00:00'}, {'conta_id': 190333, 'valor': 1855.65, 'dt_transacao': '2021-01-08 16:00:00'}, {'conta_id': 190333, 'valor': 599.39, 'dt_transacao': '2021-03-28 10:00:00'}, {'conta_id': 190333, 'valor': 1110.8, 'dt_transacao': '2021-01-03 06:00:00'}, {'conta_id': 190333, 'valor': 1155.22, 'dt_transacao': '2021-02-12 11:00:00'}, {'conta_id': 190334, 'valor': 1190.79, 'dt_transacao': '2021-02-09 22:00:00'}, {'conta_id': 190334, 'valor': 1715.78, 'dt_transacao': '2021-04-01 01:00:00'}, {'conta_id': 190335, 'valor': 1050.68, 'dt_transacao': '2021-01-23 03:00:00'}, {'conta_id': 190335, 'valor': 749.06, 'dt_transacao': '2021-01-05 06:00:00'}, {'conta_id': 190336, 'valor': 1465.15, 'dt_transacao': '2021-03-09 09:00:00'}, {'conta_id': 190336, 'valor': 1551.29, 'dt_transacao': '2021-01-19 03:00:00'}, {'conta_id': 190336, 'valor': 1644.45, 'dt_transacao': '2021-03-08 21:00:00'}, {'conta_id': 190336, 'valor': 1985.45, 'dt_transacao': '2021-01-27 23:00:00'}, {'conta_id': 190337, 'valor': 934.28, 'dt_transacao': '2021-01-06 03:00:00'}, {'conta_id': 190337, 'valor': 1041.63, 'dt_transacao': '2021-03-26 23:00:00'}, {'conta_id': 190337, 'valor': 625.65, 'dt_transacao': '2021-03-15 09:00:00'}, {'conta_id': 190338, 'valor': 546.04, 'dt_transacao': '2021-01-21 08:00:00'}, {'conta_id': 190339, 'valor': 1886.61, 'dt_transacao': '2021-02-06 18:00:00'}, {'conta_id': 190339, 'valor': 490.41, 'dt_transacao': '2021-01-10 09:00:00'}, {'conta_id': 190340, 'valor': 767.82, 'dt_transacao': '2021-01-12 13:00:00'}, {'conta_id': 190341, 'valor': 1136.01, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190341, 'valor': 937.08, 'dt_transacao': '2021-02-28 09:00:00'}, {'conta_id': 190341, 'valor': 1369.27, 'dt_transacao': '2021-04-01 14:00:00'}, {'conta_id': 190341, 'valor': 941.62, 'dt_transacao': '2021-01-29 04:00:00'}, {'conta_id': 190342, 'valor': 61.5, 'dt_transacao': '2021-03-02 03:00:00'}, {'conta_id': 190342, 'valor': 703.53, 'dt_transacao': '2021-03-01 05:00:00'}, {'conta_id': 190342, 'valor': 565.23, 'dt_transacao': '2021-02-14 19:00:00'}, {'conta_id': 190342, 'valor': 1149.78, 'dt_transacao': '2021-03-07 03:00:00'}, {'conta_id': 190343, 'valor': 1242.13, 'dt_transacao': '2021-03-12 05:00:00'}, {'conta_id': 190343, 'valor': 1947.81, 'dt_transacao': '2021-02-19 02:00:00'}, {'conta_id': 190344, 'valor': 1478.99, 'dt_transacao': '2021-03-15 15:00:00'}, {'conta_id': 190345, 'valor': 1196.92, 'dt_transacao': '2021-03-29 19:00:00'}, {'conta_id': 190346, 'valor': 1322.1, 'dt_transacao': '2021-02-27 17:00:00'}, {'conta_id': 190346, 'valor': 583.72, 'dt_transacao': '2021-03-20 18:00:00'}, {'conta_id': 190347, 'valor': 1165.87, 'dt_transacao': '2021-03-20 16:00:00'}, {'conta_id': 190348, 'valor': 1722.63, 'dt_transacao': '2021-01-04 07:00:00'}, {'conta_id': 190349, 'valor': 950.27, 'dt_transacao': '2021-01-25 17:00:00'}, {'conta_id': 190349, 'valor': 769.81, 'dt_transacao': '2021-02-16 13:00:00'}, {'conta_id': 190350, 'valor': 1316.2, 'dt_transacao': '2021-01-18 02:00:00'}, {'conta_id': 190350, 'valor': 275.3, 'dt_transacao': '2021-02-23 14:00:00'}, {'conta_id': 190350, 'valor': 1787.27, 'dt_transacao': '2021-03-31 19:00:00'}, {'conta_id': 190350, 'valor': 1860.69, 'dt_transacao': '2021-03-21 05:00:00'}, {'conta_id': 190351, 'valor': 15.85, 'dt_transacao': '2021-03-05 23:00:00'}, {'conta_id': 190351, 'valor': 474.26, 'dt_transacao': '2021-03-06 08:00:00'}, {'conta_id': 190351, 'valor': 822.26, 'dt_transacao': '2021-03-11 18:00:00'}, {'conta_id': 190351, 'valor': 734.57, 'dt_transacao': '2021-01-05 18:00:00'}, {'conta_id': 190351, 'valor': 967.05, 'dt_transacao': '2021-02-17 03:00:00'}, {'conta_id': 190352, 'valor': 1779.94, 'dt_transacao': '2021-01-12 07:00:00'}, {'conta_id': 190352, 'valor': 945.43, 'dt_transacao': '2021-03-26 21:00:00'}, {'conta_id': 190352, 'valor': 157.09, 'dt_transacao': '2021-03-23 17:00:00'}, {'conta_id': 190352, 'valor': 1596.4, 'dt_transacao': '2021-02-09 19:00:00'}, {'conta_id': 190352, 'valor': 106.5, 'dt_transacao': '2021-02-13 17:00:00'}, {'conta_id': 190353, 'valor': 579.13, 'dt_transacao': '2021-01-05 22:00:00'}, {'conta_id': 190353, 'valor': 1845.81, 'dt_transacao': '2021-02-08 23:00:00'}, {'conta_id': 190353, 'valor': 1570.26, 'dt_transacao': '2021-02-03 01:00:00'}, {'conta_id': 190354, 'valor': 254.52, 'dt_transacao': '2021-01-10 08:00:00'}, {'conta_id': 190355, 'valor': 1478.2, 'dt_transacao': '2021-03-26 14:00:00'}, {'conta_id': 190355, 'valor': 740.16, 'dt_transacao': '2021-01-23 08:00:00'}, {'conta_id': 190355, 'valor': 1565.27, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190355, 'valor': 362.44, 'dt_transacao': '2021-03-09 01:00:00'}, {'conta_id': 190355, 'valor': 686.35, 'dt_transacao': '2021-01-11 04:00:00'}, {'conta_id': 190356, 'valor': 1155.94, 'dt_transacao': '2021-02-14 01:00:00'}, {'conta_id': 190356, 'valor': 1284.88, 'dt_transacao': '2021-01-15 01:00:00'}, {'conta_id': 190357, 'valor': 949.7, 'dt_transacao': '2021-01-19 16:00:00'}, {'conta_id': 190357, 'valor': 1740.69, 'dt_transacao': '2021-01-02 09:00:00'}, {'conta_id': 190357, 'valor': 1854.52, 'dt_transacao': '2021-02-27 12:00:00'}, {'conta_id': 190357, 'valor': 1357.31, 'dt_transacao': '2021-03-09 21:00:00'}, {'conta_id': 190358, 'valor': 1604.83, 'dt_transacao': '2021-02-02 20:00:00'}, {'conta_id': 190358, 'valor': 1168.77, 'dt_transacao': '2021-03-01 21:00:00'}, {'conta_id': 190358, 'valor': 1498.43, 'dt_transacao': '2021-03-28 18:00:00'}, {'conta_id': 190359, 'valor': 823.09, 'dt_transacao': '2021-03-09 03:00:00'}, {'conta_id': 190360, 'valor': 184.79, 'dt_transacao': '2021-03-18 21:00:00'}, {'conta_id': 190360, 'valor': 1788.04, 'dt_transacao': '2021-01-28 19:00:00'}, {'conta_id': 190360, 'valor': 1276.53, 'dt_transacao': '2021-04-01 03:00:00'}, {'conta_id': 190360, 'valor': 173.69, 'dt_transacao': '2021-03-19 04:00:00'}, {'conta_id': 190360, 'valor': 752.54, 'dt_transacao': '2021-03-24 18:00:00'}, {'conta_id': 190361, 'valor': 1538.95, 'dt_transacao': '2021-03-30 21:00:00'}, {'conta_id': 190361, 'valor': 1539.99, 'dt_transacao': '2021-02-11 08:00:00'}, {'conta_id': 190361, 'valor': 324.77, 'dt_transacao': '2021-01-10 08:00:00'}, {'conta_id': 190362, 'valor': 139.73, 'dt_transacao': '2021-01-26 19:00:00'}, {'conta_id': 190362, 'valor': 785.88, 'dt_transacao': '2021-03-24 15:00:00'}, {'conta_id': 190362, 'valor': 217.75, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190362, 'valor': 437.55, 'dt_transacao': '2021-03-24 03:00:00'}, {'conta_id': 190363, 'valor': 1793.13, 'dt_transacao': '2021-01-20 20:00:00'}, {'conta_id': 190364, 'valor': 1415.5, 'dt_transacao': '2021-03-04 15:00:00'}, {'conta_id': 190364, 'valor': 872.68, 'dt_transacao': '2021-01-09 17:00:00'}, {'conta_id': 190364, 'valor': 452.47, 'dt_transacao': '2021-02-19 22:00:00'}, {'conta_id': 190364, 'valor': 1636.14, 'dt_transacao': '2021-03-03 03:00:00'}, {'conta_id': 190364, 'valor': 1134.92, 'dt_transacao': '2021-02-15 10:00:00'}, {'conta_id': 190365, 'valor': 1682.52, 'dt_transacao': '2021-02-07 20:00:00'}, {'conta_id': 190366, 'valor': 1261.6, 'dt_transacao': '2021-02-15 15:00:00'}, {'conta_id': 190367, 'valor': 778.15, 'dt_transacao': '2021-03-11 20:00:00'}, {'conta_id': 190367, 'valor': 616.15, 'dt_transacao': '2021-03-10 14:00:00'}, {'conta_id': 190367, 'valor': 1933.18, 'dt_transacao': '2021-01-03 01:00:00'}, {'conta_id': 190367, 'valor': 679.23, 'dt_transacao': '2021-02-13 23:00:00'}, {'conta_id': 190368, 'valor': 74.72, 'dt_transacao': '2021-01-19 15:00:00'}, {'conta_id': 190369, 'valor': 1809.42, 'dt_transacao': '2021-01-13 14:00:00'}, {'conta_id': 190370, 'valor': 853.0, 'dt_transacao': '2021-03-28 12:00:00'}, {'conta_id': 190370, 'valor': 649.15, 'dt_transacao': '2021-03-29 18:00:00'}, {'conta_id': 190371, 'valor': 548.51, 'dt_transacao': '2021-02-23 05:00:00'}, {'conta_id': 190372, 'valor': 752.89, 'dt_transacao': '2021-01-21 00:00:00'}, {'conta_id': 190372, 'valor': 1759.37, 'dt_transacao': '2021-01-11 00:00:00'}, {'conta_id': 190372, 'valor': 15.44, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190372, 'valor': 1303.86, 'dt_transacao': '2021-02-06 13:00:00'}, {'conta_id': 190372, 'valor': 1387.31, 'dt_transacao': '2021-03-02 22:00:00'}, {'conta_id': 190373, 'valor': 1221.47, 'dt_transacao': '2021-01-05 19:00:00'}, {'conta_id': 190373, 'valor': 360.82, 'dt_transacao': '2021-03-05 19:00:00'}, {'conta_id': 190374, 'valor': 1953.61, 'dt_transacao': '2021-03-21 11:00:00'}, {'conta_id': 190374, 'valor': 1725.6, 'dt_transacao': '2021-03-11 16:00:00'}, {'conta_id': 190374, 'valor': 439.33, 'dt_transacao': '2021-03-22 21:00:00'}, {'conta_id': 190374, 'valor': 417.01, 'dt_transacao': '2021-01-07 06:00:00'}, {'conta_id': 190374, 'valor': 226.25, 'dt_transacao': '2021-01-20 21:00:00'}, {'conta_id': 190375, 'valor': 419.97, 'dt_transacao': '2021-02-02 10:00:00'}, {'conta_id': 190375, 'valor': 1234.02, 'dt_transacao': '2021-02-28 01:00:00'}, {'conta_id': 190375, 'valor': 209.74, 'dt_transacao': '2021-01-07 17:00:00'}, {'conta_id': 190376, 'valor': 1185.82, 'dt_transacao': '2021-02-11 13:00:00'}, {'conta_id': 190376, 'valor': 398.3, 'dt_transacao': '2021-03-28 10:00:00'}, {'conta_id': 190376, 'valor': 460.9, 'dt_transacao': '2021-01-15 10:00:00'}, {'conta_id': 190376, 'valor': 456.34, 'dt_transacao': '2021-01-31 23:00:00'}, {'conta_id': 190377, 'valor': 290.74, 'dt_transacao': '2021-03-20 14:00:00'}, {'conta_id': 190377, 'valor': 356.94, 'dt_transacao': '2021-02-17 04:00:00'}, {'conta_id': 190377, 'valor': 916.02, 'dt_transacao': '2021-01-25 20:00:00'}, {'conta_id': 190377, 'valor': 717.82, 'dt_transacao': '2021-02-04 06:00:00'}, {'conta_id': 190378, 'valor': 1703.45, 'dt_transacao': '2021-02-28 13:00:00'}, {'conta_id': 190379, 'valor': 823.98, 'dt_transacao': '2021-03-08 15:00:00'}, {'conta_id': 190380, 'valor': 869.06, 'dt_transacao': '2021-01-29 18:00:00'}, {'conta_id': 190380, 'valor': 1892.56, 'dt_transacao': '2021-03-15 00:00:00'}, {'conta_id': 190380, 'valor': 836.13, 'dt_transacao': '2021-01-06 01:00:00'}, {'conta_id': 190381, 'valor': 167.1, 'dt_transacao': '2021-01-02 22:00:00'}, {'conta_id': 190381, 'valor': 1463.84, 'dt_transacao': '2021-01-28 04:00:00'}, {'conta_id': 190382, 'valor': 1715.99, 'dt_transacao': '2021-03-16 00:00:00'}, {'conta_id': 190382, 'valor': 1952.87, 'dt_transacao': '2021-03-01 04:00:00'}, {'conta_id': 190382, 'valor': 1748.52, 'dt_transacao': '2021-02-21 05:00:00'}, {'conta_id': 190382, 'valor': 1579.93, 'dt_transacao': '2021-02-17 20:00:00'}, {'conta_id': 190383, 'valor': 1506.02, 'dt_transacao': '2021-03-31 19:00:00'}, {'conta_id': 190383, 'valor': 1085.33, 'dt_transacao': '2021-01-27 21:00:00'}, {'conta_id': 190383, 'valor': 162.84, 'dt_transacao': '2021-02-12 23:00:00'}, {'conta_id': 190383, 'valor': 1864.55, 'dt_transacao': '2021-03-07 21:00:00'}, {'conta_id': 190384, 'valor': 351.36, 'dt_transacao': '2021-03-13 22:00:00'}, {'conta_id': 190384, 'valor': 134.94, 'dt_transacao': '2021-04-01 06:00:00'}, {'conta_id': 190384, 'valor': 277.67, 'dt_transacao': '2021-02-12 07:00:00'}, {'conta_id': 190385, 'valor': 1360.77, 'dt_transacao': '2021-02-05 01:00:00'}, {'conta_id': 190385, 'valor': 127.67, 'dt_transacao': '2021-02-14 14:00:00'}, {'conta_id': 190385, 'valor': 972.97, 'dt_transacao': '2021-03-09 03:00:00'}, {'conta_id': 190385, 'valor': 940.86, 'dt_transacao': '2021-03-10 21:00:00'}, {'conta_id': 190385, 'valor': 1650.93, 'dt_transacao': '2021-02-07 14:00:00'}, {'conta_id': 190386, 'valor': 577.46, 'dt_transacao': '2021-01-10 03:00:00'}, {'conta_id': 190386, 'valor': 330.37, 'dt_transacao': '2021-01-15 22:00:00'}, {'conta_id': 190387, 'valor': 1863.51, 'dt_transacao': '2021-03-02 11:00:00'}, {'conta_id': 190387, 'valor': 840.84, 'dt_transacao': '2021-03-24 00:00:00'}, {'conta_id': 190387, 'valor': 583.74, 'dt_transacao': '2021-01-09 00:00:00'}, {'conta_id': 190387, 'valor': 1743.72, 'dt_transacao': '2021-01-30 16:00:00'}, {'conta_id': 190387, 'valor': 1289.7, 'dt_transacao': '2021-02-22 18:00:00'}, {'conta_id': 190388, 'valor': 1578.75, 'dt_transacao': '2021-01-30 15:00:00'}, {'conta_id': 190388, 'valor': 744.43, 'dt_transacao': '2021-02-06 01:00:00'}, {'conta_id': 190389, 'valor': 1678.82, 'dt_transacao': '2021-03-30 21:00:00'}, {'conta_id': 190389, 'valor': 1589.12, 'dt_transacao': '2021-01-21 07:00:00'}, {'conta_id': 190390, 'valor': 1125.41, 'dt_transacao': '2021-01-05 21:00:00'}, {'conta_id': 190390, 'valor': 634.57, 'dt_transacao': '2021-02-14 04:00:00'}, {'conta_id': 190391, 'valor': 1537.27, 'dt_transacao': '2021-02-28 10:00:00'}, {'conta_id': 190391, 'valor': 312.95, 'dt_transacao': '2021-03-11 16:00:00'}, {'conta_id': 190391, 'valor': 663.23, 'dt_transacao': '2021-04-01 09:00:00'}, {'conta_id': 190391, 'valor': 948.35, 'dt_transacao': '2021-02-07 18:00:00'}, {'conta_id': 190392, 'valor': 50.57, 'dt_transacao': '2021-03-21 07:00:00'}, {'conta_id': 190392, 'valor': 892.5, 'dt_transacao': '2021-01-12 10:00:00'}, {'conta_id': 190392, 'valor': 111.01, 'dt_transacao': '2021-03-24 06:00:00'}, {'conta_id': 190393, 'valor': 789.45, 'dt_transacao': '2021-03-04 10:00:00'}, {'conta_id': 190393, 'valor': 1206.74, 'dt_transacao': '2021-03-07 01:00:00'}, {'conta_id': 190393, 'valor': 1816.99, 'dt_transacao': '2021-02-16 12:00:00'}, {'conta_id': 190393, 'valor': 296.74, 'dt_transacao': '2021-02-24 11:00:00'}, {'conta_id': 190393, 'valor': 772.84, 'dt_transacao': '2021-01-02 09:00:00'}, {'conta_id': 190394, 'valor': 1607.57, 'dt_transacao': '2021-02-06 05:00:00'}, {'conta_id': 190394, 'valor': 1611.54, 'dt_transacao': '2021-02-24 23:00:00'}, {'conta_id': 190394, 'valor': 1792.96, 'dt_transacao': '2021-01-18 18:00:00'}, {'conta_id': 190394, 'valor': 1196.27, 'dt_transacao': '2021-01-19 06:00:00'}, {'conta_id': 190395, 'valor': 1277.97, 'dt_transacao': '2021-02-04 13:00:00'}, {'conta_id': 190395, 'valor': 350.61, 'dt_transacao': '2021-03-01 22:00:00'}, {'conta_id': 190395, 'valor': 1136.26, 'dt_transacao': '2021-01-18 06:00:00'}, {'conta_id': 190395, 'valor': 372.83, 'dt_transacao': '2021-02-18 09:00:00'}, {'conta_id': 190396, 'valor': 54.65, 'dt_transacao': '2021-01-01 04:00:00'}, {'conta_id': 190396, 'valor': 526.09, 'dt_transacao': '2021-03-23 20:00:00'}, {'conta_id': 190397, 'valor': 1529.67, 'dt_transacao': '2021-03-05 02:00:00'}, {'conta_id': 190398, 'valor': 1466.18, 'dt_transacao': '2021-01-03 08:00:00'}, {'conta_id': 190398, 'valor': 326.53, 'dt_transacao': '2021-01-08 04:00:00'}, {'conta_id': 190398, 'valor': 640.04, 'dt_transacao': '2021-03-30 04:00:00'}, {'conta_id': 190398, 'valor': 715.41, 'dt_transacao': '2021-01-07 08:00:00'}, {'conta_id': 190398, 'valor': 722.97, 'dt_transacao': '2021-03-23 11:00:00'}, {'conta_id': 190399, 'valor': 1505.5, 'dt_transacao': '2021-03-11 10:00:00'}, {'conta_id': 190399, 'valor': 1745.01, 'dt_transacao': '2021-03-29 09:00:00'}, {'conta_id': 190399, 'valor': 543.65, 'dt_transacao': '2021-03-31 22:00:00'}, {'conta_id': 190399, 'valor': 725.68, 'dt_transacao': '2021-03-10 18:00:00'}, {'conta_id': 190399, 'valor': 703.91, 'dt_transacao': '2021-01-19 00:00:00'}, {'conta_id': 190400, 'valor': 309.96, 'dt_transacao': '2021-01-29 06:00:00'}, {'conta_id': 190400, 'valor': 1590.42, 'dt_transacao': '2021-03-13 19:00:00'}, {'conta_id': 190401, 'valor': 697.07, 'dt_transacao': '2021-03-30 11:00:00'}, {'conta_id': 190401, 'valor': 278.89, 'dt_transacao': '2021-02-26 19:00:00'}, {'conta_id': 190401, 'valor': 1046.72, 'dt_transacao': '2021-02-13 05:00:00'}, {'conta_id': 190401, 'valor': 1420.08, 'dt_transacao': '2021-01-04 23:00:00'}, {'conta_id': 190402, 'valor': 261.31, 'dt_transacao': '2021-01-11 13:00:00'}, {'conta_id': 190402, 'valor': 1381.95, 'dt_transacao': '2021-01-15 02:00:00'}, {'conta_id': 190403, 'valor': 238.73, 'dt_transacao': '2021-02-02 01:00:00'}, {'conta_id': 190403, 'valor': 133.43, 'dt_transacao': '2021-03-27 00:00:00'}, {'conta_id': 190403, 'valor': 1509.35, 'dt_transacao': '2021-01-16 19:00:00'}, {'conta_id': 190403, 'valor': 411.6, 'dt_transacao': '2021-01-26 05:00:00'}, {'conta_id': 190404, 'valor': 1561.82, 'dt_transacao': '2021-03-20 07:00:00'}, {'conta_id': 190404, 'valor': 1010.31, 'dt_transacao': '2021-01-09 03:00:00'}, {'conta_id': 190404, 'valor': 1639.35, 'dt_transacao': '2021-01-02 16:00:00'}, {'conta_id': 190404, 'valor': 434.78, 'dt_transacao': '2021-01-25 09:00:00'}, {'conta_id': 190404, 'valor': 344.38, 'dt_transacao': '2021-03-10 00:00:00'}, {'conta_id': 190405, 'valor': 1669.65, 'dt_transacao': '2021-02-18 13:00:00'}, {'conta_id': 190405, 'valor': 429.33, 'dt_transacao': '2021-01-30 13:00:00'}, {'conta_id': 190406, 'valor': 1114.71, 'dt_transacao': '2021-01-06 02:00:00'}, {'conta_id': 190406, 'valor': 656.9, 'dt_transacao': '2021-01-15 12:00:00'}, {'conta_id': 190406, 'valor': 567.94, 'dt_transacao': '2021-03-18 23:00:00'}, {'conta_id': 190407, 'valor': 1878.22, 'dt_transacao': '2021-02-17 05:00:00'}, {'conta_id': 190407, 'valor': 26.56, 'dt_transacao': '2021-03-28 15:00:00'}, {'conta_id': 190407, 'valor': 1344.52, 'dt_transacao': '2021-02-20 14:00:00'}, {'conta_id': 190407, 'valor': 1045.8, 'dt_transacao': '2021-03-28 02:00:00'}, {'conta_id': 190408, 'valor': 1950.44, 'dt_transacao': '2021-02-13 06:00:00'}, {'conta_id': 190408, 'valor': 1190.33, 'dt_transacao': '2021-02-21 03:00:00'}, {'conta_id': 190408, 'valor': 466.51, 'dt_transacao': '2021-02-24 23:00:00'}, {'conta_id': 190409, 'valor': 558.5, 'dt_transacao': '2021-02-28 03:00:00'}, {'conta_id': 190409, 'valor': 245.26, 'dt_transacao': '2021-03-04 15:00:00'}, {'conta_id': 190409, 'valor': 462.41, 'dt_transacao': '2021-01-18 00:00:00'}, {'conta_id': 190409, 'valor': 200.11, 'dt_transacao': '2021-01-07 02:00:00'}, {'conta_id': 190409, 'valor': 996.77, 'dt_transacao': '2021-01-04 05:00:00'}, {'conta_id': 190410, 'valor': 404.66, 'dt_transacao': '2021-03-26 15:00:00'}, {'conta_id': 190410, 'valor': 981.01, 'dt_transacao': '2021-02-03 17:00:00'}, {'conta_id': 190410, 'valor': 687.89, 'dt_transacao': '2021-03-11 17:00:00'}, {'conta_id': 190410, 'valor': 36.32, 'dt_transacao': '2021-01-05 13:00:00'}, {'conta_id': 190411, 'valor': 354.56, 'dt_transacao': '2021-03-26 09:00:00'}, {'conta_id': 190412, 'valor': 1350.37, 'dt_transacao': '2021-03-26 17:00:00'}, {'conta_id': 190412, 'valor': 1371.06, 'dt_transacao': '2021-03-15 15:00:00'}, {'conta_id': 190412, 'valor': 952.35, 'dt_transacao': '2021-01-21 16:00:00'}, {'conta_id': 190412, 'valor': 112.42, 'dt_transacao': '2021-02-03 15:00:00'}, {'conta_id': 190412, 'valor': 1182.5, 'dt_transacao': '2021-02-18 19:00:00'}, {'conta_id': 190413, 'valor': 1619.3, 'dt_transacao': '2021-03-16 00:00:00'}, {'conta_id': 190413, 'valor': 1530.62, 'dt_transacao': '2021-03-22 10:00:00'}, {'conta_id': 190413, 'valor': 761.77, 'dt_transacao': '2021-02-24 11:00:00'}, {'conta_id': 190414, 'valor': 1185.47, 'dt_transacao': '2021-03-30 02:00:00'}, {'conta_id': 190414, 'valor': 1738.01, 'dt_transacao': '2021-01-15 10:00:00'}, {'conta_id': 190414, 'valor': 340.39, 'dt_transacao': '2021-03-10 10:00:00'}, {'conta_id': 190415, 'valor': 756.1, 'dt_transacao': '2021-02-19 10:00:00'}, {'conta_id': 190415, 'valor': 1145.5, 'dt_transacao': '2021-03-04 03:00:00'}, {'conta_id': 190415, 'valor': 1382.04, 'dt_transacao': '2021-01-27 10:00:00'}, {'conta_id': 190416, 'valor': 1271.25, 'dt_transacao': '2021-01-10 17:00:00'}, {'conta_id': 190416, 'valor': 428.95, 'dt_transacao': '2021-02-03 01:00:00'}, {'conta_id': 190416, 'valor': 595.34, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190416, 'valor': 1609.4, 'dt_transacao': '2021-03-21 23:00:00'}, {'conta_id': 190417, 'valor': 1591.51, 'dt_transacao': '2021-03-25 09:00:00'}, {'conta_id': 190417, 'valor': 1009.77, 'dt_transacao': '2021-02-08 13:00:00'}, {'conta_id': 190417, 'valor': 1327.25, 'dt_transacao': '2021-02-10 06:00:00'}, {'conta_id': 190418, 'valor': 998.91, 'dt_transacao': '2021-03-26 23:00:00'}, {'conta_id': 190418, 'valor': 1876.27, 'dt_transacao': '2021-02-04 20:00:00'}, {'conta_id': 190418, 'valor': 1065.91, 'dt_transacao': '2021-03-20 13:00:00'}, {'conta_id': 190418, 'valor': 1869.34, 'dt_transacao': '2021-03-22 01:00:00'}, {'conta_id': 190418, 'valor': 1045.02, 'dt_transacao': '2021-02-16 14:00:00'}, {'conta_id': 190419, 'valor': 1720.51, 'dt_transacao': '2021-01-09 20:00:00'}, {'conta_id': 190419, 'valor': 407.91, 'dt_transacao': '2021-02-22 08:00:00'}, {'conta_id': 190419, 'valor': 1647.6, 'dt_transacao': '2021-03-24 05:00:00'}, {'conta_id': 190420, 'valor': 1594.29, 'dt_transacao': '2021-02-09 14:00:00'}, {'conta_id': 190420, 'valor': 971.28, 'dt_transacao': '2021-02-25 13:00:00'}, {'conta_id': 190421, 'valor': 352.79, 'dt_transacao': '2021-01-06 23:00:00'}, {'conta_id': 190421, 'valor': 972.2, 'dt_transacao': '2021-01-03 01:00:00'}, {'conta_id': 190421, 'valor': 1719.31, 'dt_transacao': '2021-01-11 10:00:00'}, {'conta_id': 190421, 'valor': 1203.6, 'dt_transacao': '2021-01-09 13:00:00'}, {'conta_id': 190421, 'valor': 1793.65, 'dt_transacao': '2021-03-27 21:00:00'}, {'conta_id': 190422, 'valor': 476.92, 'dt_transacao': '2021-01-16 10:00:00'}, {'conta_id': 190423, 'valor': 1939.88, 'dt_transacao': '2021-02-26 13:00:00'}, {'conta_id': 190423, 'valor': 509.36, 'dt_transacao': '2021-03-17 01:00:00'}, {'conta_id': 190424, 'valor': 1758.99, 'dt_transacao': '2021-01-19 11:00:00'}, {'conta_id': 190424, 'valor': 950.97, 'dt_transacao': '2021-02-17 09:00:00'}, {'conta_id': 190424, 'valor': 43.13, 'dt_transacao': '2021-03-19 01:00:00'}, {'conta_id': 190425, 'valor': 777.55, 'dt_transacao': '2021-02-20 08:00:00'}, {'conta_id': 190425, 'valor': 602.48, 'dt_transacao': '2021-01-31 09:00:00'}, {'conta_id': 190425, 'valor': 421.56, 'dt_transacao': '2021-01-21 14:00:00'}, {'conta_id': 190425, 'valor': 1263.07, 'dt_transacao': '2021-03-24 13:00:00'}, {'conta_id': 190426, 'valor': 1648.98, 'dt_transacao': '2021-03-31 08:00:00'}, {'conta_id': 190427, 'valor': 1283.1, 'dt_transacao': '2021-01-24 20:00:00'}, {'conta_id': 190428, 'valor': 316.49, 'dt_transacao': '2021-03-08 22:00:00'}, {'conta_id': 190429, 'valor': 1653.71, 'dt_transacao': '2021-02-06 06:00:00'}, {'conta_id': 190429, 'valor': 169.71, 'dt_transacao': '2021-02-17 05:00:00'}, {'conta_id': 190429, 'valor': 787.59, 'dt_transacao': '2021-02-18 18:00:00'}, {'conta_id': 190429, 'valor': 449.75, 'dt_transacao': '2021-01-03 17:00:00'}, {'conta_id': 190429, 'valor': 1294.57, 'dt_transacao': '2021-03-22 07:00:00'}, {'conta_id': 190430, 'valor': 858.38, 'dt_transacao': '2021-02-03 04:00:00'}, {'conta_id': 190430, 'valor': 179.03, 'dt_transacao': '2021-02-17 05:00:00'}, {'conta_id': 190430, 'valor': 1894.54, 'dt_transacao': '2021-01-16 00:00:00'}, {'conta_id': 190430, 'valor': 1437.45, 'dt_transacao': '2021-02-25 14:00:00'}, {'conta_id': 190430, 'valor': 1654.69, 'dt_transacao': '2021-03-05 03:00:00'}, {'conta_id': 190431, 'valor': 1712.82, 'dt_transacao': '2021-03-18 06:00:00'}, {'conta_id': 190431, 'valor': 1990.42, 'dt_transacao': '2021-02-06 23:00:00'}, {'conta_id': 190431, 'valor': 1354.7, 'dt_transacao': '2021-02-23 05:00:00'}, {'conta_id': 190431, 'valor': 817.64, 'dt_transacao': '2021-01-19 00:00:00'}, {'conta_id': 190432, 'valor': 938.29, 'dt_transacao': '2021-02-11 22:00:00'}, {'conta_id': 190433, 'valor': 118.45, 'dt_transacao': '2021-02-12 07:00:00'}, {'conta_id': 190433, 'valor': 942.39, 'dt_transacao': '2021-02-15 19:00:00'}, {'conta_id': 190433, 'valor': 1878.7, 'dt_transacao': '2021-03-18 22:00:00'}, {'conta_id': 190433, 'valor': 1496.74, 'dt_transacao': '2021-01-06 00:00:00'}, {'conta_id': 190433, 'valor': 508.06, 'dt_transacao': '2021-03-21 12:00:00'}, {'conta_id': 190434, 'valor': 1007.99, 'dt_transacao': '2021-02-13 14:00:00'}, {'conta_id': 190434, 'valor': 264.71, 'dt_transacao': '2021-02-10 22:00:00'}, {'conta_id': 190435, 'valor': 1743.49, 'dt_transacao': '2021-02-05 23:00:00'}, {'conta_id': 190435, 'valor': 721.82, 'dt_transacao': '2021-02-25 09:00:00'}, {'conta_id': 190435, 'valor': 1805.68, 'dt_transacao': '2021-02-03 19:00:00'}, {'conta_id': 190435, 'valor': 1659.35, 'dt_transacao': '2021-02-03 16:00:00'}, {'conta_id': 190436, 'valor': 298.09, 'dt_transacao': '2021-01-30 05:00:00'}, {'conta_id': 190436, 'valor': 72.69, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190437, 'valor': 986.48, 'dt_transacao': '2021-01-19 03:00:00'}, {'conta_id': 190437, 'valor': 699.86, 'dt_transacao': '2021-01-27 09:00:00'}, {'conta_id': 190437, 'valor': 362.13, 'dt_transacao': '2021-01-22 23:00:00'}, {'conta_id': 190437, 'valor': 1017.97, 'dt_transacao': '2021-02-04 18:00:00'}, {'conta_id': 190438, 'valor': 1536.74, 'dt_transacao': '2021-03-28 03:00:00'}, {'conta_id': 190438, 'valor': 1774.85, 'dt_transacao': '2021-01-04 09:00:00'}, {'conta_id': 190439, 'valor': 1661.57, 'dt_transacao': '2021-03-26 15:00:00'}, {'conta_id': 190439, 'valor': 908.66, 'dt_transacao': '2021-02-04 19:00:00'}, {'conta_id': 190439, 'valor': 1316.35, 'dt_transacao': '2021-02-12 01:00:00'}, {'conta_id': 190439, 'valor': 1312.69, 'dt_transacao': '2021-02-27 10:00:00'}, {'conta_id': 190440, 'valor': 1094.77, 'dt_transacao': '2021-03-19 12:00:00'}, {'conta_id': 190441, 'valor': 326.72, 'dt_transacao': '2021-02-21 09:00:00'}, {'conta_id': 190441, 'valor': 1294.69, 'dt_transacao': '2021-03-25 04:00:00'}, {'conta_id': 190442, 'valor': 1025.79, 'dt_transacao': '2021-03-04 14:00:00'}, {'conta_id': 190442, 'valor': 1770.94, 'dt_transacao': '2021-03-02 08:00:00'}, {'conta_id': 190442, 'valor': 1126.55, 'dt_transacao': '2021-02-24 02:00:00'}, {'conta_id': 190443, 'valor': 684.15, 'dt_transacao': '2021-03-01 21:00:00'}, {'conta_id': 190443, 'valor': 449.24, 'dt_transacao': '2021-03-29 22:00:00'}, {'conta_id': 190443, 'valor': 310.98, 'dt_transacao': '2021-03-05 16:00:00'}, {'conta_id': 190443, 'valor': 1259.49, 'dt_transacao': '2021-03-09 02:00:00'}, {'conta_id': 190443, 'valor': 582.12, 'dt_transacao': '2021-02-25 02:00:00'}, {'conta_id': 190444, 'valor': 1137.5, 'dt_transacao': '2021-03-12 02:00:00'}, {'conta_id': 190444, 'valor': 1834.69, 'dt_transacao': '2021-03-07 03:00:00'}, {'conta_id': 190444, 'valor': 1021.79, 'dt_transacao': '2021-02-22 03:00:00'}, {'conta_id': 190445, 'valor': 359.92, 'dt_transacao': '2021-01-24 12:00:00'}, {'conta_id': 190445, 'valor': 91.47, 'dt_transacao': '2021-02-23 22:00:00'}, {'conta_id': 190445, 'valor': 596.0, 'dt_transacao': '2021-03-17 23:00:00'}, {'conta_id': 190445, 'valor': 910.54, 'dt_transacao': '2021-03-27 14:00:00'}, {'conta_id': 190446, 'valor': 868.16, 'dt_transacao': '2021-01-03 00:00:00'}, {'conta_id': 190447, 'valor': 1452.1, 'dt_transacao': '2021-02-08 19:00:00'}, {'conta_id': 190447, 'valor': 1626.45, 'dt_transacao': '2021-03-20 11:00:00'}, {'conta_id': 190447, 'valor': 1381.63, 'dt_transacao': '2021-03-30 21:00:00'}, {'conta_id': 190447, 'valor': 790.6, 'dt_transacao': '2021-02-08 02:00:00'}, {'conta_id': 190447, 'valor': 251.05, 'dt_transacao': '2021-01-02 06:00:00'}, {'conta_id': 190448, 'valor': 1684.34, 'dt_transacao': '2021-03-10 06:00:00'}, {'conta_id': 190448, 'valor': 173.1, 'dt_transacao': '2021-03-01 04:00:00'}, {'conta_id': 190448, 'valor': 948.5, 'dt_transacao': '2021-01-03 03:00:00'}, {'conta_id': 190448, 'valor': 333.04, 'dt_transacao': '2021-01-20 23:00:00'}, {'conta_id': 190449, 'valor': 1659.07, 'dt_transacao': '2021-02-20 01:00:00'}, {'conta_id': 190449, 'valor': 668.37, 'dt_transacao': '2021-01-14 20:00:00'}, {'conta_id': 190449, 'valor': 1695.41, 'dt_transacao': '2021-02-25 13:00:00'}, {'conta_id': 190449, 'valor': 1697.38, 'dt_transacao': '2021-03-16 19:00:00'}, {'conta_id': 190449, 'valor': 1462.43, 'dt_transacao': '2021-02-07 01:00:00'}, {'conta_id': 190450, 'valor': 632.38, 'dt_transacao': '2021-01-09 08:00:00'}, {'conta_id': 190450, 'valor': 1785.86, 'dt_transacao': '2021-02-23 15:00:00'}, {'conta_id': 190450, 'valor': 234.76, 'dt_transacao': '2021-03-21 20:00:00'}, {'conta_id': 190450, 'valor': 694.18, 'dt_transacao': '2021-03-31 13:00:00'}, {'conta_id': 190450, 'valor': 42.59, 'dt_transacao': '2021-02-06 23:00:00'}, {'conta_id': 190451, 'valor': 1766.46, 'dt_transacao': '2021-02-19 13:00:00'}, {'conta_id': 190451, 'valor': 1666.33, 'dt_transacao': '2021-03-28 20:00:00'}, {'conta_id': 190452, 'valor': 347.41, 'dt_transacao': '2021-01-21 19:00:00'}, {'conta_id': 190452, 'valor': 984.88, 'dt_transacao': '2021-02-23 15:00:00'}, {'conta_id': 190453, 'valor': 350.25, 'dt_transacao': '2021-03-06 01:00:00'}, {'conta_id': 190453, 'valor': 1613.82, 'dt_transacao': '2021-02-13 13:00:00'}, {'conta_id': 190454, 'valor': 1538.98, 'dt_transacao': '2021-03-13 08:00:00'}, {'conta_id': 190454, 'valor': 1792.74, 'dt_transacao': '2021-03-30 16:00:00'}, {'conta_id': 190454, 'valor': 1208.97, 'dt_transacao': '2021-01-14 22:00:00'}, {'conta_id': 190455, 'valor': 1260.22, 'dt_transacao': '2021-01-27 11:00:00'}, {'conta_id': 190455, 'valor': 314.91, 'dt_transacao': '2021-02-07 22:00:00'}, {'conta_id': 190456, 'valor': 1697.34, 'dt_transacao': '2021-02-26 13:00:00'}, {'conta_id': 190457, 'valor': 1430.68, 'dt_transacao': '2021-02-23 11:00:00'}, {'conta_id': 190457, 'valor': 1259.25, 'dt_transacao': '2021-02-01 04:00:00'}, {'conta_id': 190457, 'valor': 1506.66, 'dt_transacao': '2021-03-09 13:00:00'}, {'conta_id': 190458, 'valor': 292.02, 'dt_transacao': '2021-01-02 22:00:00'}, {'conta_id': 190458, 'valor': 1612.69, 'dt_transacao': '2021-01-09 02:00:00'}, {'conta_id': 190458, 'valor': 677.27, 'dt_transacao': '2021-02-21 05:00:00'}, {'conta_id': 190458, 'valor': 868.71, 'dt_transacao': '2021-02-20 01:00:00'}, {'conta_id': 190459, 'valor': 1654.18, 'dt_transacao': '2021-03-09 19:00:00'}, {'conta_id': 190460, 'valor': 1762.07, 'dt_transacao': '2021-01-11 01:00:00'}, {'conta_id': 190460, 'valor': 1075.33, 'dt_transacao': '2021-02-19 23:00:00'}, {'conta_id': 190461, 'valor': 56.75, 'dt_transacao': '2021-03-11 18:00:00'}, {'conta_id': 190462, 'valor': 1254.35, 'dt_transacao': '2021-02-15 03:00:00'}, {'conta_id': 190462, 'valor': 96.42, 'dt_transacao': '2021-03-25 03:00:00'}, {'conta_id': 190462, 'valor': 898.03, 'dt_transacao': '2021-03-15 23:00:00'}, {'conta_id': 190463, 'valor': 1334.18, 'dt_transacao': '2021-03-15 21:00:00'}, {'conta_id': 190463, 'valor': 280.8, 'dt_transacao': '2021-02-18 16:00:00'}, {'conta_id': 190463, 'valor': 1475.29, 'dt_transacao': '2021-01-14 10:00:00'}, {'conta_id': 190464, 'valor': 185.4, 'dt_transacao': '2021-02-24 04:00:00'}, {'conta_id': 190465, 'valor': 998.16, 'dt_transacao': '2021-02-24 14:00:00'}, {'conta_id': 190465, 'valor': 1378.35, 'dt_transacao': '2021-03-04 23:00:00'}, {'conta_id': 190465, 'valor': 1946.57, 'dt_transacao': '2021-01-21 04:00:00'}, {'conta_id': 190465, 'valor': 1012.07, 'dt_transacao': '2021-03-04 22:00:00'}, {'conta_id': 190465, 'valor': 915.97, 'dt_transacao': '2021-04-01 16:00:00'}, {'conta_id': 190466, 'valor': 1203.6, 'dt_transacao': '2021-02-22 03:00:00'}, {'conta_id': 190467, 'valor': 1982.05, 'dt_transacao': '2021-02-08 09:00:00'}, {'conta_id': 190467, 'valor': 972.37, 'dt_transacao': '2021-01-19 04:00:00'}, {'conta_id': 190467, 'valor': 1567.8, 'dt_transacao': '2021-02-23 01:00:00'}, {'conta_id': 190468, 'valor': 1740.98, 'dt_transacao': '2021-02-23 15:00:00'}, {'conta_id': 190469, 'valor': 1273.28, 'dt_transacao': '2021-02-27 21:00:00'}, {'conta_id': 190469, 'valor': 898.7, 'dt_transacao': '2021-02-11 02:00:00'}, {'conta_id': 190469, 'valor': 1599.82, 'dt_transacao': '2021-03-27 19:00:00'}, {'conta_id': 190469, 'valor': 819.85, 'dt_transacao': '2021-01-30 12:00:00'}, {'conta_id': 190469, 'valor': 678.66, 'dt_transacao': '2021-01-06 21:00:00'}, {'conta_id': 190470, 'valor': 171.22, 'dt_transacao': '2021-03-21 17:00:00'}, {'conta_id': 190470, 'valor': 1154.13, 'dt_transacao': '2021-01-07 18:00:00'}, {'conta_id': 190470, 'valor': 1043.99, 'dt_transacao': '2021-03-02 16:00:00'}, {'conta_id': 190470, 'valor': 1793.83, 'dt_transacao': '2021-01-16 21:00:00'}, {'conta_id': 190470, 'valor': 1872.43, 'dt_transacao': '2021-01-24 00:00:00'}, {'conta_id': 190471, 'valor': 1724.09, 'dt_transacao': '2021-01-03 10:00:00'}, {'conta_id': 190471, 'valor': 874.6, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190471, 'valor': 1719.44, 'dt_transacao': '2021-02-10 12:00:00'}, {'conta_id': 190471, 'valor': 515.1, 'dt_transacao': '2021-01-15 03:00:00'}, {'conta_id': 190472, 'valor': 1904.38, 'dt_transacao': '2021-03-28 05:00:00'}, {'conta_id': 190472, 'valor': 118.77, 'dt_transacao': '2021-02-11 05:00:00'}, {'conta_id': 190472, 'valor': 55.32, 'dt_transacao': '2021-01-08 09:00:00'}, {'conta_id': 190473, 'valor': 104.7, 'dt_transacao': '2021-01-31 19:00:00'}, {'conta_id': 190474, 'valor': 1106.69, 'dt_transacao': '2021-01-09 15:00:00'}, {'conta_id': 190474, 'valor': 58.78, 'dt_transacao': '2021-03-23 07:00:00'}, {'conta_id': 190474, 'valor': 1827.56, 'dt_transacao': '2021-01-03 08:00:00'}, {'conta_id': 190474, 'valor': 1108.6, 'dt_transacao': '2021-03-17 21:00:00'}, {'conta_id': 190475, 'valor': 407.71, 'dt_transacao': '2021-01-19 11:00:00'}, {'conta_id': 190475, 'valor': 1081.34, 'dt_transacao': '2021-01-22 07:00:00'}, {'conta_id': 190475, 'valor': 1075.53, 'dt_transacao': '2021-02-24 17:00:00'}, {'conta_id': 190475, 'valor': 904.03, 'dt_transacao': '2021-01-01 14:00:00'}, {'conta_id': 190475, 'valor': 1922.27, 'dt_transacao': '2021-03-21 09:00:00'}, {'conta_id': 190476, 'valor': 1170.73, 'dt_transacao': '2021-03-07 09:00:00'}, {'conta_id': 190476, 'valor': 91.92, 'dt_transacao': '2021-03-10 18:00:00'}, {'conta_id': 190477, 'valor': 422.49, 'dt_transacao': '2021-03-06 10:00:00'}, {'conta_id': 190477, 'valor': 1767.98, 'dt_transacao': '2021-02-13 20:00:00'}, {'conta_id': 190477, 'valor': 523.46, 'dt_transacao': '2021-04-01 14:00:00'}, {'conta_id': 190477, 'valor': 546.75, 'dt_transacao': '2021-03-26 10:00:00'}, {'conta_id': 190478, 'valor': 1802.99, 'dt_transacao': '2021-03-28 05:00:00'}, {'conta_id': 190479, 'valor': 706.64, 'dt_transacao': '2021-02-18 19:00:00'}, {'conta_id': 190480, 'valor': 889.71, 'dt_transacao': '2021-01-29 23:00:00'}, {'conta_id': 190480, 'valor': 40.54, 'dt_transacao': '2021-02-05 21:00:00'}, {'conta_id': 190481, 'valor': 1934.63, 'dt_transacao': '2021-01-05 08:00:00'}, {'conta_id': 190481, 'valor': 986.18, 'dt_transacao': '2021-04-01 22:00:00'}, {'conta_id': 190481, 'valor': 1212.45, 'dt_transacao': '2021-01-01 06:00:00'}, {'conta_id': 190481, 'valor': 1968.54, 'dt_transacao': '2021-03-23 12:00:00'}, {'conta_id': 190482, 'valor': 257.36, 'dt_transacao': '2021-01-09 04:00:00'}, {'conta_id': 190482, 'valor': 519.99, 'dt_transacao': '2021-03-14 07:00:00'}, {'conta_id': 190482, 'valor': 1762.13, 'dt_transacao': '2021-02-05 06:00:00'}, {'conta_id': 190482, 'valor': 1695.49, 'dt_transacao': '2021-01-03 22:00:00'}, {'conta_id': 190482, 'valor': 1355.79, 'dt_transacao': '2021-03-20 08:00:00'}, {'conta_id': 190483, 'valor': 548.8, 'dt_transacao': '2021-03-29 16:00:00'}, {'conta_id': 190483, 'valor': 933.71, 'dt_transacao': '2021-02-16 16:00:00'}, {'conta_id': 190483, 'valor': 1518.28, 'dt_transacao': '2021-01-29 01:00:00'}, {'conta_id': 190483, 'valor': 556.59, 'dt_transacao': '2021-02-23 17:00:00'}, {'conta_id': 190484, 'valor': 1956.21, 'dt_transacao': '2021-02-24 02:00:00'}, {'conta_id': 190484, 'valor': 1834.07, 'dt_transacao': '2021-01-26 05:00:00'}, {'conta_id': 190485, 'valor': 1616.31, 'dt_transacao': '2021-03-17 20:00:00'}, {'conta_id': 190485, 'valor': 1870.19, 'dt_transacao': '2021-02-02 07:00:00'}, {'conta_id': 190485, 'valor': 216.57, 'dt_transacao': '2021-01-31 10:00:00'}, {'conta_id': 190485, 'valor': 1665.2, 'dt_transacao': '2021-02-12 16:00:00'}, {'conta_id': 190486, 'valor': 1833.51, 'dt_transacao': '2021-02-18 10:00:00'}, {'conta_id': 190487, 'valor': 1355.45, 'dt_transacao': '2021-03-13 01:00:00'}, {'conta_id': 190488, 'valor': 768.46, 'dt_transacao': '2021-02-24 19:00:00'}, {'conta_id': 190488, 'valor': 1238.84, 'dt_transacao': '2021-02-13 20:00:00'}, {'conta_id': 190488, 'valor': 1875.23, 'dt_transacao': '2021-01-04 14:00:00'}, {'conta_id': 190488, 'valor': 1034.91, 'dt_transacao': '2021-03-17 06:00:00'}, {'conta_id': 190489, 'valor': 513.32, 'dt_transacao': '2021-02-23 15:00:00'}, {'conta_id': 190489, 'valor': 1449.52, 'dt_transacao': '2021-02-26 05:00:00'}, {'conta_id': 190489, 'valor': 487.06, 'dt_transacao': '2021-01-06 01:00:00'}, {'conta_id': 190489, 'valor': 256.23, 'dt_transacao': '2021-03-16 13:00:00'}, {'conta_id': 190489, 'valor': 1026.74, 'dt_transacao': '2021-03-25 11:00:00'}, {'conta_id': 190490, 'valor': 1035.84, 'dt_transacao': '2021-03-26 21:00:00'}, {'conta_id': 190491, 'valor': 1570.74, 'dt_transacao': '2021-02-13 06:00:00'}, {'conta_id': 190491, 'valor': 1336.05, 'dt_transacao': '2021-03-30 22:00:00'}, {'conta_id': 190492, 'valor': 1752.33, 'dt_transacao': '2021-01-06 15:00:00'}, {'conta_id': 190492, 'valor': 439.59, 'dt_transacao': '2021-01-01 20:00:00'}, {'conta_id': 190492, 'valor': 446.83, 'dt_transacao': '2021-01-30 14:00:00'}, {'conta_id': 190492, 'valor': 1248.22, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190493, 'valor': 716.42, 'dt_transacao': '2021-03-13 03:00:00'}, {'conta_id': 190494, 'valor': 396.45, 'dt_transacao': '2021-01-23 07:00:00'}, {'conta_id': 190494, 'valor': 1770.88, 'dt_transacao': '2021-01-23 10:00:00'}, {'conta_id': 190495, 'valor': 1663.06, 'dt_transacao': '2021-01-18 12:00:00'}, {'conta_id': 190496, 'valor': 1882.38, 'dt_transacao': '2021-03-21 13:00:00'}, {'conta_id': 190496, 'valor': 1756.9, 'dt_transacao': '2021-01-09 03:00:00'}, {'conta_id': 190497, 'valor': 859.65, 'dt_transacao': '2021-03-23 07:00:00'}, {'conta_id': 190497, 'valor': 1842.11, 'dt_transacao': '2021-03-20 18:00:00'}, {'conta_id': 190497, 'valor': 379.02, 'dt_transacao': '2021-03-10 09:00:00'}, {'conta_id': 190497, 'valor': 1579.39, 'dt_transacao': '2021-02-11 07:00:00'}, {'conta_id': 190497, 'valor': 107.46, 'dt_transacao': '2021-02-05 04:00:00'}, {'conta_id': 190498, 'valor': 749.16, 'dt_transacao': '2021-03-08 14:00:00'}, {'conta_id': 190498, 'valor': 358.82, 'dt_transacao': '2021-02-17 16:00:00'}, {'conta_id': 190498, 'valor': 1389.32, 'dt_transacao': '2021-03-05 15:00:00'}, {'conta_id': 190498, 'valor': 537.4, 'dt_transacao': '2021-02-21 20:00:00'}, {'conta_id': 190498, 'valor': 1842.45, 'dt_transacao': '2021-03-16 20:00:00'}, {'conta_id': 190499, 'valor': 850.62, 'dt_transacao': '2021-02-18 18:00:00'}, {'conta_id': 190499, 'valor': 315.26, 'dt_transacao': '2021-02-17 08:00:00'}, {'conta_id': 190499, 'valor': 1101.55, 'dt_transacao': '2021-01-18 02:00:00'}, {'conta_id': 190500, 'valor': 902.85, 'dt_transacao': '2021-02-25 13:00:00'}, {'conta_id': 190500, 'valor': 735.66, 'dt_transacao': '2021-03-05 10:00:00'}, {'conta_id': 190500, 'valor': 546.77, 'dt_transacao': '2021-01-04 05:00:00'}, {'conta_id': 190500, 'valor': 883.94, 'dt_transacao': '2021-02-03 20:00:00'}, {'conta_id': 190501, 'valor': 238.06, 'dt_transacao': '2021-02-16 01:00:00'}, {'conta_id': 190502, 'valor': 382.09, 'dt_transacao': '2021-03-19 04:00:00'}, {'conta_id': 190502, 'valor': 1996.81, 'dt_transacao': '2021-03-11 08:00:00'}, {'conta_id': 190502, 'valor': 18.06, 'dt_transacao': '2021-02-08 09:00:00'}, {'conta_id': 190503, 'valor': 1446.95, 'dt_transacao': '2021-01-27 15:00:00'}, {'conta_id': 190503, 'valor': 468.21, 'dt_transacao': '2021-03-17 17:00:00'}, {'conta_id': 190504, 'valor': 877.92, 'dt_transacao': '2021-03-13 00:00:00'}, {'conta_id': 190504, 'valor': 750.03, 'dt_transacao': '2021-03-15 04:00:00'}, {'conta_id': 190505, 'valor': 1525.92, 'dt_transacao': '2021-03-30 17:00:00'}, {'conta_id': 190505, 'valor': 599.97, 'dt_transacao': '2021-03-17 00:00:00'}, {'conta_id': 190505, 'valor': 1135.59, 'dt_transacao': '2021-01-05 04:00:00'}, {'conta_id': 190505, 'valor': 308.23, 'dt_transacao': '2021-02-12 18:00:00'}, {'conta_id': 190506, 'valor': 218.17, 'dt_transacao': '2021-03-14 14:00:00'}, {'conta_id': 190506, 'valor': 1278.37, 'dt_transacao': '2021-02-08 02:00:00'}, {'conta_id': 190506, 'valor': 343.68, 'dt_transacao': '2021-03-28 17:00:00'}, {'conta_id': 190506, 'valor': 356.09, 'dt_transacao': '2021-03-30 19:00:00'}, {'conta_id': 190507, 'valor': 582.92, 'dt_transacao': '2021-03-09 02:00:00'}, {'conta_id': 190507, 'valor': 909.87, 'dt_transacao': '2021-02-17 16:00:00'}, {'conta_id': 190508, 'valor': 837.41, 'dt_transacao': '2021-01-09 12:00:00'}, {'conta_id': 190508, 'valor': 1910.24, 'dt_transacao': '2021-01-27 04:00:00'}, {'conta_id': 190508, 'valor': 1741.03, 'dt_transacao': '2021-02-09 13:00:00'}, {'conta_id': 190508, 'valor': 1635.97, 'dt_transacao': '2021-03-27 15:00:00'}, {'conta_id': 190508, 'valor': 318.4, 'dt_transacao': '2021-03-15 09:00:00'}, {'conta_id': 190509, 'valor': 1649.34, 'dt_transacao': '2021-03-19 07:00:00'}, {'conta_id': 190510, 'valor': 694.22, 'dt_transacao': '2021-02-08 12:00:00'}, {'conta_id': 190510, 'valor': 1226.44, 'dt_transacao': '2021-01-23 13:00:00'}, {'conta_id': 190510, 'valor': 499.02, 'dt_transacao': '2021-03-21 23:00:00'}, {'conta_id': 190510, 'valor': 864.72, 'dt_transacao': '2021-02-19 21:00:00'}, {'conta_id': 190511, 'valor': 1396.39, 'dt_transacao': '2021-01-28 15:00:00'}, {'conta_id': 190512, 'valor': 506.28, 'dt_transacao': '2021-03-06 01:00:00'}, {'conta_id': 190512, 'valor': 924.87, 'dt_transacao': '2021-01-10 20:00:00'}, {'conta_id': 190513, 'valor': 230.25, 'dt_transacao': '2021-01-18 10:00:00'}, {'conta_id': 190513, 'valor': 1340.92, 'dt_transacao': '2021-03-01 15:00:00'}, {'conta_id': 190513, 'valor': 923.25, 'dt_transacao': '2021-01-11 17:00:00'}, {'conta_id': 190514, 'valor': 201.93, 'dt_transacao': '2021-02-07 03:00:00'}, {'conta_id': 190514, 'valor': 1069.77, 'dt_transacao': '2021-02-20 05:00:00'}, {'conta_id': 190514, 'valor': 156.09, 'dt_transacao': '2021-01-02 21:00:00'}, {'conta_id': 190514, 'valor': 547.23, 'dt_transacao': '2021-02-26 11:00:00'}, {'conta_id': 190514, 'valor': 356.7, 'dt_transacao': '2021-01-05 21:00:00'}, {'conta_id': 190515, 'valor': 105.78, 'dt_transacao': '2021-01-01 01:00:00'}, {'conta_id': 190515, 'valor': 1545.05, 'dt_transacao': '2021-01-26 07:00:00'}, {'conta_id': 190515, 'valor': 749.91, 'dt_transacao': '2021-01-31 06:00:00'}, {'conta_id': 190515, 'valor': 183.68, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190516, 'valor': 831.94, 'dt_transacao': '2021-01-14 10:00:00'}, {'conta_id': 190517, 'valor': 713.8, 'dt_transacao': '2021-01-23 00:00:00'}, {'conta_id': 190518, 'valor': 1300.24, 'dt_transacao': '2021-02-17 01:00:00'}, {'conta_id': 190518, 'valor': 135.6, 'dt_transacao': '2021-01-28 13:00:00'}, {'conta_id': 190518, 'valor': 603.77, 'dt_transacao': '2021-03-18 15:00:00'}, {'conta_id': 190519, 'valor': 351.49, 'dt_transacao': '2021-03-30 10:00:00'}, {'conta_id': 190519, 'valor': 1956.07, 'dt_transacao': '2021-03-27 15:00:00'}, {'conta_id': 190519, 'valor': 135.96, 'dt_transacao': '2021-01-03 19:00:00'}, {'conta_id': 190520, 'valor': 1518.12, 'dt_transacao': '2021-01-16 09:00:00'}, {'conta_id': 190520, 'valor': 473.56, 'dt_transacao': '2021-02-24 07:00:00'}, {'conta_id': 190520, 'valor': 1239.91, 'dt_transacao': '2021-01-08 04:00:00'}, {'conta_id': 190520, 'valor': 749.67, 'dt_transacao': '2021-02-07 06:00:00'}, {'conta_id': 190520, 'valor': 304.19, 'dt_transacao': '2021-01-27 20:00:00'}, {'conta_id': 190521, 'valor': 1583.77, 'dt_transacao': '2021-02-23 20:00:00'}, {'conta_id': 190521, 'valor': 1625.61, 'dt_transacao': '2021-01-04 07:00:00'}, {'conta_id': 190521, 'valor': 1694.02, 'dt_transacao': '2021-02-12 10:00:00'}, {'conta_id': 190521, 'valor': 485.26, 'dt_transacao': '2021-03-11 01:00:00'}, {'conta_id': 190521, 'valor': 1813.36, 'dt_transacao': '2021-03-27 07:00:00'}, {'conta_id': 190522, 'valor': 1907.37, 'dt_transacao': '2021-01-07 20:00:00'}, {'conta_id': 190523, 'valor': 1450.25, 'dt_transacao': '2021-02-09 05:00:00'}, {'conta_id': 190523, 'valor': 1443.63, 'dt_transacao': '2021-02-01 13:00:00'}, {'conta_id': 190523, 'valor': 353.69, 'dt_transacao': '2021-02-01 09:00:00'}, {'conta_id': 190523, 'valor': 571.42, 'dt_transacao': '2021-02-27 20:00:00'}, {'conta_id': 190524, 'valor': 883.31, 'dt_transacao': '2021-03-06 19:00:00'}, {'conta_id': 190524, 'valor': 1470.12, 'dt_transacao': '2021-01-20 15:00:00'}, {'conta_id': 190524, 'valor': 40.6, 'dt_transacao': '2021-02-18 13:00:00'}, {'conta_id': 190524, 'valor': 1708.23, 'dt_transacao': '2021-02-08 19:00:00'}, {'conta_id': 190525, 'valor': 750.92, 'dt_transacao': '2021-03-19 15:00:00'}, {'conta_id': 190525, 'valor': 144.81, 'dt_transacao': '2021-01-18 22:00:00'}, {'conta_id': 190525, 'valor': 939.05, 'dt_transacao': '2021-01-20 06:00:00'}, {'conta_id': 190526, 'valor': 979.61, 'dt_transacao': '2021-03-24 18:00:00'}, {'conta_id': 190526, 'valor': 143.4, 'dt_transacao': '2021-01-05 03:00:00'}, {'conta_id': 190526, 'valor': 770.27, 'dt_transacao': '2021-03-14 05:00:00'}, {'conta_id': 190526, 'valor': 389.26, 'dt_transacao': '2021-02-25 08:00:00'}, {'conta_id': 190527, 'valor': 955.97, 'dt_transacao': '2021-02-16 20:00:00'}, {'conta_id': 190527, 'valor': 1413.41, 'dt_transacao': '2021-01-04 22:00:00'}, {'conta_id': 190528, 'valor': 1542.17, 'dt_transacao': '2021-03-24 22:00:00'}, {'conta_id': 190529, 'valor': 1383.98, 'dt_transacao': '2021-03-06 17:00:00'}, {'conta_id': 190529, 'valor': 1317.31, 'dt_transacao': '2021-02-18 18:00:00'}, {'conta_id': 190529, 'valor': 616.65, 'dt_transacao': '2021-02-22 23:00:00'}, {'conta_id': 190530, 'valor': 1974.06, 'dt_transacao': '2021-03-04 00:00:00'}, {'conta_id': 190531, 'valor': 749.93, 'dt_transacao': '2021-04-01 04:00:00'}, {'conta_id': 190531, 'valor': 396.84, 'dt_transacao': '2021-01-24 16:00:00'}, {'conta_id': 190532, 'valor': 1965.91, 'dt_transacao': '2021-02-22 14:00:00'}, {'conta_id': 190532, 'valor': 1254.05, 'dt_transacao': '2021-03-21 03:00:00'}, {'conta_id': 190532, 'valor': 1736.86, 'dt_transacao': '2021-01-22 02:00:00'}, {'conta_id': 190533, 'valor': 470.33, 'dt_transacao': '2021-03-04 21:00:00'}, {'conta_id': 190533, 'valor': 1521.31, 'dt_transacao': '2021-01-02 01:00:00'}, {'conta_id': 190533, 'valor': 532.9, 'dt_transacao': '2021-01-01 21:00:00'}, {'conta_id': 190533, 'valor': 1165.31, 'dt_transacao': '2021-01-06 03:00:00'}, {'conta_id': 190533, 'valor': 422.93, 'dt_transacao': '2021-01-01 12:00:00'}, {'conta_id': 190534, 'valor': 1998.89, 'dt_transacao': '2021-01-31 11:00:00'}, {'conta_id': 190534, 'valor': 1330.49, 'dt_transacao': '2021-02-18 19:00:00'}, {'conta_id': 190534, 'valor': 115.43, 'dt_transacao': '2021-02-15 10:00:00'}, {'conta_id': 190535, 'valor': 737.49, 'dt_transacao': '2021-03-21 21:00:00'}, {'conta_id': 190535, 'valor': 1318.09, 'dt_transacao': '2021-03-02 16:00:00'}, {'conta_id': 190536, 'valor': 1356.18, 'dt_transacao': '2021-01-30 19:00:00'}, {'conta_id': 190536, 'valor': 969.59, 'dt_transacao': '2021-03-20 02:00:00'}, {'conta_id': 190536, 'valor': 697.32, 'dt_transacao': '2021-03-02 21:00:00'}, {'conta_id': 190537, 'valor': 1593.84, 'dt_transacao': '2021-02-08 21:00:00'}, {'conta_id': 190537, 'valor': 1992.61, 'dt_transacao': '2021-03-01 05:00:00'}, {'conta_id': 190537, 'valor': 1792.92, 'dt_transacao': '2021-01-15 15:00:00'}, {'conta_id': 190537, 'valor': 1325.62, 'dt_transacao': '2021-01-31 07:00:00'}, {'conta_id': 190538, 'valor': 1532.85, 'dt_transacao': '2021-01-29 23:00:00'}, {'conta_id': 190538, 'valor': 516.0, 'dt_transacao': '2021-03-02 21:00:00'}, {'conta_id': 190538, 'valor': 247.07, 'dt_transacao': '2021-01-18 17:00:00'}, {'conta_id': 190538, 'valor': 1267.46, 'dt_transacao': '2021-03-04 02:00:00'}, {'conta_id': 190539, 'valor': 447.88, 'dt_transacao': '2021-01-15 09:00:00'}, {'conta_id': 190539, 'valor': 176.27, 'dt_transacao': '2021-01-12 10:00:00'}, {'conta_id': 190540, 'valor': 1474.1, 'dt_transacao': '2021-02-09 01:00:00'}, {'conta_id': 190540, 'valor': 1180.53, 'dt_transacao': '2021-01-25 12:00:00'}, {'conta_id': 190540, 'valor': 1959.72, 'dt_transacao': '2021-02-10 11:00:00'}, {'conta_id': 190540, 'valor': 1595.24, 'dt_transacao': '2021-01-17 16:00:00'}, {'conta_id': 190540, 'valor': 1327.82, 'dt_transacao': '2021-03-17 07:00:00'}, {'conta_id': 190541, 'valor': 1000.17, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190541, 'valor': 1203.33, 'dt_transacao': '2021-01-19 23:00:00'}, {'conta_id': 190542, 'valor': 1408.58, 'dt_transacao': '2021-02-18 01:00:00'}, {'conta_id': 190542, 'valor': 1962.3, 'dt_transacao': '2021-02-12 04:00:00'}, {'conta_id': 190542, 'valor': 1655.94, 'dt_transacao': '2021-01-27 23:00:00'}, {'conta_id': 190543, 'valor': 785.18, 'dt_transacao': '2021-02-24 21:00:00'}, {'conta_id': 190543, 'valor': 947.37, 'dt_transacao': '2021-02-14 12:00:00'}, {'conta_id': 190543, 'valor': 782.74, 'dt_transacao': '2021-03-09 23:00:00'}, {'conta_id': 190543, 'valor': 566.8, 'dt_transacao': '2021-03-31 06:00:00'}, {'conta_id': 190543, 'valor': 1402.52, 'dt_transacao': '2021-03-12 12:00:00'}, {'conta_id': 190544, 'valor': 275.98, 'dt_transacao': '2021-03-31 15:00:00'}, {'conta_id': 190544, 'valor': 582.87, 'dt_transacao': '2021-01-19 06:00:00'}, {'conta_id': 190544, 'valor': 437.05, 'dt_transacao': '2021-01-26 23:00:00'}, {'conta_id': 190544, 'valor': 1302.82, 'dt_transacao': '2021-02-16 23:00:00'}, {'conta_id': 190544, 'valor': 1712.43, 'dt_transacao': '2021-01-14 21:00:00'}, {'conta_id': 190545, 'valor': 1890.18, 'dt_transacao': '2021-01-16 16:00:00'}, {'conta_id': 190545, 'valor': 1300.55, 'dt_transacao': '2021-03-17 13:00:00'}, {'conta_id': 190546, 'valor': 1376.28, 'dt_transacao': '2021-01-23 20:00:00'}, {'conta_id': 190546, 'valor': 1490.66, 'dt_transacao': '2021-03-30 21:00:00'}, {'conta_id': 190546, 'valor': 1317.38, 'dt_transacao': '2021-03-01 20:00:00'}, {'conta_id': 190547, 'valor': 978.95, 'dt_transacao': '2021-03-27 16:00:00'}, {'conta_id': 190548, 'valor': 1999.91, 'dt_transacao': '2021-03-31 13:00:00'}, {'conta_id': 190548, 'valor': 597.43, 'dt_transacao': '2021-01-17 03:00:00'}, {'conta_id': 190548, 'valor': 651.13, 'dt_transacao': '2021-03-10 05:00:00'}, {'conta_id': 190548, 'valor': 811.26, 'dt_transacao': '2021-03-31 00:00:00'}, {'conta_id': 190549, 'valor': 1724.14, 'dt_transacao': '2021-03-30 05:00:00'}, {'conta_id': 190549, 'valor': 11.64, 'dt_transacao': '2021-02-27 10:00:00'}, {'conta_id': 190550, 'valor': 170.14, 'dt_transacao': '2021-01-16 16:00:00'}, {'conta_id': 190551, 'valor': 1892.48, 'dt_transacao': '2021-03-08 09:00:00'}, {'conta_id': 190552, 'valor': 1325.57, 'dt_transacao': '2021-01-06 07:00:00'}, {'conta_id': 190553, 'valor': 257.86, 'dt_transacao': '2021-03-21 13:00:00'}, {'conta_id': 190553, 'valor': 149.64, 'dt_transacao': '2021-02-02 08:00:00'}, {'conta_id': 190553, 'valor': 764.18, 'dt_transacao': '2021-01-21 12:00:00'}, {'conta_id': 190554, 'valor': 1990.72, 'dt_transacao': '2021-03-31 20:00:00'}, {'conta_id': 190554, 'valor': 386.06, 'dt_transacao': '2021-02-17 18:00:00'}, {'conta_id': 190554, 'valor': 1314.98, 'dt_transacao': '2021-03-07 19:00:00'}, {'conta_id': 190555, 'valor': 540.98, 'dt_transacao': '2021-03-10 03:00:00'}, {'conta_id': 190556, 'valor': 293.51, 'dt_transacao': '2021-01-29 08:00:00'}, {'conta_id': 190556, 'valor': 20.72, 'dt_transacao': '2021-02-18 07:00:00'}, {'conta_id': 190556, 'valor': 757.89, 'dt_transacao': '2021-01-14 03:00:00'}, {'conta_id': 190557, 'valor': 199.03, 'dt_transacao': '2021-01-07 05:00:00'}, {'conta_id': 190557, 'valor': 636.67, 'dt_transacao': '2021-02-19 05:00:00'}, {'conta_id': 190557, 'valor': 691.0, 'dt_transacao': '2021-03-28 17:00:00'}, {'conta_id': 190557, 'valor': 1103.79, 'dt_transacao': '2021-02-21 15:00:00'}, {'conta_id': 190557, 'valor': 1185.03, 'dt_transacao': '2021-02-03 00:00:00'}, {'conta_id': 190558, 'valor': 1356.75, 'dt_transacao': '2021-02-23 22:00:00'}, {'conta_id': 190558, 'valor': 916.12, 'dt_transacao': '2021-02-26 11:00:00'}, {'conta_id': 190559, 'valor': 813.62, 'dt_transacao': '2021-02-10 02:00:00'}, {'conta_id': 190559, 'valor': 1269.72, 'dt_transacao': '2021-01-10 23:00:00'}, {'conta_id': 190559, 'valor': 480.83, 'dt_transacao': '2021-01-12 05:00:00'}, {'conta_id': 190560, 'valor': 1864.95, 'dt_transacao': '2021-01-18 15:00:00'}, {'conta_id': 190560, 'valor': 996.11, 'dt_transacao': '2021-03-29 21:00:00'}, {'conta_id': 190560, 'valor': 1776.27, 'dt_transacao': '2021-03-31 03:00:00'}, {'conta_id': 190560, 'valor': 811.54, 'dt_transacao': '2021-03-20 06:00:00'}, {'conta_id': 190560, 'valor': 1821.86, 'dt_transacao': '2021-01-01 22:00:00'}, {'conta_id': 190561, 'valor': 151.17, 'dt_transacao': '2021-01-18 16:00:00'}, {'conta_id': 190561, 'valor': 707.23, 'dt_transacao': '2021-01-07 13:00:00'}, {'conta_id': 190562, 'valor': 599.06, 'dt_transacao': '2021-02-07 08:00:00'}, {'conta_id': 190562, 'valor': 426.92, 'dt_transacao': '2021-03-22 18:00:00'}, {'conta_id': 190562, 'valor': 787.85, 'dt_transacao': '2021-02-09 20:00:00'}, {'conta_id': 190563, 'valor': 597.4, 'dt_transacao': '2021-01-14 00:00:00'}, {'conta_id': 190563, 'valor': 700.09, 'dt_transacao': '2021-01-24 12:00:00'}, {'conta_id': 190563, 'valor': 709.71, 'dt_transacao': '2021-01-13 13:00:00'}, {'conta_id': 190564, 'valor': 14.81, 'dt_transacao': '2021-03-26 08:00:00'}, {'conta_id': 190564, 'valor': 452.96, 'dt_transacao': '2021-02-20 11:00:00'}, {'conta_id': 190564, 'valor': 599.27, 'dt_transacao': '2021-02-24 05:00:00'}, {'conta_id': 190564, 'valor': 1073.32, 'dt_transacao': '2021-02-04 03:00:00'}, {'conta_id': 190565, 'valor': 222.22, 'dt_transacao': '2021-02-06 01:00:00'}, {'conta_id': 190565, 'valor': 268.23, 'dt_transacao': '2021-02-23 23:00:00'}, {'conta_id': 190566, 'valor': 978.65, 'dt_transacao': '2021-03-05 21:00:00'}, {'conta_id': 190566, 'valor': 1428.93, 'dt_transacao': '2021-02-11 12:00:00'}, {'conta_id': 190566, 'valor': 1004.22, 'dt_transacao': '2021-02-19 04:00:00'}, {'conta_id': 190566, 'valor': 220.36, 'dt_transacao': '2021-01-14 00:00:00'}, {'conta_id': 190567, 'valor': 1952.9, 'dt_transacao': '2021-01-02 13:00:00'}, {'conta_id': 190567, 'valor': 960.91, 'dt_transacao': '2021-02-06 15:00:00'}, {'conta_id': 190567, 'valor': 166.15, 'dt_transacao': '2021-02-24 07:00:00'}, {'conta_id': 190567, 'valor': 1235.61, 'dt_transacao': '2021-03-07 05:00:00'}, {'conta_id': 190568, 'valor': 1413.22, 'dt_transacao': '2021-01-14 22:00:00'}, {'conta_id': 190568, 'valor': 1461.18, 'dt_transacao': '2021-03-20 05:00:00'}, {'conta_id': 190568, 'valor': 378.15, 'dt_transacao': '2021-03-17 20:00:00'}, {'conta_id': 190568, 'valor': 1269.32, 'dt_transacao': '2021-01-30 03:00:00'}, {'conta_id': 190569, 'valor': 400.09, 'dt_transacao': '2021-01-07 12:00:00'}, {'conta_id': 190569, 'valor': 815.19, 'dt_transacao': '2021-02-09 22:00:00'}, {'conta_id': 190569, 'valor': 1266.99, 'dt_transacao': '2021-01-30 07:00:00'}, {'conta_id': 190569, 'valor': 673.02, 'dt_transacao': '2021-03-03 01:00:00'}, {'conta_id': 190569, 'valor': 1020.22, 'dt_transacao': '2021-03-24 01:00:00'}, {'conta_id': 190570, 'valor': 1790.32, 'dt_transacao': '2021-01-24 01:00:00'}, {'conta_id': 190570, 'valor': 2.07, 'dt_transacao': '2021-03-26 23:00:00'}, {'conta_id': 190570, 'valor': 643.96, 'dt_transacao': '2021-02-06 15:00:00'}, {'conta_id': 190571, 'valor': 223.95, 'dt_transacao': '2021-03-27 00:00:00'}, {'conta_id': 190571, 'valor': 1366.24, 'dt_transacao': '2021-01-17 10:00:00'}, {'conta_id': 190571, 'valor': 721.78, 'dt_transacao': '2021-02-16 23:00:00'}, {'conta_id': 190572, 'valor': 1259.49, 'dt_transacao': '2021-03-19 03:00:00'}, {'conta_id': 190572, 'valor': 83.04, 'dt_transacao': '2021-03-25 06:00:00'}, {'conta_id': 190573, 'valor': 1067.58, 'dt_transacao': '2021-01-20 01:00:00'}, {'conta_id': 190573, 'valor': 256.54, 'dt_transacao': '2021-02-23 17:00:00'}, {'conta_id': 190574, 'valor': 838.59, 'dt_transacao': '2021-03-06 00:00:00'}, {'conta_id': 190574, 'valor': 334.7, 'dt_transacao': '2021-02-05 22:00:00'}, {'conta_id': 190574, 'valor': 1935.01, 'dt_transacao': '2021-02-07 01:00:00'}, {'conta_id': 190574, 'valor': 1911.65, 'dt_transacao': '2021-01-08 22:00:00'}, {'conta_id': 190575, 'valor': 1214.83, 'dt_transacao': '2021-01-27 05:00:00'}, {'conta_id': 190575, 'valor': 8.2, 'dt_transacao': '2021-03-15 06:00:00'}, {'conta_id': 190575, 'valor': 1184.63, 'dt_transacao': '2021-01-02 16:00:00'}, {'conta_id': 190575, 'valor': 911.43, 'dt_transacao': '2021-04-01 23:00:00'}, {'conta_id': 190576, 'valor': 92.18, 'dt_transacao': '2021-01-31 09:00:00'}, {'conta_id': 190576, 'valor': 79.14, 'dt_transacao': '2021-01-19 08:00:00'}, {'conta_id': 190577, 'valor': 1265.73, 'dt_transacao': '2021-02-17 10:00:00'}, {'conta_id': 190577, 'valor': 167.41, 'dt_transacao': '2021-04-01 17:00:00'}, {'conta_id': 190577, 'valor': 528.81, 'dt_transacao': '2021-01-18 13:00:00'}, {'conta_id': 190578, 'valor': 354.29, 'dt_transacao': '2021-01-03 17:00:00'}, {'conta_id': 190578, 'valor': 1314.98, 'dt_transacao': '2021-03-16 03:00:00'}, {'conta_id': 190578, 'valor': 1445.48, 'dt_transacao': '2021-03-30 12:00:00'}, {'conta_id': 190579, 'valor': 892.67, 'dt_transacao': '2021-02-25 11:00:00'}, {'conta_id': 190579, 'valor': 297.52, 'dt_transacao': '2021-03-03 02:00:00'}, {'conta_id': 190580, 'valor': 43.07, 'dt_transacao': '2021-01-21 05:00:00'}, {'conta_id': 190580, 'valor': 1635.18, 'dt_transacao': '2021-03-29 19:00:00'}, {'conta_id': 190580, 'valor': 376.51, 'dt_transacao': '2021-01-19 02:00:00'}, {'conta_id': 190581, 'valor': 768.6, 'dt_transacao': '2021-01-15 00:00:00'}, {'conta_id': 190581, 'valor': 1527.13, 'dt_transacao': '2021-03-24 01:00:00'}, {'conta_id': 190581, 'valor': 265.23, 'dt_transacao': '2021-01-11 08:00:00'}, {'conta_id': 190581, 'valor': 286.13, 'dt_transacao': '2021-03-22 19:00:00'}, {'conta_id': 190582, 'valor': 1519.58, 'dt_transacao': '2021-02-26 05:00:00'}, {'conta_id': 190582, 'valor': 615.46, 'dt_transacao': '2021-03-03 04:00:00'}, {'conta_id': 190582, 'valor': 488.37, 'dt_transacao': '2021-03-11 22:00:00'}, {'conta_id': 190582, 'valor': 535.65, 'dt_transacao': '2021-03-11 03:00:00'}, {'conta_id': 190583, 'valor': 1824.42, 'dt_transacao': '2021-01-04 07:00:00'}, {'conta_id': 190584, 'valor': 984.15, 'dt_transacao': '2021-01-15 07:00:00'}, {'conta_id': 190585, 'valor': 1791.28, 'dt_transacao': '2021-03-19 04:00:00'}, {'conta_id': 190586, 'valor': 1635.31, 'dt_transacao': '2021-02-14 03:00:00'}, {'conta_id': 190586, 'valor': 542.92, 'dt_transacao': '2021-03-05 21:00:00'}, {'conta_id': 190586, 'valor': 738.86, 'dt_transacao': '2021-02-03 22:00:00'}, {'conta_id': 190586, 'valor': 1597.57, 'dt_transacao': '2021-01-01 04:00:00'}, {'conta_id': 190586, 'valor': 1555.79, 'dt_transacao': '2021-01-19 10:00:00'}, {'conta_id': 190587, 'valor': 181.8, 'dt_transacao': '2021-01-17 05:00:00'}, {'conta_id': 190587, 'valor': 1535.3, 'dt_transacao': '2021-02-15 10:00:00'}, {'conta_id': 190588, 'valor': 837.9, 'dt_transacao': '2021-03-05 17:00:00'}, {'conta_id': 190588, 'valor': 1068.01, 'dt_transacao': '2021-03-10 17:00:00'}, {'conta_id': 190588, 'valor': 1642.99, 'dt_transacao': '2021-02-19 02:00:00'}, {'conta_id': 190588, 'valor': 1162.35, 'dt_transacao': '2021-02-23 16:00:00'}, {'conta_id': 190589, 'valor': 1117.19, 'dt_transacao': '2021-01-18 03:00:00'}, {'conta_id': 190589, 'valor': 120.1, 'dt_transacao': '2021-02-14 11:00:00'}, {'conta_id': 190589, 'valor': 1697.65, 'dt_transacao': '2021-01-18 09:00:00'}, {'conta_id': 190590, 'valor': 1924.14, 'dt_transacao': '2021-04-01 04:00:00'}, {'conta_id': 190591, 'valor': 1741.08, 'dt_transacao': '2021-01-04 04:00:00'}, {'conta_id': 190591, 'valor': 731.12, 'dt_transacao': '2021-01-11 07:00:00'}, {'conta_id': 190591, 'valor': 155.05, 'dt_transacao': '2021-03-14 13:00:00'}, {'conta_id': 190591, 'valor': 300.28, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190591, 'valor': 1487.37, 'dt_transacao': '2021-03-09 22:00:00'}, {'conta_id': 190592, 'valor': 120.94, 'dt_transacao': '2021-03-24 01:00:00'}, {'conta_id': 190592, 'valor': 1208.74, 'dt_transacao': '2021-01-23 16:00:00'}, {'conta_id': 190593, 'valor': 847.92, 'dt_transacao': '2021-01-21 08:00:00'}, {'conta_id': 190593, 'valor': 1946.27, 'dt_transacao': '2021-03-08 12:00:00'}, {'conta_id': 190594, 'valor': 955.36, 'dt_transacao': '2021-03-25 20:00:00'}, {'conta_id': 190594, 'valor': 810.79, 'dt_transacao': '2021-02-18 05:00:00'}, {'conta_id': 190594, 'valor': 110.29, 'dt_transacao': '2021-02-17 20:00:00'}, {'conta_id': 190594, 'valor': 849.02, 'dt_transacao': '2021-02-04 03:00:00'}, {'conta_id': 190595, 'valor': 213.82, 'dt_transacao': '2021-01-28 19:00:00'}, {'conta_id': 190595, 'valor': 22.94, 'dt_transacao': '2021-02-09 06:00:00'}, {'conta_id': 190595, 'valor': 1103.83, 'dt_transacao': '2021-03-23 01:00:00'}, {'conta_id': 190595, 'valor': 481.43, 'dt_transacao': '2021-01-11 18:00:00'}, {'conta_id': 190595, 'valor': 1130.03, 'dt_transacao': '2021-03-14 15:00:00'}, {'conta_id': 190596, 'valor': 1000.93, 'dt_transacao': '2021-02-10 12:00:00'}, {'conta_id': 190596, 'valor': 1451.65, 'dt_transacao': '2021-01-17 22:00:00'}, {'conta_id': 190596, 'valor': 1349.7, 'dt_transacao': '2021-02-12 13:00:00'}, {'conta_id': 190596, 'valor': 1793.13, 'dt_transacao': '2021-03-06 03:00:00'}, {'conta_id': 190597, 'valor': 303.92, 'dt_transacao': '2021-03-01 02:00:00'}, {'conta_id': 190597, 'valor': 1770.52, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190598, 'valor': 582.39, 'dt_transacao': '2021-03-21 13:00:00'}, {'conta_id': 190598, 'valor': 80.99, 'dt_transacao': '2021-03-29 12:00:00'}, {'conta_id': 190598, 'valor': 478.98, 'dt_transacao': '2021-03-20 19:00:00'}, {'conta_id': 190599, 'valor': 189.13, 'dt_transacao': '2021-02-10 05:00:00'}, {'conta_id': 190599, 'valor': 1868.18, 'dt_transacao': '2021-01-21 02:00:00'}, {'conta_id': 190599, 'valor': 560.95, 'dt_transacao': '2021-02-16 11:00:00'}, {'conta_id': 190599, 'valor': 179.37, 'dt_transacao': '2021-01-23 22:00:00'}, {'conta_id': 190600, 'valor': 727.58, 'dt_transacao': '2021-02-06 12:00:00'}, {'conta_id': 190601, 'valor': 1012.38, 'dt_transacao': '2021-02-03 21:00:00'}, {'conta_id': 190601, 'valor': 86.31, 'dt_transacao': '2021-03-24 01:00:00'}, {'conta_id': 190601, 'valor': 1482.34, 'dt_transacao': '2021-02-01 22:00:00'}, {'conta_id': 190601, 'valor': 1642.65, 'dt_transacao': '2021-01-02 04:00:00'}, {'conta_id': 190602, 'valor': 1748.44, 'dt_transacao': '2021-03-27 02:00:00'}, {'conta_id': 190602, 'valor': 698.53, 'dt_transacao': '2021-03-01 20:00:00'}, {'conta_id': 190602, 'valor': 1070.7, 'dt_transacao': '2021-03-01 03:00:00'}, {'conta_id': 190602, 'valor': 172.56, 'dt_transacao': '2021-01-14 12:00:00'}, {'conta_id': 190603, 'valor': 1404.88, 'dt_transacao': '2021-03-02 03:00:00'}, {'conta_id': 190603, 'valor': 557.47, 'dt_transacao': '2021-01-05 17:00:00'}, {'conta_id': 190604, 'valor': 1872.98, 'dt_transacao': '2021-02-16 13:00:00'}, {'conta_id': 190605, 'valor': 168.1, 'dt_transacao': '2021-01-09 08:00:00'}, {'conta_id': 190605, 'valor': 810.73, 'dt_transacao': '2021-02-08 22:00:00'}, {'conta_id': 190606, 'valor': 1926.77, 'dt_transacao': '2021-02-08 05:00:00'}, {'conta_id': 190606, 'valor': 398.01, 'dt_transacao': '2021-02-23 15:00:00'}, {'conta_id': 190606, 'valor': 951.52, 'dt_transacao': '2021-03-30 05:00:00'}, {'conta_id': 190607, 'valor': 1328.46, 'dt_transacao': '2021-02-10 00:00:00'}, {'conta_id': 190607, 'valor': 301.86, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190607, 'valor': 999.39, 'dt_transacao': '2021-02-10 03:00:00'}, {'conta_id': 190608, 'valor': 1491.67, 'dt_transacao': '2021-01-25 13:00:00'}, {'conta_id': 190608, 'valor': 1393.2, 'dt_transacao': '2021-01-22 19:00:00'}, {'conta_id': 190609, 'valor': 604.59, 'dt_transacao': '2021-02-09 16:00:00'}, {'conta_id': 190610, 'valor': 1948.17, 'dt_transacao': '2021-02-12 03:00:00'}, {'conta_id': 190610, 'valor': 1852.77, 'dt_transacao': '2021-02-28 13:00:00'}, {'conta_id': 190610, 'valor': 746.87, 'dt_transacao': '2021-03-12 19:00:00'}, {'conta_id': 190610, 'valor': 326.15, 'dt_transacao': '2021-02-25 12:00:00'}, {'conta_id': 190610, 'valor': 570.77, 'dt_transacao': '2021-03-26 22:00:00'}, {'conta_id': 190611, 'valor': 993.31, 'dt_transacao': '2021-03-21 17:00:00'}, {'conta_id': 190612, 'valor': 1103.62, 'dt_transacao': '2021-01-15 06:00:00'}, {'conta_id': 190612, 'valor': 1712.83, 'dt_transacao': '2021-01-16 11:00:00'}, {'conta_id': 190612, 'valor': 1935.26, 'dt_transacao': '2021-03-01 13:00:00'}, {'conta_id': 190612, 'valor': 1816.08, 'dt_transacao': '2021-03-24 22:00:00'}, {'conta_id': 190613, 'valor': 1677.13, 'dt_transacao': '2021-02-23 21:00:00'}, {'conta_id': 190613, 'valor': 483.87, 'dt_transacao': '2021-02-23 01:00:00'}, {'conta_id': 190613, 'valor': 32.04, 'dt_transacao': '2021-01-20 11:00:00'}, {'conta_id': 190614, 'valor': 1232.92, 'dt_transacao': '2021-02-20 01:00:00'}, {'conta_id': 190614, 'valor': 1602.86, 'dt_transacao': '2021-03-15 13:00:00'}, {'conta_id': 190615, 'valor': 1420.34, 'dt_transacao': '2021-01-17 16:00:00'}, {'conta_id': 190616, 'valor': 875.4, 'dt_transacao': '2021-03-22 18:00:00'}, {'conta_id': 190617, 'valor': 373.92, 'dt_transacao': '2021-03-25 07:00:00'}, {'conta_id': 190617, 'valor': 594.55, 'dt_transacao': '2021-03-10 07:00:00'}, {'conta_id': 190617, 'valor': 1452.9, 'dt_transacao': '2021-02-18 08:00:00'}, {'conta_id': 190618, 'valor': 1854.47, 'dt_transacao': '2021-01-16 03:00:00'}, {'conta_id': 190618, 'valor': 284.53, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190618, 'valor': 186.46, 'dt_transacao': '2021-03-05 11:00:00'}, {'conta_id': 190618, 'valor': 1825.53, 'dt_transacao': '2021-01-27 12:00:00'}, {'conta_id': 190619, 'valor': 751.13, 'dt_transacao': '2021-02-10 20:00:00'}, {'conta_id': 190619, 'valor': 1816.7, 'dt_transacao': '2021-01-21 05:00:00'}, {'conta_id': 190619, 'valor': 598.21, 'dt_transacao': '2021-01-23 21:00:00'}, {'conta_id': 190619, 'valor': 1135.02, 'dt_transacao': '2021-03-19 01:00:00'}, {'conta_id': 190620, 'valor': 386.5, 'dt_transacao': '2021-01-04 17:00:00'}, {'conta_id': 190621, 'valor': 440.3, 'dt_transacao': '2021-03-30 04:00:00'}, {'conta_id': 190621, 'valor': 535.91, 'dt_transacao': '2021-03-03 06:00:00'}, {'conta_id': 190622, 'valor': 346.75, 'dt_transacao': '2021-03-29 22:00:00'}, {'conta_id': 190622, 'valor': 562.88, 'dt_transacao': '2021-02-20 03:00:00'}, {'conta_id': 190622, 'valor': 1167.71, 'dt_transacao': '2021-03-17 09:00:00'}, {'conta_id': 190623, 'valor': 216.01, 'dt_transacao': '2021-02-19 13:00:00'}, {'conta_id': 190623, 'valor': 400.24, 'dt_transacao': '2021-02-15 11:00:00'}, {'conta_id': 190623, 'valor': 1795.89, 'dt_transacao': '2021-01-09 14:00:00'}, {'conta_id': 190623, 'valor': 335.33, 'dt_transacao': '2021-01-05 16:00:00'}, {'conta_id': 190623, 'valor': 506.87, 'dt_transacao': '2021-02-22 09:00:00'}, {'conta_id': 190624, 'valor': 716.13, 'dt_transacao': '2021-01-10 06:00:00'}, {'conta_id': 190624, 'valor': 1314.25, 'dt_transacao': '2021-02-17 04:00:00'}, {'conta_id': 190625, 'valor': 1135.37, 'dt_transacao': '2021-02-11 07:00:00'}, {'conta_id': 190625, 'valor': 1123.72, 'dt_transacao': '2021-02-19 09:00:00'}, {'conta_id': 190625, 'valor': 1204.0, 'dt_transacao': '2021-03-09 06:00:00'}, {'conta_id': 190625, 'valor': 206.74, 'dt_transacao': '2021-02-05 22:00:00'}, {'conta_id': 190626, 'valor': 1605.65, 'dt_transacao': '2021-03-05 01:00:00'}, {'conta_id': 190627, 'valor': 1153.25, 'dt_transacao': '2021-02-06 00:00:00'}, {'conta_id': 190627, 'valor': 1287.69, 'dt_transacao': '2021-03-15 00:00:00'}, {'conta_id': 190628, 'valor': 940.32, 'dt_transacao': '2021-02-23 23:00:00'}, {'conta_id': 190629, 'valor': 95.1, 'dt_transacao': '2021-02-08 22:00:00'}, {'conta_id': 190629, 'valor': 1967.95, 'dt_transacao': '2021-03-05 20:00:00'}, {'conta_id': 190629, 'valor': 1934.34, 'dt_transacao': '2021-01-16 11:00:00'}, {'conta_id': 190629, 'valor': 1875.47, 'dt_transacao': '2021-01-14 00:00:00'}, {'conta_id': 190629, 'valor': 448.8, 'dt_transacao': '2021-01-02 04:00:00'}, {'conta_id': 190630, 'valor': 1979.14, 'dt_transacao': '2021-03-30 01:00:00'}, {'conta_id': 190630, 'valor': 520.12, 'dt_transacao': '2021-01-06 03:00:00'}, {'conta_id': 190630, 'valor': 96.96, 'dt_transacao': '2021-02-08 02:00:00'}, {'conta_id': 190630, 'valor': 528.19, 'dt_transacao': '2021-03-30 19:00:00'}, {'conta_id': 190630, 'valor': 1583.46, 'dt_transacao': '2021-02-18 08:00:00'}, {'conta_id': 190631, 'valor': 1336.8, 'dt_transacao': '2021-03-08 16:00:00'}, {'conta_id': 190631, 'valor': 481.81, 'dt_transacao': '2021-03-03 21:00:00'}, {'conta_id': 190631, 'valor': 980.72, 'dt_transacao': '2021-02-05 10:00:00'}, {'conta_id': 190631, 'valor': 1545.52, 'dt_transacao': '2021-03-03 07:00:00'}, {'conta_id': 190632, 'valor': 1943.56, 'dt_transacao': '2021-01-08 06:00:00'}, {'conta_id': 190633, 'valor': 1628.18, 'dt_transacao': '2021-03-09 16:00:00'}, {'conta_id': 190633, 'valor': 1717.06, 'dt_transacao': '2021-02-09 22:00:00'}, {'conta_id': 190634, 'valor': 1027.41, 'dt_transacao': '2021-02-21 06:00:00'}, {'conta_id': 190634, 'valor': 138.21, 'dt_transacao': '2021-02-24 05:00:00'}, {'conta_id': 190634, 'valor': 1372.85, 'dt_transacao': '2021-03-15 20:00:00'}, {'conta_id': 190634, 'valor': 205.03, 'dt_transacao': '2021-03-01 03:00:00'}, {'conta_id': 190634, 'valor': 1490.31, 'dt_transacao': '2021-01-31 00:00:00'}, {'conta_id': 190635, 'valor': 1853.93, 'dt_transacao': '2021-01-11 11:00:00'}, {'conta_id': 190636, 'valor': 545.91, 'dt_transacao': '2021-01-13 01:00:00'}, {'conta_id': 190636, 'valor': 1297.59, 'dt_transacao': '2021-02-24 06:00:00'}, {'conta_id': 190636, 'valor': 588.72, 'dt_transacao': '2021-02-21 05:00:00'}, {'conta_id': 190636, 'valor': 1188.5, 'dt_transacao': '2021-03-09 19:00:00'}, {'conta_id': 190637, 'valor': 107.22, 'dt_transacao': '2021-01-05 18:00:00'}, {'conta_id': 190638, 'valor': 786.38, 'dt_transacao': '2021-01-29 23:00:00'}, {'conta_id': 190638, 'valor': 1648.31, 'dt_transacao': '2021-03-30 08:00:00'}, {'conta_id': 190638, 'valor': 990.85, 'dt_transacao': '2021-01-22 06:00:00'}, {'conta_id': 190638, 'valor': 695.92, 'dt_transacao': '2021-02-06 22:00:00'}, {'conta_id': 190639, 'valor': 630.91, 'dt_transacao': '2021-01-09 05:00:00'}, {'conta_id': 190639, 'valor': 375.5, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190640, 'valor': 1358.73, 'dt_transacao': '2021-01-02 19:00:00'}, {'conta_id': 190640, 'valor': 704.94, 'dt_transacao': '2021-03-02 18:00:00'}, {'conta_id': 190641, 'valor': 1051.01, 'dt_transacao': '2021-01-22 04:00:00'}, {'conta_id': 190641, 'valor': 1550.74, 'dt_transacao': '2021-01-13 09:00:00'}, {'conta_id': 190642, 'valor': 918.04, 'dt_transacao': '2021-02-15 09:00:00'}, {'conta_id': 190642, 'valor': 1420.6, 'dt_transacao': '2021-03-08 21:00:00'}, {'conta_id': 190642, 'valor': 1714.06, 'dt_transacao': '2021-03-18 15:00:00'}, {'conta_id': 190643, 'valor': 62.78, 'dt_transacao': '2021-03-15 23:00:00'}, {'conta_id': 190643, 'valor': 1391.92, 'dt_transacao': '2021-02-25 11:00:00'}, {'conta_id': 190643, 'valor': 413.41, 'dt_transacao': '2021-01-12 06:00:00'}, {'conta_id': 190643, 'valor': 1841.37, 'dt_transacao': '2021-02-18 20:00:00'}, {'conta_id': 190644, 'valor': 835.17, 'dt_transacao': '2021-02-05 09:00:00'}, {'conta_id': 190644, 'valor': 381.95, 'dt_transacao': '2021-03-16 16:00:00'}, {'conta_id': 190644, 'valor': 1403.97, 'dt_transacao': '2021-03-20 03:00:00'}, {'conta_id': 190644, 'valor': 867.49, 'dt_transacao': '2021-02-28 18:00:00'}, {'conta_id': 190644, 'valor': 1936.34, 'dt_transacao': '2021-02-14 09:00:00'}, {'conta_id': 190645, 'valor': 974.16, 'dt_transacao': '2021-01-09 09:00:00'}, {'conta_id': 190645, 'valor': 32.17, 'dt_transacao': '2021-01-06 06:00:00'}, {'conta_id': 190645, 'valor': 566.75, 'dt_transacao': '2021-03-17 03:00:00'}, {'conta_id': 190646, 'valor': 276.3, 'dt_transacao': '2021-02-06 16:00:00'}, {'conta_id': 190646, 'valor': 1330.99, 'dt_transacao': '2021-03-13 04:00:00'}, {'conta_id': 190646, 'valor': 1907.42, 'dt_transacao': '2021-02-04 04:00:00'}, {'conta_id': 190647, 'valor': 1623.61, 'dt_transacao': '2021-03-23 12:00:00'}, {'conta_id': 190647, 'valor': 845.7, 'dt_transacao': '2021-01-13 08:00:00'}, {'conta_id': 190648, 'valor': 272.25, 'dt_transacao': '2021-03-12 06:00:00'}, {'conta_id': 190648, 'valor': 602.82, 'dt_transacao': '2021-01-11 09:00:00'}, {'conta_id': 190648, 'valor': 1133.76, 'dt_transacao': '2021-03-22 14:00:00'}, {'conta_id': 190649, 'valor': 1869.82, 'dt_transacao': '2021-02-15 14:00:00'}, {'conta_id': 190649, 'valor': 709.03, 'dt_transacao': '2021-03-01 13:00:00'}, {'conta_id': 190649, 'valor': 1922.93, 'dt_transacao': '2021-03-01 00:00:00'}, {'conta_id': 190649, 'valor': 1825.2, 'dt_transacao': '2021-01-04 13:00:00'}, {'conta_id': 190649, 'valor': 177.77, 'dt_transacao': '2021-01-18 19:00:00'}, {'conta_id': 190650, 'valor': 806.48, 'dt_transacao': '2021-01-01 14:00:00'}, {'conta_id': 190650, 'valor': 1615.15, 'dt_transacao': '2021-03-08 14:00:00'}, {'conta_id': 190651, 'valor': 1131.32, 'dt_transacao': '2021-03-30 10:00:00'}, {'conta_id': 190651, 'valor': 1728.76, 'dt_transacao': '2021-01-16 23:00:00'}, {'conta_id': 190651, 'valor': 1169.65, 'dt_transacao': '2021-03-07 23:00:00'}, {'conta_id': 190652, 'valor': 1256.85, 'dt_transacao': '2021-01-01 15:00:00'}, {'conta_id': 190652, 'valor': 1247.7, 'dt_transacao': '2021-01-26 03:00:00'}, {'conta_id': 190652, 'valor': 135.22, 'dt_transacao': '2021-01-20 19:00:00'}, {'conta_id': 190653, 'valor': 112.25, 'dt_transacao': '2021-01-16 09:00:00'}, {'conta_id': 190653, 'valor': 809.71, 'dt_transacao': '2021-01-24 07:00:00'}, {'conta_id': 190653, 'valor': 1683.6, 'dt_transacao': '2021-03-11 08:00:00'}, {'conta_id': 190653, 'valor': 1643.55, 'dt_transacao': '2021-03-29 14:00:00'}, {'conta_id': 190654, 'valor': 342.38, 'dt_transacao': '2021-03-31 17:00:00'}, {'conta_id': 190654, 'valor': 401.5, 'dt_transacao': '2021-03-03 09:00:00'}, {'conta_id': 190655, 'valor': 928.46, 'dt_transacao': '2021-02-24 04:00:00'}, {'conta_id': 190656, 'valor': 1290.57, 'dt_transacao': '2021-03-26 19:00:00'}, {'conta_id': 190657, 'valor': 716.14, 'dt_transacao': '2021-03-06 06:00:00'}, {'conta_id': 190657, 'valor': 698.32, 'dt_transacao': '2021-04-01 16:00:00'}, {'conta_id': 190658, 'valor': 557.4, 'dt_transacao': '2021-03-26 09:00:00'}, {'conta_id': 190658, 'valor': 1847.03, 'dt_transacao': '2021-01-09 20:00:00'}, {'conta_id': 190659, 'valor': 1910.43, 'dt_transacao': '2021-02-14 18:00:00'}, {'conta_id': 190659, 'valor': 1235.33, 'dt_transacao': '2021-02-21 02:00:00'}, {'conta_id': 190659, 'valor': 90.81, 'dt_transacao': '2021-01-21 07:00:00'}, {'conta_id': 190659, 'valor': 193.98, 'dt_transacao': '2021-01-14 15:00:00'}, {'conta_id': 190659, 'valor': 1868.53, 'dt_transacao': '2021-03-02 10:00:00'}, {'conta_id': 190660, 'valor': 501.72, 'dt_transacao': '2021-01-05 23:00:00'}, {'conta_id': 190660, 'valor': 146.69, 'dt_transacao': '2021-02-17 19:00:00'}, {'conta_id': 190660, 'valor': 1262.96, 'dt_transacao': '2021-01-04 05:00:00'}, {'conta_id': 190660, 'valor': 643.0, 'dt_transacao': '2021-03-31 14:00:00'}, {'conta_id': 190660, 'valor': 755.88, 'dt_transacao': '2021-03-23 18:00:00'}, {'conta_id': 190661, 'valor': 1484.57, 'dt_transacao': '2021-02-25 16:00:00'}, {'conta_id': 190661, 'valor': 1617.54, 'dt_transacao': '2021-01-25 02:00:00'}, {'conta_id': 190662, 'valor': 1137.02, 'dt_transacao': '2021-01-10 10:00:00'}, {'conta_id': 190662, 'valor': 371.42, 'dt_transacao': '2021-03-24 16:00:00'}, {'conta_id': 190662, 'valor': 992.85, 'dt_transacao': '2021-01-27 18:00:00'}, {'conta_id': 190662, 'valor': 1655.32, 'dt_transacao': '2021-03-06 11:00:00'}, {'conta_id': 190663, 'valor': 1564.99, 'dt_transacao': '2021-02-08 03:00:00'}, {'conta_id': 190663, 'valor': 236.74, 'dt_transacao': '2021-01-30 01:00:00'}, {'conta_id': 190664, 'valor': 1745.01, 'dt_transacao': '2021-01-21 09:00:00'}, {'conta_id': 190665, 'valor': 1306.81, 'dt_transacao': '2021-01-05 10:00:00'}, {'conta_id': 190665, 'valor': 843.39, 'dt_transacao': '2021-01-26 10:00:00'}, {'conta_id': 190665, 'valor': 1776.17, 'dt_transacao': '2021-01-08 04:00:00'}, {'conta_id': 190666, 'valor': 1899.87, 'dt_transacao': '2021-03-18 23:00:00'}, {'conta_id': 190667, 'valor': 1015.7, 'dt_transacao': '2021-03-25 11:00:00'}, {'conta_id': 190667, 'valor': 863.6, 'dt_transacao': '2021-02-08 07:00:00'}, {'conta_id': 190667, 'valor': 43.87, 'dt_transacao': '2021-01-08 00:00:00'}, {'conta_id': 190667, 'valor': 1547.45, 'dt_transacao': '2021-01-26 17:00:00'}, {'conta_id': 190668, 'valor': 860.01, 'dt_transacao': '2021-02-24 17:00:00'}, {'conta_id': 190668, 'valor': 276.4, 'dt_transacao': '2021-02-26 03:00:00'}, {'conta_id': 190669, 'valor': 184.89, 'dt_transacao': '2021-01-02 07:00:00'}, {'conta_id': 190669, 'valor': 826.72, 'dt_transacao': '2021-03-10 18:00:00'}, {'conta_id': 190670, 'valor': 565.91, 'dt_transacao': '2021-02-10 11:00:00'}, {'conta_id': 190670, 'valor': 1205.23, 'dt_transacao': '2021-02-01 09:00:00'}, {'conta_id': 190671, 'valor': 412.52, 'dt_transacao': '2021-04-01 23:00:00'}, {'conta_id': 190671, 'valor': 1627.04, 'dt_transacao': '2021-01-29 14:00:00'}, {'conta_id': 190671, 'valor': 1426.55, 'dt_transacao': '2021-02-16 17:00:00'}, {'conta_id': 190671, 'valor': 586.07, 'dt_transacao': '2021-03-25 07:00:00'}, {'conta_id': 190671, 'valor': 1054.16, 'dt_transacao': '2021-02-04 10:00:00'}, {'conta_id': 190672, 'valor': 1824.19, 'dt_transacao': '2021-03-08 07:00:00'}, {'conta_id': 190672, 'valor': 610.57, 'dt_transacao': '2021-03-12 06:00:00'}, {'conta_id': 190673, 'valor': 998.32, 'dt_transacao': '2021-01-05 08:00:00'}, {'conta_id': 190674, 'valor': 1725.68, 'dt_transacao': '2021-02-19 14:00:00'}, {'conta_id': 190675, 'valor': 1083.56, 'dt_transacao': '2021-02-14 00:00:00'}, {'conta_id': 190676, 'valor': 1614.99, 'dt_transacao': '2021-03-12 12:00:00'}, {'conta_id': 190677, 'valor': 1815.52, 'dt_transacao': '2021-02-05 21:00:00'}, {'conta_id': 190677, 'valor': 1901.98, 'dt_transacao': '2021-01-11 07:00:00'}, {'conta_id': 190678, 'valor': 1565.07, 'dt_transacao': '2021-03-19 00:00:00'}, {'conta_id': 190678, 'valor': 249.49, 'dt_transacao': '2021-03-14 21:00:00'}, {'conta_id': 190678, 'valor': 642.66, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190679, 'valor': 1275.85, 'dt_transacao': '2021-03-13 06:00:00'}, {'conta_id': 190679, 'valor': 817.81, 'dt_transacao': '2021-03-16 02:00:00'}, {'conta_id': 190679, 'valor': 553.61, 'dt_transacao': '2021-03-20 17:00:00'}, {'conta_id': 190679, 'valor': 628.74, 'dt_transacao': '2021-01-09 14:00:00'}, {'conta_id': 190680, 'valor': 1742.61, 'dt_transacao': '2021-03-25 01:00:00'}, {'conta_id': 190680, 'valor': 466.11, 'dt_transacao': '2021-03-25 01:00:00'}, {'conta_id': 190681, 'valor': 33.22, 'dt_transacao': '2021-02-15 17:00:00'}, {'conta_id': 190681, 'valor': 1436.92, 'dt_transacao': '2021-03-18 07:00:00'}, {'conta_id': 190681, 'valor': 700.11, 'dt_transacao': '2021-01-10 16:00:00'}, {'conta_id': 190682, 'valor': 1284.13, 'dt_transacao': '2021-02-06 04:00:00'}, {'conta_id': 190683, 'valor': 63.28, 'dt_transacao': '2021-02-17 01:00:00'}, {'conta_id': 190683, 'valor': 582.45, 'dt_transacao': '2021-01-20 18:00:00'}, {'conta_id': 190683, 'valor': 52.05, 'dt_transacao': '2021-03-19 06:00:00'}, {'conta_id': 190683, 'valor': 1616.26, 'dt_transacao': '2021-03-07 22:00:00'}, {'conta_id': 190683, 'valor': 1561.97, 'dt_transacao': '2021-03-07 18:00:00'}, {'conta_id': 190684, 'valor': 1917.29, 'dt_transacao': '2021-01-29 18:00:00'}, {'conta_id': 190685, 'valor': 390.85, 'dt_transacao': '2021-03-15 09:00:00'}, {'conta_id': 190685, 'valor': 1822.23, 'dt_transacao': '2021-02-12 07:00:00'}, {'conta_id': 190685, 'valor': 1598.21, 'dt_transacao': '2021-02-16 07:00:00'}, {'conta_id': 190686, 'valor': 704.62, 'dt_transacao': '2021-03-06 02:00:00'}, {'conta_id': 190687, 'valor': 1348.54, 'dt_transacao': '2021-03-27 04:00:00'}, {'conta_id': 190687, 'valor': 1211.49, 'dt_transacao': '2021-02-06 10:00:00'}, {'conta_id': 190687, 'valor': 628.37, 'dt_transacao': '2021-01-11 13:00:00'}, {'conta_id': 190687, 'valor': 842.62, 'dt_transacao': '2021-01-19 18:00:00'}, {'conta_id': 190688, 'valor': 1531.25, 'dt_transacao': '2021-03-02 23:00:00'}, {'conta_id': 190688, 'valor': 202.89, 'dt_transacao': '2021-01-05 13:00:00'}, {'conta_id': 190688, 'valor': 48.84, 'dt_transacao': '2021-01-22 02:00:00'}, {'conta_id': 190689, 'valor': 674.28, 'dt_transacao': '2021-02-13 18:00:00'}, {'conta_id': 190690, 'valor': 579.29, 'dt_transacao': '2021-03-03 22:00:00'}, {'conta_id': 190690, 'valor': 270.08, 'dt_transacao': '2021-01-24 23:00:00'}, {'conta_id': 190690, 'valor': 1532.15, 'dt_transacao': '2021-03-07 02:00:00'}, {'conta_id': 190690, 'valor': 305.36, 'dt_transacao': '2021-01-01 18:00:00'}, {'conta_id': 190691, 'valor': 465.29, 'dt_transacao': '2021-03-05 10:00:00'}, {'conta_id': 190692, 'valor': 1628.16, 'dt_transacao': '2021-01-27 13:00:00'}, {'conta_id': 190693, 'valor': 548.42, 'dt_transacao': '2021-02-26 07:00:00'}, {'conta_id': 190693, 'valor': 1770.81, 'dt_transacao': '2021-02-06 08:00:00'}, {'conta_id': 190693, 'valor': 1977.62, 'dt_transacao': '2021-01-14 07:00:00'}, {'conta_id': 190694, 'valor': 1076.29, 'dt_transacao': '2021-01-25 00:00:00'}, {'conta_id': 190694, 'valor': 977.24, 'dt_transacao': '2021-01-04 11:00:00'}, {'conta_id': 190694, 'valor': 407.08, 'dt_transacao': '2021-03-15 13:00:00'}, {'conta_id': 190694, 'valor': 1464.45, 'dt_transacao': '2021-03-08 04:00:00'}, {'conta_id': 190694, 'valor': 1662.24, 'dt_transacao': '2021-02-07 03:00:00'}, {'conta_id': 190695, 'valor': 1936.91, 'dt_transacao': '2021-01-11 23:00:00'}, {'conta_id': 190696, 'valor': 1344.49, 'dt_transacao': '2021-01-10 22:00:00'}, {'conta_id': 190696, 'valor': 1376.19, 'dt_transacao': '2021-01-05 13:00:00'}, {'conta_id': 190696, 'valor': 201.78, 'dt_transacao': '2021-04-01 16:00:00'}, {'conta_id': 190696, 'valor': 635.74, 'dt_transacao': '2021-03-13 21:00:00'}, {'conta_id': 190696, 'valor': 1675.43, 'dt_transacao': '2021-03-22 01:00:00'}, {'conta_id': 190697, 'valor': 303.71, 'dt_transacao': '2021-01-24 23:00:00'}, {'conta_id': 190697, 'valor': 993.96, 'dt_transacao': '2021-01-17 06:00:00'}, {'conta_id': 190697, 'valor': 1269.91, 'dt_transacao': '2021-01-12 16:00:00'}, {'conta_id': 190697, 'valor': 1714.46, 'dt_transacao': '2021-01-28 00:00:00'}, {'conta_id': 190698, 'valor': 1227.14, 'dt_transacao': '2021-03-22 18:00:00'}, {'conta_id': 190698, 'valor': 470.2, 'dt_transacao': '2021-02-14 15:00:00'}, {'conta_id': 190698, 'valor': 1061.39, 'dt_transacao': '2021-01-24 09:00:00'}, {'conta_id': 190699, 'valor': 193.97, 'dt_transacao': '2021-01-15 09:00:00'}, {'conta_id': 190699, 'valor': 839.77, 'dt_transacao': '2021-02-27 10:00:00'}, {'conta_id': 190699, 'valor': 495.55, 'dt_transacao': '2021-02-24 03:00:00'}, {'conta_id': 190700, 'valor': 1202.52, 'dt_transacao': '2021-03-29 07:00:00'}, {'conta_id': 190700, 'valor': 58.9, 'dt_transacao': '2021-02-04 16:00:00'}, {'conta_id': 190700, 'valor': 431.0, 'dt_transacao': '2021-02-22 01:00:00'}, {'conta_id': 190700, 'valor': 1124.95, 'dt_transacao': '2021-02-11 08:00:00'}, {'conta_id': 190701, 'valor': 971.53, 'dt_transacao': '2021-01-29 10:00:00'}, {'conta_id': 190701, 'valor': 1272.9, 'dt_transacao': '2021-01-06 09:00:00'}, {'conta_id': 190701, 'valor': 895.83, 'dt_transacao': '2021-01-29 18:00:00'}, {'conta_id': 190702, 'valor': 1605.02, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190703, 'valor': 1676.46, 'dt_transacao': '2021-02-20 22:00:00'}, {'conta_id': 190703, 'valor': 459.44, 'dt_transacao': '2021-01-21 11:00:00'}, {'conta_id': 190704, 'valor': 796.48, 'dt_transacao': '2021-03-04 01:00:00'}, {'conta_id': 190704, 'valor': 314.73, 'dt_transacao': '2021-02-08 00:00:00'}, {'conta_id': 190704, 'valor': 460.78, 'dt_transacao': '2021-03-30 07:00:00'}, {'conta_id': 190704, 'valor': 68.52, 'dt_transacao': '2021-01-16 12:00:00'}, {'conta_id': 190704, 'valor': 1310.9, 'dt_transacao': '2021-01-17 23:00:00'}, {'conta_id': 190705, 'valor': 1210.14, 'dt_transacao': '2021-01-20 03:00:00'}, {'conta_id': 190705, 'valor': 1472.01, 'dt_transacao': '2021-02-20 06:00:00'}, {'conta_id': 190705, 'valor': 1663.11, 'dt_transacao': '2021-02-09 10:00:00'}, {'conta_id': 190705, 'valor': 850.88, 'dt_transacao': '2021-01-03 09:00:00'}, {'conta_id': 190705, 'valor': 548.83, 'dt_transacao': '2021-01-26 03:00:00'}, {'conta_id': 190706, 'valor': 557.84, 'dt_transacao': '2021-02-06 22:00:00'}, {'conta_id': 190706, 'valor': 1066.87, 'dt_transacao': '2021-02-01 12:00:00'}, {'conta_id': 190706, 'valor': 35.41, 'dt_transacao': '2021-03-05 23:00:00'}, {'conta_id': 190706, 'valor': 787.9, 'dt_transacao': '2021-02-17 17:00:00'}, {'conta_id': 190706, 'valor': 1429.44, 'dt_transacao': '2021-02-27 07:00:00'}, {'conta_id': 190707, 'valor': 665.54, 'dt_transacao': '2021-03-22 02:00:00'}, {'conta_id': 190707, 'valor': 191.68, 'dt_transacao': '2021-01-22 06:00:00'}, {'conta_id': 190707, 'valor': 1729.79, 'dt_transacao': '2021-03-04 18:00:00'}, {'conta_id': 190708, 'valor': 504.8, 'dt_transacao': '2021-02-09 23:00:00'}, {'conta_id': 190708, 'valor': 676.4, 'dt_transacao': '2021-01-20 01:00:00'}, {'conta_id': 190708, 'valor': 643.56, 'dt_transacao': '2021-03-14 19:00:00'}, {'conta_id': 190708, 'valor': 813.66, 'dt_transacao': '2021-01-01 08:00:00'}, {'conta_id': 190709, 'valor': 1029.35, 'dt_transacao': '2021-01-11 03:00:00'}, {'conta_id': 190709, 'valor': 851.98, 'dt_transacao': '2021-01-10 12:00:00'}, {'conta_id': 190709, 'valor': 255.44, 'dt_transacao': '2021-01-02 05:00:00'}, {'conta_id': 190709, 'valor': 1040.36, 'dt_transacao': '2021-01-16 05:00:00'}, {'conta_id': 190710, 'valor': 1498.24, 'dt_transacao': '2021-03-12 13:00:00'}, {'conta_id': 190710, 'valor': 799.41, 'dt_transacao': '2021-02-07 03:00:00'}, {'conta_id': 190711, 'valor': 1046.98, 'dt_transacao': '2021-01-27 19:00:00'}, {'conta_id': 190711, 'valor': 1904.04, 'dt_transacao': '2021-02-17 16:00:00'}, {'conta_id': 190711, 'valor': 1209.43, 'dt_transacao': '2021-03-02 14:00:00'}, {'conta_id': 190712, 'valor': 359.27, 'dt_transacao': '2021-03-02 02:00:00'}, {'conta_id': 190712, 'valor': 1986.63, 'dt_transacao': '2021-01-01 09:00:00'}, {'conta_id': 190712, 'valor': 1403.23, 'dt_transacao': '2021-01-26 22:00:00'}, {'conta_id': 190712, 'valor': 188.6, 'dt_transacao': '2021-02-28 10:00:00'}, {'conta_id': 190713, 'valor': 1855.83, 'dt_transacao': '2021-02-05 09:00:00'}, {'conta_id': 190714, 'valor': 1461.59, 'dt_transacao': '2021-02-05 05:00:00'}, {'conta_id': 190715, 'valor': 1337.33, 'dt_transacao': '2021-03-02 06:00:00'}, {'conta_id': 190715, 'valor': 337.03, 'dt_transacao': '2021-01-30 06:00:00'}, {'conta_id': 190715, 'valor': 183.59, 'dt_transacao': '2021-03-20 04:00:00'}, {'conta_id': 190715, 'valor': 287.52, 'dt_transacao': '2021-03-02 00:00:00'}, {'conta_id': 190715, 'valor': 647.35, 'dt_transacao': '2021-01-07 04:00:00'}, {'conta_id': 190716, 'valor': 787.1, 'dt_transacao': '2021-02-19 22:00:00'}, {'conta_id': 190716, 'valor': 949.48, 'dt_transacao': '2021-02-09 16:00:00'}, {'conta_id': 190716, 'valor': 634.63, 'dt_transacao': '2021-01-11 19:00:00'}, {'conta_id': 190716, 'valor': 1684.54, 'dt_transacao': '2021-01-22 20:00:00'}, {'conta_id': 190716, 'valor': 1667.38, 'dt_transacao': '2021-01-19 13:00:00'}, {'conta_id': 190717, 'valor': 51.57, 'dt_transacao': '2021-02-16 14:00:00'}, {'conta_id': 190717, 'valor': 1541.72, 'dt_transacao': '2021-02-14 23:00:00'}, {'conta_id': 190718, 'valor': 722.62, 'dt_transacao': '2021-01-25 00:00:00'}, {'conta_id': 190718, 'valor': 1172.79, 'dt_transacao': '2021-02-12 03:00:00'}, {'conta_id': 190718, 'valor': 1822.17, 'dt_transacao': '2021-02-02 16:00:00'}, {'conta_id': 190718, 'valor': 1843.21, 'dt_transacao': '2021-03-19 08:00:00'}, {'conta_id': 190719, 'valor': 1646.83, 'dt_transacao': '2021-03-02 19:00:00'}, {'conta_id': 190719, 'valor': 1264.85, 'dt_transacao': '2021-03-04 05:00:00'}, {'conta_id': 190719, 'valor': 947.42, 'dt_transacao': '2021-03-24 17:00:00'}, {'conta_id': 190719, 'valor': 1615.05, 'dt_transacao': '2021-01-05 11:00:00'}, {'conta_id': 190719, 'valor': 58.65, 'dt_transacao': '2021-01-20 22:00:00'}, {'conta_id': 190720, 'valor': 195.85, 'dt_transacao': '2021-03-11 08:00:00'}, {'conta_id': 190720, 'valor': 1966.82, 'dt_transacao': '2021-01-25 20:00:00'}, {'conta_id': 190720, 'valor': 1228.39, 'dt_transacao': '2021-01-03 10:00:00'}, {'conta_id': 190721, 'valor': 386.69, 'dt_transacao': '2021-03-06 04:00:00'}, {'conta_id': 190722, 'valor': 1270.96, 'dt_transacao': '2021-01-01 09:00:00'}, {'conta_id': 190722, 'valor': 548.25, 'dt_transacao': '2021-02-14 03:00:00'}, {'conta_id': 190723, 'valor': 1459.37, 'dt_transacao': '2021-01-24 10:00:00'}, {'conta_id': 190723, 'valor': 1182.71, 'dt_transacao': '2021-01-03 05:00:00'}, {'conta_id': 190724, 'valor': 621.66, 'dt_transacao': '2021-02-18 22:00:00'}, {'conta_id': 190724, 'valor': 574.87, 'dt_transacao': '2021-02-05 04:00:00'}, {'conta_id': 190725, 'valor': 519.45, 'dt_transacao': '2021-02-26 08:00:00'}, {'conta_id': 190725, 'valor': 185.73, 'dt_transacao': '2021-02-15 18:00:00'}, {'conta_id': 190725, 'valor': 727.86, 'dt_transacao': '2021-03-24 16:00:00'}, {'conta_id': 190726, 'valor': 1292.37, 'dt_transacao': '2021-03-29 09:00:00'}, {'conta_id': 190726, 'valor': 415.69, 'dt_transacao': '2021-03-31 16:00:00'}, {'conta_id': 190726, 'valor': 1103.67, 'dt_transacao': '2021-02-04 02:00:00'}, {'conta_id': 190727, 'valor': 1317.01, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190727, 'valor': 280.49, 'dt_transacao': '2021-03-17 10:00:00'}, {'conta_id': 190727, 'valor': 745.19, 'dt_transacao': '2021-02-18 21:00:00'}, {'conta_id': 190727, 'valor': 1177.19, 'dt_transacao': '2021-01-05 10:00:00'}, {'conta_id': 190727, 'valor': 1402.88, 'dt_transacao': '2021-03-22 11:00:00'}, {'conta_id': 190728, 'valor': 613.24, 'dt_transacao': '2021-03-01 17:00:00'}, {'conta_id': 190728, 'valor': 373.14, 'dt_transacao': '2021-01-03 14:00:00'}, {'conta_id': 190728, 'valor': 1618.63, 'dt_transacao': '2021-02-28 18:00:00'}, {'conta_id': 190729, 'valor': 1357.43, 'dt_transacao': '2021-02-08 01:00:00'}, {'conta_id': 190729, 'valor': 1941.56, 'dt_transacao': '2021-01-30 16:00:00'}, {'conta_id': 190729, 'valor': 216.6, 'dt_transacao': '2021-02-24 08:00:00'}, {'conta_id': 190729, 'valor': 1908.01, 'dt_transacao': '2021-02-21 18:00:00'}, {'conta_id': 190730, 'valor': 1577.35, 'dt_transacao': '2021-01-07 02:00:00'}, {'conta_id': 190730, 'valor': 619.24, 'dt_transacao': '2021-01-19 11:00:00'}, {'conta_id': 190730, 'valor': 917.97, 'dt_transacao': '2021-01-22 14:00:00'}, {'conta_id': 190731, 'valor': 1715.67, 'dt_transacao': '2021-02-25 02:00:00'}, {'conta_id': 190731, 'valor': 1448.54, 'dt_transacao': '2021-01-20 21:00:00'}, {'conta_id': 190732, 'valor': 1373.89, 'dt_transacao': '2021-03-31 06:00:00'}, {'conta_id': 190732, 'valor': 1825.75, 'dt_transacao': '2021-03-27 12:00:00'}, {'conta_id': 190732, 'valor': 1662.86, 'dt_transacao': '2021-04-01 07:00:00'}, {'conta_id': 190732, 'valor': 1526.35, 'dt_transacao': '2021-01-13 12:00:00'}, {'conta_id': 190732, 'valor': 257.88, 'dt_transacao': '2021-01-18 12:00:00'}, {'conta_id': 190733, 'valor': 54.77, 'dt_transacao': '2021-03-19 05:00:00'}, {'conta_id': 190733, 'valor': 1535.92, 'dt_transacao': '2021-02-17 11:00:00'}, {'conta_id': 190733, 'valor': 680.36, 'dt_transacao': '2021-01-01 19:00:00'}, {'conta_id': 190733, 'valor': 1198.74, 'dt_transacao': '2021-02-20 02:00:00'}, {'conta_id': 190733, 'valor': 1291.51, 'dt_transacao': '2021-01-18 15:00:00'}, {'conta_id': 190734, 'valor': 852.0, 'dt_transacao': '2021-02-06 09:00:00'}, {'conta_id': 190734, 'valor': 774.24, 'dt_transacao': '2021-03-03 01:00:00'}, {'conta_id': 190735, 'valor': 1372.45, 'dt_transacao': '2021-01-13 20:00:00'}, {'conta_id': 190735, 'valor': 1725.07, 'dt_transacao': '2021-01-02 05:00:00'}, {'conta_id': 190735, 'valor': 638.73, 'dt_transacao': '2021-03-15 16:00:00'}, {'conta_id': 190736, 'valor': 850.54, 'dt_transacao': '2021-03-23 03:00:00'}, {'conta_id': 190736, 'valor': 1207.67, 'dt_transacao': '2021-02-27 16:00:00'}, {'conta_id': 190736, 'valor': 746.28, 'dt_transacao': '2021-03-02 22:00:00'}, {'conta_id': 190737, 'valor': 674.89, 'dt_transacao': '2021-04-01 18:00:00'}, {'conta_id': 190738, 'valor': 1735.93, 'dt_transacao': '2021-02-28 11:00:00'}, {'conta_id': 190739, 'valor': 587.19, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190739, 'valor': 894.09, 'dt_transacao': '2021-02-08 20:00:00'}, {'conta_id': 190740, 'valor': 1070.44, 'dt_transacao': '2021-01-17 23:00:00'}, {'conta_id': 190741, 'valor': 154.0, 'dt_transacao': '2021-02-17 01:00:00'}, {'conta_id': 190742, 'valor': 368.68, 'dt_transacao': '2021-02-02 23:00:00'}, {'conta_id': 190743, 'valor': 749.1, 'dt_transacao': '2021-03-02 06:00:00'}, {'conta_id': 190744, 'valor': 546.09, 'dt_transacao': '2021-02-22 06:00:00'}, {'conta_id': 190744, 'valor': 38.42, 'dt_transacao': '2021-02-23 21:00:00'}, {'conta_id': 190744, 'valor': 260.4, 'dt_transacao': '2021-02-04 14:00:00'}, {'conta_id': 190745, 'valor': 1223.16, 'dt_transacao': '2021-03-19 14:00:00'}, {'conta_id': 190746, 'valor': 693.39, 'dt_transacao': '2021-02-26 04:00:00'}, {'conta_id': 190746, 'valor': 924.43, 'dt_transacao': '2021-03-05 06:00:00'}, {'conta_id': 190746, 'valor': 1064.95, 'dt_transacao': '2021-01-22 21:00:00'}, {'conta_id': 190747, 'valor': 415.89, 'dt_transacao': '2021-03-09 11:00:00'}, {'conta_id': 190748, 'valor': 1170.29, 'dt_transacao': '2021-03-21 11:00:00'}, {'conta_id': 190748, 'valor': 1130.22, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190748, 'valor': 1503.03, 'dt_transacao': '2021-02-06 17:00:00'}, {'conta_id': 190748, 'valor': 1488.36, 'dt_transacao': '2021-01-31 12:00:00'}, {'conta_id': 190748, 'valor': 1842.43, 'dt_transacao': '2021-01-17 12:00:00'}, {'conta_id': 190749, 'valor': 1729.92, 'dt_transacao': '2021-02-21 17:00:00'}, {'conta_id': 190749, 'valor': 817.62, 'dt_transacao': '2021-01-28 13:00:00'}, {'conta_id': 190750, 'valor': 1718.25, 'dt_transacao': '2021-03-05 14:00:00'}, {'conta_id': 190750, 'valor': 573.28, 'dt_transacao': '2021-02-27 05:00:00'}, {'conta_id': 190751, 'valor': 767.32, 'dt_transacao': '2021-03-25 13:00:00'}, {'conta_id': 190752, 'valor': 1141.6, 'dt_transacao': '2021-02-22 00:00:00'}, {'conta_id': 190752, 'valor': 650.72, 'dt_transacao': '2021-02-01 05:00:00'}, {'conta_id': 190752, 'valor': 1139.04, 'dt_transacao': '2021-03-02 21:00:00'}, {'conta_id': 190752, 'valor': 182.84, 'dt_transacao': '2021-02-17 00:00:00'}, {'conta_id': 190752, 'valor': 1486.19, 'dt_transacao': '2021-03-25 12:00:00'}, {'conta_id': 190753, 'valor': 646.13, 'dt_transacao': '2021-02-07 02:00:00'}, {'conta_id': 190753, 'valor': 1789.0, 'dt_transacao': '2021-01-27 18:00:00'}, {'conta_id': 190754, 'valor': 1675.91, 'dt_transacao': '2021-01-21 07:00:00'}, {'conta_id': 190754, 'valor': 924.96, 'dt_transacao': '2021-01-09 00:00:00'}, {'conta_id': 190754, 'valor': 1497.47, 'dt_transacao': '2021-03-29 21:00:00'}, {'conta_id': 190754, 'valor': 813.29, 'dt_transacao': '2021-03-08 15:00:00'}, {'conta_id': 190754, 'valor': 1724.48, 'dt_transacao': '2021-01-10 13:00:00'}, {'conta_id': 190755, 'valor': 727.38, 'dt_transacao': '2021-03-07 20:00:00'}, {'conta_id': 190756, 'valor': 826.37, 'dt_transacao': '2021-02-13 04:00:00'}, {'conta_id': 190756, 'valor': 1826.04, 'dt_transacao': '2021-03-03 06:00:00'}, {'conta_id': 190756, 'valor': 529.75, 'dt_transacao': '2021-02-12 20:00:00'}, {'conta_id': 190757, 'valor': 1574.96, 'dt_transacao': '2021-03-31 08:00:00'}, {'conta_id': 190757, 'valor': 1136.89, 'dt_transacao': '2021-03-18 01:00:00'}, {'conta_id': 190757, 'valor': 1522.88, 'dt_transacao': '2021-01-19 16:00:00'}, {'conta_id': 190758, 'valor': 1135.71, 'dt_transacao': '2021-02-09 15:00:00'}, {'conta_id': 190758, 'valor': 1426.44, 'dt_transacao': '2021-03-28 09:00:00'}, {'conta_id': 190759, 'valor': 1701.99, 'dt_transacao': '2021-03-16 09:00:00'}, {'conta_id': 190760, 'valor': 1288.06, 'dt_transacao': '2021-03-17 16:00:00'}, {'conta_id': 190760, 'valor': 629.28, 'dt_transacao': '2021-02-17 12:00:00'}, {'conta_id': 190760, 'valor': 1730.95, 'dt_transacao': '2021-03-04 20:00:00'}, {'conta_id': 190761, 'valor': 619.31, 'dt_transacao': '2021-01-29 04:00:00'}, {'conta_id': 190761, 'valor': 557.35, 'dt_transacao': '2021-01-16 20:00:00'}, {'conta_id': 190762, 'valor': 1328.82, 'dt_transacao': '2021-02-21 20:00:00'}, {'conta_id': 190762, 'valor': 198.58, 'dt_transacao': '2021-02-12 16:00:00'}, {'conta_id': 190762, 'valor': 135.33, 'dt_transacao': '2021-03-12 17:00:00'}, {'conta_id': 190763, 'valor': 1379.71, 'dt_transacao': '2021-03-25 07:00:00'}, {'conta_id': 190763, 'valor': 457.45, 'dt_transacao': '2021-02-28 05:00:00'}, {'conta_id': 190764, 'valor': 416.26, 'dt_transacao': '2021-02-01 18:00:00'}, {'conta_id': 190765, 'valor': 957.12, 'dt_transacao': '2021-02-26 02:00:00'}, {'conta_id': 190765, 'valor': 952.6, 'dt_transacao': '2021-01-13 17:00:00'}, {'conta_id': 190766, 'valor': 1555.01, 'dt_transacao': '2021-01-25 18:00:00'}, {'conta_id': 190766, 'valor': 1611.73, 'dt_transacao': '2021-03-20 14:00:00'}, {'conta_id': 190766, 'valor': 1984.48, 'dt_transacao': '2021-02-02 13:00:00'}, {'conta_id': 190766, 'valor': 1203.11, 'dt_transacao': '2021-03-05 20:00:00'}, {'conta_id': 190767, 'valor': 551.3, 'dt_transacao': '2021-03-13 16:00:00'}, {'conta_id': 190767, 'valor': 618.3, 'dt_transacao': '2021-02-26 03:00:00'}, {'conta_id': 190767, 'valor': 1954.48, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190768, 'valor': 190.04, 'dt_transacao': '2021-02-26 10:00:00'}, {'conta_id': 190768, 'valor': 1276.13, 'dt_transacao': '2021-03-10 05:00:00'}, {'conta_id': 190768, 'valor': 388.0, 'dt_transacao': '2021-01-14 07:00:00'}, {'conta_id': 190769, 'valor': 421.37, 'dt_transacao': '2021-03-28 04:00:00'}, {'conta_id': 190769, 'valor': 648.21, 'dt_transacao': '2021-01-25 12:00:00'}, {'conta_id': 190769, 'valor': 1571.84, 'dt_transacao': '2021-01-21 19:00:00'}, {'conta_id': 190769, 'valor': 1325.52, 'dt_transacao': '2021-02-11 16:00:00'}, {'conta_id': 190769, 'valor': 1962.33, 'dt_transacao': '2021-03-25 19:00:00'}, {'conta_id': 190770, 'valor': 668.11, 'dt_transacao': '2021-02-15 23:00:00'}, {'conta_id': 190770, 'valor': 631.38, 'dt_transacao': '2021-02-09 13:00:00'}, {'conta_id': 190770, 'valor': 1127.01, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190771, 'valor': 1637.25, 'dt_transacao': '2021-01-26 06:00:00'}, {'conta_id': 190772, 'valor': 1120.38, 'dt_transacao': '2021-01-26 06:00:00'}, {'conta_id': 190772, 'valor': 1235.49, 'dt_transacao': '2021-02-26 08:00:00'}, {'conta_id': 190772, 'valor': 214.56, 'dt_transacao': '2021-02-01 05:00:00'}, {'conta_id': 190772, 'valor': 651.64, 'dt_transacao': '2021-03-03 23:00:00'}, {'conta_id': 190772, 'valor': 1186.29, 'dt_transacao': '2021-01-06 21:00:00'}, {'conta_id': 190773, 'valor': 1279.62, 'dt_transacao': '2021-02-05 17:00:00'}, {'conta_id': 190774, 'valor': 1648.8, 'dt_transacao': '2021-01-20 18:00:00'}, {'conta_id': 190774, 'valor': 598.41, 'dt_transacao': '2021-01-26 17:00:00'}, {'conta_id': 190774, 'valor': 176.92, 'dt_transacao': '2021-02-01 10:00:00'}, {'conta_id': 190774, 'valor': 1145.89, 'dt_transacao': '2021-03-26 08:00:00'}, {'conta_id': 190775, 'valor': 369.1, 'dt_transacao': '2021-03-01 07:00:00'}, {'conta_id': 190775, 'valor': 922.15, 'dt_transacao': '2021-02-12 06:00:00'}, {'conta_id': 190776, 'valor': 1596.4, 'dt_transacao': '2021-03-09 01:00:00'}, {'conta_id': 190776, 'valor': 1826.05, 'dt_transacao': '2021-03-07 11:00:00'}, {'conta_id': 190776, 'valor': 33.39, 'dt_transacao': '2021-02-25 01:00:00'}, {'conta_id': 190776, 'valor': 1919.62, 'dt_transacao': '2021-02-11 17:00:00'}, {'conta_id': 190777, 'valor': 1692.75, 'dt_transacao': '2021-01-12 18:00:00'}, {'conta_id': 190777, 'valor': 1845.07, 'dt_transacao': '2021-03-20 20:00:00'}, {'conta_id': 190777, 'valor': 1985.91, 'dt_transacao': '2021-01-10 21:00:00'}, {'conta_id': 190777, 'valor': 862.46, 'dt_transacao': '2021-02-05 14:00:00'}, {'conta_id': 190777, 'valor': 903.13, 'dt_transacao': '2021-01-20 20:00:00'}, {'conta_id': 190778, 'valor': 803.53, 'dt_transacao': '2021-02-07 00:00:00'}, {'conta_id': 190779, 'valor': 1711.29, 'dt_transacao': '2021-03-16 21:00:00'}, {'conta_id': 190779, 'valor': 1535.79, 'dt_transacao': '2021-03-10 14:00:00'}, {'conta_id': 190779, 'valor': 1307.57, 'dt_transacao': '2021-01-26 18:00:00'}, {'conta_id': 190780, 'valor': 1067.31, 'dt_transacao': '2021-01-06 04:00:00'}, {'conta_id': 190781, 'valor': 327.41, 'dt_transacao': '2021-02-04 07:00:00'}, {'conta_id': 190781, 'valor': 1872.67, 'dt_transacao': '2021-01-28 11:00:00'}, {'conta_id': 190781, 'valor': 55.57, 'dt_transacao': '2021-02-26 03:00:00'}, {'conta_id': 190782, 'valor': 398.08, 'dt_transacao': '2021-02-18 18:00:00'}, {'conta_id': 190782, 'valor': 1629.18, 'dt_transacao': '2021-01-12 05:00:00'}, {'conta_id': 190782, 'valor': 1013.9, 'dt_transacao': '2021-03-16 23:00:00'}, {'conta_id': 190782, 'valor': 1033.48, 'dt_transacao': '2021-01-21 06:00:00'}, {'conta_id': 190783, 'valor': 720.36, 'dt_transacao': '2021-02-02 16:00:00'}, {'conta_id': 190783, 'valor': 224.87, 'dt_transacao': '2021-02-28 00:00:00'}, {'conta_id': 190783, 'valor': 15.5, 'dt_transacao': '2021-02-23 12:00:00'}, {'conta_id': 190783, 'valor': 1923.18, 'dt_transacao': '2021-01-13 11:00:00'}, {'conta_id': 190783, 'valor': 721.42, 'dt_transacao': '2021-02-09 02:00:00'}, {'conta_id': 190784, 'valor': 1459.53, 'dt_transacao': '2021-02-11 01:00:00'}, {'conta_id': 190784, 'valor': 282.33, 'dt_transacao': '2021-02-01 05:00:00'}, {'conta_id': 190784, 'valor': 1129.59, 'dt_transacao': '2021-01-17 16:00:00'}, {'conta_id': 190785, 'valor': 830.34, 'dt_transacao': '2021-01-21 13:00:00'}, {'conta_id': 190785, 'valor': 345.67, 'dt_transacao': '2021-03-08 04:00:00'}, {'conta_id': 190785, 'valor': 1484.19, 'dt_transacao': '2021-03-27 05:00:00'}, {'conta_id': 190785, 'valor': 730.77, 'dt_transacao': '2021-01-15 05:00:00'}, {'conta_id': 190785, 'valor': 60.93, 'dt_transacao': '2021-01-27 09:00:00'}, {'conta_id': 190786, 'valor': 69.41, 'dt_transacao': '2021-01-19 13:00:00'}, {'conta_id': 190786, 'valor': 93.95, 'dt_transacao': '2021-02-18 19:00:00'}, {'conta_id': 190786, 'valor': 1445.32, 'dt_transacao': '2021-01-22 05:00:00'}, {'conta_id': 190786, 'valor': 201.13, 'dt_transacao': '2021-02-14 07:00:00'}, {'conta_id': 190787, 'valor': 1713.97, 'dt_transacao': '2021-03-23 00:00:00'}, {'conta_id': 190787, 'valor': 623.55, 'dt_transacao': '2021-03-02 08:00:00'}, {'conta_id': 190788, 'valor': 1029.95, 'dt_transacao': '2021-01-19 07:00:00'}, {'conta_id': 190788, 'valor': 1318.76, 'dt_transacao': '2021-01-13 22:00:00'}, {'conta_id': 190789, 'valor': 1421.21, 'dt_transacao': '2021-03-01 21:00:00'}, {'conta_id': 190789, 'valor': 1181.98, 'dt_transacao': '2021-03-08 18:00:00'}, {'conta_id': 190789, 'valor': 1975.3, 'dt_transacao': '2021-02-04 04:00:00'}, {'conta_id': 190790, 'valor': 1959.08, 'dt_transacao': '2021-01-02 04:00:00'}, {'conta_id': 190790, 'valor': 1269.63, 'dt_transacao': '2021-01-31 05:00:00'}, {'conta_id': 190791, 'valor': 811.91, 'dt_transacao': '2021-01-28 14:00:00'}, {'conta_id': 190791, 'valor': 1243.0, 'dt_transacao': '2021-03-11 14:00:00'}, {'conta_id': 190791, 'valor': 236.52, 'dt_transacao': '2021-02-05 20:00:00'}, {'conta_id': 190791, 'valor': 1537.25, 'dt_transacao': '2021-02-13 10:00:00'}, {'conta_id': 190791, 'valor': 1993.37, 'dt_transacao': '2021-01-24 08:00:00'}, {'conta_id': 190792, 'valor': 853.6, 'dt_transacao': '2021-01-04 17:00:00'}, {'conta_id': 190792, 'valor': 253.33, 'dt_transacao': '2021-01-30 14:00:00'}, {'conta_id': 190792, 'valor': 1660.33, 'dt_transacao': '2021-01-16 17:00:00'}, {'conta_id': 190793, 'valor': 1640.23, 'dt_transacao': '2021-03-07 06:00:00'}, {'conta_id': 190793, 'valor': 284.28, 'dt_transacao': '2021-03-29 07:00:00'}, {'conta_id': 190793, 'valor': 1552.05, 'dt_transacao': '2021-01-26 23:00:00'}, {'conta_id': 190793, 'valor': 1489.45, 'dt_transacao': '2021-03-09 19:00:00'}, {'conta_id': 190794, 'valor': 1361.11, 'dt_transacao': '2021-03-23 20:00:00'}, {'conta_id': 190795, 'valor': 1918.15, 'dt_transacao': '2021-02-21 09:00:00'}, {'conta_id': 190796, 'valor': 503.41, 'dt_transacao': '2021-03-13 21:00:00'}, {'conta_id': 190797, 'valor': 1529.42, 'dt_transacao': '2021-02-01 12:00:00'}, {'conta_id': 190797, 'valor': 1572.84, 'dt_transacao': '2021-03-24 13:00:00'}, {'conta_id': 190798, 'valor': 247.82, 'dt_transacao': '2021-03-16 21:00:00'}, {'conta_id': 190798, 'valor': 304.54, 'dt_transacao': '2021-02-26 04:00:00'}, {'conta_id': 190798, 'valor': 1344.8, 'dt_transacao': '2021-02-11 00:00:00'}, {'conta_id': 190798, 'valor': 1273.88, 'dt_transacao': '2021-03-08 09:00:00'}, {'conta_id': 190799, 'valor': 1486.08, 'dt_transacao': '2021-02-28 04:00:00'}, {'conta_id': 190799, 'valor': 141.74, 'dt_transacao': '2021-02-13 08:00:00'}, {'conta_id': 190799, 'valor': 1899.03, 'dt_transacao': '2021-01-06 12:00:00'}, {'conta_id': 190799, 'valor': 1676.12, 'dt_transacao': '2021-02-02 03:00:00'}, {'conta_id': 190799, 'valor': 1254.17, 'dt_transacao': '2021-02-24 18:00:00'}, {'conta_id': 190800, 'valor': 1738.85, 'dt_transacao': '2021-03-27 15:00:00'}, {'conta_id': 190800, 'valor': 1673.58, 'dt_transacao': '2021-02-25 13:00:00'}, {'conta_id': 190801, 'valor': 167.1, 'dt_transacao': '2021-01-13 22:00:00'}, {'conta_id': 190801, 'valor': 1612.18, 'dt_transacao': '2021-02-01 18:00:00'}, {'conta_id': 190801, 'valor': 990.46, 'dt_transacao': '2021-03-17 13:00:00'}, {'conta_id': 190802, 'valor': 1676.57, 'dt_transacao': '2021-03-24 03:00:00'}, {'conta_id': 190802, 'valor': 1151.68, 'dt_transacao': '2021-01-16 07:00:00'}, {'conta_id': 190802, 'valor': 722.04, 'dt_transacao': '2021-03-28 22:00:00'}, {'conta_id': 190802, 'valor': 1807.68, 'dt_transacao': '2021-02-23 12:00:00'}, {'conta_id': 190802, 'valor': 1436.86, 'dt_transacao': '2021-01-06 11:00:00'}, {'conta_id': 190803, 'valor': 362.29, 'dt_transacao': '2021-03-22 10:00:00'}, {'conta_id': 190803, 'valor': 198.51, 'dt_transacao': '2021-02-22 00:00:00'}, {'conta_id': 190803, 'valor': 1458.39, 'dt_transacao': '2021-02-02 11:00:00'}, {'conta_id': 190803, 'valor': 1636.92, 'dt_transacao': '2021-02-14 11:00:00'}, {'conta_id': 190803, 'valor': 1947.19, 'dt_transacao': '2021-02-01 08:00:00'}, {'conta_id': 190804, 'valor': 1541.85, 'dt_transacao': '2021-03-12 20:00:00'}, {'conta_id': 190804, 'valor': 644.44, 'dt_transacao': '2021-02-21 11:00:00'}, {'conta_id': 190804, 'valor': 1562.16, 'dt_transacao': '2021-01-23 15:00:00'}, {'conta_id': 190804, 'valor': 1713.44, 'dt_transacao': '2021-02-12 11:00:00'}, {'conta_id': 190804, 'valor': 1233.19, 'dt_transacao': '2021-01-30 10:00:00'}, {'conta_id': 190805, 'valor': 405.1, 'dt_transacao': '2021-04-01 15:00:00'}, {'conta_id': 190805, 'valor': 701.84, 'dt_transacao': '2021-02-26 11:00:00'}, {'conta_id': 190805, 'valor': 1509.79, 'dt_transacao': '2021-01-22 10:00:00'}, {'conta_id': 190805, 'valor': 32.84, 'dt_transacao': '2021-01-25 18:00:00'}, {'conta_id': 190806, 'valor': 1423.62, 'dt_transacao': '2021-03-26 12:00:00'}, {'conta_id': 190807, 'valor': 1227.23, 'dt_transacao': '2021-01-11 10:00:00'}, {'conta_id': 190807, 'valor': 1998.88, 'dt_transacao': '2021-02-26 01:00:00'}, {'conta_id': 190807, 'valor': 492.56, 'dt_transacao': '2021-01-20 21:00:00'}, {'conta_id': 190808, 'valor': 1296.14, 'dt_transacao': '2021-01-28 04:00:00'}, {'conta_id': 190808, 'valor': 25.82, 'dt_transacao': '2021-02-01 00:00:00'}, {'conta_id': 190808, 'valor': 1715.21, 'dt_transacao': '2021-03-03 14:00:00'}, {'conta_id': 190808, 'valor': 829.64, 'dt_transacao': '2021-03-12 18:00:00'}, {'conta_id': 190809, 'valor': 1607.05, 'dt_transacao': '2021-02-03 13:00:00'}, {'conta_id': 190809, 'valor': 741.09, 'dt_transacao': '2021-01-31 10:00:00'}, {'conta_id': 190809, 'valor': 1081.2, 'dt_transacao': '2021-01-10 22:00:00'}, {'conta_id': 190810, 'valor': 456.42, 'dt_transacao': '2021-02-14 22:00:00'}, {'conta_id': 190810, 'valor': 1404.59, 'dt_transacao': '2021-03-12 07:00:00'}, {'conta_id': 190810, 'valor': 931.11, 'dt_transacao': '2021-03-15 01:00:00'}, {'conta_id': 190810, 'valor': 921.02, 'dt_transacao': '2021-02-26 15:00:00'}, {'conta_id': 190810, 'valor': 1182.53, 'dt_transacao': '2021-01-09 05:00:00'}, {'conta_id': 190811, 'valor': 1909.51, 'dt_transacao': '2021-02-05 18:00:00'}, {'conta_id': 190812, 'valor': 620.34, 'dt_transacao': '2021-03-31 21:00:00'}, {'conta_id': 190812, 'valor': 610.02, 'dt_transacao': '2021-01-30 18:00:00'}, {'conta_id': 190812, 'valor': 1597.03, 'dt_transacao': '2021-03-07 13:00:00'}, {'conta_id': 190812, 'valor': 289.32, 'dt_transacao': '2021-03-11 21:00:00'}, {'conta_id': 190813, 'valor': 128.19, 'dt_transacao': '2021-02-22 23:00:00'}, {'conta_id': 190813, 'valor': 323.25, 'dt_transacao': '2021-01-25 07:00:00'}, {'conta_id': 190813, 'valor': 1959.58, 'dt_transacao': '2021-01-23 05:00:00'}, {'conta_id': 190813, 'valor': 59.95, 'dt_transacao': '2021-03-05 23:00:00'}, {'conta_id': 190813, 'valor': 239.88, 'dt_transacao': '2021-01-12 03:00:00'}, {'conta_id': 190814, 'valor': 192.63, 'dt_transacao': '2021-02-15 05:00:00'}, {'conta_id': 190814, 'valor': 1414.82, 'dt_transacao': '2021-02-02 15:00:00'}, {'conta_id': 190815, 'valor': 904.0, 'dt_transacao': '2021-01-13 16:00:00'}, {'conta_id': 190815, 'valor': 516.28, 'dt_transacao': '2021-03-19 01:00:00'}, {'conta_id': 190815, 'valor': 1964.04, 'dt_transacao': '2021-01-08 21:00:00'}, {'conta_id': 190816, 'valor': 613.22, 'dt_transacao': '2021-03-14 18:00:00'}, {'conta_id': 190816, 'valor': 750.3, 'dt_transacao': '2021-01-09 05:00:00'}, {'conta_id': 190816, 'valor': 377.78, 'dt_transacao': '2021-03-21 15:00:00'}, {'conta_id': 190816, 'valor': 364.39, 'dt_transacao': '2021-01-28 11:00:00'}, {'conta_id': 190817, 'valor': 1244.54, 'dt_transacao': '2021-03-10 03:00:00'}, {'conta_id': 190818, 'valor': 387.13, 'dt_transacao': '2021-01-20 19:00:00'}, {'conta_id': 190818, 'valor': 1840.08, 'dt_transacao': '2021-02-28 21:00:00'}, {'conta_id': 190818, 'valor': 1096.41, 'dt_transacao': '2021-02-19 14:00:00'}, {'conta_id': 190818, 'valor': 1888.08, 'dt_transacao': '2021-03-27 17:00:00'}, {'conta_id': 190819, 'valor': 84.14, 'dt_transacao': '2021-01-08 15:00:00'}, {'conta_id': 190819, 'valor': 1622.0, 'dt_transacao': '2021-03-26 22:00:00'}, {'conta_id': 190819, 'valor': 1509.47, 'dt_transacao': '2021-03-17 03:00:00'}, {'conta_id': 190819, 'valor': 1695.7, 'dt_transacao': '2021-01-12 21:00:00'}, {'conta_id': 190820, 'valor': 1888.55, 'dt_transacao': '2021-02-24 12:00:00'}, {'conta_id': 190820, 'valor': 870.84, 'dt_transacao': '2021-01-11 15:00:00'}, {'conta_id': 190820, 'valor': 48.64, 'dt_transacao': '2021-01-14 10:00:00'}, {'conta_id': 190821, 'valor': 1893.61, 'dt_transacao': '2021-02-09 18:00:00'}, {'conta_id': 190821, 'valor': 1495.95, 'dt_transacao': '2021-03-15 17:00:00'}, {'conta_id': 190822, 'valor': 704.18, 'dt_transacao': '2021-03-10 01:00:00'}, {'conta_id': 190822, 'valor': 1647.67, 'dt_transacao': '2021-03-29 13:00:00'}, {'conta_id': 190823, 'valor': 1425.11, 'dt_transacao': '2021-01-10 22:00:00'}, {'conta_id': 190823, 'valor': 408.01, 'dt_transacao': '2021-01-27 18:00:00'}, {'conta_id': 190824, 'valor': 1319.53, 'dt_transacao': '2021-02-12 05:00:00'}, {'conta_id': 190824, 'valor': 1974.18, 'dt_transacao': '2021-01-30 11:00:00'}, {'conta_id': 190825, 'valor': 1420.63, 'dt_transacao': '2021-03-22 09:00:00'}, {'conta_id': 190825, 'valor': 410.55, 'dt_transacao': '2021-03-25 23:00:00'}, {'conta_id': 190825, 'valor': 1827.56, 'dt_transacao': '2021-02-22 22:00:00'}, {'conta_id': 190825, 'valor': 21.44, 'dt_transacao': '2021-02-22 12:00:00'}, {'conta_id': 190825, 'valor': 556.9, 'dt_transacao': '2021-01-25 15:00:00'}, {'conta_id': 190826, 'valor': 1275.56, 'dt_transacao': '2021-01-15 05:00:00'}, {'conta_id': 190827, 'valor': 1072.98, 'dt_transacao': '2021-02-06 18:00:00'}, {'conta_id': 190827, 'valor': 523.74, 'dt_transacao': '2021-01-16 12:00:00'}, {'conta_id': 190827, 'valor': 1645.93, 'dt_transacao': '2021-01-26 02:00:00'}, {'conta_id': 190828, 'valor': 223.7, 'dt_transacao': '2021-01-05 12:00:00'}, {'conta_id': 190829, 'valor': 1194.41, 'dt_transacao': '2021-01-06 21:00:00'}, {'conta_id': 190829, 'valor': 1867.3, 'dt_transacao': '2021-01-25 06:00:00'}, {'conta_id': 190829, 'valor': 1450.1, 'dt_transacao': '2021-01-06 18:00:00'}, {'conta_id': 190829, 'valor': 29.41, 'dt_transacao': '2021-01-20 10:00:00'}, {'conta_id': 190830, 'valor': 1893.7, 'dt_transacao': '2021-03-06 02:00:00'}, {'conta_id': 190830, 'valor': 451.25, 'dt_transacao': '2021-03-22 06:00:00'}, {'conta_id': 190831, 'valor': 1094.7, 'dt_transacao': '2021-02-10 01:00:00'}, {'conta_id': 190831, 'valor': 223.62, 'dt_transacao': '2021-01-09 13:00:00'}, {'conta_id': 190832, 'valor': 1368.95, 'dt_transacao': '2021-01-01 12:00:00'}, {'conta_id': 190833, 'valor': 1874.48, 'dt_transacao': '2021-03-24 12:00:00'}, {'conta_id': 190833, 'valor': 565.65, 'dt_transacao': '2021-03-20 12:00:00'}, {'conta_id': 190833, 'valor': 899.89, 'dt_transacao': '2021-01-31 21:00:00'}, {'conta_id': 190833, 'valor': 981.63, 'dt_transacao': '2021-02-02 11:00:00'}, {'conta_id': 190834, 'valor': 1274.55, 'dt_transacao': '2021-01-23 12:00:00'}, {'conta_id': 190834, 'valor': 1998.28, 'dt_transacao': '2021-01-21 11:00:00'}, {'conta_id': 190834, 'valor': 1480.27, 'dt_transacao': '2021-02-25 09:00:00'}, {'conta_id': 190835, 'valor': 416.99, 'dt_transacao': '2021-02-21 13:00:00'}, {'conta_id': 190835, 'valor': 1767.36, 'dt_transacao': '2021-02-28 23:00:00'}, {'conta_id': 190835, 'valor': 905.17, 'dt_transacao': '2021-02-19 14:00:00'}, {'conta_id': 190836, 'valor': 1137.2, 'dt_transacao': '2021-03-26 19:00:00'}, {'conta_id': 190836, 'valor': 727.47, 'dt_transacao': '2021-03-01 17:00:00'}, {'conta_id': 190837, 'valor': 1304.82, 'dt_transacao': '2021-02-08 19:00:00'}, {'conta_id': 190837, 'valor': 824.82, 'dt_transacao': '2021-01-26 06:00:00'}, {'conta_id': 190837, 'valor': 676.03, 'dt_transacao': '2021-01-21 21:00:00'}, {'conta_id': 190837, 'valor': 769.2, 'dt_transacao': '2021-03-08 13:00:00'}, {'conta_id': 190837, 'valor': 361.45, 'dt_transacao': '2021-02-26 00:00:00'}, {'conta_id': 190838, 'valor': 823.17, 'dt_transacao': '2021-03-01 07:00:00'}, {'conta_id': 190838, 'valor': 1736.19, 'dt_transacao': '2021-01-17 15:00:00'}, {'conta_id': 190839, 'valor': 1705.35, 'dt_transacao': '2021-02-10 11:00:00'}, {'conta_id': 190839, 'valor': 406.81, 'dt_transacao': '2021-01-20 06:00:00'}, {'conta_id': 190839, 'valor': 615.82, 'dt_transacao': '2021-01-09 11:00:00'}, {'conta_id': 190840, 'valor': 1604.87, 'dt_transacao': '2021-01-21 04:00:00'}, {'conta_id': 190841, 'valor': 1380.24, 'dt_transacao': '2021-02-01 03:00:00'}, {'conta_id': 190841, 'valor': 1328.07, 'dt_transacao': '2021-01-19 14:00:00'}, {'conta_id': 190841, 'valor': 195.04, 'dt_transacao': '2021-01-06 04:00:00'}, {'conta_id': 190842, 'valor': 1144.25, 'dt_transacao': '2021-02-27 00:00:00'}, {'conta_id': 190842, 'valor': 1940.62, 'dt_transacao': '2021-03-27 18:00:00'}, {'conta_id': 190842, 'valor': 688.31, 'dt_transacao': '2021-02-13 02:00:00'}, {'conta_id': 190842, 'valor': 1773.09, 'dt_transacao': '2021-01-09 05:00:00'}, {'conta_id': 190842, 'valor': 270.75, 'dt_transacao': '2021-02-08 21:00:00'}, {'conta_id': 190843, 'valor': 1206.02, 'dt_transacao': '2021-03-05 00:00:00'}, {'conta_id': 190843, 'valor': 1244.1, 'dt_transacao': '2021-03-17 13:00:00'}, {'conta_id': 190843, 'valor': 1983.41, 'dt_transacao': '2021-02-16 08:00:00'}, {'conta_id': 190844, 'valor': 526.96, 'dt_transacao': '2021-02-25 10:00:00'}, {'conta_id': 190844, 'valor': 81.87, 'dt_transacao': '2021-03-26 14:00:00'}, {'conta_id': 190844, 'valor': 1319.16, 'dt_transacao': '2021-02-12 17:00:00'}, {'conta_id': 190845, 'valor': 1809.17, 'dt_transacao': '2021-02-18 12:00:00'}, {'conta_id': 190845, 'valor': 72.93, 'dt_transacao': '2021-03-28 08:00:00'}, {'conta_id': 190846, 'valor': 1105.21, 'dt_transacao': '2021-03-04 09:00:00'}, {'conta_id': 190846, 'valor': 534.08, 'dt_transacao': '2021-02-27 21:00:00'}, {'conta_id': 190846, 'valor': 170.51, 'dt_transacao': '2021-03-27 10:00:00'}, {'conta_id': 190847, 'valor': 130.85, 'dt_transacao': '2021-03-15 07:00:00'}, {'conta_id': 190847, 'valor': 1557.97, 'dt_transacao': '2021-01-10 00:00:00'}, {'conta_id': 190847, 'valor': 1446.49, 'dt_transacao': '2021-02-13 19:00:00'}, {'conta_id': 190848, 'valor': 399.84, 'dt_transacao': '2021-01-11 05:00:00'}, {'conta_id': 190848, 'valor': 184.32, 'dt_transacao': '2021-01-12 00:00:00'}, {'conta_id': 190849, 'valor': 1635.28, 'dt_transacao': '2021-04-01 15:00:00'}, {'conta_id': 190849, 'valor': 1211.31, 'dt_transacao': '2021-01-24 15:00:00'}, {'conta_id': 190849, 'valor': 576.84, 'dt_transacao': '2021-03-28 18:00:00'}, {'conta_id': 190849, 'valor': 1730.15, 'dt_transacao': '2021-01-08 14:00:00'}, {'conta_id': 190850, 'valor': 1406.03, 'dt_transacao': '2021-01-10 06:00:00'}, {'conta_id': 190851, 'valor': 853.62, 'dt_transacao': '2021-02-14 23:00:00'}, {'conta_id': 190851, 'valor': 482.3, 'dt_transacao': '2021-01-01 20:00:00'}, {'conta_id': 190852, 'valor': 1240.07, 'dt_transacao': '2021-02-25 02:00:00'}, {'conta_id': 190853, 'valor': 902.12, 'dt_transacao': '2021-03-03 11:00:00'}, {'conta_id': 190853, 'valor': 999.89, 'dt_transacao': '2021-03-04 10:00:00'}, {'conta_id': 190853, 'valor': 1092.29, 'dt_transacao': '2021-01-25 07:00:00'}, {'conta_id': 190854, 'valor': 1846.97, 'dt_transacao': '2021-02-21 14:00:00'}, {'conta_id': 190854, 'valor': 1737.5, 'dt_transacao': '2021-03-01 14:00:00'}, {'conta_id': 190854, 'valor': 306.22, 'dt_transacao': '2021-02-19 13:00:00'}, {'conta_id': 190855, 'valor': 869.33, 'dt_transacao': '2021-03-26 05:00:00'}, {'conta_id': 190856, 'valor': 675.29, 'dt_transacao': '2021-01-03 15:00:00'}, {'conta_id': 190856, 'valor': 785.07, 'dt_transacao': '2021-02-08 17:00:00'}, {'conta_id': 190856, 'valor': 556.06, 'dt_transacao': '2021-01-25 15:00:00'}, {'conta_id': 190857, 'valor': 134.75, 'dt_transacao': '2021-02-11 22:00:00'}, {'conta_id': 190858, 'valor': 26.92, 'dt_transacao': '2021-02-09 20:00:00'}, {'conta_id': 190858, 'valor': 318.05, 'dt_transacao': '2021-03-22 09:00:00'}, {'conta_id': 190858, 'valor': 1368.43, 'dt_transacao': '2021-02-24 19:00:00'}, {'conta_id': 190858, 'valor': 244.57, 'dt_transacao': '2021-01-17 16:00:00'}, {'conta_id': 190859, 'valor': 740.65, 'dt_transacao': '2021-02-01 06:00:00'}, {'conta_id': 190859, 'valor': 1889.21, 'dt_transacao': '2021-03-30 07:00:00'}, {'conta_id': 190859, 'valor': 1820.38, 'dt_transacao': '2021-02-10 20:00:00'}, {'conta_id': 190859, 'valor': 1831.56, 'dt_transacao': '2021-03-02 08:00:00'}, {'conta_id': 190860, 'valor': 977.69, 'dt_transacao': '2021-03-02 00:00:00'}, {'conta_id': 190860, 'valor': 627.61, 'dt_transacao': '2021-01-13 12:00:00'}, {'conta_id': 190861, 'valor': 779.76, 'dt_transacao': '2021-01-25 08:00:00'}, {'conta_id': 190861, 'valor': 1424.1, 'dt_transacao': '2021-02-16 01:00:00'}, {'conta_id': 190861, 'valor': 948.02, 'dt_transacao': '2021-01-29 20:00:00'}, {'conta_id': 190861, 'valor': 7.1, 'dt_transacao': '2021-02-15 19:00:00'}, {'conta_id': 190862, 'valor': 1210.56, 'dt_transacao': '2021-02-07 23:00:00'}, {'conta_id': 190862, 'valor': 122.07, 'dt_transacao': '2021-01-25 16:00:00'}, {'conta_id': 190862, 'valor': 1597.9, 'dt_transacao': '2021-02-06 07:00:00'}, {'conta_id': 190862, 'valor': 1581.63, 'dt_transacao': '2021-02-24 22:00:00'}, {'conta_id': 190862, 'valor': 1637.35, 'dt_transacao': '2021-01-02 09:00:00'}, {'conta_id': 190863, 'valor': 930.41, 'dt_transacao': '2021-01-19 14:00:00'}, {'conta_id': 190863, 'valor': 1252.15, 'dt_transacao': '2021-03-13 05:00:00'}, {'conta_id': 190863, 'valor': 1188.1, 'dt_transacao': '2021-02-10 03:00:00'}, {'conta_id': 190864, 'valor': 257.94, 'dt_transacao': '2021-01-03 03:00:00'}, {'conta_id': 190864, 'valor': 105.79, 'dt_transacao': '2021-02-11 17:00:00'}, {'conta_id': 190864, 'valor': 954.73, 'dt_transacao': '2021-01-19 10:00:00'}, {'conta_id': 190864, 'valor': 1663.78, 'dt_transacao': '2021-02-18 17:00:00'}, {'conta_id': 190864, 'valor': 51.91, 'dt_transacao': '2021-02-25 20:00:00'}, {'conta_id': 190865, 'valor': 1513.23, 'dt_transacao': '2021-01-06 16:00:00'}, {'conta_id': 190865, 'valor': 112.66, 'dt_transacao': '2021-01-17 15:00:00'}, {'conta_id': 190865, 'valor': 128.69, 'dt_transacao': '2021-01-09 04:00:00'}, {'conta_id': 190865, 'valor': 303.99, 'dt_transacao': '2021-03-08 05:00:00'}, {'conta_id': 190865, 'valor': 1880.31, 'dt_transacao': '2021-01-31 02:00:00'}, {'conta_id': 190866, 'valor': 1090.68, 'dt_transacao': '2021-03-09 13:00:00'}, {'conta_id': 190866, 'valor': 1284.0, 'dt_transacao': '2021-02-20 22:00:00'}, {'conta_id': 190866, 'valor': 310.81, 'dt_transacao': '2021-01-12 06:00:00'}, {'conta_id': 190866, 'valor': 897.55, 'dt_transacao': '2021-01-15 08:00:00'}, {'conta_id': 190867, 'valor': 85.75, 'dt_transacao': '2021-01-29 07:00:00'}, {'conta_id': 190867, 'valor': 1899.98, 'dt_transacao': '2021-01-18 16:00:00'}, {'conta_id': 190867, 'valor': 639.6, 'dt_transacao': '2021-02-19 20:00:00'}, {'conta_id': 190867, 'valor': 1340.86, 'dt_transacao': '2021-01-03 16:00:00'}, {'conta_id': 190868, 'valor': 1179.75, 'dt_transacao': '2021-01-27 07:00:00'}, {'conta_id': 190868, 'valor': 1165.9, 'dt_transacao': '2021-01-01 11:00:00'}, {'conta_id': 190869, 'valor': 1112.97, 'dt_transacao': '2021-03-07 12:00:00'}, {'conta_id': 190869, 'valor': 320.57, 'dt_transacao': '2021-01-26 19:00:00'}, {'conta_id': 190869, 'valor': 513.3, 'dt_transacao': '2021-01-26 10:00:00'}, {'conta_id': 190870, 'valor': 1935.5, 'dt_transacao': '2021-03-27 16:00:00'}, {'conta_id': 190870, 'valor': 1540.26, 'dt_transacao': '2021-01-27 05:00:00'}, {'conta_id': 190870, 'valor': 226.27, 'dt_transacao': '2021-02-19 03:00:00'}, {'conta_id': 190870, 'valor': 1485.88, 'dt_transacao': '2021-01-27 06:00:00'}, {'conta_id': 190870, 'valor': 1519.76, 'dt_transacao': '2021-02-08 22:00:00'}, {'conta_id': 190871, 'valor': 48.6, 'dt_transacao': '2021-01-02 06:00:00'}, {'conta_id': 190871, 'valor': 1659.34, 'dt_transacao': '2021-02-12 06:00:00'}, {'conta_id': 190871, 'valor': 999.69, 'dt_transacao': '2021-02-19 23:00:00'}, {'conta_id': 190871, 'valor': 655.39, 'dt_transacao': '2021-02-20 19:00:00'}, {'conta_id': 190871, 'valor': 1507.96, 'dt_transacao': '2021-03-04 15:00:00'}, {'conta_id': 190872, 'valor': 101.01, 'dt_transacao': '2021-01-02 08:00:00'}, {'conta_id': 190872, 'valor': 1860.39, 'dt_transacao': '2021-02-08 14:00:00'}, {'conta_id': 190872, 'valor': 763.46, 'dt_transacao': '2021-03-14 05:00:00'}, {'conta_id': 190872, 'valor': 195.37, 'dt_transacao': '2021-02-17 06:00:00'}, {'conta_id': 190872, 'valor': 1291.9, 'dt_transacao': '2021-02-24 23:00:00'}, {'conta_id': 190873, 'valor': 604.8, 'dt_transacao': '2021-02-03 14:00:00'}, {'conta_id': 190874, 'valor': 911.03, 'dt_transacao': '2021-03-16 22:00:00'}, {'conta_id': 190874, 'valor': 1115.33, 'dt_transacao': '2021-02-04 00:00:00'}, {'conta_id': 190875, 'valor': 1356.36, 'dt_transacao': '2021-01-04 01:00:00'}, {'conta_id': 190875, 'valor': 995.35, 'dt_transacao': '2021-03-01 13:00:00'}, {'conta_id': 190875, 'valor': 350.87, 'dt_transacao': '2021-01-02 18:00:00'}, {'conta_id': 190875, 'valor': 1794.92, 'dt_transacao': '2021-03-26 02:00:00'}, {'conta_id': 190876, 'valor': 1379.66, 'dt_transacao': '2021-01-01 20:00:00'}, {'conta_id': 190876, 'valor': 89.2, 'dt_transacao': '2021-03-14 13:00:00'}, {'conta_id': 190877, 'valor': 1641.61, 'dt_transacao': '2021-03-11 05:00:00'}, {'conta_id': 190877, 'valor': 747.16, 'dt_transacao': '2021-02-17 23:00:00'}, {'conta_id': 190877, 'valor': 1102.5, 'dt_transacao': '2021-01-19 10:00:00'}, {'conta_id': 190877, 'valor': 1470.65, 'dt_transacao': '2021-02-14 13:00:00'}, {'conta_id': 190878, 'valor': 816.24, 'dt_transacao': '2021-02-25 02:00:00'}, {'conta_id': 190878, 'valor': 1204.7, 'dt_transacao': '2021-01-18 10:00:00'}, {'conta_id': 190878, 'valor': 1849.51, 'dt_transacao': '2021-03-24 14:00:00'}, {'conta_id': 190878, 'valor': 902.09, 'dt_transacao': '2021-03-11 21:00:00'}, {'conta_id': 190878, 'valor': 329.39, 'dt_transacao': '2021-01-06 10:00:00'}, {'conta_id': 190879, 'valor': 1647.06, 'dt_transacao': '2021-01-15 16:00:00'}, {'conta_id': 190880, 'valor': 978.87, 'dt_transacao': '2021-02-11 23:00:00'}, {'conta_id': 190880, 'valor': 1673.1, 'dt_transacao': '2021-02-17 17:00:00'}, {'conta_id': 190880, 'valor': 1873.92, 'dt_transacao': '2021-03-08 21:00:00'}, {'conta_id': 190881, 'valor': 1668.86, 'dt_transacao': '2021-01-25 22:00:00'}, {'conta_id': 190881, 'valor': 2.19, 'dt_transacao': '2021-03-06 03:00:00'}, {'conta_id': 190882, 'valor': 1084.39, 'dt_transacao': '2021-01-13 06:00:00'}, {'conta_id': 190882, 'valor': 47.99, 'dt_transacao': '2021-01-04 09:00:00'}, {'conta_id': 190883, 'valor': 844.41, 'dt_transacao': '2021-02-15 21:00:00'}, {'conta_id': 190884, 'valor': 1238.9, 'dt_transacao': '2021-03-03 20:00:00'}, {'conta_id': 190884, 'valor': 1229.99, 'dt_transacao': '2021-01-06 21:00:00'}, {'conta_id': 190884, 'valor': 983.18, 'dt_transacao': '2021-01-05 15:00:00'}, {'conta_id': 190884, 'valor': 639.94, 'dt_transacao': '2021-01-04 08:00:00'}, {'conta_id': 190885, 'valor': 78.3, 'dt_transacao': '2021-03-04 02:00:00'}, {'conta_id': 190885, 'valor': 950.27, 'dt_transacao': '2021-03-11 02:00:00'}, {'conta_id': 190885, 'valor': 166.85, 'dt_transacao': '2021-03-22 03:00:00'}, {'conta_id': 190885, 'valor': 909.92, 'dt_transacao': '2021-02-13 07:00:00'}, {'conta_id': 190886, 'valor': 21.58, 'dt_transacao': '2021-01-17 18:00:00'}, {'conta_id': 190887, 'valor': 1307.75, 'dt_transacao': '2021-01-07 08:00:00'}, {'conta_id': 190887, 'valor': 1171.74, 'dt_transacao': '2021-03-31 16:00:00'}, {'conta_id': 190887, 'valor': 982.71, 'dt_transacao': '2021-02-27 09:00:00'}, {'conta_id': 190887, 'valor': 1707.26, 'dt_transacao': '2021-01-06 09:00:00'}, {'conta_id': 190887, 'valor': 1202.64, 'dt_transacao': '2021-02-24 03:00:00'}, {'conta_id': 190888, 'valor': 1545.15, 'dt_transacao': '2021-01-06 09:00:00'}, {'conta_id': 190888, 'valor': 366.17, 'dt_transacao': '2021-02-13 21:00:00'}, {'conta_id': 190889, 'valor': 264.87, 'dt_transacao': '2021-03-19 18:00:00'}, {'conta_id': 190889, 'valor': 48.33, 'dt_transacao': '2021-03-05 11:00:00'}, {'conta_id': 190890, 'valor': 1115.01, 'dt_transacao': '2021-02-24 02:00:00'}, {'conta_id': 190890, 'valor': 131.29, 'dt_transacao': '2021-02-03 21:00:00'}, {'conta_id': 190890, 'valor': 715.93, 'dt_transacao': '2021-02-21 16:00:00'}, {'conta_id': 190890, 'valor': 1838.63, 'dt_transacao': '2021-01-18 03:00:00'}, {'conta_id': 190891, 'valor': 526.75, 'dt_transacao': '2021-01-09 21:00:00'}, {'conta_id': 190891, 'valor': 371.83, 'dt_transacao': '2021-02-22 03:00:00'}, {'conta_id': 190891, 'valor': 1119.77, 'dt_transacao': '2021-01-30 06:00:00'}, {'conta_id': 190891, 'valor': 295.63, 'dt_transacao': '2021-01-18 23:00:00'}, {'conta_id': 190891, 'valor': 1403.25, 'dt_transacao': '2021-01-02 11:00:00'}, {'conta_id': 190892, 'valor': 639.74, 'dt_transacao': '2021-02-01 15:00:00'}, {'conta_id': 190892, 'valor': 1602.65, 'dt_transacao': '2021-01-25 21:00:00'}, {'conta_id': 190893, 'valor': 1334.5, 'dt_transacao': '2021-03-07 04:00:00'}, {'conta_id': 190893, 'valor': 1722.51, 'dt_transacao': '2021-01-09 15:00:00'}, {'conta_id': 190894, 'valor': 813.38, 'dt_transacao': '2021-02-25 01:00:00'}, {'conta_id': 190894, 'valor': 1741.47, 'dt_transacao': '2021-02-20 00:00:00'}, {'conta_id': 190894, 'valor': 386.02, 'dt_transacao': '2021-02-17 12:00:00'}, {'conta_id': 190895, 'valor': 1043.0, 'dt_transacao': '2021-03-05 02:00:00'}, {'conta_id': 190895, 'valor': 866.05, 'dt_transacao': '2021-01-04 13:00:00'}, {'conta_id': 190895, 'valor': 133.6, 'dt_transacao': '2021-01-11 22:00:00'}, {'conta_id': 190895, 'valor': 1783.41, 'dt_transacao': '2021-01-19 20:00:00'}, {'conta_id': 190895, 'valor': 80.86, 'dt_transacao': '2021-02-05 12:00:00'}, {'conta_id': 190896, 'valor': 976.09, 'dt_transacao': '2021-03-30 18:00:00'}, {'conta_id': 190896, 'valor': 1390.8, 'dt_transacao': '2021-03-29 19:00:00'}, {'conta_id': 190896, 'valor': 1199.08, 'dt_transacao': '2021-03-15 04:00:00'}, {'conta_id': 190896, 'valor': 939.19, 'dt_transacao': '2021-01-12 22:00:00'}, {'conta_id': 190896, 'valor': 979.86, 'dt_transacao': '2021-02-13 17:00:00'}, {'conta_id': 190897, 'valor': 1814.53, 'dt_transacao': '2021-03-14 06:00:00'}, {'conta_id': 190897, 'valor': 697.68, 'dt_transacao': '2021-02-09 06:00:00'}, {'conta_id': 190897, 'valor': 1651.64, 'dt_transacao': '2021-03-08 21:00:00'}, {'conta_id': 190898, 'valor': 1374.52, 'dt_transacao': '2021-02-16 22:00:00'}, {'conta_id': 190898, 'valor': 1086.01, 'dt_transacao': '2021-02-11 06:00:00'}, {'conta_id': 190899, 'valor': 1774.66, 'dt_transacao': '2021-03-05 11:00:00'}, {'conta_id': 190899, 'valor': 492.02, 'dt_transacao': '2021-03-03 22:00:00'}, {'conta_id': 190900, 'valor': 1626.42, 'dt_transacao': '2021-02-20 00:00:00'}, {'conta_id': 190900, 'valor': 134.03, 'dt_transacao': '2021-02-02 15:00:00'}, {'conta_id': 190900, 'valor': 910.07, 'dt_transacao': '2021-03-12 18:00:00'}, {'conta_id': 190900, 'valor': 1642.5, 'dt_transacao': '2021-03-07 18:00:00'}, {'conta_id': 190901, 'valor': 774.38, 'dt_transacao': '2021-01-16 04:00:00'}, {'conta_id': 190901, 'valor': 363.03, 'dt_transacao': '2021-02-16 01:00:00'}, {'conta_id': 190901, 'valor': 83.03, 'dt_transacao': '2021-01-09 08:00:00'}, {'conta_id': 190901, 'valor': 215.82, 'dt_transacao': '2021-03-10 08:00:00'}, {'conta_id': 190902, 'valor': 1953.7, 'dt_transacao': '2021-02-05 09:00:00'}, {'conta_id': 190902, 'valor': 860.18, 'dt_transacao': '2021-01-30 07:00:00'}, {'conta_id': 190902, 'valor': 1237.01, 'dt_transacao': '2021-03-15 08:00:00'}, {'conta_id': 190903, 'valor': 407.08, 'dt_transacao': '2021-03-20 00:00:00'}, {'conta_id': 190903, 'valor': 1908.99, 'dt_transacao': '2021-01-12 18:00:00'}, {'conta_id': 190903, 'valor': 1481.17, 'dt_transacao': '2021-02-04 15:00:00'}, {'conta_id': 190903, 'valor': 250.0, 'dt_transacao': '2021-02-01 13:00:00'}, {'conta_id': 190904, 'valor': 478.3, 'dt_transacao': '2021-02-09 13:00:00'}, {'conta_id': 190905, 'valor': 885.61, 'dt_transacao': '2021-02-19 19:00:00'}, {'conta_id': 190905, 'valor': 1501.64, 'dt_transacao': '2021-01-03 17:00:00'}, {'conta_id': 190905, 'valor': 1530.35, 'dt_transacao': '2021-01-27 21:00:00'}, {'conta_id': 190905, 'valor': 983.55, 'dt_transacao': '2021-03-09 10:00:00'}, {'conta_id': 190906, 'valor': 1202.31, 'dt_transacao': '2021-03-31 21:00:00'}, {'conta_id': 190907, 'valor': 498.28, 'dt_transacao': '2021-02-07 07:00:00'}, {'conta_id': 190907, 'valor': 759.51, 'dt_transacao': '2021-03-02 00:00:00'}, {'conta_id': 190907, 'valor': 1373.3, 'dt_transacao': '2021-03-14 07:00:00'}, {'conta_id': 190908, 'valor': 982.42, 'dt_transacao': '2021-02-27 13:00:00'}, {'conta_id': 190909, 'valor': 317.04, 'dt_transacao': '2021-01-29 18:00:00'}, {'conta_id': 190909, 'valor': 1290.37, 'dt_transacao': '2021-03-18 16:00:00'}, {'conta_id': 190909, 'valor': 1620.47, 'dt_transacao': '2021-01-05 12:00:00'}, {'conta_id': 190909, 'valor': 729.78, 'dt_transacao': '2021-01-30 02:00:00'}, {'conta_id': 190910, 'valor': 284.13, 'dt_transacao': '2021-02-25 08:00:00'}, {'conta_id': 190910, 'valor': 1742.15, 'dt_transacao': '2021-01-24 07:00:00'}, {'conta_id': 190910, 'valor': 819.52, 'dt_transacao': '2021-01-22 16:00:00'}, {'conta_id': 190911, 'valor': 1789.95, 'dt_transacao': '2021-01-31 00:00:00'}, {'conta_id': 190911, 'valor': 1615.65, 'dt_transacao': '2021-01-26 19:00:00'}, {'conta_id': 190911, 'valor': 1425.35, 'dt_transacao': '2021-01-09 03:00:00'}, {'conta_id': 190911, 'valor': 538.82, 'dt_transacao': '2021-01-23 17:00:00'}, {'conta_id': 190912, 'valor': 1710.96, 'dt_transacao': '2021-02-04 16:00:00'}, {'conta_id': 190912, 'valor': 1626.8, 'dt_transacao': '2021-03-06 16:00:00'}, {'conta_id': 190912, 'valor': 1789.78, 'dt_transacao': '2021-02-20 04:00:00'}, {'conta_id': 190913, 'valor': 492.33, 'dt_transacao': '2021-01-23 22:00:00'}, {'conta_id': 190913, 'valor': 220.64, 'dt_transacao': '2021-02-01 05:00:00'}, {'conta_id': 190913, 'valor': 1280.63, 'dt_transacao': '2021-01-06 02:00:00'}, {'conta_id': 190913, 'valor': 1965.73, 'dt_transacao': '2021-01-18 00:00:00'}, {'conta_id': 190914, 'valor': 1155.37, 'dt_transacao': '2021-02-21 08:00:00'}, {'conta_id': 190914, 'valor': 1047.42, 'dt_transacao': '2021-02-22 05:00:00'}, {'conta_id': 190914, 'valor': 1006.8, 'dt_transacao': '2021-03-05 04:00:00'}, {'conta_id': 190914, 'valor': 1609.87, 'dt_transacao': '2021-01-27 02:00:00'}, {'conta_id': 190914, 'valor': 1200.0, 'dt_transacao': '2021-02-17 09:00:00'}, {'conta_id': 190915, 'valor': 398.21, 'dt_transacao': '2021-01-03 06:00:00'}, {'conta_id': 190915, 'valor': 871.64, 'dt_transacao': '2021-03-23 04:00:00'}, {'conta_id': 190915, 'valor': 900.43, 'dt_transacao': '2021-03-17 23:00:00'}, {'conta_id': 190915, 'valor': 280.55, 'dt_transacao': '2021-02-08 12:00:00'}, {'conta_id': 190916, 'valor': 1426.07, 'dt_transacao': '2021-02-17 06:00:00'}, {'conta_id': 190916, 'valor': 927.14, 'dt_transacao': '2021-02-23 01:00:00'}, {'conta_id': 190916, 'valor': 491.49, 'dt_transacao': '2021-01-19 20:00:00'}, {'conta_id': 190916, 'valor': 1845.57, 'dt_transacao': '2021-02-07 11:00:00'}, {'conta_id': 190917, 'valor': 1642.37, 'dt_transacao': '2021-03-12 15:00:00'}, {'conta_id': 190917, 'valor': 82.01, 'dt_transacao': '2021-03-03 02:00:00'}, {'conta_id': 190917, 'valor': 705.61, 'dt_transacao': '2021-01-25 09:00:00'}, {'conta_id': 190918, 'valor': 438.17, 'dt_transacao': '2021-03-08 11:00:00'}, {'conta_id': 190918, 'valor': 49.26, 'dt_transacao': '2021-02-21 10:00:00'}, {'conta_id': 190918, 'valor': 1597.22, 'dt_transacao': '2021-02-28 11:00:00'}, {'conta_id': 190918, 'valor': 1243.92, 'dt_transacao': '2021-02-06 11:00:00'}, {'conta_id': 190918, 'valor': 102.78, 'dt_transacao': '2021-03-08 18:00:00'}, {'conta_id': 190919, 'valor': 660.97, 'dt_transacao': '2021-02-26 23:00:00'}, {'conta_id': 190919, 'valor': 1445.36, 'dt_transacao': '2021-02-04 04:00:00'}, {'conta_id': 190919, 'valor': 285.28, 'dt_transacao': '2021-01-26 02:00:00'}, {'conta_id': 190919, 'valor': 1234.13, 'dt_transacao': '2021-02-05 12:00:00'}, {'conta_id': 190920, 'valor': 388.13, 'dt_transacao': '2021-01-20 13:00:00'}, {'conta_id': 190920, 'valor': 1094.56, 'dt_transacao': '2021-02-02 06:00:00'}, {'conta_id': 190920, 'valor': 2.13, 'dt_transacao': '2021-01-09 05:00:00'}, {'conta_id': 190920, 'valor': 1717.82, 'dt_transacao': '2021-02-03 23:00:00'}, {'conta_id': 190920, 'valor': 965.85, 'dt_transacao': '2021-01-09 07:00:00'}, {'conta_id': 190921, 'valor': 1911.38, 'dt_transacao': '2021-02-03 18:00:00'}, {'conta_id': 190921, 'valor': 1117.58, 'dt_transacao': '2021-02-12 20:00:00'}, {'conta_id': 190921, 'valor': 1330.22, 'dt_transacao': '2021-03-09 04:00:00'}, {'conta_id': 190921, 'valor': 1335.84, 'dt_transacao': '2021-03-17 16:00:00'}, {'conta_id': 190922, 'valor': 601.84, 'dt_transacao': '2021-02-21 01:00:00'}, {'conta_id': 190922, 'valor': 1904.25, 'dt_transacao': '2021-02-08 08:00:00'}, {'conta_id': 190922, 'valor': 213.89, 'dt_transacao': '2021-01-19 17:00:00'}, {'conta_id': 190922, 'valor': 448.03, 'dt_transacao': '2021-01-29 07:00:00'}, {'conta_id': 190922, 'valor': 929.23, 'dt_transacao': '2021-01-20 03:00:00'}, {'conta_id': 190923, 'valor': 325.73, 'dt_transacao': '2021-03-03 21:00:00'}, {'conta_id': 190923, 'valor': 133.57, 'dt_transacao': '2021-02-10 01:00:00'}, {'conta_id': 190924, 'valor': 237.33, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190924, 'valor': 1758.33, 'dt_transacao': '2021-03-04 12:00:00'}, {'conta_id': 190924, 'valor': 1177.26, 'dt_transacao': '2021-02-21 13:00:00'}, {'conta_id': 190925, 'valor': 202.22, 'dt_transacao': '2021-01-11 23:00:00'}, {'conta_id': 190926, 'valor': 355.97, 'dt_transacao': '2021-01-18 09:00:00'}, {'conta_id': 190926, 'valor': 1967.42, 'dt_transacao': '2021-02-16 21:00:00'}, {'conta_id': 190926, 'valor': 1371.64, 'dt_transacao': '2021-03-31 11:00:00'}, {'conta_id': 190926, 'valor': 1332.45, 'dt_transacao': '2021-01-12 01:00:00'}, {'conta_id': 190927, 'valor': 1579.3, 'dt_transacao': '2021-02-20 15:00:00'}, {'conta_id': 190927, 'valor': 215.97, 'dt_transacao': '2021-03-17 09:00:00'}, {'conta_id': 190928, 'valor': 193.14, 'dt_transacao': '2021-03-19 07:00:00'}, {'conta_id': 190928, 'valor': 1378.38, 'dt_transacao': '2021-01-05 12:00:00'}, {'conta_id': 190928, 'valor': 1704.46, 'dt_transacao': '2021-03-27 05:00:00'}, {'conta_id': 190928, 'valor': 1390.12, 'dt_transacao': '2021-02-27 11:00:00'}, {'conta_id': 190928, 'valor': 1903.46, 'dt_transacao': '2021-02-09 09:00:00'}, {'conta_id': 190929, 'valor': 298.29, 'dt_transacao': '2021-01-20 01:00:00'}, {'conta_id': 190929, 'valor': 159.83, 'dt_transacao': '2021-03-17 03:00:00'}, {'conta_id': 190929, 'valor': 296.21, 'dt_transacao': '2021-02-27 04:00:00'}, {'conta_id': 190929, 'valor': 299.5, 'dt_transacao': '2021-01-10 09:00:00'}, {'conta_id': 190930, 'valor': 1791.28, 'dt_transacao': '2021-03-25 01:00:00'}, {'conta_id': 190930, 'valor': 729.05, 'dt_transacao': '2021-02-20 06:00:00'}, {'conta_id': 190931, 'valor': 1317.0, 'dt_transacao': '2021-02-20 22:00:00'}, {'conta_id': 190931, 'valor': 1725.29, 'dt_transacao': '2021-02-27 04:00:00'}, {'conta_id': 190931, 'valor': 1631.75, 'dt_transacao': '2021-04-01 13:00:00'}, {'conta_id': 190931, 'valor': 1317.15, 'dt_transacao': '2021-03-14 22:00:00'}, {'conta_id': 190932, 'valor': 85.01, 'dt_transacao': '2021-01-18 07:00:00'}, {'conta_id': 190932, 'valor': 1026.06, 'dt_transacao': '2021-03-05 08:00:00'}, {'conta_id': 190933, 'valor': 6.47, 'dt_transacao': '2021-03-03 16:00:00'}, {'conta_id': 190933, 'valor': 404.22, 'dt_transacao': '2021-01-19 10:00:00'}, {'conta_id': 190933, 'valor': 1361.03, 'dt_transacao': '2021-03-06 08:00:00'}, {'conta_id': 190934, 'valor': 569.69, 'dt_transacao': '2021-01-03 13:00:00'}, {'conta_id': 190935, 'valor': 530.14, 'dt_transacao': '2021-03-25 01:00:00'}, {'conta_id': 190935, 'valor': 1633.66, 'dt_transacao': '2021-03-27 18:00:00'}, {'conta_id': 190935, 'valor': 509.03, 'dt_transacao': '2021-02-03 00:00:00'}, {'conta_id': 190935, 'valor': 1061.95, 'dt_transacao': '2021-01-06 07:00:00'}, {'conta_id': 190936, 'valor': 1629.03, 'dt_transacao': '2021-03-11 13:00:00'}, {'conta_id': 190936, 'valor': 33.67, 'dt_transacao': '2021-03-23 09:00:00'}, {'conta_id': 190936, 'valor': 1538.03, 'dt_transacao': '2021-03-26 01:00:00'}, {'conta_id': 190936, 'valor': 993.77, 'dt_transacao': '2021-03-16 06:00:00'}, {'conta_id': 190936, 'valor': 1068.65, 'dt_transacao': '2021-03-12 09:00:00'}, {'conta_id': 190937, 'valor': 658.22, 'dt_transacao': '2021-02-01 15:00:00'}, {'conta_id': 190938, 'valor': 324.56, 'dt_transacao': '2021-02-24 06:00:00'}, {'conta_id': 190938, 'valor': 1458.59, 'dt_transacao': '2021-02-16 03:00:00'}, {'conta_id': 190938, 'valor': 1530.42, 'dt_transacao': '2021-02-06 09:00:00'}, {'conta_id': 190938, 'valor': 1995.77, 'dt_transacao': '2021-03-30 17:00:00'}, {'conta_id': 190939, 'valor': 227.63, 'dt_transacao': '2021-02-27 10:00:00'}, {'conta_id': 190939, 'valor': 243.76, 'dt_transacao': '2021-03-10 14:00:00'}, {'conta_id': 190940, 'valor': 776.37, 'dt_transacao': '2021-01-25 17:00:00'}, {'conta_id': 190940, 'valor': 1889.94, 'dt_transacao': '2021-01-03 19:00:00'}, {'conta_id': 190941, 'valor': 60.55, 'dt_transacao': '2021-02-19 13:00:00'}, {'conta_id': 190941, 'valor': 208.04, 'dt_transacao': '2021-03-07 12:00:00'}, {'conta_id': 190941, 'valor': 425.51, 'dt_transacao': '2021-01-25 19:00:00'}, {'conta_id': 190942, 'valor': 399.03, 'dt_transacao': '2021-01-06 01:00:00'}, {'conta_id': 190942, 'valor': 906.37, 'dt_transacao': '2021-01-01 18:00:00'}, {'conta_id': 190942, 'valor': 711.49, 'dt_transacao': '2021-03-27 11:00:00'}, {'conta_id': 190943, 'valor': 1041.51, 'dt_transacao': '2021-01-15 18:00:00'}, {'conta_id': 190943, 'valor': 272.4, 'dt_transacao': '2021-03-05 17:00:00'}, {'conta_id': 190943, 'valor': 1992.28, 'dt_transacao': '2021-02-13 03:00:00'}, {'conta_id': 190943, 'valor': 1250.27, 'dt_transacao': '2021-01-05 20:00:00'}, {'conta_id': 190943, 'valor': 634.82, 'dt_transacao': '2021-02-09 17:00:00'}, {'conta_id': 190944, 'valor': 1998.56, 'dt_transacao': '2021-02-05 10:00:00'}, {'conta_id': 190944, 'valor': 1699.62, 'dt_transacao': '2021-01-10 16:00:00'}, {'conta_id': 190944, 'valor': 728.21, 'dt_transacao': '2021-03-05 09:00:00'}, {'conta_id': 190944, 'valor': 1477.52, 'dt_transacao': '2021-02-22 00:00:00'}, {'conta_id': 190945, 'valor': 227.63, 'dt_transacao': '2021-02-03 10:00:00'}, {'conta_id': 190945, 'valor': 637.31, 'dt_transacao': '2021-01-16 05:00:00'}, {'conta_id': 190945, 'valor': 1153.25, 'dt_transacao': '2021-03-16 02:00:00'}, {'conta_id': 190946, 'valor': 1790.76, 'dt_transacao': '2021-01-06 09:00:00'}, {'conta_id': 190947, 'valor': 9.01, 'dt_transacao': '2021-03-03 01:00:00'}, {'conta_id': 190947, 'valor': 938.98, 'dt_transacao': '2021-02-16 12:00:00'}, {'conta_id': 190947, 'valor': 1084.39, 'dt_transacao': '2021-02-03 05:00:00'}, {'conta_id': 190947, 'valor': 1182.12, 'dt_transacao': '2021-03-13 07:00:00'}, {'conta_id': 190947, 'valor': 164.94, 'dt_transacao': '2021-01-22 05:00:00'}, {'conta_id': 190948, 'valor': 11.57, 'dt_transacao': '2021-03-08 08:00:00'}, {'conta_id': 190948, 'valor': 1153.8, 'dt_transacao': '2021-01-25 15:00:00'}, {'conta_id': 190948, 'valor': 1095.58, 'dt_transacao': '2021-01-08 13:00:00'}, {'conta_id': 190948, 'valor': 422.8, 'dt_transacao': '2021-03-27 12:00:00'}, {'conta_id': 190949, 'valor': 1233.15, 'dt_transacao': '2021-01-16 22:00:00'}, {'conta_id': 190949, 'valor': 841.03, 'dt_transacao': '2021-03-24 19:00:00'}, {'conta_id': 190950, 'valor': 369.53, 'dt_transacao': '2021-03-24 18:00:00'}, {'conta_id': 190950, 'valor': 240.83, 'dt_transacao': '2021-03-27 07:00:00'}, {'conta_id': 190950, 'valor': 265.76, 'dt_transacao': '2021-01-06 07:00:00'}, {'conta_id': 190950, 'valor': 1996.29, 'dt_transacao': '2021-01-06 23:00:00'}, {'conta_id': 190950, 'valor': 145.33, 'dt_transacao': '2021-02-20 04:00:00'}, {'conta_id': 190951, 'valor': 214.8, 'dt_transacao': '2021-01-27 14:00:00'}, {'conta_id': 190951, 'valor': 289.62, 'dt_transacao': '2021-03-24 03:00:00'}, {'conta_id': 190951, 'valor': 1737.73, 'dt_transacao': '2021-02-11 13:00:00'}, {'conta_id': 190951, 'valor': 758.07, 'dt_transacao': '2021-03-08 21:00:00'}, {'conta_id': 190952, 'valor': 980.34, 'dt_transacao': '2021-02-05 13:00:00'}, {'conta_id': 190952, 'valor': 98.82, 'dt_transacao': '2021-01-01 13:00:00'}, {'conta_id': 190953, 'valor': 1762.73, 'dt_transacao': '2021-01-19 09:00:00'}, {'conta_id': 190954, 'valor': 1288.86, 'dt_transacao': '2021-02-27 06:00:00'}, {'conta_id': 190954, 'valor': 1377.17, 'dt_transacao': '2021-02-05 15:00:00'}, {'conta_id': 190954, 'valor': 1927.05, 'dt_transacao': '2021-02-04 05:00:00'}, {'conta_id': 190955, 'valor': 1973.0, 'dt_transacao': '2021-03-12 16:00:00'}, {'conta_id': 190955, 'valor': 1284.14, 'dt_transacao': '2021-01-19 03:00:00'}, {'conta_id': 190956, 'valor': 1134.74, 'dt_transacao': '2021-01-14 07:00:00'}, {'conta_id': 190957, 'valor': 655.43, 'dt_transacao': '2021-03-30 17:00:00'}, {'conta_id': 190958, 'valor': 503.08, 'dt_transacao': '2021-02-04 18:00:00'}, {'conta_id': 190958, 'valor': 399.73, 'dt_transacao': '2021-02-21 21:00:00'}, {'conta_id': 190958, 'valor': 1683.98, 'dt_transacao': '2021-03-19 18:00:00'}, {'conta_id': 190958, 'valor': 1128.48, 'dt_transacao': '2021-01-13 12:00:00'}, {'conta_id': 190959, 'valor': 1065.44, 'dt_transacao': '2021-03-05 07:00:00'}, {'conta_id': 190959, 'valor': 862.22, 'dt_transacao': '2021-03-13 00:00:00'}, {'conta_id': 190960, 'valor': 1070.43, 'dt_transacao': '2021-01-10 03:00:00'}, {'conta_id': 190960, 'valor': 1362.24, 'dt_transacao': '2021-03-12 14:00:00'}, {'conta_id': 190960, 'valor': 1437.14, 'dt_transacao': '2021-01-29 23:00:00'}, {'conta_id': 190960, 'valor': 429.85, 'dt_transacao': '2021-02-17 03:00:00'}, {'conta_id': 190961, 'valor': 187.87, 'dt_transacao': '2021-01-31 19:00:00'}, {'conta_id': 190961, 'valor': 238.44, 'dt_transacao': '2021-02-02 19:00:00'}, {'conta_id': 190962, 'valor': 1634.31, 'dt_transacao': '2021-01-28 13:00:00'}, {'conta_id': 190962, 'valor': 1775.05, 'dt_transacao': '2021-01-13 02:00:00'}, {'conta_id': 190962, 'valor': 1007.9, 'dt_transacao': '2021-01-30 18:00:00'}, {'conta_id': 190962, 'valor': 556.06, 'dt_transacao': '2021-02-24 22:00:00'}, {'conta_id': 190963, 'valor': 344.96, 'dt_transacao': '2021-01-21 07:00:00'}, {'conta_id': 190963, 'valor': 1475.21, 'dt_transacao': '2021-01-23 06:00:00'}, {'conta_id': 190964, 'valor': 269.9, 'dt_transacao': '2021-01-16 23:00:00'}, {'conta_id': 190964, 'valor': 1824.57, 'dt_transacao': '2021-01-27 11:00:00'}, {'conta_id': 190964, 'valor': 1254.63, 'dt_transacao': '2021-02-08 21:00:00'}, {'conta_id': 190964, 'valor': 864.21, 'dt_transacao': '2021-03-30 02:00:00'}, {'conta_id': 190964, 'valor': 1667.41, 'dt_transacao': '2021-01-21 15:00:00'}, {'conta_id': 190965, 'valor': 1529.07, 'dt_transacao': '2021-01-28 02:00:00'}, {'conta_id': 190966, 'valor': 315.64, 'dt_transacao': '2021-03-18 17:00:00'}, {'conta_id': 190966, 'valor': 1321.37, 'dt_transacao': '2021-01-28 14:00:00'}, {'conta_id': 190966, 'valor': 850.35, 'dt_transacao': '2021-03-20 14:00:00'}, {'conta_id': 190966, 'valor': 559.07, 'dt_transacao': '2021-02-03 19:00:00'}, {'conta_id': 190967, 'valor': 231.58, 'dt_transacao': '2021-03-30 04:00:00'}, {'conta_id': 190967, 'valor': 1801.76, 'dt_transacao': '2021-01-30 23:00:00'}, {'conta_id': 190967, 'valor': 1400.84, 'dt_transacao': '2021-01-15 17:00:00'}, {'conta_id': 190967, 'valor': 959.16, 'dt_transacao': '2021-03-25 18:00:00'}, {'conta_id': 190967, 'valor': 1425.19, 'dt_transacao': '2021-01-18 01:00:00'}, {'conta_id': 190968, 'valor': 1105.0, 'dt_transacao': '2021-02-13 16:00:00'}, {'conta_id': 190968, 'valor': 1075.6, 'dt_transacao': '2021-02-03 08:00:00'}, {'conta_id': 190968, 'valor': 664.82, 'dt_transacao': '2021-01-18 05:00:00'}, {'conta_id': 190968, 'valor': 88.99, 'dt_transacao': '2021-03-04 06:00:00'}, {'conta_id': 190968, 'valor': 280.29, 'dt_transacao': '2021-01-29 22:00:00'}, {'conta_id': 190969, 'valor': 1363.0, 'dt_transacao': '2021-02-13 00:00:00'}, {'conta_id': 190969, 'valor': 67.18, 'dt_transacao': '2021-01-02 00:00:00'}, {'conta_id': 190970, 'valor': 1906.22, 'dt_transacao': '2021-01-13 14:00:00'}, {'conta_id': 190970, 'valor': 1567.52, 'dt_transacao': '2021-03-10 20:00:00'}, {'conta_id': 190970, 'valor': 743.71, 'dt_transacao': '2021-03-22 23:00:00'}, {'conta_id': 190970, 'valor': 1937.32, 'dt_transacao': '2021-01-04 23:00:00'}, {'conta_id': 190971, 'valor': 319.39, 'dt_transacao': '2021-03-11 04:00:00'}, {'conta_id': 190971, 'valor': 1201.1, 'dt_transacao': '2021-03-15 17:00:00'}, {'conta_id': 190971, 'valor': 954.08, 'dt_transacao': '2021-03-04 16:00:00'}, {'conta_id': 190972, 'valor': 81.73, 'dt_transacao': '2021-02-25 14:00:00'}, {'conta_id': 190972, 'valor': 53.48, 'dt_transacao': '2021-03-18 01:00:00'}, {'conta_id': 190972, 'valor': 6.59, 'dt_transacao': '2021-02-03 07:00:00'}, {'conta_id': 190972, 'valor': 756.41, 'dt_transacao': '2021-03-23 10:00:00'}, {'conta_id': 190973, 'valor': 299.4, 'dt_transacao': '2021-02-04 16:00:00'}, {'conta_id': 190973, 'valor': 456.46, 'dt_transacao': '2021-02-11 11:00:00'}, {'conta_id': 190973, 'valor': 1322.99, 'dt_transacao': '2021-02-16 09:00:00'}, {'conta_id': 190973, 'valor': 1643.33, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190974, 'valor': 1712.2, 'dt_transacao': '2021-03-13 14:00:00'}, {'conta_id': 190974, 'valor': 1236.56, 'dt_transacao': '2021-01-03 19:00:00'}, {'conta_id': 190974, 'valor': 1557.45, 'dt_transacao': '2021-03-09 05:00:00'}, {'conta_id': 190974, 'valor': 252.41, 'dt_transacao': '2021-01-08 08:00:00'}, {'conta_id': 190974, 'valor': 1604.19, 'dt_transacao': '2021-01-16 12:00:00'}, {'conta_id': 190975, 'valor': 1761.03, 'dt_transacao': '2021-02-01 16:00:00'}, {'conta_id': 190975, 'valor': 388.52, 'dt_transacao': '2021-01-25 16:00:00'}, {'conta_id': 190975, 'valor': 1295.84, 'dt_transacao': '2021-02-21 08:00:00'}, {'conta_id': 190975, 'valor': 485.21, 'dt_transacao': '2021-03-28 02:00:00'}, {'conta_id': 190976, 'valor': 42.4, 'dt_transacao': '2021-02-23 22:00:00'}, {'conta_id': 190976, 'valor': 261.89, 'dt_transacao': '2021-03-09 13:00:00'}, {'conta_id': 190977, 'valor': 1890.96, 'dt_transacao': '2021-03-22 12:00:00'}, {'conta_id': 190977, 'valor': 314.19, 'dt_transacao': '2021-02-17 09:00:00'}, {'conta_id': 190977, 'valor': 1732.68, 'dt_transacao': '2021-01-06 05:00:00'}, {'conta_id': 190978, 'valor': 1337.66, 'dt_transacao': '2021-03-02 12:00:00'}, {'conta_id': 190979, 'valor': 483.65, 'dt_transacao': '2021-03-31 16:00:00'}, {'conta_id': 190979, 'valor': 1037.98, 'dt_transacao': '2021-02-05 10:00:00'}, {'conta_id': 190979, 'valor': 1451.26, 'dt_transacao': '2021-01-18 14:00:00'}, {'conta_id': 190980, 'valor': 1833.06, 'dt_transacao': '2021-01-27 11:00:00'}, {'conta_id': 190980, 'valor': 389.46, 'dt_transacao': '2021-03-09 08:00:00'}, {'conta_id': 190981, 'valor': 1422.9, 'dt_transacao': '2021-02-28 16:00:00'}, {'conta_id': 190982, 'valor': 434.35, 'dt_transacao': '2021-02-21 20:00:00'}, {'conta_id': 190982, 'valor': 646.72, 'dt_transacao': '2021-01-29 01:00:00'}, {'conta_id': 190982, 'valor': 1764.88, 'dt_transacao': '2021-01-05 23:00:00'}, {'conta_id': 190982, 'valor': 154.6, 'dt_transacao': '2021-01-11 19:00:00'}, {'conta_id': 190983, 'valor': 1823.11, 'dt_transacao': '2021-01-27 00:00:00'}, {'conta_id': 190984, 'valor': 668.37, 'dt_transacao': '2021-03-23 22:00:00'}, {'conta_id': 190984, 'valor': 995.21, 'dt_transacao': '2021-02-06 19:00:00'}, {'conta_id': 190984, 'valor': 240.28, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190984, 'valor': 1764.68, 'dt_transacao': '2021-03-29 06:00:00'}, {'conta_id': 190985, 'valor': 1741.81, 'dt_transacao': '2021-03-23 18:00:00'}, {'conta_id': 190986, 'valor': 1173.53, 'dt_transacao': '2021-02-22 06:00:00'}, {'conta_id': 190987, 'valor': 676.05, 'dt_transacao': '2021-01-10 02:00:00'}, {'conta_id': 190987, 'valor': 511.04, 'dt_transacao': '2021-01-05 22:00:00'}, {'conta_id': 190987, 'valor': 1492.29, 'dt_transacao': '2021-02-07 04:00:00'}, {'conta_id': 190988, 'valor': 1871.25, 'dt_transacao': '2021-02-24 13:00:00'}, {'conta_id': 190988, 'valor': 1932.61, 'dt_transacao': '2021-01-06 16:00:00'}, {'conta_id': 190988, 'valor': 1387.68, 'dt_transacao': '2021-01-15 23:00:00'}, {'conta_id': 190988, 'valor': 258.44, 'dt_transacao': '2021-01-03 01:00:00'}, {'conta_id': 190989, 'valor': 308.83, 'dt_transacao': '2021-02-28 02:00:00'}, {'conta_id': 190989, 'valor': 641.44, 'dt_transacao': '2021-01-28 03:00:00'}, {'conta_id': 190989, 'valor': 1905.62, 'dt_transacao': '2021-01-28 08:00:00'}, {'conta_id': 190989, 'valor': 1795.09, 'dt_transacao': '2021-02-15 01:00:00'}, {'conta_id': 190990, 'valor': 422.51, 'dt_transacao': '2021-03-09 08:00:00'}, {'conta_id': 190990, 'valor': 581.51, 'dt_transacao': '2021-01-06 03:00:00'}, {'conta_id': 190990, 'valor': 484.96, 'dt_transacao': '2021-03-12 00:00:00'}, {'conta_id': 190990, 'valor': 1587.83, 'dt_transacao': '2021-02-27 06:00:00'}, {'conta_id': 190990, 'valor': 685.88, 'dt_transacao': '2021-01-31 15:00:00'}, {'conta_id': 190991, 'valor': 205.85, 'dt_transacao': '2021-02-17 14:00:00'}, {'conta_id': 190992, 'valor': 1983.8, 'dt_transacao': '2021-03-16 05:00:00'}, {'conta_id': 190992, 'valor': 806.31, 'dt_transacao': '2021-03-18 22:00:00'}, {'conta_id': 190992, 'valor': 1146.0, 'dt_transacao': '2021-03-22 13:00:00'}, {'conta_id': 190992, 'valor': 1533.49, 'dt_transacao': '2021-03-15 12:00:00'}, {'conta_id': 190993, 'valor': 497.15, 'dt_transacao': '2021-02-12 18:00:00'}, {'conta_id': 190993, 'valor': 206.79, 'dt_transacao': '2021-01-03 16:00:00'}, {'conta_id': 190993, 'valor': 1774.79, 'dt_transacao': '2021-03-14 19:00:00'}, {'conta_id': 190993, 'valor': 1584.8, 'dt_transacao': '2021-01-25 05:00:00'}, {'conta_id': 190993, 'valor': 1781.19, 'dt_transacao': '2021-01-14 05:00:00'}, {'conta_id': 190994, 'valor': 946.48, 'dt_transacao': '2021-02-20 20:00:00'}, {'conta_id': 190994, 'valor': 236.32, 'dt_transacao': '2021-02-23 00:00:00'}, {'conta_id': 190994, 'valor': 1284.23, 'dt_transacao': '2021-01-23 23:00:00'}, {'conta_id': 190994, 'valor': 432.6, 'dt_transacao': '2021-03-13 23:00:00'}, {'conta_id': 190994, 'valor': 456.99, 'dt_transacao': '2021-03-19 07:00:00'}, {'conta_id': 190995, 'valor': 1920.08, 'dt_transacao': '2021-03-26 03:00:00'}, {'conta_id': 190995, 'valor': 1963.21, 'dt_transacao': '2021-03-30 18:00:00'}, {'conta_id': 190995, 'valor': 1220.4, 'dt_transacao': '2021-02-13 00:00:00'}, {'conta_id': 190996, 'valor': 1567.42, 'dt_transacao': '2021-02-18 18:00:00'}, {'conta_id': 190997, 'valor': 1344.35, 'dt_transacao': '2021-03-29 04:00:00'}, {'conta_id': 190998, 'valor': 298.76, 'dt_transacao': '2021-03-17 09:00:00'}, {'conta_id': 190998, 'valor': 1699.68, 'dt_transacao': '2021-02-04 20:00:00'}, {'conta_id': 190998, 'valor': 1798.09, 'dt_transacao': '2021-01-04 05:00:00'}, {'conta_id': 190998, 'valor': 957.36, 'dt_transacao': '2021-01-29 00:00:00'}, {'conta_id': 190998, 'valor': 14.02, 'dt_transacao': '2021-03-07 13:00:00'}, {'conta_id': 190999, 'valor': 163.35, 'dt_transacao': '2021-01-25 19:00:00'}, {'conta_id': 191000, 'valor': 1260.09, 'dt_transacao': '2021-01-10 23:00:00'}, {'conta_id': 191000, 'valor': 643.46, 'dt_transacao': '2021-02-28 13:00:00'}, {'conta_id': 191000, 'valor': 1547.53, 'dt_transacao': '2021-03-22 16:00:00'}]))
